In [1]:
import numpy as np
import pandas as pd

In [2]:
extension = '20220719'

In [3]:
EyeChart = '/Users/nicolebrye/Desktop/HGC/Data_Management/REDCapMerge/20220719_EyeChartREDCapMerge/EyeChart/'
redcapeyechart = '/Users/nicolebrye/Desktop/HGC/Data_Management/REDCapMerge/20220719_EyeChartREDCapMerge/20220719_REDCapExports/'


In [4]:
import os
L_name = os.listdir(EyeChart)

In [5]:
import os
from os import path
#trying to make shift_graphs directory if it does not already exist:

if not os.path.exists(redcapeyechart + 'Results'):
    os.mkdir(redcapeyechart + 'Results')

In [6]:
#/Users/mhunsicker/Desktop/20190125EyeChartREDCapMerge

In [7]:
L1 = [i for i in L_name if '_ResearchIDStudyIndex' in i]

In [8]:
L1

['20170322_ResearchIDStudyIndex_CLEAN.csv']

In [9]:
ind = pd.read_csv(EyeChart + L1[0])
ind = ind.drop_duplicates(["OPHID_Text", "researchid"]).reset_index(drop=True)

In [10]:
ind

,OPHID_Text,researchid,study,anchor_date,end_date,flagged_for_exit,is_taxi_required,cStudyCodeWithDIGSFamiliesCollapsed
0,OPH-106121,AIS001G,Retinal Amyloid,09/22/2015,09/25/2015,0,0,Amyloid
1,OPH-105643,AIS002G,Retinal Amyloid,09/22/2015,09/25/2015,0,0,Amyloid
2,OPH-101003,AIS003G,Retinal Amyloid,09/23/2015,09/28/2015,0,0,Amyloid
3,OPH-105627,AIS004G,Retinal Amyloid,09/29/2015,10/02/2015,0,0,Amyloid
4,OPH-106586,AIS005G,Retinal Amyloid,09/29/2015,10/09/2015,0,0,Amyloid
...,...,...,...,...,...,...,...,...
8425,OPH-104167,TYX1438,ALG IMPL 091,NaN,NaN,0,1,ALGIMP3
8426,OPH-107071,UAR1464,ALG IMPL 091,NaN,NaN,0,0,ALGIMP3
8427,OPH-108848,UQN2048,ALG IMPL 091,NaN,NaN,0,0,ALGIMP3
8428,OPH-107070,WFJ1447,ALG IMPL 091,NaN,NaN,0,0,ALGIMP3


In [11]:
ind = ind[~ind['researchid'].str.contains('AIS')]

In [12]:
ind = ind[(ind['researchid'].str.contains('OG')) | (ind['researchid'].str.contains('ED'))]

In [13]:
ind["sd"] = np.where(ind['researchid'].str.contains('SD'), 1, 0)

In [14]:
ind["mul"] = ind.groupby('OPHID_Text').OPHID_Text.transform(lambda x:len(x))
ind = ind[~((ind['sd']==1) & (ind['mul']>1))]

In [15]:
ind = ind.drop_duplicates(['OPHID_Text']).reset_index(drop=True)
ind = ind.drop_duplicates(['researchid']).reset_index(drop=True)

In [16]:
ind['maskedid'] = np.array(ind['researchid'])
ind['OPHID_Text_1'] = np.array(ind['OPHID_Text'])

In [17]:
L_name = os.listdir(EyeChart)

In [18]:
L1 = [i for i in L_name if '20170322_FamMedHistory' in i]
df = pd.read_csv(EyeChart + L1[0])
df.rename(columns={'ophid': 'OPHID_Text'}, inplace=True)
df1 = pd.read_csv(redcapeyechart + extension +'_FamMedHistory.csv')
df1 = df1[~df1['fam_mom_avail'].isnull()]
name = 'maskedid	OPHID_Text	BioBID	site	visit_id	fmh_date	fmh_relation	fmh_cancer	fmh_diabetes	fmh_heart_disease	fmh_hypertension	fmh_stroke	fmh_skin_problems	fmh_kidney_problems	fmh_obesity	fmh_hearing_loss	fmh_extra_digits	fmh_other	source'
name = name.split('\t')
df["site"] = df['maskedid'].str[:2]
df["source"] = 'EyeChart'


df2 = pd.DataFrame(np.array(df1["maskedID"]), columns=["maskedid"])
df2["OPHID_Text"] = np.nan
df2["fmh_date"] = np.array(df1["fam_date"])
df2["fmh_relation"] = "MotherSide"


df1["fam_cancer___1"] = np.where(df1["fam_cancer___1"]==1, 'YES', 'NO')
df2["fmh_cancer"] = np.where(df1['fam_mom_avail']=='Yes', df1["fam_cancer___1"], np.nan)
df2["fmh_cancer"] = np.where(df1['fam_mom_avail']=='No', 'UNKNOWN TO PT', df2["fmh_cancer"])
df2["fmh_cancer"] = np.where(df1['fam_mom_avail']=='Decline', 'DECLINED', df2["fmh_cancer"])

##


df1["fam_diabetes___1"] = np.where(df1["fam_diabetes___1"]==1, 'YES', 'NO')
df2["fmh_diabetes"] = np.where(df1['fam_mom_avail']=='Yes', df1["fam_diabetes___1"], np.nan)
df2["fmh_diabetes"] = np.where(df1['fam_mom_avail']=='No', 'UNKNOWN TO PT', df2["fmh_diabetes"])
df2["fmh_diabetes"] = np.where(df1['fam_mom_avail']=='Decline', 'DECLINED', df2["fmh_diabetes"])

##


df1["fam_heartd___1"] = np.where(df1["fam_heartd___1"]==1, 'YES', 'NO')
df2["fmh_heart_disease"] = np.where(df1['fam_mom_avail']=='Yes', df1["fam_heartd___1"], np.nan)
df2["fmh_heart_disease"] = np.where(df1['fam_mom_avail']=='No', 'UNKNOWN TO PT', df2["fmh_heart_disease"])
df2["fmh_heart_disease"] = np.where(df1['fam_mom_avail']=='Decline', 'DECLINED', df2["fmh_heart_disease"])

##


df1["fam_hyper___1"] = np.where(df1["fam_hyper___1"]==1, 'YES', 'NO')
df2["fmh_hypertension"] = np.where(df1['fam_mom_avail']=='Yes', df1["fam_hyper___1"], np.nan)
df2["fmh_hypertension"] = np.where(df1['fam_mom_avail']=='No', 'UNKNOWN TO PT', df2["fmh_hypertension"])
df2["fmh_hypertension"] = np.where(df1['fam_mom_avail']=='Decline', 'DECLINED', df2["fmh_hypertension"])

##



df1["fam_stroke___1"] = np.where(df1["fam_stroke___1"]==1, 'YES', 'NO')
df2["fmh_stroke"] = np.where(df1['fam_mom_avail']=='Yes', df1["fam_stroke___1"], np.nan)
df2["fmh_stroke"] = np.where(df1['fam_mom_avail']=='No', 'UNKNOWN TO PT', df2["fmh_stroke"])
df2["fmh_stroke"] = np.where(df1['fam_mom_avail']=='Decline', 'DECLINED', df2["fmh_stroke"])

##


df1["fam_skin___1"] = np.where(df1["fam_skin___1"]==1, 'YES', 'NO')
df2["fmh_skin_problems"] = np.where(df1['fam_mom_avail']=='Yes', df1["fam_skin___1"], np.nan)
df2["fmh_skin_problems"] = np.where(df1['fam_mom_avail']=='No', 'UNKNOWN TO PT', df2["fmh_skin_problems"])
df2["fmh_skin_problems"] = np.where(df1['fam_mom_avail']=='Decline', 'DECLINED', df2["fmh_skin_problems"])

##



df1["fam_kidney___1"] = np.where(df1["fam_kidney___1"]==1, 'YES', 'NO')
df2["fmh_kidney_problems"] = np.where(df1['fam_mom_avail']=='Yes', df1["fam_kidney___1"], np.nan)
df2["fmh_kidney_problems"] = np.where(df1['fam_mom_avail']=='No', 'UNKNOWN TO PT', df2["fmh_kidney_problems"])
df2["fmh_kidney_problems"] = np.where(df1['fam_mom_avail']=='Decline', 'DECLINED', df2["fmh_kidney_problems"])

##


df1["fam_obese___1"] = np.where(df1["fam_obese___1"]==1, 'YES', 'NO')
df2["fmh_obesity"] = np.where(df1['fam_mom_avail']=='Yes', df1["fam_obese___1"], np.nan)
df2["fmh_obesity"] = np.where(df1['fam_mom_avail']=='No', 'UNKNOWN TO PT', df2["fmh_obesity"])
df2["fmh_obesity"] = np.where(df1['fam_mom_avail']=='Decline', 'DECLINED', df2["fmh_obesity"])

##


df1["fam_hear___1"] = np.where(df1["fam_hear___1"]==1, 'YES', 'NO')
df2["fmh_hearing_loss"] = np.where(df1['fam_mom_avail']=='Yes', df1["fam_hear___1"], np.nan)
df2["fmh_hearing_loss"] = np.where(df1['fam_mom_avail']=='No', 'UNKNOWN TO PT', df2["fmh_hearing_loss"])
df2["fmh_hearing_loss"] = np.where(df1['fam_mom_avail']=='Decline', 'DECLINED', df2["fmh_hearing_loss"])

##



df1["fam_extras___1"] = np.where(df1["fam_extras___1"]==1, 'YES', 'NO')
df2["fmh_extra_digits"] = np.where(df1['fam_mom_avail']=='Yes', df1["fam_extras___1"], np.nan)
df2["fmh_extra_digits"] = np.where(df1['fam_mom_avail']=='No', 'UNKNOWN TO PT', df2["fmh_extra_digits"])
df2["fmh_extra_digits"] = np.where(df1['fam_mom_avail']=='Decline', 'Decline', df2["fmh_extra_digits"])

##

df1["fam_others___1"] = np.where(df1["fam_others___1"]==1, df1['fam_mom_others_spec'], 'NO')
df2["fmh_other"] = np.where(df1['fam_mom_avail']=='Yes', df1["fam_others___1"], np.nan)
df2["fmh_other"] = np.where(df1['fam_mom_avail']=='No', 'UNKNOWN TO PT', df2["fmh_other"])
df2["fmh_other"] = np.where(df1['fam_mom_avail']=='Decline', 'DECLINED', df2["fmh_other"])

##

df2["site"] = np.array(df1["site"])
df2["visit_id"] = np.array(df1["visit_id"])
df2["source"] = 'REDCap'

df3 = df2.copy(deep=True)


df2 = pd.DataFrame(np.array(df1["maskedID"]), columns=["maskedid"])
df2["OPHID_Text"] = np.nan

df2["fmh_date"] = np.array(df1["fam_date"])
df2["fmh_relation"] = "FatherSide"

df1["fam_cancer___2"] = np.where(df1["fam_cancer___2"]==1, 'YES', 'NO')
df2["fmh_cancer"] = np.where(df1['fam_dad_avail']=='Yes', df1["fam_cancer___2"], np.nan)
df2["fmh_cancer"] = np.where(df1['fam_dad_avail']=='No', 'UNKNOWN TO PT', df2["fmh_cancer"])
df2["fmh_cancer"] = np.where(df1['fam_dad_avail']=='Decline', 'DECLINED', df2["fmh_cancer"])
##


df1["fam_diabetes___2"] = np.where(df1["fam_diabetes___2"]==1, 'YES', 'NO')
df2["fmh_diabetes"] = np.where(df1['fam_dad_avail']=='Yes', df1["fam_diabetes___2"], np.nan)
df2["fmh_diabetes"] = np.where(df1['fam_dad_avail']=='No', 'UNKNOWN TO PT', df2["fmh_diabetes"])
df2["fmh_diabetes"] = np.where(df1['fam_dad_avail']=='Decline', 'DECLINED', df2["fmh_diabetes"])

##


df1["fam_heartd___2"] = np.where(df1["fam_heartd___2"]==1, 'YES', 'NO')
df2["fmh_heart_disease"] = np.where(df1['fam_dad_avail']=='Yes', df1["fam_heartd___2"], np.nan)
df2["fmh_heart_disease"] = np.where(df1['fam_dad_avail']=='No', 'UNKNOWN TO PT', df2["fmh_heart_disease"])
df2["fmh_heart_disease"] = np.where(df1['fam_dad_avail']=='Decline', 'DECLINED', df2["fmh_heart_disease"])

##


df1["fam_hyper___2"] = np.where(df1["fam_hyper___2"]==1, 'YES', 'NO')
df2["fmh_hypertension"] = np.where(df1['fam_dad_avail']=='Yes', df1["fam_hyper___2"], np.nan)
df2["fmh_hypertension"] = np.where(df1['fam_dad_avail']=='No', 'UNKNOWN TO PT', df2["fmh_hypertension"])
df2["fmh_hypertension"] = np.where(df1['fam_dad_avail']=='Decline', 'DECLINED', df2["fmh_hypertension"])

##



df1["fam_stroke___2"] = np.where(df1["fam_stroke___2"]==1, 'YES', 'NO')
df2["fmh_stroke"] = np.where(df1['fam_dad_avail']=='Yes', df1["fam_stroke___2"], np.nan)
df2["fmh_stroke"] = np.where(df1['fam_dad_avail']=='No', 'UNKNOWN TO PT', df2["fmh_stroke"])
df2["fmh_stroke"] = np.where(df1['fam_dad_avail']=='Decline', 'DECLINED', df2["fmh_stroke"])

##

df1["fam_skin___2"] = np.where(df1["fam_skin___2"]==1, 'YES','NO')
df2["fmh_skin_problems"] = np.where(df1['fam_dad_avail']=='Yes', df1["fam_skin___2"], np.nan)
df2["fmh_skin_problems"] = np.where(df1['fam_dad_avail']=='No', 'UNKNOWN TO PT', df2["fmh_skin_problems"])
df2["fmh_skin_problems"] = np.where(df1['fam_dad_avail']=='Decline', 'DECLINED', df2["fmh_skin_problems"])

##



df1["fam_kidney___2"] = np.where(df1["fam_kidney___2"]==1, 'YES', 'NO')
df2["fmh_kidney_problems"] = np.where(df1['fam_dad_avail']=='Yes', df1["fam_kidney___2"], np.nan)
df2["fmh_kidney_problems"] = np.where(df1['fam_dad_avail']=='No', 'UNKNOWN TO PT', df2["fmh_kidney_problems"])
df2["fmh_kidney_problems"] = np.where(df1['fam_dad_avail']=='Decline', 'DECLINED', df2["fmh_kidney_problems"])

##


df1["fam_obese___2"] = np.where(df1["fam_obese___2"]==1, 'YES', 'NO')
df2["fmh_obesity"] = np.where(df1['fam_dad_avail']=='Yes', df1["fam_obese___2"], np.nan)
df2["fmh_obesity"] = np.where(df1['fam_dad_avail']=='No', 'UNKNOWN TO PT', df2["fmh_obesity"])
df2["fmh_obesity"] = np.where(df1['fam_dad_avail']=='Decline', 'DECLINED', df2["fmh_obesity"])

##


df1["fam_hear___2"] = np.where(df1["fam_hear___2"]==1, 'YES', 'NO')
df2["fmh_hearing_loss"] = np.where(df1['fam_dad_avail']=='Yes', df1["fam_hear___2"], np.nan)
df2["fmh_hearing_loss"] = np.where(df1['fam_dad_avail']=='No', 'UNKNOWN TO PT', df2["fmh_hearing_loss"])
df2["fmh_hearing_loss"] = np.where(df1['fam_dad_avail']=='Decline', 'DECLINED', df2["fmh_hearing_loss"])

##


df1["fam_extras___2"] = np.where(df1["fam_extras___2"]==1, 'YES', 'NO')
df2["fmh_extra_digits"] = np.where(df1['fam_dad_avail']=='Yes', df1["fam_extras___2"], np.nan)
df2["fmh_extra_digits"] = np.where(df1['fam_dad_avail']=='No', 'UNKNOWN TO PT', df2["fmh_extra_digits"])
df2["fmh_extra_digits"] = np.where(df1['fam_dad_avail']=='Decline', 'DECLINED', df2["fmh_extra_digits"])

##

df1["fam_others___2"] = np.where(df1["fam_others___2"]==1, df1['fam_dad_others_spec'], 'NO')
df2["fmh_other"] = np.where(df1['fam_dad_avail']=='Yes', df1["fam_others___2"], np.nan)
df2["fmh_other"] = np.where(df1['fam_dad_avail']=='No', 'UNKNOWN TO PT', df2["fmh_other"])
df2["fmh_other"] = np.where(df1['fam_dad_avail']=='Decline', 'DECLINED', df2["fmh_other"])

##

df2["site"] = np.array(df1["site"])
df2["visit_id"] = np.array(df1["visit_id"])
df2["source"] = 'REDCap'

df = df[df['maskedid'].notnull()]
H = list(df.columns.values) + ['visit_id']
FIN = pd.concat([df, df2[H]], ignore_index=True)
FIN = pd.concat([FIN, df3[H]], ignore_index=True)
FIN = pd.merge(FIN, ind[["OPHID_Text", "researchid"]], how='left', on='OPHID_Text')
FIN = pd.merge(FIN, ind[["OPHID_Text_1", "maskedid"]], how='left', on='maskedid')
FIN["OPHID_Text"] = np.where(FIN["OPHID_Text"].isnull(), FIN["OPHID_Text_1"], FIN["OPHID_Text"])

FIN['maskedid'] = np.where(FIN['maskedid'].isnull(), FIN['researchid'], FIN['maskedid'])
FIN['BioBID'] = np.where(FIN['researchid'].isnull(), FIN['maskedid'], FIN['researchid'])
FIN['BioBID'] = np.where( (FIN['researchid'].str.contains('OG')) | (FIN['researchid'].str.contains('ED')), FIN['BioBID'], np.nan)
FIN["site"] = FIN['maskedid'].str[:2]
FIN["mul"] = FIN.groupby(['maskedid', 'fmh_relation']).OPHID_Text.transform(lambda x:len(x))
FIN = FIN[~((FIN['source']=='REDCap') & (FIN['mul']>1))]
FIN = FIN[name]
FIN.to_csv(redcapeyechart + 'Results/' + extension + '_FamMedHistory_RedEC.csv', index=False)

In [19]:
L1 = [i for i in L_name if '20170322_FamOcHist' in i]
df = pd.read_csv(EyeChart + L1[0])

#df = pd.read_csv(EyeChart + '20170322_FamOcHist.csv')
df1 = pd.read_csv(redcapeyechart + extension + '_FamOcularHistory.csv')
name = 'maskedid	OPHID_Text	BioBID	site	visit_id	foh_mother_date	mother_blind_affected	mother_cataract_affected	mother_diaret_affected	mother_dryeye_affected	mother_eyedrops_affected	mother_eyeinject_affected	mother_glau_affected	mother_hadsurg_affected	mother_macdeg_affected	mother_hmyopia	mother_myopia	mother_other_occond	foh_father_date	father_blind_affected	father_cataract_affected	father_diaret_affected	father_dryeye_affected	father_eyedrops_affected	father_eyeinject_affected	father_glau_affected	father_hadsurg_affected	father_macdeg_affected	father_hmyopia	father_myopia	father_other_occond	foh_sisters_date	sister_cnt	sisters_blind_affected	sisters_cataract_affected	sisters_diaret_affected	sisters_dryeye_affected	sisters_eyedrops_affected	sisters_eyeinject_affected	sisters_glau_affected	sisters_hadsurg_affected	sisters_macdeg_affected	sisters_hmyopia	sisters_myopia	sisters_other_occond	foh_brothers_date	brother_cnt	brothers_blind_affected	brothers_cataract_affected	brothers_diaret_affected	brothers_dryeye_affected	brothers_eyedrops_affected	brothers_eyeinject_affected	brothers_glau_affected	brothers_hadsurg_affected	brothers_macdeg_affected	brothers_hmyopia	brothers_myopia	brothers_other_occond	foh_spouse_date	has_spouse	spouse_blind_affected	spouse_cataract_affected	spouse_diaret_affected	spouse_dryeye_affected	spouse_eyedrops_affected	spouse_eyeinject_affected	spouse_glau_affected	spouse_hadsurg_affected	spouse_macdeg_affected	spouse_hmyopia	spouse_myopia	spouse_other_occond	bba_blind_bro	bba_eyesurgery_sis	bba_eyesurgery_bro	bba_eyedrops_sis	bba_eyedrops_bro	bba_eyeinject_sis	bba_eyeinject_bro	bba_glaucoma_sis	bba_glaucoma_bro	bba_macdgnrtn_sis	bba_macdgnrtn_bro	bba_diabrtnpthy_sis	bba_diabrtnpthy_bro	bba_cataract_sis	bba_cataract_bro	bba_dryeye_sis	bba_dryeye_bro	bba_myopia_sis	bba_myopia_bro	bba_hmyopia_sis	bba_hmyopia_bro	bba_ocother_sis	bba_ocother_bro	source'
name = name.split('\t')

df["site"] = df['maskedid'].str[:2]
df["source"] = 'EyeChart'
df["visit_id"] = np.nan
df2 = pd.DataFrame(np.array(df1["maskedID"]), columns=["maskedid"])
df2["OPHID_Text"] = np.nan
df2["foh_mother_date"] = np.array(df1["fam_date"])
df2["visit_id"] = np.array(df1["visit_id"])
df2["mother_blind_affected"] = np.where(df1["fam_blind___1"]== 1, 'YES', 'NO')
df2["mother_blind_affected"] = np.where(df1["fam_mom_oc"]=='Unknown', 'UNKNOWN TO PT', df2["mother_blind_affected"])
df2["mother_blind_affected"] = np.where(df1["fam_mom_oc"]=='No', 'DECLINED', df2["mother_blind_affected"])
df2["mother_blind_affected"] = np.where(df1["fam_mom_oc"].isnull(), 'N/A', df2["mother_blind_affected"])


df2["mother_cataract_affected"] = np.where(df1["fam_cataract___1"]== 1, 'YES', 'NO')
df2["mother_cataract_affected"] = np.where(df1["fam_mom_oc"]=='Unknown', 'UNKNOWN TO PT', df2["mother_cataract_affected"])
df2["mother_cataract_affected"] = np.where(df1["fam_mom_oc"]=='No', 'DECLINED', df2["mother_cataract_affected"])
df2["mother_cataract_affected"] = np.where(df1["fam_mom_oc"].isnull(), 'N/A', df2["mother_cataract_affected"])


df2["mother_diaret_affected"] = np.where(df1["fam_diabrtnpthy___1"]== 1, 'YES', 'NO')
df2["mother_diaret_affected"] = np.where(df1["fam_mom_oc"]=='Unknown', 'UNKNOWN TO PT', df2["mother_diaret_affected"])
df2["mother_diaret_affected"] = np.where(df1["fam_mom_oc"]=='No', 'DECLINED', df2["mother_diaret_affected"])
df2["mother_diaret_affected"] = np.where(df1["fam_mom_oc"].isnull(), 'N/A', df2["mother_diaret_affected"])


df2["mother_dryeye_affected"] = np.where(df1["fam_dryeye___1"]== 1, 'YES', 'NO')
df2["mother_dryeye_affected"] = np.where(df1["fam_mom_oc"]=='Unknown', 'UNKNOWN TO PT', df2["mother_dryeye_affected"])
df2["mother_dryeye_affected"] = np.where(df1["fam_mom_oc"]=='No', 'DECLINED', df2["mother_dryeye_affected"])
df2["mother_dryeye_affected"] = np.where(df1["fam_mom_oc"].isnull(), 'N/A', df2["mother_dryeye_affected"])


df2["mother_eyedrops_affected"] = np.where(df1["fam_eyedrops___1"]== 1, 'YES', 'NO')
df2["mother_eyedrops_affected"] = np.where(df1["fam_mom_oc"]=='Unknown', 'UNKNOWN TO PT', df2["mother_eyedrops_affected"])
df2["mother_eyedrops_affected"] = np.where(df1["fam_mom_oc"]=='No', 'DECLINED', df2["mother_eyedrops_affected"])
df2["mother_eyedrops_affected"] = np.where(df1["fam_mom_oc"].isnull(), 'N/A', df2["mother_eyedrops_affected"])


df2["mother_eyeinject_affected"] = np.where(df1["fam_eyeinject___1"]== 1, 'YES', 'NO')
df2["mother_eyeinject_affected"] = np.where(df1["fam_mom_oc"]=='Unknown', 'UNKNOWN TO PT', df2["mother_eyeinject_affected"])
df2["mother_eyeinject_affected"] = np.where(df1["fam_mom_oc"]=='No', 'DECLINED', df2["mother_eyeinject_affected"])
df2["mother_eyeinject_affected"] = np.where(df1["fam_mom_oc"].isnull(), 'N/A', df2["mother_eyeinject_affected"])


df2["mother_glau_affected"] = np.where(df1["fam_glaucoma___1"]== 1, 'YES', 'NO')
df2["mother_glau_affected"] = np.where(df1["fam_mom_oc"]=='Unknown', 'UNKNOWN TO PT', df2["mother_glau_affected"])
df2["mother_glau_affected"] = np.where(df1["fam_mom_oc"]=='No', 'DECLINED', df2["mother_glau_affected"])
df2["mother_glau_affected"] = np.where(df1["fam_mom_oc"].isnull(), 'N/A', df2["mother_glau_affected"])


df2["mother_hadsurg_affected"] = np.where(df1["fam_eyesurgery___1"]== 1, 'YES', 'NO')
df2["mother_hadsurg_affected"] = np.where(df1["fam_mom_oc"]=='Unknown', 'UNKNOWN TO PT', df2["mother_hadsurg_affected"])
df2["mother_hadsurg_affected"] = np.where(df1["fam_mom_oc"]=='No', 'DECLINED', df2["mother_hadsurg_affected"])
df2["mother_hadsurg_affected"] = np.where(df1["fam_mom_oc"].isnull(), 'N/A', df2["mother_hadsurg_affected"])



df2["mother_macdeg_affected"] = np.where(df1["fam_macdgnrtn___1"]== 1, 'YES', 'NO')
df2["mother_macdeg_affected"] = np.where(df1["fam_mom_oc"]=='Unknown', 'UNKNOWN TO PT', df2["mother_macdeg_affected"])
df2["mother_macdeg_affected"] = np.where(df1["fam_mom_oc"]=='No', 'DECLINED', df2["mother_macdeg_affected"])
df2["mother_macdeg_affected"] = np.where(df1["fam_mom_oc"].isnull(), 'N/A', df2["mother_macdeg_affected"])



df2["mother_other_occond"] = np.where(df1["fam_ocother___1"]== 1, 'YES', 'NO')
df2["mother_other_occond"] = np.where(df1["fam_mom_oc"]=='Unknown', 'UNKNOWN TO PT', df2["mother_other_occond"])
df2["mother_other_occond"] = np.where(df1["fam_mom_oc"]=='No', 'DECLINED', df2["mother_other_occond"])
df2["mother_other_occond"] = np.where(df1["fam_mom_oc"].isnull(), 'N/A', df2["mother_other_occond"])


####
df2["mother_hmyopia"] = np.where(df1["fam_hmyopia___1"]== 1, 'YES', 'NO')
df2["mother_hmyopia"] = np.where(df1["fam_mom_oc"]=='Unknown', 'UNKNOWN TO PT', df2["mother_hmyopia"])
df2["mother_hmyopia"] = np.where(df1["fam_mom_oc"]=='No', 'DECLINED', df2["mother_hmyopia"])
df2["mother_hmyopia"] = np.where(df1["fam_mom_oc"].isnull(), 'N/A', df2["mother_hmyopia"])
df["mother_hmyopia"] = np.nan


df2["mother_myopia"] = np.where(df1["fam_myopia___1"]== 1, 'YES', 'NO')
df2["mother_myopia"] = np.where(df1["fam_mom_oc"]=='Unknown', 'UNKNOWN TO PT', df2["mother_myopia"])
df2["mother_myopia"] = np.where(df1["fam_mom_oc"]=='No', 'DECLINED', df2["mother_myopia"])
df2["mother_myopia"] = np.where(df1["fam_mom_oc"].isnull(), 'N/A', df2["mother_myopia"])
df["mother_myopia"] = np.nan

####

df2["foh_father_date"] = np.array(df1["fam_date"])

df2["father_blind_affected"] = np.where(df1["fam_blind___2"]== 1, 'YES', 'NO')
df2["father_blind_affected"] = np.where(df1["fam_dad_oc"]=='Unknown', 'UNKNOWN TO PT', df2["father_blind_affected"])
df2["father_blind_affected"] = np.where(df1["fam_dad_oc"]=='No', 'DECLINED', df2["father_blind_affected"])
df2["father_blind_affected"] = np.where(df1["fam_dad_oc"].isnull(), 'N/A', df2["father_blind_affected"])


df2["father_cataract_affected"] = np.where(df1["fam_cataract___2"]== 1, 'YES', 'NO')
df2["father_cataract_affected"] = np.where(df1["fam_dad_oc"]=='Unknown', 'UNKNOWN TO PT', df2["father_cataract_affected"])
df2["father_cataract_affected"] = np.where(df1["fam_dad_oc"]=='No', 'DECLINED', df2["father_cataract_affected"])
df2["father_cataract_affected"] = np.where(df1["fam_dad_oc"].isnull(), 'N/A', df2["father_cataract_affected"])


df2["father_diaret_affected"] = np.where(df1["fam_diabrtnpthy___2"]== 1, 'YES', 'NO')
df2["father_diaret_affected"] = np.where(df1["fam_dad_oc"]=='Unknown', 'UNKNOWN TO PT', df2["father_diaret_affected"])
df2["father_diaret_affected"] = np.where(df1["fam_dad_oc"]=='No', 'DECLINED', df2["father_diaret_affected"])
df2["father_diaret_affected"] = np.where(df1["fam_dad_oc"].isnull(), 'N/A', df2["father_diaret_affected"])


df2["father_dryeye_affected"] = np.where(df1["fam_dryeye___2"]== 1, 'YES', 'NO')
df2["father_dryeye_affected"] = np.where(df1["fam_dad_oc"]=='Unknown', 'UNKNOWN TO PT', df2["father_dryeye_affected"])
df2["father_dryeye_affected"] = np.where(df1["fam_dad_oc"]=='No', 'DECLINED', df2["father_dryeye_affected"])
df2["father_dryeye_affected"] = np.where(df1["fam_dad_oc"].isnull(), 'N/A', df2["father_dryeye_affected"])


df2["father_eyedrops_affected"] = np.where(df1["fam_eyedrops___2"]== 1, 'YES', 'NO')
df2["father_eyedrops_affected"] = np.where(df1["fam_dad_oc"]=='Unknown', 'UNKNOWN TO PT', df2["father_eyedrops_affected"])
df2["father_eyedrops_affected"] = np.where(df1["fam_dad_oc"]=='No', 'DECLINED', df2["father_eyedrops_affected"])
df2["father_eyedrops_affected"] = np.where(df1["fam_dad_oc"].isnull(), 'N/A', df2["father_eyedrops_affected"])


df2["father_eyeinject_affected"] = np.where(df1["fam_eyeinject___2"]== 1, 'YES', 'NO')
df2["father_eyeinject_affected"] = np.where(df1["fam_dad_oc"]=='Unknown', 'UNKNOWN TO PT', df2["father_eyeinject_affected"])
df2["father_eyeinject_affected"] = np.where(df1["fam_dad_oc"]=='No', 'DECLINED', df2["father_eyeinject_affected"])
df2["father_eyeinject_affected"] = np.where(df1["fam_dad_oc"].isnull(), 'N/A', df2["father_eyeinject_affected"])



df2["father_glau_affected"] = np.where(df1["fam_glaucoma___2"]== 1, 'YES', 'NO')
df2["father_glau_affected"] = np.where(df1["fam_dad_oc"]=='Unknown', 'UNKNOWN TO PT', df2["father_glau_affected"])
df2["father_glau_affected"] = np.where(df1["fam_dad_oc"]=='No', 'DECLINED', df2["father_glau_affected"])
df2["father_glau_affected"] = np.where(df1["fam_dad_oc"].isnull(), 'N/A', df2["father_glau_affected"])


df2["father_hadsurg_affected"] = np.where(df1["fam_eyesurgery___2"]== 1, 'YES', 'NO')
df2["father_hadsurg_affected"] = np.where(df1["fam_dad_oc"]=='Unknown', 'UNKNOWN TO PT', df2["father_hadsurg_affected"])
df2["father_hadsurg_affected"] = np.where(df1["fam_dad_oc"]=='No', 'DECLINED', df2["father_hadsurg_affected"])
df2["father_hadsurg_affected"] = np.where(df1["fam_dad_oc"].isnull(), 'N/A', df2["father_hadsurg_affected"])



df2["father_macdeg_affected"] = np.where(df1["fam_macdgnrtn___2"]== 1, 'YES', 'NO')
df2["father_macdeg_affected"] = np.where(df1["fam_dad_oc"]=='Unknown', 'UNKNOWN TO PT', df2["father_macdeg_affected"])
df2["father_macdeg_affected"] = np.where(df1["fam_dad_oc"]=='No', 'DECLINED', df2["father_macdeg_affected"])
df2["father_macdeg_affected"] = np.where(df1["fam_dad_oc"].isnull(), 'N/A', df2["father_macdeg_affected"])



df2["father_other_occond"] = np.where(df1["fam_ocother___2"]== 1, 'YES', 'NO')
df2["father_other_occond"] = np.where(df1["fam_dad_oc"]=='Unknown', 'UNKNOWN TO PT', df2["father_other_occond"])
df2["father_other_occond"] = np.where(df1["fam_dad_oc"]=='No', 'DECLINED', df2["father_other_occond"])
df2["father_other_occond"] = np.where(df1["fam_dad_oc"].isnull(), 'N/A', df2["father_other_occond"])


####
df2["father_hmyopia"] = np.where(df1["fam_hmyopia___2"]== 1, 'YES', 'NO')
df2["father_hmyopia"] = np.where(df1["fam_dad_oc"].isnull(), 'N/A', df2["father_hmyopia"])
df2["father_hmyopia"] = np.where(df1["fam_dad_oc"]=='No', 'DECLINED', df2["father_hmyopia"])
df2["father_hmyopia"] = np.where(df1["fam_dad_oc"].isnull(), 'N/A', df2["father_hmyopia"])
df["father_hmyopia"] = np.nan

df2["father_myopia"] = np.where(df1["fam_myopia___2"]== 1, 'YES', 'NO')
df2["father_myopia"] = np.where(df1["fam_dad_oc"].isnull(), 'N/A', df2["father_myopia"])
df2["father_hmyopia"] = np.where(df1["fam_dad_oc"]=='No', 'DECLINED', df2["father_hmyopia"])
df2["father_hmyopia"] = np.where(df1["fam_dad_oc"].isnull(), 'N/A', df2["father_hmyopia"])
df["father_myopia"] = np.nan

####

df2["foh_sisters_date"] = np.array(df1["fam_date"])
df2["sister_cnt"] = np.array(df1["fam_sisters"])
df2["sisters_blind_affected"] = np.where(df1["fam_blind___3"]== 1, 'YES', 'NO')
df2["sisters_blind_affected"] = np.where(df1["fam_sis_oc"]=='Unknown', 'UNKNOWN TO PT', df2["sisters_blind_affected"])
df2["sisters_blind_affected"] = np.where(df1["fam_sis_oc"]=='No', 'DECLINED', df2["sisters_blind_affected"])
df2["sisters_blind_affected"] = np.where(df1["fam_sis_oc"].isnull(), 'N/A', df2["sisters_blind_affected"])


df2["sisters_cataract_affected"] = np.where(df1["fam_cataract___3"]== 1, 'YES', 'NO')
df2["sisters_cataract_affected"] = np.where(df1["fam_sis_oc"]=='Unknown', 'UNKNOWN TO PT', df2["sisters_cataract_affected"])
df2["sisters_cataract_affected"] = np.where(df1["fam_sis_oc"]=='No', 'DECLINED', df2["sisters_cataract_affected"])
df2["sisters_cataract_affected"] = np.where(df1["fam_sis_oc"].isnull(), 'N/A', df2["sisters_cataract_affected"])


df2["sisters_diaret_affected"] = np.where(df1["fam_diabrtnpthy___3"]== 1, 'YES', 'NO')
df2["sisters_diaret_affected"] = np.where(df1["fam_sis_oc"]=='Unknown', 'UNKNOWN TO PT', df2["sisters_diaret_affected"])
df2["sisters_diaret_affected"] = np.where(df1["fam_sis_oc"]=='No', 'DECLINED', df2["sisters_diaret_affected"])
df2["sisters_diaret_affected"] = np.where(df1["fam_sis_oc"].isnull(), 'N/A', df2["sisters_diaret_affected"])


df2["sisters_dryeye_affected"] = np.where(df1["fam_dryeye___3"]== 1, 'YES', 'NO')
df2["sisters_dryeye_affected"] = np.where(df1["fam_sis_oc"]=='Unknown', 'UNKNOWN TO PT', df2["sisters_dryeye_affected"])
df2["sisters_dryeye_affected"] = np.where(df1["fam_sis_oc"]=='No', 'DECLINED', df2["sisters_dryeye_affected"])
df2["sisters_dryeye_affected"] = np.where(df1["fam_sis_oc"].isnull(), 'N/A', df2["sisters_dryeye_affected"])


df2["sisters_eyedrops_affected"] = np.where(df1["fam_eyedrops___3"]== 1, 'YES', 'NO')
df2["sisters_eyedrops_affected"] = np.where(df1["fam_sis_oc"]=='Unknown', 'UNKNOWN TO PT', df2["sisters_eyedrops_affected"])
df2["sisters_eyedrops_affected"] = np.where(df1["fam_sis_oc"]=='No', 'DECLINED', df2["sisters_eyedrops_affected"])
df2["sisters_eyedrops_affected"] = np.where(df1["fam_sis_oc"].isnull(), 'N/A', df2["sisters_eyedrops_affected"])


df2["sisters_eyeinject_affected"] = np.where(df1["fam_eyeinject___3"]== 1, 'YES', 'NO')
df2["sisters_eyeinject_affected"] = np.where(df1["fam_sis_oc"]=='Unknown', 'UNKNOWN TO PT', df2["sisters_eyeinject_affected"])
df2["sisters_eyeinject_affected"] = np.where(df1["fam_sis_oc"]=='No', 'DECLINED', df2["sisters_eyeinject_affected"])
df2["sisters_eyeinject_affected"] = np.where(df1["fam_sis_oc"].isnull(), 'N/A', df2["sisters_eyeinject_affected"])


df2["sisters_glau_affected"] = np.where(df1["fam_glaucoma___3"]== 1, 'YES', 'NO')
df2["sisters_glau_affected"] = np.where(df1["fam_sis_oc"]=='Unknown', 'UNKNOWN TO PT', df2["sisters_glau_affected"])
df2["sisters_glau_affected"] = np.where(df1["fam_sis_oc"]=='No', 'DECLINED', df2["sisters_glau_affected"])
df2["sisters_glau_affected"] = np.where(df1["fam_sis_oc"].isnull(), 'N/A', df2["sisters_glau_affected"])


df2["sisters_hadsurg_affected"] = np.where(df1["fam_eyesurgery___3"]== 1, 'YES', 'NO')
df2["sisters_hadsurg_affected"] = np.where(df1["fam_sis_oc"]=='Unknown', 'UNKNOWN TO PT', df2["sisters_hadsurg_affected"])
df2["sisters_hadsurg_affected"] = np.where(df1["fam_sis_oc"]=='No', 'DECLINED', df2["sisters_hadsurg_affected"])
df2["sisters_hadsurg_affected"] = np.where(df1["fam_sis_oc"].isnull(), 'N/A', df2["sisters_hadsurg_affected"])


df2["sisters_macdeg_affected"] = np.where(df1["fam_macdgnrtn___3"]== 1, 'YES', 'NO')
df2["sisters_macdeg_affected"] = np.where(df1["fam_sis_oc"]=='Unknown', 'UNKNOWN TO PT', df2["sisters_macdeg_affected"])
df2["sisters_macdeg_affected"] = np.where(df1["fam_sis_oc"]=='No', 'DECLINED', df2["sisters_macdeg_affected"])
df2["sisters_macdeg_affected"] = np.where(df1["fam_sis_oc"].isnull(), 'N/A', df2["sisters_macdeg_affected"])



df2["sisters_other_occond"] = np.where(df1["fam_ocother___3"]== 1, 'YES', 'NO')
df2["sisters_other_occond"] = np.where(df1["fam_sis_oc"]=='Unknown', 'UNKNOWN TO PT', df2["sisters_other_occond"])
df2["sisters_other_occond"] = np.where(df1["fam_sis_oc"]=='No', 'DECLINED', df2["sisters_other_occond"])
df2["sisters_other_occond"] = np.where(df1["fam_sis_oc"].isnull(), 'N/A', df2["sisters_other_occond"])



####
df2["sisters_hmyopia"] = np.where(df1["fam_hmyopia___3"]== 1, 'YES', 'NO')
df2["sisters_hmyopia"] = np.where(df1["fam_sis_oc"]=='Unknown', 'UNKNOWN TO PT', df2["sisters_hmyopia"])
df2["sisters_hmyopia"] = np.where(df1["fam_sis_oc"]=='No', 'DECLINED', df2["sisters_hmyopia"])
df2["sisters_hmyopia"] = np.where(df1["fam_sis_oc"].isnull(), 'N/A', df2["sisters_hmyopia"])
df["sisters_hmyopia"] = np.nan

df2["sisters_myopia"] = np.where(df1["fam_myopia___3"]== 1, 'YES', 'NO')
df2["sisters_myopia"] = np.where(df1["fam_sis_oc"]=='Unknown', 'UNKNOWN TO PT', df2["sisters_myopia"])
df2["sisters_myopia"] = np.where(df1["fam_sis_oc"]=='No', 'DECLINED', df2["sisters_myopia"])
df2["sisters_myopia"] = np.where(df1["fam_sis_oc"].isnull(), 'N/A', df2["sisters_myopia"])
df["sisters_myopia"] = np.nan

####


df2["foh_brothers_date"] = np.array(df1["fam_date"])
df2["brother_cnt"] = np.array(df1["fam_brothers"])
df2["brothers_blind_affected"] = np.where(df1["fam_blind___4"]== 1, 'YES', 'NO')
df2["brothers_blind_affected"] = np.where(df1["fam_bro_oc"]=='Unknown', 'UNKNOWN TO PT', df2["brothers_blind_affected"])
df2["brothers_blind_affected"] = np.where(df1["fam_bro_oc"]=='No', 'DECLINED', df2["brothers_blind_affected"])
df2["brothers_blind_affected"] = np.where(df1["fam_bro_oc"].isnull(), 'N/A', df2["brothers_blind_affected"])



df2["brothers_cataract_affected"] = np.where(df1["fam_cataract___4"]== 1, 'YES', 'NO')
df2["brothers_cataract_affected"] = np.where(df1["fam_bro_oc"]=='Unknown', 'UNKNOWN TO PT', df2["brothers_cataract_affected"])
df2["brothers_cataract_affected"] = np.where(df1["fam_bro_oc"]=='No', 'DECLINED', df2["brothers_cataract_affected"])
df2["brothers_cataract_affected"] = np.where(df1["fam_bro_oc"].isnull(), 'N/A', df2["brothers_cataract_affected"])



df2["brothers_diaret_affected"] = np.where(df1["fam_diabrtnpthy___4"]== 1, 'YES', 'NO')
df2["brothers_diaret_affected"] = np.where(df1["fam_bro_oc"]=='Unknown', 'UNKNOWN TO PT', df2["brothers_diaret_affected"])
df2["brothers_diaret_affected"] = np.where(df1["fam_bro_oc"]=='No', 'DECLINED', df2["brothers_diaret_affected"])
df2["brothers_diaret_affected"] = np.where(df1["fam_bro_oc"].isnull(), 'N/A', df2["brothers_diaret_affected"])



df2["brothers_dryeye_affected"] = np.where(df1["fam_dryeye___4"]== 1, 'YES', 'NO')
df2["brothers_dryeye_affected"] = np.where(df1["fam_bro_oc"]=='Unknown', 'UNKNOWN TO PT', df2["brothers_dryeye_affected"])
df2["brothers_dryeye_affected"] = np.where(df1["fam_bro_oc"]=='No', 'DECLINED', df2["brothers_dryeye_affected"])
df2["brothers_dryeye_affected"] = np.where(df1["fam_bro_oc"].isnull(), 'N/A', df2["brothers_dryeye_affected"])



df2["brothers_eyedrops_affected"] = np.where(df1["fam_eyedrops___4"]== 1, 'YES', 'NO')
df2["brothers_eyedrops_affected"] = np.where(df1["fam_bro_oc"]=='Unknown', 'UNKNOWN TO PT', df2["brothers_eyedrops_affected"])
df2["brothers_eyedrops_affected"] = np.where(df1["fam_bro_oc"]=='No', 'DECLINED', df2["brothers_eyedrops_affected"])
df2["brothers_eyedrops_affected"] = np.where(df1["fam_bro_oc"].isnull(), 'N/A', df2["brothers_eyedrops_affected"])



df2["brothers_eyeinject_affected"] = np.where(df1["fam_eyeinject___4"]== 1, 'YES', 'NO')
df2["brothers_eyeinject_affected"] = np.where(df1["fam_bro_oc"]=='Unknown', 'UNKNOWN TO PT', df2["brothers_eyeinject_affected"])
df2["brothers_eyeinject_affected"] = np.where(df1["fam_bro_oc"]=='No', 'DECLINED', df2["brothers_eyeinject_affected"])
df2["brothers_eyeinject_affected"] = np.where(df1["fam_bro_oc"].isnull(), 'N/A', df2["brothers_eyeinject_affected"])



df2["brothers_glau_affected"] = np.where(df1["fam_glaucoma___4"]== 1, 'YES', 'NO')
df2["brothers_glau_affected"] = np.where(df1["fam_bro_oc"]=='Unknown', 'UNKNOWN TO PT', df2["brothers_glau_affected"])
df2["brothers_glau_affected"] = np.where(df1["fam_bro_oc"]=='No', 'DECLINED', df2["brothers_glau_affected"])
df2["brothers_glau_affected"] = np.where(df1["fam_bro_oc"].isnull(), 'N/A', df2["brothers_glau_affected"])



df2["brothers_hadsurg_affected"] = np.where(df1["fam_eyesurgery___4"]== 1, 'YES', 'NO')
df2["brothers_hadsurg_affected"] = np.where(df1["fam_bro_oc"]=='Unknown', 'UNKNOWN TO PT', df2["brothers_hadsurg_affected"])
df2["brothers_hadsurg_affected"] = np.where(df1["fam_bro_oc"]=='No', 'DECLINED', df2["brothers_hadsurg_affected"])
df2["brothers_hadsurg_affected"] = np.where(df1["fam_bro_oc"].isnull(), 'N/A', df2["brothers_hadsurg_affected"])




df2["brothers_macdeg_affected"] = np.where(df1["fam_macdgnrtn___4"]== 1, 'YES', 'NO')
df2["brothers_macdeg_affected"] = np.where(df1["fam_bro_oc"]=='Unknown', 'UNKNOWN TO PT', df2["brothers_macdeg_affected"])
df2["brothers_macdeg_affected"] = np.where(df1["fam_bro_oc"]=='No', 'DECLINED', df2["brothers_macdeg_affected"])
df2["brothers_macdeg_affected"] = np.where(df1["fam_bro_oc"].isnull(), 'N/A', df2["brothers_macdeg_affected"])




df2["brothers_other_occond"] = np.where(df1["fam_ocother___4"]== 1, 'YES', 'NO')
df2["brothers_other_occond"] = np.where(df1["fam_bro_oc"]=='Unknown', 'UNKNOWN TO PT', df2["brothers_other_occond"])
df2["brothers_other_occond"] = np.where(df1["fam_bro_oc"]=='No', 'DECLINED', df2["brothers_other_occond"])
df2["brothers_other_occond"] = np.where(df1["fam_bro_oc"].isnull(), 'N/A', df2["brothers_other_occond"])




####
df2["brothers_hmyopia"] = np.where(df1["fam_hmyopia___4"]== 1, 'YES', 'NO')
df2["brothers_hmyopia"] = np.where(df1["fam_bro_oc"]=='Unknown', 'UNKNOWN TO PT', df2["brothers_hmyopia"])
df2["brothers_hmyopia"] = np.where(df1["fam_bro_oc"]=='No', 'DECLINED', df2["brothers_hmyopia"])
df2["brothers_hmyopia"] = np.where(df1["fam_bro_oc"].isnull(), 'N/A', df2["brothers_hmyopia"])
df["brothers_hmyopia"] = np.nan

df2["brothers_myopia"] = np.where(df1["fam_myopia___4"]== 1, 'YES', 'NO')
df2["brothers_myopia"] = np.where(df1["fam_bro_oc"]=='Unknown', 'UNKNOWN TO PT', df2["brothers_myopia"])
df2["brothers_myopia"] = np.where(df1["fam_bro_oc"]=='No', 'DECLINED', df2["brothers_myopia"])
df2["brothers_myopia"] = np.where(df1["fam_bro_oc"].isnull(), 'N/A', df2["brothers_myopia"])
df["brothers_myopia"] = np.nan

####


df2["foh_spouse_date"] = np.array(df1["fam_date"])
df2["has_spouse"] = np.nan
df2["spouse_blind_affected"] = np.where(df1["fam_blind___5"]== 1, 'YES', 'NO')
df2["spouse_blind_affected"] = np.where(df1["fam_spouse_oc"]=='Unknown', 'UNKNOWN TO PT', df2["spouse_blind_affected"])
df2["spouse_blind_affected"] = np.where(df1["fam_spouse_oc"]=='No', 'DECLINED', df2["spouse_blind_affected"])
df2["spouse_blind_affected"] = np.where(df1["fam_spouse_oc"].isnull(), 'N/A', df2["spouse_blind_affected"])



df2["spouse_cataract_affected"] = np.where(df1["fam_cataract___5"]== 1, 'YES', 'NO')
df2["spouse_cataract_affected"] = np.where(df1["fam_spouse_oc"]=='Unknown', 'UNKNOWN TO PT', df2["spouse_cataract_affected"])
df2["spouse_cataract_affected"] = np.where(df1["fam_spouse_oc"]=='No', 'DECLINED', df2["spouse_cataract_affected"])
df2["spouse_cataract_affected"] = np.where(df1["fam_spouse_oc"].isnull(), 'N/A', df2["spouse_cataract_affected"])



df2["spouse_diaret_affected"] = np.where(df1["fam_diabrtnpthy___5"]== 1, 'YES', 'NO')
df2["spouse_diaret_affected"] = np.where(df1["fam_spouse_oc"]=='Unknown', 'UNKNOWN TO PT', df2["spouse_diaret_affected"])
df2["spouse_diaret_affected"] = np.where(df1["fam_spouse_oc"]=='No', 'DECLINED', df2["spouse_diaret_affected"])
df2["spouse_diaret_affected"] = np.where(df1["fam_spouse_oc"].isnull(), 'N/A', df2["spouse_diaret_affected"])



df2["spouse_dryeye_affected"] = np.where(df1["fam_dryeye___5"]== 1, 'YES', 'NO')
df2["spouse_dryeye_affected"] = np.where(df1["fam_spouse_oc"]=='Unknown', 'UNKNOWN TO PT', df2["spouse_dryeye_affected"])
df2["spouse_dryeye_affected"] = np.where(df1["fam_spouse_oc"]=='No', 'DECLINED', df2["spouse_dryeye_affected"])
df2["spouse_dryeye_affected"] = np.where(df1["fam_spouse_oc"].isnull(), 'N/A', df2["spouse_dryeye_affected"])



df2["spouse_eyedrops_affected"] = np.where(df1["fam_eyedrops___5"]== 1, 'YES', 'NO')
df2["spouse_eyedrops_affected"] = np.where(df1["fam_spouse_oc"]=='Unknown', 'UNKNOWN TO PT', df2["spouse_eyedrops_affected"])
df2["spouse_eyedrops_affected"] = np.where(df1["fam_spouse_oc"]=='No', 'DECLINED', df2["spouse_eyedrops_affected"])
df2["spouse_eyedrops_affected"] = np.where(df1["fam_spouse_oc"].isnull(), 'N/A', df2["spouse_eyedrops_affected"])



df2["spouse_eyeinject_affected"] = np.where(df1["fam_eyeinject___5"]== 1, 'YES', 'NO')
df2["spouse_eyeinject_affected"] = np.where(df1["fam_spouse_oc"]=='Unknown', 'UNKNOWN TO PT', df2["spouse_eyeinject_affected"])
df2["spouse_eyeinject_affected"] = np.where(df1["fam_spouse_oc"]=='No', 'DECLINED', df2["spouse_eyeinject_affected"])
df2["spouse_eyeinject_affected"] = np.where(df1["fam_spouse_oc"].isnull(), 'N/A', df2["spouse_eyeinject_affected"])



df2["spouse_glau_affected"] = np.where(df1["fam_glaucoma___5"]== 1, 'YES', 'NO')
df2["spouse_glau_affected"] = np.where(df1["fam_spouse_oc"]=='Unknown', 'UNKNOWN TO PT', df2["spouse_glau_affected"])
df2["spouse_glau_affected"] = np.where(df1["fam_spouse_oc"]=='No', 'DECLINED', df2["spouse_glau_affected"])
df2["spouse_glau_affected"] = np.where(df1["fam_spouse_oc"].isnull(), 'N/A', df2["spouse_glau_affected"])



df2["spouse_hadsurg_affected"] = np.where(df1["fam_eyesurgery___5"]== 1, 'YES', 'NO')
df2["spouse_hadsurg_affected"] = np.where(df1["fam_spouse_oc"]=='Unknown', 'UNKNOWN TO PT', df2["spouse_hadsurg_affected"])
df2["spouse_hadsurg_affected"] = np.where(df1["fam_spouse_oc"]=='No', 'DECLINED', df2["spouse_hadsurg_affected"])
df2["spouse_hadsurg_affected"] = np.where(df1["fam_spouse_oc"].isnull(), 'N/A', df2["spouse_hadsurg_affected"])



df2["spouse_macdeg_affected"] = np.where(df1["fam_macdgnrtn___5"]== 1, 'YES', 'NO')
df2["spouse_macdeg_affected"] = np.where(df1["fam_spouse_oc"]=='Unknown', 'UNKNOWN TO PT', df2["spouse_macdeg_affected"])
df2["spouse_macdeg_affected"] = np.where(df1["fam_spouse_oc"]=='No', 'DECLINED', df2["spouse_macdeg_affected"])
df2["spouse_macdeg_affected"] = np.where(df1["fam_spouse_oc"].isnull(), 'N/A', df2["spouse_macdeg_affected"])



df2["spouse_other_occond"] = np.where(df1["fam_ocother___5"]== 1, 'YES', 'NO')
df2["spouse_other_occond"] = np.where(df1["fam_spouse_oc"]=='Unknown', 'UNKNOWN TO PT', df2["spouse_other_occond"])
df2["spouse_other_occond"] = np.where(df1["fam_spouse_oc"]=='No', 'DECLINED', df2["spouse_other_occond"])
df2["spouse_other_occond"] = np.where(df1["fam_spouse_oc"].isnull(), 'N/A', df2["spouse_other_occond"])


####
df2["spouse_hmyopia"] = np.where(df1["fam_hmyopia___5"]== 1, 'YES', 'NO')
df2["spouse_hmyopia"] = np.where(df1["fam_spouse_oc"]=='Unknown', 'UNKNOWN TO PT', df2["spouse_hmyopia"])
df2["spouse_hmyopia"] = np.where(df1["fam_spouse_oc"].isnull(), 'N/A', df2["spouse_hmyopia"])
df2["spouse_hmyopia"] = np.where(df1["fam_spouse_oc"].isnull(), 'N/A', df2["spouse_hmyopia"])
df["spouse_hmyopia"] = np.nan

df2["spouse_myopia"] = np.where(df1["fam_myopia___5"]== 1, 'YES', 'NO')
df2["spouse_myopia"] = np.where(df1["fam_spouse_oc"]=='Unknown', 'UNKNOWN TO PT', df2["spouse_myopia"])
df2["spouse_myopia"] = np.where(df1["fam_spouse_oc"]=='No', 'DECLINED', df2["spouse_myopia"])
df2["spouse_myopia"] = np.where(df1["fam_spouse_oc"].isnull(), 'N/A', df2["spouse_myopia"])
df["spouse_myopia"] = np.nan

####
Z = ['bba_blind_bro',
 'bba_eyesurgery_sis',
 'bba_eyesurgery_bro',
 'bba_eyedrops_sis',
 'bba_eyedrops_bro',
 'bba_eyeinject_sis',
 'bba_eyeinject_bro',
 'bba_glaucoma_sis',
 'bba_glaucoma_bro',
 'bba_macdgnrtn_sis',
 'bba_macdgnrtn_bro',
 'bba_diabrtnpthy_sis',
 'bba_diabrtnpthy_bro',
 'bba_cataract_sis',
 'bba_cataract_bro',
 'bba_dryeye_sis',
 'bba_dryeye_bro',
 'bba_myopia_sis',
 'bba_myopia_bro',
 'bba_hmyopia_sis',
 'bba_hmyopia_bro',
 'bba_ocother_sis',
 'bba_ocother_bro']

for i in Z:
    df2[i] = np.array(df1[i])
    df[i] = np.nan


 

df2["site"] = np.array(df1["site"])
df2["source"] = 'REDCap'


H = list(df.columns.values)

FIN = pd.concat([df, df2[H]], ignore_index=True)
FIN = pd.merge(FIN, ind[["OPHID_Text", "researchid"]], how='left', on='OPHID_Text')

FIN = FIN[~FIN['maskedid'].isnull()]

FIN = pd.merge(FIN, ind[["OPHID_Text_1", "maskedid"]], how='left', on='maskedid')
FIN["OPHID_Text"] = np.where(FIN["OPHID_Text"].isnull(), FIN["OPHID_Text_1"], FIN["OPHID_Text"])

FIN['maskedid'] = np.where(FIN['maskedid'].isnull(), FIN['researchid'], FIN['maskedid'])
FIN['BioBID'] = np.where(FIN['researchid'].isnull(), FIN['maskedid'], FIN['researchid'])
FIN['BioBID'] = np.where( (FIN['researchid'].str.contains('OG')) | (FIN['researchid'].str.contains('ED')), FIN['BioBID'], np.nan)
FIN["site"] = FIN['maskedid'].str[:2]
FIN["foh_mother_date"] = pd.to_datetime(FIN["foh_mother_date"],  errors='coerce').dt.strftime('%m/%d/%Y')
FIN['foh_mother_date'] = FIN['foh_mother_date'].astype(str)
FIN["mul"] = FIN.groupby(['maskedid', 'foh_mother_date']).maskedid.transform(lambda x:len(x))
FIN = FIN[~((FIN['source']=='REDCap') & (FIN['mul']>1))]
FIN = FIN[name]



FIN.to_csv(redcapeyechart + 'Results/' + extension + '_FamOcHistory_RedEC.csv', index=False)

In [20]:
L1 = [i for i in L_name if '20170322_NonOcularConditions' in i]
df = pd.read_csv(EyeChart + L1[0], encoding = "ISO-8859-1")
#df = pd.read_csv(EyeChart + '20170322_NonOcularConditions.csv', encoding = "ISO-8859-1")
df1 = pd.read_csv(redcapeyechart + extension + '_NonOcularConditions.csv')
df1["noc_date"] = pd.to_datetime(df1["noc_date"],  errors='coerce').dt.strftime('%m/%d/%Y')

name = ['maskedid',
'OPHID_Text',
'BioBID',
'site',
'visit_id',
'sys_noc_id',
'name',
'noc_date_rep',
'noc_date_dx',
'noc_da',
'noc_treat',
'noc_date_first_treated',
'noc_a',
'noc_details',
'source']

df["site"] = df['maskedid'].str[:2]
df["source"] = 'EyeChart'
df["visit_id"] = np.nan
Allergy = df1[df1["noc_allergy"] == 'Yes'][['noc_allergy_food',
 'noc_allergy_drugmed',
 'noc_allergy_env',
 'noc_allergy_insect',
 'noc_allergy_latex',
 'noc_allergy_spec_o',]].fillna("")


VAR = "allergy"

Allergy2 = df1[df1["noc_"+VAR] == 'Yes']
[['noc_'+VAR+'_dd'
, 'noc_'+VAR+'_dbd'
,'visit_id'
,'noc_'+VAR+'_a'
,'noc_'+VAR+'_da'
,'noc_'+VAR+'_treat'
, 'noc_'+VAR+'_d'
, 'noc_'+VAR+'_bd'
, "noc_date"
, 'maskedID'
, 'site']]



Allergy2["noc_" + VAR + "_dd"] = pd.to_datetime(df1["noc_" + VAR + "_dd"],  errors='coerce').dt.strftime('%m/%d/%Y')
Allergy2["noc_" + VAR + "_d"] = pd.to_datetime(df1["noc_" + VAR + "_d"],  errors='coerce').dt.strftime('%m/%d/%Y')

A = Allergy["noc_allergy_food"] + Allergy["noc_allergy_drugmed"] + Allergy["noc_allergy_env"] + Allergy["noc_allergy_insect"] + Allergy["noc_allergy_latex"] + Allergy["noc_allergy_spec_o"]
A = pd.DataFrame(A, columns = ["noc_details"])
A["name"] = 'Allergy'
A['sys_noc_id'] = 127
A["noc_date_dx"] = np.where(Allergy2["noc_"+VAR+"_dbd"]== 'No', 'UNKNOWN', Allergy2["noc_" + VAR + "_dd"])
A["noc_date_dx"] = np.where(Allergy2["noc_"+VAR+"_dbd"].isnull(), 'Not Reported', A["noc_date_dx"])
A["noc_date_first_treated"] = np.where( (Allergy2["noc_"+VAR+"_bd"] == 'Yes') & (Allergy2["noc_"+VAR+"_treat"] == 'Yes') , Allergy2["noc_" + VAR + "_d"], np.nan)
A["noc_date_first_treated"] = np.where( (Allergy2["noc_"+VAR+"_bd"] == 'No') & (Allergy2["noc_"+VAR+"_treat"] == 'Yes'), 'UNKNOWN', A["noc_date_first_treated"])
A["noc_date_first_treated"] = np.where( Allergy2["noc_"+VAR+"_bd"].isnull() , 'Not Reported', A["noc_date_first_treated"])

A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"] == 'Yes' , "Yes", np.nan)
A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"] == 'No' ,'UNTREATED', A["noc_treat"])
A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"].isnull() , 'Not Reported', A["noc_treat"])



A["OPHID_Text"] = np.nan
A["maskedid"] = np.array(Allergy2["maskedID"])
A["noc_date_rep"] = np.array(Allergy2["noc_date"])
A["site"] = Allergy2["site"]
A["source"] = 'REDCap'
A["noc_da"] = np.array(Allergy2["noc_" + VAR + "_da"])
A["noc_a"] = np.array(Allergy2["noc_" + VAR + "_a"])
A["visit_id"] = np.array(Allergy2["visit_id"])
A1 = A.copy(deep=True)


VAR = "arthritis"

Allergy2 = df1[df1["noc_"+VAR] == 'Yes']
[['noc_'+VAR+'_dd'
, 'noc_'+VAR+'_dbd'
,'visit_id'
,'noc_'+VAR+'_a'
,'noc_'+VAR+'_da'
,'noc_'+VAR+'_treat'
, 'noc_'+VAR+'_d'
, 'noc_'+VAR+'_bd'
, "noc_date"
, 'maskedID', 'noce_arthritis_type___1', 'noce_arthritis_type___2'
, 'site']]

A = pd.DataFrame(Allergy2["site"], columns = ["site"])
A["name"] = 'Arthritis'
A['sys_noc_id'] = 6

Allergy2["noc_" + VAR + "_dd"] = pd.to_datetime(df1["noc_" + VAR + "_dd"],  errors='coerce').dt.strftime('%m/%d/%Y')
Allergy2["noc_" + VAR + "_d"] = pd.to_datetime(df1["noc_" + VAR + "_d"],  errors='coerce').dt.strftime('%m/%d/%Y')




A["noc_date_dx"] = np.where(Allergy2["noc_"+VAR+"_dbd"]== 'No', 'UNKNOWN', Allergy2["noc_" + VAR + "_dd"])
A["noc_date_dx"] = np.where(Allergy2["noc_"+VAR+"_dbd"].isnull(), 'Not Reported', A["noc_date_dx"])
A["noc_date_first_treated"] = np.where( (Allergy2["noc_"+VAR+"_bd"] == 'Yes') & (Allergy2["noc_"+VAR+"_treat"] == 'Yes') , Allergy2["noc_" + VAR + "_d"], np.nan)
A["noc_date_first_treated"] = np.where( (Allergy2["noc_"+VAR+"_bd"] == 'No') & (Allergy2["noc_"+VAR+"_treat"] == 'Yes'), 'UNKNOWN', A["noc_date_first_treated"])
A["noc_date_first_treated"] = np.where( Allergy2["noc_"+VAR+"_bd"].isnull() , 'Not Reported', A["noc_date_first_treated"])

A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"] == 'Yes' , "Yes", np.nan)
A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"] == 'No' ,'UNTREATED', A["noc_treat"])
A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"].isnull() , 'Not Reported', A["noc_treat"])



A["OPHID_Text"] = np.nan
A["maskedid"] = np.array(Allergy2["maskedID"])
A["noc_date_rep"] = np.array(Allergy2["noc_date"])
A["site"] = Allergy2["site"]
A["source"] = 'REDCap'
A["noc_da"] = np.array(Allergy2["noc_" + VAR + "_da"])
A["noc_a"] = np.array(Allergy2["noc_" + VAR + "_a"])
A["visit_id"] = np.array(Allergy2["visit_id"]);

A['noc_details'] = np.where(Allergy2['noce_arthritis_type___1']==1, 'arthritis_type_type_1', np.nan)
A['noc_details'] = np.where(Allergy2['noce_arthritis_type___2']==1, 'arthritis_type_type_2', A['noc_details'])
A2 = A.copy(deep=True)


VAR = "asthma"

Allergy2 = df1[df1["noc_"+VAR] == 'Yes']
[['noc_'+VAR+'_dd'
, 'noc_'+VAR+'_dbd'
,'visit_id'
,'noc_'+VAR+'_a'
,'noc_'+VAR+'_da'
,'noc_'+VAR+'_treat'
, 'noc_'+VAR+'_d'
, 'noc_'+VAR+'_bd'
, "noc_date"
, 'maskedID'
, 'site']]

Allergy2["noc_" + VAR + "_dd"] = pd.to_datetime(df1["noc_" + VAR + "_dd"],  errors='coerce').dt.strftime('%m/%d/%Y')
Allergy2["noc_" + VAR + "_d"] = pd.to_datetime(df1["noc_" + VAR + "_d"],  errors='coerce').dt.strftime('%m/%d/%Y')


A = pd.DataFrame(Allergy2["site"], columns = ["site"])
A["name"] = 'Asthma'
A['sys_noc_id'] = 7

A["noc_date_dx"] = np.where(Allergy2["noc_"+VAR+"_dbd"]== 'No', 'UNKNOWN', Allergy2["noc_" + VAR + "_dd"])
A["noc_date_dx"] = np.where(Allergy2["noc_"+VAR+"_dbd"].isnull(), 'Not Reported', A["noc_date_dx"])
A["noc_date_first_treated"] = np.where( (Allergy2["noc_"+VAR+"_bd"] == 'Yes') & (Allergy2["noc_"+VAR+"_treat"] == 'Yes') , Allergy2["noc_" + VAR + "_d"], np.nan)
A["noc_date_first_treated"] = np.where( (Allergy2["noc_"+VAR+"_bd"] == 'No') & (Allergy2["noc_"+VAR+"_treat"] == 'Yes'), 'UNKNOWN', A["noc_date_first_treated"])
A["noc_date_first_treated"] = np.where( Allergy2["noc_"+VAR+"_bd"].isnull() , 'Not Reported', A["noc_date_first_treated"])

A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"] == 'Yes' , "Yes", np.nan)
A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"] == 'No' ,'UNTREATED', A["noc_treat"])
A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"].isnull() , 'Not Reported', A["noc_treat"])



A["OPHID_Text"] = np.nan
A["maskedid"] = np.array(Allergy2["maskedID"])
A["noc_date_rep"] = np.array(Allergy2["noc_date"])
A["site"] = Allergy2["site"]
A["source"] = 'REDCap'
A["noc_da"] = np.array(Allergy2["noc_" + VAR + "_da"])
A["noc_a"] = np.array(Allergy2["noc_" + VAR + "_a"])
A["visit_id"] = np.array(Allergy2["visit_id"]); A['noc_details'] = np.nan

A3 = A.copy(deep=True)

Allergy = df1[df1["noc_cancer"] == 'Yes'][['noc_cancer_spec1',
 'noc_cancer_other1',]].fillna("")


VAR = "cancer"

Allergy2 = df1[df1["noc_"+VAR] == 'Yes']
[['noc_'+VAR+'_dd'
, 'noc_'+VAR+'_dbd'
,'visit_id'
,'noc_'+VAR+'_a'
,'noc_'+VAR+'_da'
,'noc_'+VAR+'_treat'
, 'noc_'+VAR+'_d'
, 'noc_'+VAR+'_bd'
, "noc_date"
, 'maskedID'
, 'site']]

Allergy2["noc_" + VAR + "_dd"] = pd.to_datetime(df1["noc_" + VAR + "_dd"],  errors='coerce').dt.strftime('%m/%d/%Y')
Allergy2["noc_" + VAR + "_d"] = pd.to_datetime(df1["noc_" + VAR + "_d"],  errors='coerce').dt.strftime('%m/%d/%Y')


A = Allergy["noc_cancer_spec1"] + Allergy["noc_cancer_other1"]
A = pd.DataFrame(A, columns = ["noc_details"])
A["name"] = 'Cancer'
A['sys_noc_id'] = np.nan

A["noc_date_dx"] = np.where(Allergy2["noc_"+VAR+"_dbd"]== 'No', 'UNKNOWN', Allergy2["noc_" + VAR + "_dd"])
A["noc_date_dx"] = np.where(Allergy2["noc_"+VAR+"_dbd"].isnull(), 'Not Reported', A["noc_date_dx"])
A["noc_date_first_treated"] = np.where( (Allergy2["noc_"+VAR+"_bd"] == 'Yes') & (Allergy2["noc_"+VAR+"_treat"] == 'Yes') , Allergy2["noc_" + VAR + "_d"], np.nan)
A["noc_date_first_treated"] = np.where( (Allergy2["noc_"+VAR+"_bd"] == 'No') & (Allergy2["noc_"+VAR+"_treat"] == 'Yes'), 'UNKNOWN', A["noc_date_first_treated"])
A["noc_date_first_treated"] = np.where( Allergy2["noc_"+VAR+"_bd"].isnull() , 'Not Reported', A["noc_date_first_treated"])

A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"] == 'Yes' , "Yes", np.nan)
A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"] == 'No' ,'UNTREATED', A["noc_treat"])
A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"].isnull() , 'Not Reported', A["noc_treat"])



A["OPHID_Text"] = np.nan
A["maskedid"] = np.array(Allergy2["maskedID"])
A["noc_date_rep"] = np.array(Allergy2["noc_date"])
A["site"] = Allergy2["site"]
A["source"] = 'REDCap'
A["noc_da"] = np.array(Allergy2["noc_" + VAR + "_da"])
A["noc_a"] = np.array(Allergy2["noc_" + VAR + "_a"])
A["visit_id"] = np.array(Allergy2["visit_id"]); A['noc_details'] = np.nan


A4 = A.copy(deep=True)


VAR = "diabetes1"

Allergy2 = df1[df1["noc_"+VAR] == 'Yes']
[['noc_'+VAR+'_dd'
, 'noc_'+VAR+'_dbd'
,'visit_id'
,'noc_'+VAR+'_a'
,'noc_'+VAR+'_da'
,'noc_'+VAR+'_treat'
, 'noc_'+VAR+'_d'
, 'noc_'+VAR+'_bd'
, "noc_date"
, 'maskedID'
, 'site']]

Allergy2["noc_" + VAR + "_dd"] = pd.to_datetime(df1["noc_" + VAR + "_dd"],  errors='coerce').dt.strftime('%m/%d/%Y')
Allergy2["noc_" + VAR + "_d"] = pd.to_datetime(df1["noc_" + VAR + "_d"],  errors='coerce').dt.strftime('%m/%d/%Y')

A = pd.DataFrame(Allergy2["site"], columns = ["site"])
A["name"] = 'Diabetes - Type 1'
A['sys_noc_id'] = 259


A["noc_date_dx"] = np.where(Allergy2["noc_"+VAR+"_dbd"]== 'No', 'UNKNOWN', Allergy2["noc_" + VAR + "_dd"])
A["noc_date_dx"] = np.where(Allergy2["noc_"+VAR+"_dbd"].isnull(), 'Not Reported', A["noc_date_dx"])
A["noc_date_first_treated"] = np.where( (Allergy2["noc_"+VAR+"_bd"] == 'Yes') & (Allergy2["noc_"+VAR+"_treat"] == 'Yes') , Allergy2["noc_" + VAR + "_d"], np.nan)
A["noc_date_first_treated"] = np.where( (Allergy2["noc_"+VAR+"_bd"] == 'No') & (Allergy2["noc_"+VAR+"_treat"] == 'Yes'), 'UNKNOWN', A["noc_date_first_treated"])
A["noc_date_first_treated"] = np.where( Allergy2["noc_"+VAR+"_bd"].isnull() , 'Not Reported', A["noc_date_first_treated"])

A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"] == 'Yes' , "Yes", np.nan)
A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"] == 'No' ,'UNTREATED', A["noc_treat"])
A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"].isnull() , 'Not Reported', A["noc_treat"])



A["OPHID_Text"] = np.nan
A["maskedid"] = np.array(Allergy2["maskedID"])
A["noc_date_rep"] = np.array(Allergy2["noc_date"])
A["site"] = Allergy2["site"]
A["source"] = 'REDCap'
A["noc_da"] = np.array(Allergy2["noc_" + VAR + "_da"])
A["noc_a"] = np.array(Allergy2["noc_" + VAR + "_a"])
A["visit_id"] = np.array(Allergy2["visit_id"]); A['noc_details'] = np.nan
A5 = A.copy(deep=True)



VAR = "diabetes2"

Allergy2 = df1[df1["noc_"+VAR] == 'Yes']
[['noc_'+VAR+'_dd'
, 'noc_'+VAR+'_dbd'
,'visit_id'
,'noc_'+VAR+'_a'
,'noc_'+VAR+'_da'
,'noc_'+VAR+'_treat'
, 'noc_'+VAR+'_d'
, 'noc_'+VAR+'_bd'
, "noc_date"
, 'maskedID'
, 'site']]

Allergy2["noc_" + VAR + "_dd"] = pd.to_datetime(df1["noc_" + VAR + "_dd"],  errors='coerce').dt.strftime('%m/%d/%Y')
Allergy2["noc_" + VAR + "_d"] = pd.to_datetime(df1["noc_" + VAR + "_d"],  errors='coerce').dt.strftime('%m/%d/%Y')


A = pd.DataFrame(Allergy2["site"], columns = ["site"])
A["name"] = 'Diabetes - Type 2'
A['sys_noc_id'] = 260



A["noc_date_dx"] = np.where(Allergy2["noc_"+VAR+"_dbd"]== 'No', 'UNKNOWN', Allergy2["noc_" + VAR + "_dd"])
A["noc_date_dx"] = np.where(Allergy2["noc_"+VAR+"_dbd"].isnull(), 'Not Reported', A["noc_date_dx"])
A["noc_date_first_treated"] = np.where( (Allergy2["noc_"+VAR+"_bd"] == 'Yes') & (Allergy2["noc_"+VAR+"_treat"] == 'Yes') , Allergy2["noc_" + VAR + "_d"], np.nan)
A["noc_date_first_treated"] = np.where( (Allergy2["noc_"+VAR+"_bd"] == 'No') & (Allergy2["noc_"+VAR+"_treat"] == 'Yes'), 'UNKNOWN', A["noc_date_first_treated"])
A["noc_date_first_treated"] = np.where( Allergy2["noc_"+VAR+"_bd"].isnull() , 'Not Reported', A["noc_date_first_treated"])

A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"] == 'Yes' , "Yes", np.nan)
A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"] == 'No' ,'UNTREATED', A["noc_treat"])
A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"].isnull() , 'Not Reported', A["noc_treat"])



A["OPHID_Text"] = np.nan
A["maskedid"] = np.array(Allergy2["maskedID"])
A["noc_date_rep"] = np.array(Allergy2["noc_date"])
A["site"] = Allergy2["site"]
A["source"] = 'REDCap'
A["noc_da"] = np.array(Allergy2["noc_" + VAR + "_da"])
A["noc_a"] = np.array(Allergy2["noc_" + VAR + "_a"])
A["visit_id"] = np.array(Allergy2["visit_id"]); A['noc_details'] = np.nan


A6 = A.copy(deep=True)



VAR = "heardis"

Allergy2 = df1[df1["noc_"+VAR] == 'Yes']
[['noc_'+VAR+'_dd'
, 'noc_'+VAR+'_dbd'
,'visit_id'
,'noc_'+VAR+'_a'
,'noc_'+VAR+'_da'
,'noc_'+VAR+'_treat'
, 'noc_'+VAR+'_d'
, 'noc_'+VAR+'_bd'
, "noc_date"
, 'maskedID'
, 'site']]

Allergy2["noc_" + VAR + "_dd"] = pd.to_datetime(df1["noc_" + VAR + "_dd"],  errors='coerce').dt.strftime('%m/%d/%Y')
Allergy2["noc_" + VAR + "_d"] = pd.to_datetime(df1["noc_" + VAR + "_d"],  errors='coerce').dt.strftime('%m/%d/%Y')


A = pd.DataFrame(Allergy2["site"], columns = ["site"])
A["name"] = 'Hearing Disorders 	'
A['sys_noc_id'] = 266


A["noc_date_dx"] = np.where(Allergy2["noc_"+VAR+"_dbd"]== 'No', 'UNKNOWN', Allergy2["noc_" + VAR + "_dd"])
A["noc_date_dx"] = np.where(Allergy2["noc_"+VAR+"_dbd"].isnull(), 'Not Reported', A["noc_date_dx"])
A["noc_date_first_treated"] = np.where( (Allergy2["noc_"+VAR+"_bd"] == 'Yes') & (Allergy2["noc_"+VAR+"_treat"] == 'Yes') , Allergy2["noc_" + VAR + "_d"], np.nan)
A["noc_date_first_treated"] = np.where( (Allergy2["noc_"+VAR+"_bd"] == 'No') & (Allergy2["noc_"+VAR+"_treat"] == 'Yes'), 'UNKNOWN', A["noc_date_first_treated"])
A["noc_date_first_treated"] = np.where( Allergy2["noc_"+VAR+"_bd"].isnull() , 'Not Reported', A["noc_date_first_treated"])

A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"] == 'Yes' , "Yes", np.nan)
A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"] == 'No' ,'UNTREATED', A["noc_treat"])
A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"].isnull() , 'Not Reported', A["noc_treat"])



A["OPHID_Text"] = np.nan
A["maskedid"] = np.array(Allergy2["maskedID"])
A["noc_date_rep"] = np.array(Allergy2["noc_date"])
A["site"] = Allergy2["site"]
A["source"] = 'REDCap'
A["noc_da"] = np.array(Allergy2["noc_" + VAR + "_da"])
A["noc_a"] = np.array(Allergy2["noc_" + VAR + "_a"])
A["visit_id"] = np.array(Allergy2["visit_id"]); A['noc_details'] = np.nan

A7 = A.copy(deep=True)


VAR = "hrtattack"

Allergy2 = df1[df1["noc_"+VAR] == 'Yes']
[['noc_'+VAR+'_dd'
, 'noc_'+VAR+'_dbd'
,'visit_id'
,'noc_'+VAR+'_a'
,'noc_'+VAR+'_da'
,'noc_'+VAR+'_treat'
, 'noc_'+VAR+'_d'
, 'noc_'+VAR+'_bd'
, "noc_date"
, 'maskedID'
, 'site']]

Allergy2["noc_" + VAR + "_dd"] = pd.to_datetime(df1["noc_" + VAR + "_dd"],  errors='coerce').dt.strftime('%m/%d/%Y')
Allergy2["noc_" + VAR + "_d"] = pd.to_datetime(df1["noc_" + VAR + "_d"],  errors='coerce').dt.strftime('%m/%d/%Y')

A = pd.DataFrame(Allergy2["site"], columns = ["site"])
A["name"] = 'Heart Attack'
A['sys_noc_id'] = 268

A["noc_date_dx"] = np.where(Allergy2["noc_"+VAR+"_dbd"]== 'No', 'UNKNOWN', Allergy2["noc_" + VAR + "_dd"])
A["noc_date_dx"] = np.where(Allergy2["noc_"+VAR+"_dbd"].isnull(), 'Not Reported', A["noc_date_dx"])
A["noc_date_first_treated"] = np.where( (Allergy2["noc_"+VAR+"_bd"] == 'Yes') & (Allergy2["noc_"+VAR+"_treat"] == 'Yes') , Allergy2["noc_" + VAR + "_d"], np.nan)
A["noc_date_first_treated"] = np.where( (Allergy2["noc_"+VAR+"_bd"] == 'No') & (Allergy2["noc_"+VAR+"_treat"] == 'Yes'), 'UNKNOWN', A["noc_date_first_treated"])
A["noc_date_first_treated"] = np.where( Allergy2["noc_"+VAR+"_bd"].isnull() , 'Not Reported', A["noc_date_first_treated"])

A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"] == 'Yes' , "Yes", np.nan)
A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"] == 'No' ,'UNTREATED', A["noc_treat"])
A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"].isnull() , 'Not Reported', A["noc_treat"])



A["OPHID_Text"] = np.nan
A["maskedid"] = np.array(Allergy2["maskedID"])
A["noc_date_rep"] = np.array(Allergy2["noc_date"])
A["site"] = Allergy2["site"]
A["source"] = 'REDCap'
A["noc_da"] = np.array(Allergy2["noc_" + VAR + "_da"])
A["noc_a"] = np.array(Allergy2["noc_" + VAR + "_a"])
A["visit_id"] = np.array(Allergy2["visit_id"]); A['noc_details'] = np.nan

A8 = A.copy(deep=True)



VAR = "hrtdisease"

Allergy2 = df1[df1["noc_"+VAR] == 'Yes']
[['noc_'+VAR+'_dd'
, 'noc_'+VAR+'_dbd'
,'visit_id'
,'noc_'+VAR+'_a'
,'noc_'+VAR+'_da'
,'noc_'+VAR+'_treat'
, 'noc_'+VAR+'_d'
, 'noc_'+VAR+'_bd'
, "noc_date"
, 'maskedID'
, 'site']]

Allergy2["noc_" + VAR + "_dd"] = pd.to_datetime(df1["noc_" + VAR + "_dd"],  errors='coerce').dt.strftime('%m/%d/%Y')
Allergy2["noc_" + VAR + "_d"] = pd.to_datetime(df1["noc_" + VAR + "_d"],  errors='coerce').dt.strftime('%m/%d/%Y')

A = pd.DataFrame(Allergy2["site"], columns = ["site"])
A["name"] = 'Heart Disease'
A['sys_noc_id'] = 34


A["noc_date_dx"] = np.where(Allergy2["noc_"+VAR+"_dbd"]== 'No', 'UNKNOWN', Allergy2["noc_" + VAR + "_dd"])
A["noc_date_dx"] = np.where(Allergy2["noc_"+VAR+"_dbd"].isnull(), 'Not Reported', A["noc_date_dx"])
A["noc_date_first_treated"] = np.where( (Allergy2["noc_"+VAR+"_bd"] == 'Yes') & (Allergy2["noc_"+VAR+"_treat"] == 'Yes') , Allergy2["noc_" + VAR + "_d"], np.nan)
A["noc_date_first_treated"] = np.where( (Allergy2["noc_"+VAR+"_bd"] == 'No') & (Allergy2["noc_"+VAR+"_treat"] == 'Yes'), 'UNKNOWN', A["noc_date_first_treated"])
A["noc_date_first_treated"] = np.where( Allergy2["noc_"+VAR+"_bd"].isnull() , 'Not Reported', A["noc_date_first_treated"])

A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"] == 'Yes' , "Yes", np.nan)
A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"] == 'No' ,'UNTREATED', A["noc_treat"])
A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"].isnull() , 'Not Reported', A["noc_treat"])



A["OPHID_Text"] = np.nan
A["maskedid"] = np.array(Allergy2["maskedID"])
A["noc_date_rep"] = np.array(Allergy2["noc_date"])
A["site"] = Allergy2["site"]
A["source"] = 'REDCap'
A["noc_da"] = np.array(Allergy2["noc_" + VAR + "_da"])
A["noc_a"] = np.array(Allergy2["noc_" + VAR + "_a"])
A["visit_id"] = np.array(Allergy2["visit_id"]); A['noc_details'] = np.nan

A9 = A.copy(deep=True)



VAR = "bloodclots"

Allergy2 = df1[df1["noc_"+VAR] == 'Yes']
[['noc_'+VAR+'_dd'
, 'noc_'+VAR+'_dbd'
,'visit_id'
,'noc_'+VAR+'_a'
,'noc_'+VAR+'_da'
,'noc_'+VAR+'_treat'
, 'noc_'+VAR+'_d'
, 'noc_'+VAR+'_bd'
, "noc_date"
, 'maskedID'
, 'site']]

Allergy2["noc_" + VAR + "_dd"] = pd.to_datetime(df1["noc_" + VAR + "_dd"],  errors='coerce').dt.strftime('%m/%d/%Y')
Allergy2["noc_" + VAR + "_d"] = pd.to_datetime(df1["noc_" + VAR + "_d"],  errors='coerce').dt.strftime('%m/%d/%Y')

A = pd.DataFrame(Allergy2["site"], columns = ["site"])
A["name"] = 'Blood Clots (Lungs or Legs)'
A['sys_noc_id'] = 269

A = pd.DataFrame(Allergy2["site"], columns = ["site"])
A["name"] = 'Heart Disease'
A['sys_noc_id'] = 34


A["noc_date_dx"] = np.where(Allergy2["noc_"+VAR+"_dbd"]== 'No', 'UNKNOWN', Allergy2["noc_" + VAR + "_dd"])
A["noc_date_dx"] = np.where(Allergy2["noc_"+VAR+"_dbd"].isnull(), 'Not Reported', A["noc_date_dx"])
A["noc_date_first_treated"] = np.where( (Allergy2["noc_"+VAR+"_bd"] == 'Yes') & (Allergy2["noc_"+VAR+"_treat"] == 'Yes') , Allergy2["noc_" + VAR + "_d"], np.nan)
A["noc_date_first_treated"] = np.where( (Allergy2["noc_"+VAR+"_bd"] == 'No') & (Allergy2["noc_"+VAR+"_treat"] == 'Yes'), 'UNKNOWN', A["noc_date_first_treated"])
A["noc_date_first_treated"] = np.where( Allergy2["noc_"+VAR+"_bd"].isnull() , 'Not Reported', A["noc_date_first_treated"])

A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"] == 'Yes' , "Yes", np.nan)
A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"] == 'No' ,'UNTREATED', A["noc_treat"])
A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"].isnull() , 'Not Reported', A["noc_treat"])



A["OPHID_Text"] = np.nan
A["maskedid"] = np.array(Allergy2["maskedID"])
A["noc_date_rep"] = np.array(Allergy2["noc_date"])
A["site"] = Allergy2["site"]
A["source"] = 'REDCap'
A["noc_da"] = np.array(Allergy2["noc_" + VAR + "_da"])
A["noc_a"] = np.array(Allergy2["noc_" + VAR + "_a"])
A["visit_id"] = np.array(Allergy2["visit_id"]); A['noc_details'] = np.nan

A10 = A.copy(deep=True)


VAR = "highbp"

Allergy2 = df1[df1["noc_"+VAR] == 'Yes']
[['noc_'+VAR+'_dd'
, 'noc_'+VAR+'_dbd'
,'visit_id'
,'noc_'+VAR+'_a'
,'noc_'+VAR+'_da'
,'noc_'+VAR+'_treat'
, 'noc_'+VAR+'_d'
, 'noc_'+VAR+'_bd'
, "noc_date"
, 'maskedID'
, 'site']]


Allergy2["noc_" + VAR + "_dd"] = pd.to_datetime(df1["noc_" + VAR + "_dd"],  errors='coerce').dt.strftime('%m/%d/%Y')
Allergy2["noc_" + VAR + "_d"] = pd.to_datetime(df1["noc_" + VAR + "_d"],  errors='coerce').dt.strftime('%m/%d/%Y')

A = pd.DataFrame(Allergy2["site"], columns = ["site"])
A["name"] = 'High Blood Pressure'
A['sys_noc_id'] = 35



A["noc_date_dx"] = np.where(Allergy2["noc_"+VAR+"_dbd"]== 'No', 'UNKNOWN', Allergy2["noc_" + VAR + "_dd"])
A["noc_date_dx"] = np.where(Allergy2["noc_"+VAR+"_dbd"].isnull(), 'Not Reported', A["noc_date_dx"])
A["noc_date_first_treated"] = np.where( (Allergy2["noc_"+VAR+"_bd"] == 'Yes') & (Allergy2["noc_"+VAR+"_treat"] == 'Yes') , Allergy2["noc_" + VAR + "_d"], np.nan)
A["noc_date_first_treated"] = np.where( (Allergy2["noc_"+VAR+"_bd"] == 'No') & (Allergy2["noc_"+VAR+"_treat"] == 'Yes'), 'UNKNOWN', A["noc_date_first_treated"])
A["noc_date_first_treated"] = np.where( Allergy2["noc_"+VAR+"_bd"].isnull() , 'Not Reported', A["noc_date_first_treated"])


A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"] == 'Yes' , "Yes", np.nan)
A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"] == 'No' ,'UNTREATED', A["noc_treat"])
A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"].isnull() , 'Not Reported', A["noc_treat"])



A["OPHID_Text"] = np.nan
A["maskedid"] = np.array(Allergy2["maskedID"])
A["noc_date_rep"] = np.array(Allergy2["noc_date"])
A["site"] = Allergy2["site"]
A["source"] = 'REDCap'
A["noc_da"] = np.array(Allergy2["noc_" + VAR + "_da"])
A["noc_a"] = np.array(Allergy2["noc_" + VAR + "_a"])
A["visit_id"] = np.array(Allergy2["visit_id"]); A['noc_details'] = np.nan

A11 = A.copy(deep=True)


VAR = "lowbp"

Allergy2 = df1[df1["noc_"+VAR] == 'Yes']
[['noc_'+VAR+'_dd'
, 'noc_'+VAR+'_dbd'
,'visit_id'
,'noc_'+VAR+'_a'
,'noc_'+VAR+'_da'
,'noc_'+VAR+'_treat'
, 'noc_'+VAR+'_d'
, 'noc_'+VAR+'_bd'
, "noc_date"
, 'maskedID'
, 'site']]

Allergy2["noc_" + VAR + "_dd"] = pd.to_datetime(df1["noc_" + VAR + "_dd"],  errors='coerce').dt.strftime('%m/%d/%Y')
Allergy2["noc_" + VAR + "_d"] = pd.to_datetime(df1["noc_" + VAR + "_d"],  errors='coerce').dt.strftime('%m/%d/%Y')

A = pd.DataFrame(Allergy2["site"], columns = ["site"])
A["name"] = 'Low Blood Pressure'
A['sys_noc_id'] = 270


A["noc_date_dx"] = np.where(Allergy2["noc_"+VAR+"_dbd"]== 'No', 'UNKNOWN', Allergy2["noc_" + VAR + "_dd"])
A["noc_date_dx"] = np.where(Allergy2["noc_"+VAR+"_dbd"].isnull(), 'Not Reported', A["noc_date_dx"])
A["noc_date_first_treated"] = np.where( (Allergy2["noc_"+VAR+"_bd"] == 'Yes') & (Allergy2["noc_"+VAR+"_treat"] == 'Yes') , Allergy2["noc_" + VAR + "_d"], np.nan)
A["noc_date_first_treated"] = np.where( (Allergy2["noc_"+VAR+"_bd"] == 'No') & (Allergy2["noc_"+VAR+"_treat"] == 'Yes'), 'UNKNOWN', A["noc_date_first_treated"])
A["noc_date_first_treated"] = np.where( Allergy2["noc_"+VAR+"_bd"].isnull() , 'Not Reported', A["noc_date_first_treated"])

A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"] == 'Yes' , "Yes", np.nan)
A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"] == 'No' ,'UNTREATED', A["noc_treat"])
A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"].isnull() , 'Not Reported', A["noc_treat"])



A["OPHID_Text"] = np.nan
A["maskedid"] = np.array(Allergy2["maskedID"])
A["noc_date_rep"] = np.array(Allergy2["noc_date"])
A["site"] = Allergy2["site"]
A["source"] = 'REDCap'
A["noc_da"] = np.array(Allergy2["noc_" + VAR + "_da"])
A["noc_a"] = np.array(Allergy2["noc_" + VAR + "_a"])
A["visit_id"] = np.array(Allergy2["visit_id"]); A['noc_details'] = np.nan

A12 = A.copy(deep=True)


VAR = "migraine"

Allergy2 = df1[df1["noc_"+VAR] == 'Yes']
[['noc_'+VAR+'_dd'
, 'noc_'+VAR+'_dbd'
,'visit_id'
,'noc_'+VAR+'_a'
,'noc_'+VAR+'_da'
,'noc_'+VAR+'_treat'
, 'noc_'+VAR+'_d'
, 'noc_'+VAR+'_bd'
, "noc_date"
, 'maskedID'
, 'site']]

Allergy2["noc_" + VAR + "_dd"] = pd.to_datetime(df1["noc_" + VAR + "_dd"],  errors='coerce').dt.strftime('%m/%d/%Y')
Allergy2["noc_" + VAR + "_d"] = pd.to_datetime(df1["noc_" + VAR + "_d"],  errors='coerce').dt.strftime('%m/%d/%Y')

A = pd.DataFrame(Allergy2["site"], columns = ["site"])
A["name"] = 'Migraine'
A['sys_noc_id'] = 50


A["noc_date_dx"] = np.where(Allergy2["noc_"+VAR+"_dbd"]== 'No', 'UNKNOWN', Allergy2["noc_" + VAR + "_dd"])
A["noc_date_dx"] = np.where(Allergy2["noc_"+VAR+"_dbd"].isnull(), 'Not Reported', A["noc_date_dx"])
A["noc_date_first_treated"] = np.where( (Allergy2["noc_"+VAR+"_bd"] == 'Yes') & (Allergy2["noc_"+VAR+"_treat"] == 'Yes') , Allergy2["noc_" + VAR + "_d"], np.nan)
A["noc_date_first_treated"] = np.where( (Allergy2["noc_"+VAR+"_bd"] == 'No') & (Allergy2["noc_"+VAR+"_treat"] == 'Yes'), 'UNKNOWN', A["noc_date_first_treated"])
A["noc_date_first_treated"] = np.where( Allergy2["noc_"+VAR+"_bd"].isnull() , 'Not Reported', A["noc_date_first_treated"])

A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"] == 'Yes' , "Yes", np.nan)
A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"] == 'No' ,'UNTREATED', A["noc_treat"])
A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"].isnull() , 'Not Reported', A["noc_treat"])



A["OPHID_Text"] = np.nan
A["maskedid"] = np.array(Allergy2["maskedID"])
A["noc_date_rep"] = np.array(Allergy2["noc_date"])
A["site"] = Allergy2["site"]
A["source"] = 'REDCap'
A["noc_da"] = np.array(Allergy2["noc_" + VAR + "_da"])
A["noc_a"] = np.array(Allergy2["noc_" + VAR + "_a"])
A["visit_id"] = np.array(Allergy2["visit_id"]); A['noc_details'] = np.nan

A13 = A.copy(deep=True)


VAR = "obese"

Allergy2 = df1[df1["noc_"+VAR] == 'Yes']
[['noc_'+VAR+'_dd'
, 'noc_'+VAR+'_dbd'
,'visit_id'
,'noc_'+VAR+'_a'
,'noc_'+VAR+'_da'
,'noc_'+VAR+'_treat'
, 'noc_'+VAR+'_d'
, 'noc_'+VAR+'_bd'
, "noc_date"
, 'maskedID'
, 'site']]

Allergy2["noc_" + VAR + "_dd"] = pd.to_datetime(df1["noc_" + VAR + "_dd"],  errors='coerce').dt.strftime('%m/%d/%Y')
Allergy2["noc_" + VAR + "_d"] = pd.to_datetime(df1["noc_" + VAR + "_d"],  errors='coerce').dt.strftime('%m/%d/%Y')

A = pd.DataFrame(Allergy2["site"], columns = ["site"])
A["name"] = 'Obesity'
A['sys_noc_id'] = 271


A["noc_date_dx"] = np.where(Allergy2["noc_"+VAR+"_dbd"]== 'No', 'UNKNOWN', Allergy2["noc_" + VAR + "_dd"])
A["noc_date_dx"] = np.where(Allergy2["noc_"+VAR+"_dbd"].isnull(), 'Not Reported', A["noc_date_dx"])
A["noc_date_first_treated"] = np.where( (Allergy2["noc_"+VAR+"_bd"] == 'Yes') & (Allergy2["noc_"+VAR+"_treat"] == 'Yes') , Allergy2["noc_" + VAR + "_d"], np.nan)
A["noc_date_first_treated"] = np.where( (Allergy2["noc_"+VAR+"_bd"] == 'No') & (Allergy2["noc_"+VAR+"_treat"] == 'Yes'), 'UNKNOWN', A["noc_date_first_treated"])
A["noc_date_first_treated"] = np.where( Allergy2["noc_"+VAR+"_bd"].isnull() , 'Not Reported', A["noc_date_first_treated"])

A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"] == 'Yes' , "Yes", np.nan)
A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"] == 'No' ,'UNTREATED', A["noc_treat"])
A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"].isnull() , 'Not Reported', A["noc_treat"])



A["OPHID_Text"] = np.nan
A["maskedid"] = np.array(Allergy2["maskedID"])
A["noc_date_rep"] = np.array(Allergy2["noc_date"])
A["site"] = Allergy2["site"]
A["source"] = 'REDCap'
A["noc_da"] = np.array(Allergy2["noc_" + VAR + "_da"])
A["noc_a"] = np.array(Allergy2["noc_" + VAR + "_a"])
A["visit_id"] = np.array(Allergy2["visit_id"]); A['noc_details'] = np.nan

A14 = A.copy(deep=True)

VAR = "fertility"

Allergy2 = df1[df1["noc_"+VAR] == 'Yes']
[['noc_'+VAR+'_dd'
, 'noc_'+VAR+'_dbd'
,'visit_id'
,'noc_'+VAR+'_a'
,'noc_'+VAR+'_da'
,'noc_'+VAR+'_treat'
, 'noc_'+VAR+'_d'
, 'noc_'+VAR+'_bd'
, "noc_date"
, 'maskedID'
, 'site']]

Allergy2["noc_" + VAR + "_dd"] = pd.to_datetime(df1["noc_" + VAR + "_dd"],  errors='coerce').dt.strftime('%m/%d/%Y')
Allergy2["noc_" + VAR + "_d"] = pd.to_datetime(df1["noc_" + VAR + "_d"],  errors='coerce').dt.strftime('%m/%d/%Y')


A = pd.DataFrame(Allergy2["site"], columns = ["site"])
A["name"] = 'Reproductive/Fertility Disorder'
A['sys_noc_id'] = 272


A["noc_date_dx"] = np.where(Allergy2["noc_"+VAR+"_dbd"]== 'No', 'UNKNOWN', Allergy2["noc_" + VAR + "_dd"])
A["noc_date_dx"] = np.where(Allergy2["noc_"+VAR+"_dbd"].isnull(), 'Not Reported', A["noc_date_dx"])
A["noc_date_first_treated"] = np.where( (Allergy2["noc_"+VAR+"_bd"] == 'Yes') & (Allergy2["noc_"+VAR+"_treat"] == 'Yes') , Allergy2["noc_" + VAR + "_d"], np.nan)
A["noc_date_first_treated"] = np.where( (Allergy2["noc_"+VAR+"_bd"] == 'No') & (Allergy2["noc_"+VAR+"_treat"] == 'Yes'), 'UNKNOWN', A["noc_date_first_treated"])
A["noc_date_first_treated"] = np.where( Allergy2["noc_"+VAR+"_bd"].isnull() , 'Not Reported', A["noc_date_first_treated"])

A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"] == 'Yes' , "Yes", np.nan)
A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"] == 'No' ,'UNTREATED', A["noc_treat"])
A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"].isnull() , 'Not Reported', A["noc_treat"])



A["OPHID_Text"] = np.nan
A["maskedid"] = np.array(Allergy2["maskedID"])
A["noc_date_rep"] = np.array(Allergy2["noc_date"])
A["site"] = Allergy2["site"]
A["source"] = 'REDCap'
A["noc_da"] = np.array(Allergy2["noc_" + VAR + "_da"])
A["noc_a"] = np.array(Allergy2["noc_" + VAR + "_a"])
A["visit_id"] = np.array(Allergy2["visit_id"]); A['noc_details'] = np.nan

A15 = A.copy(deep=True)

VAR = "reynaud"

Allergy2 = df1[df1["noc_"+VAR] == 'Yes']
[['noc_'+VAR+'_dd'
, 'noc_'+VAR+'_dbd'
,'visit_id'
,'noc_'+VAR+'_a'
,'noc_'+VAR+'_da'
,'noc_'+VAR+'_treat'
, 'noc_'+VAR+'_d'
, 'noc_'+VAR+'_bd'
, "noc_date"
, 'maskedID'
, 'site']]

Allergy2["noc_" + VAR + "_dd"] = pd.to_datetime(df1["noc_" + VAR + "_dd"],  errors='coerce').dt.strftime('%m/%d/%Y')
Allergy2["noc_" + VAR + "_d"] = pd.to_datetime(df1["noc_" + VAR + "_d"],  errors='coerce').dt.strftime('%m/%d/%Y')

A = pd.DataFrame(Allergy2["site"], columns = ["site"])
A["name"] = "Raynaud's Disease"
A['sys_noc_id'] = 130


A["noc_date_dx"] = np.where(Allergy2["noc_"+VAR+"_dbd"]== 'No', 'UNKNOWN', Allergy2["noc_" + VAR + "_dd"])
A["noc_date_dx"] = np.where(Allergy2["noc_"+VAR+"_dbd"].isnull(), 'Not Reported', A["noc_date_dx"])
A["noc_date_first_treated"] = np.where( (Allergy2["noc_"+VAR+"_bd"] == 'Yes') & (Allergy2["noc_"+VAR+"_treat"] == 'Yes') , Allergy2["noc_" + VAR + "_d"], np.nan)
A["noc_date_first_treated"] = np.where( (Allergy2["noc_"+VAR+"_bd"] == 'No') & (Allergy2["noc_"+VAR+"_treat"] == 'Yes'), 'UNKNOWN', A["noc_date_first_treated"])
A["noc_date_first_treated"] = np.where( Allergy2["noc_"+VAR+"_bd"].isnull() , 'Not Reported', A["noc_date_first_treated"])

A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"] == 'Yes' , "Yes", np.nan)
A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"] == 'No' ,'UNTREATED', A["noc_treat"])
A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"].isnull() , 'Not Reported', A["noc_treat"])



A["OPHID_Text"] = np.nan
A["maskedid"] = np.array(Allergy2["maskedID"])
A["noc_date_rep"] = np.array(Allergy2["noc_date"])
A["site"] = Allergy2["site"]
A["source"] = 'REDCap'
A["noc_da"] = np.array(Allergy2["noc_" + VAR + "_da"])
A["noc_a"] = np.array(Allergy2["noc_" + VAR + "_a"])
A["visit_id"] = np.array(Allergy2["visit_id"]); A['noc_details'] = np.nan

A16 = A.copy(deep=True)

VAR = "stroke"

Allergy2 = df1[df1["noc_"+VAR] == 'Yes']
[['noc_'+VAR+'_dd'
, 'noc_'+VAR+'_dbd'
,'visit_id'
,'noc_'+VAR+'_a'
,'noc_'+VAR+'_da'
,'noc_'+VAR+'_treat'
, 'noc_'+VAR+'_d'
, 'noc_'+VAR+'_bd'
, "noc_date"
, 'maskedID'
, 'site']]

Allergy2["noc_" + VAR + "_dd"] = pd.to_datetime(df1["noc_" + VAR + "_dd"],  errors='coerce').dt.strftime('%m/%d/%Y')
Allergy2["noc_" + VAR + "_d"] = pd.to_datetime(df1["noc_" + VAR + "_d"],  errors='coerce').dt.strftime('%m/%d/%Y')

A = pd.DataFrame(Allergy2["site"], columns = ["site"])
A["name"] = "Stroke"
A['sys_noc_id'] = 86

A["noc_date_dx"] = np.where(Allergy2["noc_"+VAR+"_dbd"]== 'No', 'UNKNOWN', Allergy2["noc_" + VAR + "_dd"])
A["noc_date_dx"] = np.where(Allergy2["noc_"+VAR+"_dbd"].isnull(), 'Not Reported', A["noc_date_dx"])
A["noc_date_first_treated"] = np.where( (Allergy2["noc_"+VAR+"_bd"] == 'Yes') & (Allergy2["noc_"+VAR+"_treat"] == 'Yes') , Allergy2["noc_" + VAR + "_d"], np.nan)
A["noc_date_first_treated"] = np.where( (Allergy2["noc_"+VAR+"_bd"] == 'No') & (Allergy2["noc_"+VAR+"_treat"] == 'Yes'), 'UNKNOWN', A["noc_date_first_treated"])
A["noc_date_first_treated"] = np.where( Allergy2["noc_"+VAR+"_bd"].isnull() , 'Not Reported', A["noc_date_first_treated"])

A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"] == 'Yes' , "Yes", np.nan)
A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"] == 'No' ,'UNTREATED', A["noc_treat"])
A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"].isnull() , 'Not Reported', A["noc_treat"])



A["OPHID_Text"] = np.nan
A["maskedid"] = np.array(Allergy2["maskedID"])
A["noc_date_rep"] = np.array(Allergy2["noc_date"])
A["site"] = Allergy2["site"]
A["source"] = 'REDCap'
A["noc_da"] = np.array(Allergy2["noc_" + VAR + "_da"])
A["noc_a"] = np.array(Allergy2["noc_" + VAR + "_a"])
A["visit_id"] = np.array(Allergy2["visit_id"]); A['noc_details'] = np.nan

A17 = A.copy(deep=True)


VAR = "hyperthy"

Allergy2 = df1[df1["noc_"+VAR] == 'Yes']
[['noc_'+VAR+'_dd'
, 'noc_'+VAR+'_dbd'
,'visit_id'
,'noc_'+VAR+'_a'
,'noc_'+VAR+'_da'
,'noc_'+VAR+'_treat'
, 'noc_'+VAR+'_d'
, 'noc_'+VAR+'_bd'
, "noc_date"
, 'maskedID'
, 'site']]


Allergy2["noc_" + VAR + "_dd"] = pd.to_datetime(df1["noc_" + VAR + "_dd"],  errors='coerce').dt.strftime('%m/%d/%Y')
Allergy2["noc_" + VAR + "_d"] = pd.to_datetime(df1["noc_" + VAR + "_d"],  errors='coerce').dt.strftime('%m/%d/%Y')

A = pd.DataFrame(Allergy2["site"], columns = ["site"])
A["name"] = "Thyroid - Hyperthyroidism"
A['sys_noc_id'] = 274


A["noc_date_dx"] = np.where(Allergy2["noc_"+VAR+"_dbd"]== 'No', 'UNKNOWN', Allergy2["noc_" + VAR + "_dd"])
A["noc_date_dx"] = np.where(Allergy2["noc_"+VAR+"_dbd"].isnull(), 'Not Reported', A["noc_date_dx"])
A["noc_date_first_treated"] = np.where( (Allergy2["noc_"+VAR+"_bd"] == 'Yes') & (Allergy2["noc_"+VAR+"_treat"] == 'Yes') , Allergy2["noc_" + VAR + "_d"], np.nan)
A["noc_date_first_treated"] = np.where( (Allergy2["noc_"+VAR+"_bd"] == 'No') & (Allergy2["noc_"+VAR+"_treat"] == 'Yes'), 'UNKNOWN', A["noc_date_first_treated"])
A["noc_date_first_treated"] = np.where( Allergy2["noc_"+VAR+"_bd"].isnull() , 'Not Reported', A["noc_date_first_treated"])

A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"] == 'Yes' , "Yes", np.nan)
A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"] == 'No' ,'UNTREATED', A["noc_treat"])
A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"].isnull() , 'Not Reported', A["noc_treat"])



A["OPHID_Text"] = np.nan
A["maskedid"] = np.array(Allergy2["maskedID"])
A["noc_date_rep"] = np.array(Allergy2["noc_date"])
A["site"] = Allergy2["site"]
A["source"] = 'REDCap'
A["noc_da"] = np.array(Allergy2["noc_" + VAR + "_da"])
A["noc_a"] = np.array(Allergy2["noc_" + VAR + "_a"])
A["visit_id"] = np.array(Allergy2["visit_id"]); A['noc_details'] = np.nan

A18 = A.copy(deep=True)


VAR = "hypothy"

Allergy2 = df1[df1["noc_"+VAR] == 'Yes']
[['noc_'+VAR+'_dd'
, 'noc_'+VAR+'_dbd'
,'visit_id'
,'noc_'+VAR+'_a'
,'noc_'+VAR+'_da'
,'noc_'+VAR+'_treat'
, 'noc_'+VAR+'_d'
, 'noc_'+VAR+'_bd'
, "noc_date"
, 'maskedID'
, 'site']]

Allergy2["noc_" + VAR + "_dd"] = pd.to_datetime(df1["noc_" + VAR + "_dd"],  errors='coerce').dt.strftime('%m/%d/%Y')
Allergy2["noc_" + VAR + "_d"] = pd.to_datetime(df1["noc_" + VAR + "_d"],  errors='coerce').dt.strftime('%m/%d/%Y')

A = pd.DataFrame(Allergy2["site"], columns = ["site"])
A["name"] = "Thyroid - Hypothyroidism"
A['sys_noc_id'] = 275


A["noc_date_dx"] = np.where(Allergy2["noc_"+VAR+"_dbd"]== 'No', 'UNKNOWN', Allergy2["noc_" + VAR + "_dd"])
A["noc_date_dx"] = np.where(Allergy2["noc_"+VAR+"_dbd"].isnull(), 'Not Reported', A["noc_date_dx"])
A["noc_date_first_treated"] = np.where( (Allergy2["noc_"+VAR+"_bd"] == 'Yes') & (Allergy2["noc_"+VAR+"_treat"] == 'Yes') , Allergy2["noc_" + VAR + "_d"], np.nan)
A["noc_date_first_treated"] = np.where( (Allergy2["noc_"+VAR+"_bd"] == 'No') & (Allergy2["noc_"+VAR+"_treat"] == 'Yes'), 'UNKNOWN', A["noc_date_first_treated"])
A["noc_date_first_treated"] = np.where( Allergy2["noc_"+VAR+"_bd"].isnull() , 'Not Reported', A["noc_date_first_treated"])

A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"] == 'Yes' , "Yes", np.nan)
A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"] == 'No' ,'UNTREATED', A["noc_treat"])
A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"].isnull() , 'Not Reported', A["noc_treat"])



A["OPHID_Text"] = np.nan
A["maskedid"] = np.array(Allergy2["maskedID"])
A["noc_date_rep"] = np.array(Allergy2["noc_date"])
A["site"] = Allergy2["site"]
A["source"] = 'REDCap'
A["noc_da"] = np.array(Allergy2["noc_" + VAR + "_da"])
A["noc_a"] = np.array(Allergy2["noc_" + VAR + "_a"])
A["visit_id"] = np.array(Allergy2["visit_id"]); A['noc_details'] = np.nan
A19 = A.copy(deep=True)



####

VAR = "diabeteso"

Allergy2 = df1[df1["noc_"+VAR] == 'Yes']
[['noc_'+VAR+'_dd'
, 'noc_'+VAR+'_dbd'
,'visit_id'
,'noc_'+VAR+'_a'
,'noc_'+VAR+'_da'
,'noc_'+VAR+'_treat'
, 'noc_'+VAR+'_d'
, 'noc_'+VAR+'_bd'
, "noc_date"
, 'maskedID'
, 'site']]

Allergy2["noc_" + VAR + "_dd"] = pd.to_datetime(df1["noc_" + VAR + "_dd"],  errors='coerce').dt.strftime('%m/%d/%Y')
Allergy2["noc_" + VAR + "_d"] = pd.to_datetime(df1["noc_" + VAR + "_d"],  errors='coerce').dt.strftime('%m/%d/%Y')

A = pd.DataFrame(Allergy2["site"], columns = ["site"])
A["name"] = "diabetes other"
A['sys_noc_id'] = np.nan


A["noc_date_dx"] = np.where(Allergy2["noc_"+VAR+"_dbd"]== 'No', 'UNKNOWN', Allergy2["noc_" + VAR + "_dd"])
A["noc_date_dx"] = np.where(Allergy2["noc_"+VAR+"_dbd"].isnull(), 'Not Reported', A["noc_date_dx"])
A["noc_date_first_treated"] = np.where( (Allergy2["noc_"+VAR+"_bd"] == 'Yes') & (Allergy2["noc_"+VAR+"_treat"] == 'Yes') , Allergy2["noc_" + VAR + "_d"], np.nan)
A["noc_date_first_treated"] = np.where( (Allergy2["noc_"+VAR+"_bd"] == 'No') & (Allergy2["noc_"+VAR+"_treat"] == 'Yes'), 'UNKNOWN', A["noc_date_first_treated"])
A["noc_date_first_treated"] = np.where( Allergy2["noc_"+VAR+"_bd"].isnull() , 'Not Reported', A["noc_date_first_treated"])

A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"] == 'Yes' , "Yes", np.nan)
A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"] == 'No' ,'UNTREATED', A["noc_treat"])
A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"].isnull() , 'Not Reported', A["noc_treat"])



A["OPHID_Text"] = np.nan
A["maskedid"] = np.array(Allergy2["maskedID"])
A["noc_date_rep"] = np.array(Allergy2["noc_date"])
A["site"] = Allergy2["site"]
A["source"] = 'REDCap'
A["noc_da"] = np.array(Allergy2["noc_" + VAR + "_da"])
A["noc_a"] = np.array(Allergy2["noc_" + VAR + "_a"])
A["visit_id"] = np.array(Allergy2["visit_id"]);
A['noc_details'] = np.array(Allergy2['noc_'+VAR+'_spec'])
A19 = A.copy(deep=True)





###

VAR = "highcol"

Allergy2 = df1[df1["noc_"+VAR] == 'Yes']
[['noc_'+VAR+'_dd'
, 'noc_'+VAR+'_dbd'
,'visit_id'
,'noc_'+VAR+'_a'
,'noc_'+VAR+'_da'
,'noc_'+VAR+'_treat'
, 'noc_'+VAR+'_d'
, 'noc_'+VAR+'_bd'
, "noc_date"
, 'maskedID'
, 'site']]

Allergy2["noc_" + VAR + "_dd"] = pd.to_datetime(df1["noc_" + VAR + "_dd"],  errors='coerce').dt.strftime('%m/%d/%Y')
Allergy2["noc_" + VAR + "_d"] = pd.to_datetime(df1["noc_" + VAR + "_d"],  errors='coerce').dt.strftime('%m/%d/%Y')

A = pd.DataFrame(Allergy2["site"], columns = ["site"])
A["name"] = "High Cholesterol"
A['sys_noc_id'] = np.nan


A["noc_date_dx"] = np.where(Allergy2["noc_"+VAR+"_dbd"]== 'No', 'UNKNOWN', Allergy2["noc_" + VAR + "_dd"])
A["noc_date_dx"] = np.where(Allergy2["noc_"+VAR+"_dbd"].isnull(), 'Not Reported', A["noc_date_dx"])
A["noc_date_first_treated"] = np.where( (Allergy2["noc_"+VAR+"_bd"] == 'Yes') & (Allergy2["noc_"+VAR+"_treat"] == 'Yes') , Allergy2["noc_" + VAR + "_d"], np.nan)
A["noc_date_first_treated"] = np.where( (Allergy2["noc_"+VAR+"_bd"] == 'No') & (Allergy2["noc_"+VAR+"_treat"] == 'Yes'), 'UNKNOWN', A["noc_date_first_treated"])
A["noc_date_first_treated"] = np.where( Allergy2["noc_"+VAR+"_bd"].isnull() , 'Not Reported', A["noc_date_first_treated"])

A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"] == 'Yes' , "Yes", np.nan)
A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"] == 'No' ,'UNTREATED', A["noc_treat"])
A["noc_treat"] = np.where( Allergy2["noc_" + VAR + "_treat"].isnull() , 'Not Reported', A["noc_treat"])



A["OPHID_Text"] = np.nan
A["maskedid"] = np.array(Allergy2["maskedID"])
A["noc_date_rep"] = np.array(Allergy2["noc_date"])
A["site"] = Allergy2["site"]
A["source"] = 'REDCap'
A["noc_da"] = np.array(Allergy2["noc_" + VAR + "_da"])
A["noc_a"] = np.array(Allergy2["noc_" + VAR + "_a"])
A["visit_id"] = np.array(Allergy2["visit_id"]); A['noc_details'] = np.nan
A190 = A.copy(deep=True)


###

VAR = "1"

Allergy2 = df1[df1['noc_condreport'+VAR] == 'Yes']
[['noc_condreport'+VAR,
 'noc_cond'+VAR,
 'noc_othercond'+VAR,
 'noc_diag_bool_date'+VAR,
 'noc_diag_date'+VAR,
 'noc_diag_age'+VAR,
 'noc_cond' +VAR,'_treat'
 'noc_treat_bool_date'+VAR,
 'noc_treat_date'+VAR,
 'noc_treat_age'+VAR,
'visit_id'
, "noc_date"
, 'maskedID'
, 'site']]

Allergy2['noc_diag_date'+VAR] = pd.to_datetime(df1['noc_diag_date'+VAR],  errors='coerce').dt.strftime('%m/%d/%Y')
Allergy2['noc_treat_date'+VAR] = pd.to_datetime(df1['noc_treat_date'+VAR],  errors='coerce').dt.strftime('%m/%d/%Y')

A = pd.DataFrame(Allergy2["site"], columns = ["site"])
A["name"] = np.array(Allergy2['noc_cond'+VAR])
A['sys_noc_id'] = np.nan


A["noc_date_dx"] = np.where(Allergy2['noc_diag_bool_date'+VAR]== 'No', 'UNKNOWN', Allergy2['noc_diag_date'+VAR])
A["noc_date_dx"] = np.where(Allergy2['noc_diag_bool_date'+VAR].isnull(), 'Not Reported', A["noc_date_dx"])
A["noc_date_first_treated"] = np.where( Allergy2['noc_treat_bool_date'+VAR] == 'Yes' , Allergy2['noc_treat_date'+VAR], np.nan)
A["noc_date_first_treated"] = np.where( Allergy2['noc_treat_bool_date'+VAR] == 'No' ,'UNKNOWN', A["noc_date_first_treated"])
A["noc_date_first_treated"] = np.where( Allergy2['noc_treat_bool_date'+VAR].isnull() , 'Not Reported', A["noc_date_first_treated"])

A["noc_treat"] = np.where( Allergy2['noc_cond' +VAR+'_treat'] == 'Yes' , "Yes", np.nan)
A["noc_treat"] = np.where( Allergy2['noc_cond' +VAR+'_treat'] == 'No' ,'UNTREATED', A["noc_treat"])
A["noc_treat"] = np.where( Allergy2['noc_cond' +VAR+'_treat'].isnull() , 'Not Reported', A["noc_treat"])



A["OPHID_Text"] = np.nan
A["maskedid"] = np.array(Allergy2["maskedID"])
A["noc_date_rep"] = np.array(Allergy2["noc_date"])
A["site"] = Allergy2["site"]
A["source"] = 'REDCap'
A["noc_da"] = np.array(Allergy2['noc_diag_age'+VAR])
A["noc_a"] = np.array(Allergy2['noc_treat_age'+VAR])
A["visit_id"] = np.array(Allergy2["visit_id"]); A['noc_details'] = np.array(Allergy2['noc_othercond'+VAR])
A20 = A.copy(deep=True)

###


VAR = "2"

Allergy2 = df1[df1['noc_condreport'+VAR] == 'Yes']
[['noc_condreport'+VAR,
 'noc_cond'+VAR,
 'noc_othercond'+VAR,
 'noc_diag_bool_date'+VAR,
 'noc_diag_date'+VAR,
 'noc_diag_age'+VAR,
 'noc_cond' +VAR,'_treat'
 'noc_treat_bool_date'+VAR,
 'noc_treat_date'+VAR,
 'noc_treat_age'+VAR,
'visit_id'
, "noc_date"
, 'maskedID'
, 'site']]

Allergy2['noc_diag_date'+VAR] = pd.to_datetime(df1['noc_diag_date'+VAR],  errors='coerce').dt.strftime('%m/%d/%Y')
Allergy2['noc_treat_date'+VAR] = pd.to_datetime(df1['noc_treat_date'+VAR],  errors='coerce').dt.strftime('%m/%d/%Y')

A = pd.DataFrame(Allergy2["site"], columns = ["site"])
A["name"] = np.array(Allergy2['noc_cond'+VAR])
A['sys_noc_id'] = np.nan


A["noc_date_dx"] = np.where(Allergy2['noc_diag_bool_date'+VAR]== 'No', 'UNKNOWN', Allergy2['noc_diag_date'+VAR])
A["noc_date_dx"] = np.where(Allergy2['noc_diag_bool_date'+VAR].isnull(), 'Not Reported', A["noc_date_dx"])
A["noc_date_first_treated"] = np.where( Allergy2['noc_treat_bool_date'+VAR] == 'Yes' , Allergy2['noc_treat_date'+VAR], np.nan)
A["noc_date_first_treated"] = np.where( Allergy2['noc_treat_bool_date'+VAR] == 'No' ,'UNKNOWN', A["noc_date_first_treated"])
A["noc_date_first_treated"] = np.where( Allergy2['noc_treat_bool_date'+VAR].isnull() , 'Not Reported', A["noc_date_first_treated"])

A["noc_treat"] = np.where( Allergy2['noc_cond' +VAR+'_treat'] == 'Yes' , "Yes", np.nan)
A["noc_treat"] = np.where( Allergy2['noc_cond' +VAR+'_treat'] == 'No' ,'UNTREATED', A["noc_treat"])
A["noc_treat"] = np.where( Allergy2['noc_cond' +VAR+'_treat'].isnull() , 'Not Reported', A["noc_treat"])



A["OPHID_Text"] = np.nan
A["maskedid"] = np.array(Allergy2["maskedID"])
A["noc_date_rep"] = np.array(Allergy2["noc_date"])
A["site"] = Allergy2["site"]
A["source"] = 'REDCap'
A["noc_da"] = np.array(Allergy2['noc_diag_age'+VAR])
A["noc_a"] = np.array(Allergy2['noc_treat_age'+VAR])
A["visit_id"] = np.array(Allergy2["visit_id"]); A['noc_details'] = np.array(Allergy2['noc_othercond'+VAR])
A21 = A.copy(deep=True)

###


VAR = "3"

Allergy2 = df1[df1['noc_condreport'+VAR] == 'Yes']
[['noc_condreport'+VAR,
 'noc_cond'+VAR,
 'noc_othercond'+VAR,
 'noc_diag_bool_date'+VAR,
 'noc_diag_date'+VAR,
 'noc_diag_age'+VAR,
 'noc_cond' +VAR,'_treat'
 'noc_treat_bool_date'+VAR,
 'noc_treat_date'+VAR,
 'noc_treat_age'+VAR,
'visit_id'
, "noc_date"
, 'maskedID'
, 'site']]

Allergy2['noc_diag_date'+VAR] = pd.to_datetime(df1['noc_diag_date'+VAR],  errors='coerce').dt.strftime('%m/%d/%Y')
Allergy2['noc_treat_date'+VAR] = pd.to_datetime(df1['noc_treat_date'+VAR],  errors='coerce').dt.strftime('%m/%d/%Y')

A = pd.DataFrame(Allergy2["site"], columns = ["site"])
A["name"] = np.array(Allergy2['noc_cond'+VAR])
A['sys_noc_id'] = np.nan


A["noc_date_dx"] = np.where(Allergy2['noc_diag_bool_date'+VAR]== 'No', 'UNKNOWN', Allergy2['noc_diag_date'+VAR])
A["noc_date_dx"] = np.where(Allergy2['noc_diag_bool_date'+VAR].isnull(), 'Not Reported', A["noc_date_dx"])
A["noc_date_first_treated"] = np.where( Allergy2['noc_treat_bool_date'+VAR] == 'Yes' , Allergy2['noc_treat_date'+VAR], np.nan)
A["noc_date_first_treated"] = np.where( Allergy2['noc_treat_bool_date'+VAR] == 'No' ,'UNKNOWN', A["noc_date_first_treated"])
A["noc_date_first_treated"] = np.where( Allergy2['noc_treat_bool_date'+VAR].isnull() , 'Not Reported', A["noc_date_first_treated"])

A["noc_treat"] = np.where( Allergy2['noc_cond' +VAR+'_treat'] == 'Yes' , "Yes", np.nan)
A["noc_treat"] = np.where( Allergy2['noc_cond' +VAR+'_treat'] == 'No' ,'UNTREATED', A["noc_treat"])
A["noc_treat"] = np.where( Allergy2['noc_cond' +VAR+'_treat'].isnull() , 'Not Reported', A["noc_treat"])



A["OPHID_Text"] = np.nan
A["maskedid"] = np.array(Allergy2["maskedID"])
A["noc_date_rep"] = np.array(Allergy2["noc_date"])
A["site"] = Allergy2["site"]
A["source"] = 'REDCap'
A["noc_da"] = np.array(Allergy2['noc_diag_age'+VAR])
A["noc_a"] = np.array(Allergy2['noc_treat_age'+VAR])
A["visit_id"] = np.array(Allergy2["visit_id"]); A['noc_details'] = np.array(Allergy2['noc_othercond'+VAR])
A22 = A.copy(deep=True)



VAR = "4"

Allergy2 = df1[df1['noc_condreport'+VAR] == 'Yes']
[['noc_condreport'+VAR,
 'noc_cond'+VAR,
 'noc_othercond'+VAR,
 'noc_diag_bool_date'+VAR,
 'noc_diag_date'+VAR,
 'noc_diag_age'+VAR,
 'noc_cond' +VAR,'_treat'
 'noc_treat_bool_date'+VAR,
 'noc_treat_date'+VAR,
 'noc_treat_age'+VAR,
'visit_id'
, "noc_date"
, 'maskedID'
, 'site']]

Allergy2['noc_diag_date'+VAR] = pd.to_datetime(df1['noc_diag_date'+VAR],  errors='coerce').dt.strftime('%m/%d/%Y')
Allergy2['noc_treat_date'+VAR] = pd.to_datetime(df1['noc_treat_date'+VAR],  errors='coerce').dt.strftime('%m/%d/%Y')

A = pd.DataFrame(Allergy2["site"], columns = ["site"])
A["name"] = np.array(Allergy2['noc_cond'+VAR])
A['sys_noc_id'] = np.nan


A["noc_date_dx"] = np.where(Allergy2['noc_diag_bool_date'+VAR]== 'No', 'UNKNOWN', Allergy2['noc_diag_date'+VAR])
A["noc_date_dx"] = np.where(Allergy2['noc_diag_bool_date'+VAR].isnull(), 'Not Reported', A["noc_date_dx"])
A["noc_date_first_treated"] = np.where( Allergy2['noc_treat_bool_date'+VAR] == 'Yes' , Allergy2['noc_treat_date'+VAR], np.nan)
A["noc_date_first_treated"] = np.where( Allergy2['noc_treat_bool_date'+VAR] == 'No' ,'UNKNOWN', A["noc_date_first_treated"])
A["noc_date_first_treated"] = np.where( Allergy2['noc_treat_bool_date'+VAR].isnull() , 'Not Reported', A["noc_date_first_treated"])

A["noc_treat"] = np.where( Allergy2['noc_cond' +VAR+'_treat'] == 'Yes' , "Yes", np.nan)
A["noc_treat"] = np.where( Allergy2['noc_cond' +VAR+'_treat'] == 'No' ,'UNTREATED', A["noc_treat"])
A["noc_treat"] = np.where( Allergy2['noc_cond' +VAR+'_treat'].isnull() , 'Not Reported', A["noc_treat"])



A["OPHID_Text"] = np.nan
A["maskedid"] = np.array(Allergy2["maskedID"])
A["noc_date_rep"] = np.array(Allergy2["noc_date"])
A["site"] = Allergy2["site"]
A["source"] = 'REDCap'
A["noc_da"] = np.array(Allergy2['noc_diag_age'+VAR])
A["noc_a"] = np.array(Allergy2['noc_treat_age'+VAR])
A["visit_id"] = np.array(Allergy2["visit_id"]); A['noc_details'] = np.array(Allergy2['noc_othercond'+VAR])
A23 = A.copy(deep=True)



Allergy2 = df1[df1['noc_cond_add'] == 'Yes']
[['noc_cond_add',
 'noc_cond_more',
'visit_id'
, "noc_date"
, 'maskedID'
, 'site']]


A = pd.DataFrame(Allergy2["site"], columns = ["site"])
A["name"] = 'Other (specify)'
A['sys_noc_id'] = np.nan


A["noc_date_dx"] = np.nan

A["noc_date_first_treated"] = np.nan

A["noc_treat"] = np.nan



A["OPHID_Text"] = np.nan
A["maskedid"] = np.array(Allergy2["maskedID"])
A["noc_date_rep"] = np.array(Allergy2["noc_date"])
A["site"] = Allergy2["site"]
A["source"] = 'REDCap'
A["source"] = 'REDCap'
A["noc_da"] = np.nan
A["noc_a"] = np.nan
A["visit_id"] = np.array(Allergy2["visit_id"]); A['noc_details'] = np.array(Allergy2['noc_cond_more'])
A24 = A.copy(deep=True)



df["noc_da"] = np.nan
df["noc_a"] = np.nan
df['noc_treat'] = np.nan
H = list(df.columns.values)
FIN = pd.concat([df, A1[H]], ignore_index=True)
FIN = pd.concat([FIN, A2[H]], ignore_index=True)
FIN = pd.concat([FIN, A3[H]], ignore_index=True)
FIN = pd.concat([FIN, A4[H]], ignore_index=True)
FIN = pd.concat([FIN, A5[H]], ignore_index=True)
FIN = pd.concat([FIN, A6[H]], ignore_index=True)
FIN = pd.concat([FIN, A7[H]], ignore_index=True)
FIN = pd.concat([FIN, A8[H]], ignore_index=True)
FIN = pd.concat([FIN, A9[H]], ignore_index=True)
FIN = pd.concat([FIN, A10[H]], ignore_index=True)
FIN = pd.concat([FIN, A11[H]], ignore_index=True)
FIN = pd.concat([FIN, A12[H]], ignore_index=True)
FIN = pd.concat([FIN, A13[H]], ignore_index=True)
FIN = pd.concat([FIN, A14[H]], ignore_index=True)
FIN = pd.concat([FIN, A15[H]], ignore_index=True)
FIN = pd.concat([FIN, A16[H]], ignore_index=True)
FIN = pd.concat([FIN, A17[H]], ignore_index=True)
FIN = pd.concat([FIN, A18[H]], ignore_index=True)
FIN = pd.concat([FIN, A19[H]], ignore_index=True)
FIN = pd.concat([FIN, A190[H]], ignore_index=True)
FIN = pd.concat([FIN, A20[H]], ignore_index=True)
FIN = pd.concat([FIN, A21[H]], ignore_index=True)
FIN = pd.concat([FIN, A22[H]], ignore_index=True)
FIN = pd.concat([FIN, A23[H]], ignore_index=True)

FIN = pd.merge(FIN, ind[["OPHID_Text", "researchid"]], how='left', on='OPHID_Text')



FIN = FIN[~FIN['maskedid'].isnull()]

FIN = pd.merge(FIN, ind[["OPHID_Text_1", "maskedid"]], how='left', on='maskedid')
FIN["OPHID_Text"] = np.where(FIN["OPHID_Text"].isnull(), FIN["OPHID_Text_1"], FIN["OPHID_Text"])

FIN['maskedid'] = np.where(FIN['maskedid'].isnull(), FIN['researchid'], FIN['maskedid'])
FIN['BioBID'] = np.where(FIN['researchid'].isnull(), FIN['maskedid'], FIN['researchid'])
FIN['BioBID'] = np.where( (FIN['researchid'].str.contains('OG')) | (FIN['researchid'].str.contains('ED')), FIN['BioBID'], np.nan)
FIN["site"] = FIN['maskedid'].str[:2]

FIN = FIN[name]


FIN["noc_date_dx"] = np.where(FIN["noc_date_dx"]=='NaT', "", FIN["noc_date_dx"])
FIN["noc_date_first_treated"] = np.where(FIN["noc_date_first_treated"]=='NaT', "", FIN["noc_date_first_treated"])


FIN.to_csv(redcapeyechart + 'Results/' + extension  + '_NonOcularConditions_RedEC.csv', encoding = 'utf-8', index=False)


/var/folders/7m/n2zn61593k18rvyz907d2skc0000gn/T/ipykernel_30504/773164059.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Allergy2["noc_" + VAR + "_dd"] = pd.to_datetime(df1["noc_" + VAR + "_dd"],  errors='coerce').dt.strftime('%m/%d/%Y')
/var/folders/7m/n2zn61593k18rvyz907d2skc0000gn/T/ipykernel_30504/773164059.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Allergy2["noc_" + VAR + "_d"] = pd.to_datetime(df1["noc_" + VAR + "_d"],  errors='coerce').dt.strftime('%m/%d/%Y')
/var/folders/7m/n2zn6159

/var/folders/7m/n2zn61593k18rvyz907d2skc0000gn/T/ipykernel_30504/773164059.py:728: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Allergy2["noc_" + VAR + "_dd"] = pd.to_datetime(df1["noc_" + VAR + "_dd"],  errors='coerce').dt.strftime('%m/%d/%Y')
/var/folders/7m/n2zn61593k18rvyz907d2skc0000gn/T/ipykernel_30504/773164059.py:729: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Allergy2["noc_" + VAR + "_d"] = pd.to_datetime(df1["noc_" + VAR + "_d"],  errors='coerce').dt.strftime('%m/%d/%Y')
/var/folders/7m/n2zn61

In [21]:
L1 = [i for i in L_name if '20170322_NonOcularMedications_wdiabetichypertensive' in i]
df = pd.read_csv(EyeChart + L1[0], encoding = "ISO-8859-1")
#df = pd.read_csv(EyeChart + '20170322_NonOcularMedications_wdiabetichypertensive.csv', encoding = "ISO-8859-1")

df["visit_id"] = np.nan
df['nom_rep'] = np.nan
df1 = pd.read_csv(redcapeyechart + extension +'_NonOcularMedications.csv', encoding = "ISO-8859-1")
df10 = pd.read_csv(redcapeyechart + 'hyper_diab.csv', encoding = "ISO-8859-1")

name = 'maskedid	OPHID_Text	BioBID	site	visit_id	nom_rep	sys_nom_id	name	anti_diabetic	anti_hypertensive	nom_date_rep	nom_date	nomed_medage	nom_details	source'
name = name.split('\t')
Diab = df10[df10['canti-diabetic'] == 1][["name"]].reset_index(drop=True)
Diab["anti_diabetic"] = 'Yes'
hyper = df10[df10['canti-hypertensive'] == 1][["name"]].reset_index(drop=True)
hyper["anti_hypertensive"] = 'Yes'

df["site"] = df['maskedid'].str[:2]
df["source"] = 'EyeChart'
#df1 = df1[~df1["maskedID"].isnull()]
df2 = df1[(df1["nomed_medreport1"] == 'Yes') | (df1["nomed_medreport1"] == 'No')][['nomed_medreport1',
 'nomed_med1',
 'nomed_othermed1',
 'nomed_allergy1',
 'nomed_date1',
 'nomed_medstartdate1',
 'nomed_medage1','maskedID',
 'site','nomed_date', 'visit_id']]

A = pd.DataFrame(np.array(df2["nomed_med1"]), columns=["name"])
A["site"] = np.array(df2["site"])
A["maskedid"] = np.array(df2["maskedID"])
A["OPHID_Text"] = np.nan

A = pd.merge(A,Diab, how='left', on= 'name')
A["anti_diabetic"] = A["anti_diabetic"].fillna('No')
A["sys_nom_id"] = np.nan
A = pd.merge(A,hyper, how='left', on= 'name')
A["anti_hypertensive"] = A["anti_hypertensive"].fillna('No')

A["nom_date_rep"] = np.array(df2["nomed_date"])
A["nom_date"] = np.array(df2["nomed_medstartdate1"])
A["nom_details"] = np.array(df2["nomed_othermed1"])
A["nomed_medage"] = np.array(df2["nomed_medage1"])
A["visit_id"] = np.array(df2["visit_id"])
A["nom_rep"] = np.array(df2["nomed_medreport1"])

A["source"] = 'REDCap'

A1 = A.copy(deep=True)

df2 = df1[df1["nomed_medreport2"] == 'Yes'][['nomed_medreport2',
 'nomed_med2',
 'nomed_othermed2',
 'nomed_allergy2',
 'nomed_date2',
 'nomed_medstartdate2',
 'nomed_medage2','maskedID',
 'site','nomed_date', 'visit_id']]

A = pd.DataFrame(np.array(df2["nomed_med2"]), columns=["name"])
A["site"] = np.array(df2["site"])
A["maskedid"] = np.array(df2["maskedID"])
A["OPHID_Text"] = np.nan

A["visit_id"] = np.array(df2["visit_id"])
A["nom_rep"] = np.array(df2["nomed_medreport2"])

A = pd.merge(A,Diab, how='left', on= 'name')
A["anti_diabetic"] = A["anti_diabetic"].fillna('No')
A["sys_nom_id"] = np.nan
A = pd.merge(A,hyper, how='left', on= 'name')
A["anti_hypertensive"] = A["anti_hypertensive"].fillna('No')

A["nom_date_rep"] = np.array(df2["nomed_date"])
A["nom_date"] = np.array(df2["nomed_medstartdate2"])

A["nom_details"] = np.array(df2["nomed_othermed2"])
A["nomed_medage"] = np.array(df2["nomed_medage2"])
A["source"] = 'REDCap'

A2 = A.copy(deep=True)

df2 = df1[df1["nomed_medreport3"] == 'Yes'][['nomed_medreport3',
 'nomed_med3',
 'nomed_othermed3',
 'nomed_allergy3',
 'nomed_date3',
 'nomed_medstartdate3',
 'nomed_medage3','maskedID',
 'site','nomed_date', 'visit_id']]

A = pd.DataFrame(np.array(df2["nomed_med3"]), columns=["name"])
A["site"] = np.array(df2["site"])
A["maskedid"] = np.array(df2["maskedID"])
A["OPHID_Text"] = np.nan

A["visit_id"] = np.array(df2["visit_id"])
A["nom_rep"] = np.array(df2["nomed_medreport3"])

A = pd.merge(A,Diab, how='left', on= 'name')
A["anti_diabetic"] = A["anti_diabetic"].fillna('No')
A["sys_nom_id"] = np.nan
A = pd.merge(A,hyper, how='left', on= 'name')
A["anti_hypertensive"] = A["anti_hypertensive"].fillna('No')

A["nom_date_rep"] = np.array(df2["nomed_date"])
A["nom_date"] = np.array(df2["nomed_medstartdate3"])

A["nom_details"] = np.array(df2["nomed_othermed3"])
A["nomed_medage"] = np.array(df2["nomed_medage3"])
A["source"] = 'REDCap'

A3 = A.copy(deep=True)

df2 = df1[df1["nomed_medreport4"] == 'Yes'][['nomed_medreport4',
 'nomed_med4',
 'nomed_othermed4',
 'nomed_allergy4',
 'nomed_date4',
 'nomed_medstartdate4',
 'nomed_medage4','maskedID',
 'site','nomed_date', 'visit_id']]

A = pd.DataFrame(np.array(df2["nomed_med4"]), columns=["name"])
A["site"] = np.array(df2["site"])
A["maskedid"] = np.array(df2["maskedID"])
A["OPHID_Text"] = np.nan

A["visit_id"] = np.array(df2["visit_id"])
A["nom_rep"] = np.array(df2["nomed_medreport4"])

A = pd.merge(A,Diab, how='left', on= 'name')
A["anti_diabetic"] = A["anti_diabetic"].fillna('No')
A["sys_nom_id"] = np.nan
A = pd.merge(A,hyper, how='left', on= 'name')
A["anti_hypertensive"] = A["anti_hypertensive"].fillna('No')

A["nom_date_rep"] = np.array(df2["nomed_date"])
A["nom_date"] = np.array(df2["nomed_medstartdate4"])

A["nom_details"] = np.array(df2["nomed_othermed4"])
A["nomed_medage"] = np.array(df2["nomed_medage4"])
A["source"] = 'REDCap'

A4 = A.copy(deep=True)

df2 = df1[df1["nomed_medreport5"] == 'Yes'][['nomed_medreport5',
 'nomed_med5',
 'nomed_othermed5',
 'nomed_allergy5',
 'nomed_date5',
 'nomed_medstartdate5',
 'nomed_medage5','maskedID',
 'site','nomed_date', 'visit_id']]

A = pd.DataFrame(np.array(df2["nomed_med5"]), columns=["name"])
A["site"] = np.array(df2["site"])
A["maskedid"] = np.array(df2["maskedID"])
A["OPHID_Text"] = np.nan

A["visit_id"] = np.array(df2["visit_id"])
A["nom_rep"] = np.array(df2["nomed_medreport5"])

A = pd.merge(A,Diab, how='left', on= 'name')
A["anti_diabetic"] = A["anti_diabetic"].fillna('No')
A["sys_nom_id"] = np.nan
A = pd.merge(A,hyper, how='left', on= 'name')
A["anti_hypertensive"] = A["anti_hypertensive"].fillna('No')

A["nom_date_rep"] = np.array(df2["nomed_date"])
A["nom_date"] = np.array(df2["nomed_medstartdate5"])

A["nom_details"] = np.array(df2["nomed_othermed5"])
A["nomed_medage"] = np.array(df2["nomed_medage5"])
A["source"] = 'REDCap'

A5 = A.copy(deep=True)

df2 = df1[df1["nomed_medreport6"] == 'Yes'][['nomed_medreport6',
 'nomed_med6',
 'nomed_othermed6',
 'nomed_allergy6',
 'nomed_date6',
 'nomed_medstartdate6',
 'nomed_medage6','maskedID',
 'site','nomed_date', 'visit_id']]

A = pd.DataFrame(np.array(df2["nomed_med6"]), columns=["name"])
A["site"] = np.array(df2["site"])
A["maskedid"] = np.array(df2["maskedID"])
A["OPHID_Text"] = np.nan

A["visit_id"] = np.array(df2["visit_id"])
A["nom_rep"] = np.array(df2["nomed_medreport6"])

A = pd.merge(A,Diab, how='left', on= 'name')
A["anti_diabetic"] = A["anti_diabetic"].fillna('No')
A["sys_nom_id"] = np.nan
A = pd.merge(A,hyper, how='left', on= 'name')
A["anti_hypertensive"] = A["anti_hypertensive"].fillna('No')

A["nom_date_rep"] = np.array(df2["nomed_date"])
A["nom_date"] = np.array(df2["nomed_medstartdate6"])

A["nom_details"] = np.array(df2["nomed_othermed6"])
A["nomed_medage"] = np.array(df2["nomed_medage6"])
A["source"] = 'REDCap'

A6 = A.copy(deep=True)

df2 = df1[df1["nomed_medreport7"] == 'Yes'][['nomed_medreport7',
 'nomed_med7',
 'nomed_othermed7',
 'nomed_allergy7',
 'nomed_date7',
 'nomed_medstartdate7',
 'nomed_medage7','maskedID',
 'site','nomed_date', 'visit_id']]

A = pd.DataFrame(np.array(df2["nomed_med7"]), columns=["name"])
A["site"] = np.array(df2["site"])
A["maskedid"] = np.array(df2["maskedID"])
A["OPHID_Text"] = np.nan

A["visit_id"] = np.array(df2["visit_id"])
A["nom_rep"] = np.array(df2["nomed_medreport7"])

A = pd.merge(A,Diab, how='left', on= 'name')
A["anti_diabetic"] = A["anti_diabetic"].fillna('No')
A["sys_nom_id"] = np.nan
A = pd.merge(A,hyper, how='left', on= 'name')
A["anti_hypertensive"] = A["anti_hypertensive"].fillna('No')

A["nom_date_rep"] = np.array(df2["nomed_date"])
A["nom_date"] = np.array(df2["nomed_medstartdate7"])

A["nom_details"] = np.array(df2["nomed_othermed7"])
A["nomed_medage"] = np.array(df2["nomed_medage7"])
A["source"] = 'REDCap'

A7 = A.copy(deep=True)

df2 = df1[df1["nomed_medreport8"] == 'Yes'][['nomed_medreport8',
 'nomed_med8',
 'nomed_othermed8',
 'nomed_allergy8',
 'nomed_date8',
 'nomed_medstartdate8',
 'nomed_medage8','maskedID',
 'site','nomed_date', 'visit_id']]

A = pd.DataFrame(np.array(df2["nomed_med8"]), columns=["name"])
A["site"] = np.array(df2["site"])
A["maskedid"] = np.array(df2["maskedID"])
A["OPHID_Text"] = np.nan

A["visit_id"] = np.array(df2["visit_id"])
A["nom_rep"] = np.array(df2["nomed_medreport8"])

A = pd.merge(A,Diab, how='left', on= 'name')
A["anti_diabetic"] = A["anti_diabetic"].fillna('No')
A["sys_nom_id"] = np.nan
A = pd.merge(A,hyper, how='left', on= 'name')
A["anti_hypertensive"] = A["anti_hypertensive"].fillna('No')

A["nom_date_rep"] = np.array(df2["nomed_date"])
A["nom_date"] = np.array(df2["nomed_medstartdate8"])

A["nom_details"] = np.array(df2["nomed_othermed8"])
A["nomed_medage"] = np.array(df2["nomed_medage8"])
A["source"] = 'REDCap'

A8 = A.copy(deep=True)

df2 = df1[df1["nomed_medreport9"] == 'Yes'][['nomed_medreport9',
 'nomed_med9',
 'nomed_othermed9',
 'nomed_allergy9',
 'nomed_date9',
 'nomed_medstartdate9',
 'nomed_medage9','maskedID',
 'site','nomed_date', 'visit_id']]

A = pd.DataFrame(np.array(df2["nomed_med9"]), columns=["name"])
A["site"] = np.array(df2["site"])
A["maskedid"] = np.array(df2["maskedID"])
A["OPHID_Text"] = np.nan

A["visit_id"] = np.array(df2["visit_id"])
A["nom_rep"] = np.array(df2["nomed_medreport9"])

A = pd.merge(A,Diab, how='left', on= 'name')
A["anti_diabetic"] = A["anti_diabetic"].fillna('No')
A["sys_nom_id"] = np.nan
A = pd.merge(A,hyper, how='left', on= 'name')
A["anti_hypertensive"] = A["anti_hypertensive"].fillna('No')

A["nom_date_rep"] = np.array(df2["nomed_date"])
A["nom_date"] = np.array(df2["nomed_medstartdate9"])

A["nom_details"] = np.array(df2["nomed_othermed9"])
A["nomed_medage"] = np.array(df2["nomed_medage9"])
A["source"] = 'REDCap'

A9 = A.copy(deep=True)

df2 = df1[df1["nomed_medreport10"] == 'Yes'][['nomed_medreport10',
 'nomed_med10',
 'nomed_othermed10',
 'nomed_allergy10',
 'nomed_date10',
 'nomed_medstartdate10',
 'nomed_medage10','maskedID',
 'site','nomed_date', 'visit_id']]

A = pd.DataFrame(np.array(df2["nomed_med10"]), columns=["name"])
A["site"] = np.array(df2["site"])
A["maskedid"] = np.array(df2["maskedID"])
A["OPHID_Text"] = np.nan

A["visit_id"] = np.array(df2["visit_id"])
A["nom_rep"] = np.array(df2["nomed_medreport10"])

A = pd.merge(A,Diab, how='left', on= 'name')
A["anti_diabetic"] = A["anti_diabetic"].fillna('No')
A["sys_nom_id"] = np.nan
A = pd.merge(A,hyper, how='left', on= 'name')
A["anti_hypertensive"] = A["anti_hypertensive"].fillna('No')

A["nom_date_rep"] = np.array(df2["nomed_date"])
A["nom_date"] = np.array(df2["nomed_medstartdate10"])

A["nom_details"] = np.array(df2["nomed_othermed10"])
A["nomed_medage"] = np.array(df2["nomed_medage10"])
A["source"] = 'REDCap'

A10 = A.copy(deep=True)


df2 = df1[df1["nomed_supp"] == 'Yes'][['nomed_supp_notes','maskedID',
 'site','nomed_date', 'visit_id']]

A = pd.DataFrame(np.array(df2["nomed_supp_notes"]), columns=["name"])
A["site"] = np.array(df2["site"])
A["maskedid"] = np.array(df2["maskedID"])
A["OPHID_Text"] = np.nan

A["visit_id"] = np.array(df2["visit_id"])
A["nom_rep"] = np.nan

A = pd.merge(A,Diab, how='left', on= 'name')
A["anti_diabetic"] = A["anti_diabetic"].fillna('No')
A["sys_nom_id"] = np.nan
A = pd.merge(A,hyper, how='left', on= 'name')
A["anti_hypertensive"] = A["anti_hypertensive"].fillna('No')

A["nom_date_rep"] = np.array(df2["nomed_date"])
A["nom_date"] = np.nan

A["nom_details"] = np.array(df2["nomed_supp_notes"])
A["nomed_medage"] = np.nan
A["source"] = 'REDCap'
A11 = A.copy(deep=True)
df["nomed_medage"] = np.nan
H = list(df.columns.values)
FIN = pd.concat([df, A1[H]], ignore_index=True)
FIN = pd.concat([FIN, A2[H]], ignore_index=True)
FIN = pd.concat([FIN, A3[H]], ignore_index=True)
FIN = pd.concat([FIN, A4[H]], ignore_index=True)
FIN = pd.concat([FIN, A5[H]], ignore_index=True)
FIN = pd.concat([FIN, A6[H]], ignore_index=True)
FIN = pd.concat([FIN, A7[H]], ignore_index=True)
FIN = pd.concat([FIN, A8[H]], ignore_index=True)
FIN = pd.concat([FIN, A9[H]], ignore_index=True)
FIN = pd.concat([FIN, A10[H]], ignore_index=True)
FIN = pd.concat([FIN, A11[H]], ignore_index=True)

FIN = pd.merge(FIN, ind[["OPHID_Text", "researchid"]], how='left', on='OPHID_Text')
    
FIN = pd.merge(FIN, ind[["OPHID_Text_1", "maskedid"]], how='left', on='maskedid')
FIN["OPHID_Text"] = np.where(FIN["OPHID_Text"].isnull(), FIN["OPHID_Text_1"], FIN["OPHID_Text"])

FIN['maskedid'] = np.where(FIN['maskedid'].isnull(), FIN['researchid'], FIN['maskedid'])
FIN['BioBID'] = np.where(FIN['researchid'].isnull(), FIN['maskedid'], FIN['researchid'])
FIN['BioBID'] = np.where( (FIN['researchid'].str.contains('OG')) | (FIN['researchid'].str.contains('ED')), FIN['BioBID'], np.nan)
FIN["site"] = FIN['maskedid'].str[:2]

FIN = FIN[name]



FIN.to_csv(redcapeyechart + 'Results/' + extension +'_NonOcularMedications_RedEC.csv', index=False)



In [22]:
L1 = [i for i in L_name if '_NonOcularProcedures' in i]
df = pd.read_csv(EyeChart + L1[0], encoding = "ISO-8859-1")

#df = pd.read_csv(EyeChart + '20170322_NonOcularProcedures.csv', encoding = "ISO-8859-1")
df["visit_id"] = np.nan
df1 = pd.read_csv(redcapeyechart + extension + '_NonOcularSurgeriesProcedures.csv', encoding = "ISO-8859-1")
df["site"] = df['maskedid'].str[:2]
df["source"] = 'EyeChart'
df1 = df1[~df1["maskedID"].isnull()]

name = ['maskedid',
'OPHID_Text',
'BioBID',
'site',
'visit_id',
'sys_nop_id',
'name',
'nop_date_rep',
'nop_date',
'nop_age',
'nop_details',
'source']

df2 = df1[df1["nosp_angio"] == 'Yes'][['maskedID',
 'site',
 'visit_id',
 'nosp_date',
 'nosp_angio',
 'nosp_angio_db',
 'nosp_angio_d',
 'nosp_angio_a',]]




A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["name"] = 'Angioplasty'
A["OPHID_Text"] = np.nan
A["sys_nop_id"] = 125
A["nop_date_rep"] = np.array(df2["nosp_date"])
A["visit_id"] = np.array(df2["visit_id"])
A["nop_date"] = np.array(df2["nosp_angio_d"])
A["nop_date"] = pd.to_datetime(A["nop_date"],  errors='coerce').dt.strftime('%m/%d/%Y')
A["nop_date"] = np.where(df2['nosp_angio_db']=='No', 'UNKNOWN', A["nop_date"])
A["nop_date"] = np.where(df2['nosp_angio_db'].isnull(), 'Not Reported', A["nop_date"])
A["nop_details"] = np.nan
A["nop_age"] = np.array(df2["nosp_angio_a"])
A["source"] = 'REDCap'
A["nop_date_end"] = np.nan
A1 = A.copy(deep=True)

df2 = df1[df1["nosp_oheartsurg"] == 'Yes'][['maskedID',
 'site',
 'visit_id',
 'nosp_date',
 'nosp_oheartsurg',
 'nosp_oheartsurg_db',
 'nosp_oheartsurg_d',
 'nosp_oheartsurg_a',]]




A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["name"] = 'Heart Surgery (Open Heart Surgery)'
A["OPHID_Text"] = np.nan
A["sys_nop_id"] = 124
A["visit_id"] = np.array(df2["visit_id"])
A["nop_date_rep"] = np.array(df2["nosp_date"])
A["nop_date"] = np.array(df2["nosp_oheartsurg_d"])
A["nop_date"] = pd.to_datetime(A["nop_date"],  errors='coerce').dt.strftime('%m/%d/%Y')
A["nop_date"] = np.where(df2['nosp_oheartsurg_db']=='No', 'UNKNOWN', A["nop_date"])
A["nop_date"] = np.where(df2['nosp_oheartsurg_db'].isnull(), 'Not Reported', A["nop_date"])
A["nop_date"] = A["nop_date"].fillna('')
A["nop_details"] = np.nan
A["nop_age"] = np.array(df2["nosp_oheartsurg_a"])
A["source"] = 'REDCap'
A["nop_date_end"] = np.nan
A2 = A.copy(deep=True)

df2 = df1[df1["nosp_hyster"] == 'Yes'][['maskedID',
 'site',
 'visit_id',
 'nosp_date',
 'nosp_hyster',
 'nosp_hyster_db',
 'nosp_hyster_d',
 'nosp_hyster_a',]]




A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["name"] = 'Hysterectomy'
A["OPHID_Text"] = np.nan
A["sys_nop_id"] = 109
A["visit_id"] = np.array(df2["visit_id"])
A["nop_date_rep"] = np.array(df2["nosp_date"])
A["nop_date"] = np.array(df2["nosp_hyster_d"])
A["nop_date"] = pd.to_datetime(A["nop_date"],  errors='coerce').dt.strftime('%m/%d/%Y')
A["nop_date"] = np.where(df2['nosp_hyster_db']=='No', 'UNKNOWN', A["nop_date"])
A["nop_date"] = np.where(df2['nosp_hyster_db'].isnull(), 'Not Reported', A["nop_date"])
A["nop_date"] = A["nop_date"].fillna('')
A["nop_details"] = np.nan
A["nop_age"] = np.array(df2["nosp_hyster_a"])
A["source"] = 'REDCap'
A["nop_date_end"] = np.nan
A3 = A.copy(deep=True)

df2 = df1[df1["nosp_radtop"] == 'Yes'][['maskedID',
 'site',
 'visit_id',
 'nosp_date',
 'nosp_radtop',
 'nosp_radtop_db',
 'nosp_radtop_d',
 'nosp_radtop_a',]]




A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["name"] = 'Radiation Therapy (above shoulders)'
A["OPHID_Text"] = np.nan
A["sys_nop_id"] = 78
A["visit_id"] = np.array(df2["visit_id"])
A["nop_date_rep"] = np.array(df2["nosp_date"])
A["nop_date"] = np.array(df2["nosp_radtop_d"])
A["nop_date"] = pd.to_datetime(A["nop_date"],  errors='coerce').dt.strftime('%m/%d/%Y')
A["nop_date"] = np.where(df2['nosp_radtop_db']=='No', 'UNKNOWN', A["nop_date"])
A["nop_date"] = np.where(df2['nosp_radtop_db'].isnull(), 'Not Reported', A["nop_date"])
A["nop_date"] = A["nop_date"].fillna('')

A["nop_details"] = np.nan
A["nop_age"] = np.array(df2["nosp_radtop_a"])
A["source"] = 'REDCap'
A["nop_date_end"] = np.nan
A4 = A.copy(deep=True)

df2 = df1[df1["nosp_radbot"] == 'Yes'][['maskedID',
 'site',
 'visit_id',
 'nosp_date',
 'nosp_radbot',
 'nosp_radbot_db',
 'nosp_radbot_d',
 'nosp_radbot_a',]]




A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["name"] = 'Radiation Therapy (at or below shoulders)'
A["OPHID_Text"] = np.nan
A["sys_nop_id"] = 127
A["visit_id"] = np.array(df2["visit_id"])
A["nop_date_rep"] = np.array(df2["nosp_date"])
A["nop_date"] = np.array(df2["nosp_radbot_d"])

A["nop_date"] = pd.to_datetime(A["nop_date"],  errors='coerce').dt.strftime('%m/%d/%Y')
A["nop_date"] = np.where(df2['nosp_radbot_db']=='No', 'UNKNOWN', A["nop_date"])
A["nop_date"] = np.where(df2['nosp_radbot_db'].isnull(), 'Not Reported', A["nop_date"])
A["nop_date"] = A["nop_date"].fillna('')

A["nop_details"] = np.nan
A["nop_age"] = np.array(df2["nosp_radbot_a"])
A["source"] = 'REDCap'
A["nop_date_end"] = np.nan
A5 = A.copy(deep=True)

df2 = df1[df1["nosp_proc_rep1"] == 'Yes'][['maskedID',
 'site',
 'visit_id',
 'nosp_date',
 'nosp_proc_rep1',
 'nosp_proc1',
 'nosp_other1',
 'nosp_bool_date1',
 'nosp_date1',
 'nosp_age1',]]




A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["name"] = np.array(df2["nosp_proc1"])
A["OPHID_Text"] = np.nan
A["visit_id"] = np.array(df2["visit_id"])
A["sys_nop_id"] = np.nan
A["nop_date_rep"] = np.array(df2["nosp_date"])
A["nop_date"] = np.array(df2["nosp_date1"])

A["nop_date"] = pd.to_datetime(A["nop_date"],  errors='coerce').dt.strftime('%m/%d/%Y')
A["nop_date"] = np.where(df2['nosp_bool_date1']=='No', 'UNKNOWN', A["nop_date"])
A["nop_date"] = np.where(df2['nosp_bool_date1'].isnull(), 'Not Reported', A["nop_date"])
A["nop_date"] = A["nop_date"].fillna('')

A["nop_details"] = np.array(df2["nosp_other1"])
A["nop_age"] = np.array(df2["nosp_age1"])
A["source"] = 'REDCap'
A["nop_date_end"] = np.nan
A6 = A.copy(deep=True)

df2 = df1[df1["nosp_proc_rep2"] == 'Yes'][['maskedID',
 'site',
 'visit_id',
 'nosp_date',
 'nosp_proc_rep2',
 'nosp_proc2',
 'nosp_other2',
 'nosp_bool_date2',
 'nosp_date2',
 'nosp_age2',]]




A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["name"] = np.array(df2["nosp_proc2"])
A["OPHID_Text"] = np.nan
A["visit_id"] = np.array(df2["visit_id"])
A["sys_nop_id"] = np.nan
A["nop_date_rep"] = np.array(df2["nosp_date"])
A["nop_date"] = np.array(df2["nosp_date2"])


A["nop_date"] = pd.to_datetime(A["nop_date"],  errors='coerce').dt.strftime('%m/%d/%Y')
A["nop_date"] = np.where(df2['nosp_bool_date2']=='No', 'UNKNOWN', A["nop_date"])
A["nop_date"] = np.where(df2['nosp_bool_date2'].isnull(), 'Not Reported', A["nop_date"])
A["nop_date"] = A["nop_date"].fillna('')

A["nop_details"] = np.array(df2["nosp_other2"])
A["nop_age"] = np.array(df2["nosp_age2"])
A["source"] = 'REDCap'
A["nop_date_end"] = np.nan
A7 = A.copy(deep=True)

df2 = df1[df1["nosp_proc_rep3"] == 'Yes'][['maskedID',
 'site',
 'visit_id',
 'nosp_date',
 'nosp_proc_rep3',
 'nosp_proc3',
 'nosp_other3',
 'nosp_bool_date3',
 'nosp_date3',
 'nosp_age3',]]




A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["name"] = np.array(df2["nosp_proc3"])
A["OPHID_Text"] = np.nan
A["visit_id"] = np.array(df2["visit_id"])
A["sys_nop_id"] = np.nan
A["nop_date_rep"] = np.array(df2["nosp_date"])
A["nop_date"] = np.array(df2["nosp_date3"])


A["nop_date"] = pd.to_datetime(A["nop_date"],  errors='coerce').dt.strftime('%m/%d/%Y')
A["nop_date"] = np.where(df2['nosp_bool_date3']=='No', 'UNKNOWN', A["nop_date"])
A["nop_date"] = np.where(df2['nosp_bool_date3'].isnull(), 'Not Reported', A["nop_date"])
A["nop_date"] = A["nop_date"].fillna('')

A["nop_details"] = np.array(df2["nosp_other3"])
A["nop_age"] = np.array(df2["nosp_age3"])
A["source"] = 'REDCap'
A["nop_date_end"] = np.nan
A8 = A.copy(deep=True)

df2 = df1[df1["nosp_proc_rep4"] == 'Yes'][['maskedID',
 'site',
 'visit_id',
 'nosp_date',
 'nosp_proc_rep4',
 'nosp_proc4',
 'nosp_other4',
 'nosp_bool_date4',
 'nosp_date4',
 'nosp_age4',]]




A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["name"] = np.array(df2["nosp_proc4"])
A["OPHID_Text"] = np.nan
A["visit_id"] = np.array(df2["visit_id"])
A["sys_nop_id"] = np.nan
A["nop_date_rep"] = np.array(df2["nosp_date"])
A["nop_date"] = np.array(df2["nosp_date4"])


A["nop_date"] = pd.to_datetime(A["nop_date"],  errors='coerce').dt.strftime('%m/%d/%Y')
A["nop_date"] = np.where(df2['nosp_bool_date4']=='No', 'UNKNOWN', A["nop_date"])
A["nop_date"] = np.where(df2['nosp_bool_date4'].isnull(), 'Not Reported', A["nop_date"])
A["nop_date"] = A["nop_date"].fillna('')

A["nop_details"] = np.array(df2["nosp_other4"])
A["nop_age"] = np.array(df2["nosp_age4"])
A["source"] = 'REDCap'
A["nop_date_end"] = np.nan
A9 = A.copy(deep=True)

df2 = df1[df1["nosp_proc_rep5"] == 'Yes'][['maskedID',
 'site',
 'visit_id',
 'nosp_date',
 'nosp_proc_rep5',
 'nosp_proc5',
 'nosp_other5',
 'nosp_bool_date5',
 'nosp_date5',
 'nosp_age5',]]




A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["name"] = np.array(df2["nosp_proc5"])
A["OPHID_Text"] = np.nan
A["visit_id"] = np.array(df2["visit_id"])
A["sys_nop_id"] = np.nan
A["nop_date_rep"] = np.array(df2["nosp_date"])
A["nop_date"] = np.array(df2["nosp_date5"])


A["nop_date"] = pd.to_datetime(A["nop_date"],  errors='coerce').dt.strftime('%m/%d/%Y')
A["nop_date"] = np.where(df2['nosp_bool_date5']=='No', 'UNKNOWN', A["nop_date"])
A["nop_date"] = np.where(df2['nosp_bool_date5'].isnull(), 'Not Reported', A["nop_date"])
A["nop_date"] = A["nop_date"].fillna('')

A["nop_details"] = np.array(df2["nosp_other5"])
A["nop_age"] = np.array(df2["nosp_age5"])
A["source"] = 'REDCap'
A["nop_date_end"] = np.nan
A10 = A.copy(deep=True)

df2 = df1[df1["nosp_proc_rep6"] == 'Yes'][['maskedID',
 'site',
 'visit_id',
 'nosp_date',
 'nosp_proc_rep6',
 'nosp_proc6',
 'nosp_other6',
 'nosp_bool_date6',
 'nosp_date6',
 'nosp_age6',]]




A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["name"] = np.array(df2["nosp_proc6"])
A["OPHID_Text"] = np.nan
A["visit_id"] = np.array(df2["visit_id"])
A["sys_nop_id"] = np.nan
A["nop_date_rep"] = np.array(df2["nosp_date"])
A["nop_date"] = np.array(df2["nosp_date6"])


A["nop_date"] = pd.to_datetime(A["nop_date"],  errors='coerce').dt.strftime('%m/%d/%Y')
A["nop_date"] = np.where(df2['nosp_bool_date6']=='No', 'UNKNOWN', A["nop_date"])
A["nop_date"] = np.where(df2['nosp_bool_date6'].isnull(), 'Not Reported', A["nop_date"])
A["nop_date"] = A["nop_date"].fillna('')

A["nop_details"] = np.array(df2["nosp_other6"])
A["nop_age"] = np.array(df2["nosp_age6"])
A["source"] = 'REDCap'
A["nop_date_end"] = np.nan
A11 = A.copy(deep=True)


df2 = df1[df1['nosp_proc_add']=='Yes'][['maskedID',
 'site',
 'visit_id',
 'nosp_date',
 'nosp_proc_rep6',
 'nosp_proc6',
 'nosp_other6',
 'nosp_bool_date6',
 'nosp_date6',
 'nosp_age6','nosp_proc_more']]




A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["name"] = np.array(df2["nosp_proc_more"])
A["OPHID_Text"] = np.nan
A["visit_id"] = np.array(df2["visit_id"])
A["sys_nop_id"] = np.nan
A["nop_date_rep"] = np.array(df2["nosp_date"])
A["nop_date"] = np.nan


A["nop_details"] = np.nan
A["nop_age"] = np.nan
A["source"] = 'REDCap'
A["nop_date_end"] = np.nan
A12 = A.copy(deep=True)


df1[df1['nosp_proc_add']=='Yes']['nosp_proc_more']


'''
df2 = df1[df1["nosp_hosp_bool"] == 'Yes'][['maskedID',
 'site',
 'visit_id',
 'nosp_date',
 'nosp_hosp_start',
 'nosp_hosp_end',
 'nosp_hosp_reason']]

A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["name"] = 'hospitalization'
A["OPHID_Text"] = np.nan
A["sys_nop_id"] = np.nan
A["nop_date_rep"] = np.array(df2["nosp_date"])
A["nop_date"] = np.array(df2["nosp_hosp_start"])
A["nop_date_end"] = np.array(df2["nosp_hosp_end"])
A["nop_details"] = np.array(df2["nosp_hosp_reason"])
A["nop_age"] = np.nan
A["source"] = 'REDCap'

A12 = A.copy(deep=True)
'''


df["nop_age"] = np.nan
df["nop_date_end"] = np.nan
H = list(df.columns.values)
FIN = pd.concat([df, A1[H]], ignore_index=True)
FIN = pd.concat([FIN, A2[H]], ignore_index=True)
FIN = pd.concat([FIN, A3[H]], ignore_index=True)
FIN = pd.concat([FIN, A4[H]], ignore_index=True)
FIN = pd.concat([FIN, A5[H]], ignore_index=True)
FIN = pd.concat([FIN, A6[H]], ignore_index=True)
FIN = pd.concat([FIN, A7[H]], ignore_index=True)
FIN = pd.concat([FIN, A8[H]], ignore_index=True)
FIN = pd.concat([FIN, A9[H]], ignore_index=True)
FIN = pd.concat([FIN, A10[H]], ignore_index=True)
FIN = pd.concat([FIN, A11[H]], ignore_index=True)
FIN = pd.concat([FIN, A12[H]], ignore_index=True)
#FIN = pd.concat([FIN, A12[H]], ignore_index=True)
FIN['nop_date'] = FIN['nop_date'].astype(str)

FIN = pd.merge(FIN, ind[["OPHID_Text", "researchid"]], how='left', on='OPHID_Text')

FIN = pd.merge(FIN, ind[["OPHID_Text_1", "maskedid"]], how='left', on='maskedid')
FIN["OPHID_Text"] = np.where(FIN["OPHID_Text"].isnull(), FIN["OPHID_Text_1"], FIN["OPHID_Text"])

FIN['maskedid'] = np.where(FIN['maskedid'].isnull(), FIN['researchid'], FIN['maskedid'])
FIN['BioBID'] = np.where(FIN['researchid'].isnull(), FIN['maskedid'], FIN['researchid'])
FIN['BioBID'] = np.where( (FIN['researchid'].str.contains('OG')) | (FIN['researchid'].str.contains('ED')), FIN['BioBID'], np.nan)
FIN["site"] = FIN['maskedid'].str[:2]

FIN = FIN[name]


FIN.to_csv(redcapeyechart + 'Results/' + extension +'_NonOcularSurgeriesProcedures_RedEC.csv', index=False)


In [23]:
FIN

,maskedid,OPHID_Text,BioBID,site,visit_id,sys_nop_id,name,nop_date_rep,nop_date,nop_age,nop_details,source
0,AL0001,OPH-102097,NaN,AL,NaN,142.0,Bladder Procedure,11/15/2012,05/01/2012,NaN,Tumor removed from bladder,EyeChart
1,AL0005,OPH-102042,NaN,AL,NaN,49.0,Mastectomy,02/22/2001,01/01/1996,NaN,NaN,EyeChart
2,AL0008,OPH-102065,NaN,AL,NaN,147.0,Gastrointestinal Procedure,02/27/2001,nan,NaN,Colon Surgery,EyeChart
3,AL0008,OPH-102065,NaN,AL,NaN,109.0,Hysterectomy,02/27/2001,01/01/1981,NaN,NaN,EyeChart
4,AL0036,OPH-102121,NaN,AL,NaN,109.0,Hysterectomy,03/02/2001,01/01/1982,NaN,NaN,EyeChart
...,...,...,...,...,...,...,...,...,...,...,...,...
12644,OG13269,NaN,NaN,OG,Baseline,NaN,4 OTHER KNEE PROCEDURES; 3 SHOULDER PROCEDURES...,10/31/2017,nan,NaN,NaN,REDCap
12645,OG13286,NaN,NaN,OG,Baseline,NaN,SECOND SKIN PROCEDURE,12/20/2017,nan,NaN,NaN,REDCap
12646,OG13319,NaN,NaN,OG,Baseline,NaN,C-SECTION 1993,04/24/2018,nan,NaN,NaN,REDCap
12647,SD3924,NaN,NaN,SD,Month 36,NaN,"Hardware removal Left clavicle Olecranon, - un...",10/22/2020,nan,NaN,NaN,REDCap


In [24]:
L1 = [i for i in L_name if '20170322_RetExam' in i]
df = pd.read_csv(EyeChart + L1[0], encoding = "ISO-8859-1")

#df = pd.read_csv(EyeChart + '20170322_RetExam.csv', encoding = "ISO-8859-1")
df['visit_id'] = np.nan
df1 = pd.read_csv(redcapeyechart + extension + '_RetinaExam.csv', encoding = "ISO-8859-1")

name = 'maskedid	OPHID_Text	BioBID	site	visit_id	retexam_date	ret_staff	ret_dilated	retexam_eye	retexam_status	retexam_amd	retexam_dr	retexam_erm	retexam_hem	retexam_drusen	retexam_macedema	retexam_mac_change	retexam_rpe_change	retexam_pvr	retexam_retdystrophy	retexam_drusmac_change	retexam_other	retexam_comments	source'
name = name.split('\t')

df["site"] = df['maskedid'].str[:2]
df["source"] = 'EyeChart'

df4 = df1.copy(deep=True)
df4 = df4[df4["ret_od_tested"] == 'Yes']
df2 = pd.DataFrame(np.array(df4["maskedID"]), columns=["maskedid"])
df2["OPHID_Text"] = np.nan
df2["ret_staff"] = np.array(df4["ret_staff"])
df2["ret_dilated"] = np.array(df4["ret_dilated"])
df2["retexam_date"] = np.array(df4["ret_date"])
df2["visit_id"] = np.array(df4["visit_id"])

df2["retexam_eye"] = 'OD'


df2["retexam_status"] = np.where(df4["ret_od_status"]=='Normal', 'NORMAL' ,'ABNORMAL')


df2["retexam_amd"] = np.where(df4["ret_od_amd"]=='No', 'NO' ,'YES')
df2["retexam_amd"] = np.where(df4["ret_od_amd"]=='Not Tested', 'NOT TESTED',df2["retexam_amd"])
df2["retexam_amd"] = np.where(df4["ret_od_amd"].isnull(), np.nan,df2["retexam_amd"])

df2["retexam_dr"] = np.where(df4["ret_od_dr"]=='No', 'NO' ,'YES')
df2["retexam_dr"] = np.where(df4["ret_od_dr"]=='Not Tested', 'NOT TESTED',df2["retexam_dr"])
df2["retexam_dr"] = np.where(df4["ret_od_dr"].isnull(), np.nan,df2["retexam_dr"])

df2["retexam_erm"] = np.where(df4["ret_od_erm"]=='No', 'NO' ,'YES')
df2["retexam_erm"] = np.where(df4["ret_od_erm"]=='Not Tested', 'NOT TESTED',df2["retexam_erm"])
df2["retexam_erm"] = np.where(df4["ret_od_erm"].isnull(), np.nan,df2["retexam_erm"])

df2["retexam_hem"] = np.where(df4["ret_od_hem"]=='No', 'NO' ,'YES')
df2["retexam_hem"] = np.where(df4["ret_od_hem"]=='Not Tested', 'NOT TESTED',df2["retexam_hem"])
df2["retexam_hem"] = np.where(df4["ret_od_hem"].isnull(), np.nan,df2["retexam_hem"])

df2["retexam_drusen"] = np.where(df4["ret_od_drusen"]=='No', 'NO' ,'YES')
df2["retexam_drusen"] = np.where(df4["ret_od_drusen"]=='Not Tested', 'NOT TESTED',df2["retexam_drusen"])
df2["retexam_drusen"] = np.where(df4["ret_od_drusen"].isnull(), np.nan,df2["retexam_drusen"])

df2["retexam_macedema"] = np.where(df4["ret_od_macedema"]=='No', 'NO' ,'YES')
df2["retexam_macedema"] = np.where(df4["ret_od_macedema"]=='Not Tested', 'NOT TESTED',df2["retexam_macedema"])
df2["retexam_macedema"] = np.where(df4["ret_od_macedema"].isnull(), np.nan,df2["retexam_macedema"])

df2["retexam_mac_change"] = np.where(df4["ret_od_mac_change"]=='No', 'NO' ,'YES')
df2["retexam_mac_change"] = np.where(df4["ret_od_mac_change"]=='Not Tested', 'NOT TESTED',df2["retexam_mac_change"])
df2["retexam_mac_change"] = np.where(df4["ret_od_mac_change"].isnull(), np.nan,df2["retexam_mac_change"])

df2["retexam_rpe_change"] = np.where(df4["ret_od_rpe_change"]=='No', 'NO' ,'YES')
df2["retexam_rpe_change"] = np.where(df4["ret_od_rpe_change"]=='Not Tested', 'NOT TESTED',df2["retexam_rpe_change"])
df2["retexam_rpe_change"] = np.where(df4["ret_od_rpe_change"].isnull(), np.nan,df2["retexam_rpe_change"])

df2["retexam_pvr"] = np.where(df4["ret_od_pvr"]=='No', 'NO' ,'YES')
df2["retexam_pvr"] = np.where(df4["ret_od_pvr"]=='Not Tested', 'NOT TESTED',df2["retexam_pvr"])
df2["retexam_pvr"] = np.where(df4["ret_od_pvr"].isnull(), np.nan,df2["retexam_pvr"])

df2["retexam_retdystrophy"] = np.where(df4["ret_od_retdystrophy"]=='No', 'NO' ,'YES')
df2["retexam_retdystrophy"] = np.where(df4["ret_od_retdystrophy"]=='Not Tested', 'NOT TESTED',df2["retexam_retdystrophy"])
df2["retexam_retdystrophy"] = np.where(df4["ret_od_retdystrophy"].isnull(), np.nan,df2["retexam_retdystrophy"])


df2["retexam_drusmac_change"] = np.nan

df2["retexam_other"] = df4["ret_od_other"].str.upper()


df2["retexam_comments"] = np.array(df4["ret_od_other_notes"])

df2["site"] = np.array(df4["site"])
df2["source"] = 'REDCap'
df3 = df2.copy(deep=True)


df4 = df1.copy(deep=True)
df4 = df4[df4["ret_os_tested"] == 'Yes']
df2 = pd.DataFrame(np.array(df4["maskedID"]), columns=["maskedid"])
df2["OPHID_Text"] = np.nan

df2["ret_staff"] = np.array(df4["ret_staff"])
df2["ret_dilated"] = np.array(df4["ret_dilated"])

    
df2["retexam_date"] = np.array(df4["ret_date"])
df2["visit_id"] = np.array(df4["visit_id"])
df2["retexam_eye"] = 'OS'


df2["retexam_status"] = np.where(df4["ret_os_status"]=='Normal', 'NORMAL' ,'ABNORMAL')


df2["retexam_amd"] = np.where(df4["ret_os_amd"]=='No', 'NO' ,'YES')
df2["retexam_amd"] = np.where(df4["ret_os_amd"]=='Not Tested', 'NOT TESTED',df2["retexam_amd"])
df2["retexam_amd"] = np.where(df4["ret_os_amd"].isnull(), np.nan,df2["retexam_amd"])

df2["retexam_dr"] = np.where(df4["ret_os_dr"]=='No', 'NO' ,'YES')
df2["retexam_dr"] = np.where(df4["ret_os_dr"]=='Not Tested', 'NOT TESTED',df2["retexam_dr"])
df2["retexam_dr"] = np.where(df4["ret_os_dr"].isnull(), np.nan,df2["retexam_dr"])

df2["retexam_erm"] = np.where(df4["ret_os_erm"]=='No', 'NO' ,'YES')
df2["retexam_erm"] = np.where(df4["ret_os_erm"]=='Not Tested', 'NOT TESTED',df2["retexam_erm"])
df2["retexam_erm"] = np.where(df4["ret_os_erm"].isnull(), np.nan,df2["retexam_erm"])

df2["retexam_hem"] = np.where(df4["ret_os_hem"]=='No', 'NO' ,'YES')
df2["retexam_hem"] = np.where(df4["ret_os_hem"]=='Not Tested', 'NOT TESTED',df2["retexam_hem"])
df2["retexam_hem"] = np.where(df4["ret_os_hem"].isnull(), np.nan,df2["retexam_hem"])

df2["retexam_drusen"] = np.where(df4["ret_os_drusen"]=='No', 'NO' ,'YES')
df2["retexam_drusen"] = np.where(df4["ret_os_drusen"]=='Not Tested', 'NOT TESTED',df2["retexam_drusen"])
df2["retexam_drusen"] = np.where(df4["ret_os_drusen"].isnull(), np.nan,df2["retexam_drusen"])

df2["retexam_macedema"] = np.where(df4["ret_os_macedema"]=='No', 'NO' ,'YES')
df2["retexam_macedema"] = np.where(df4["ret_os_macedema"]=='Not Tested', 'NOT TESTED',df2["retexam_macedema"])
df2["retexam_macedema"] = np.where(df4["ret_os_macedema"].isnull(), np.nan,df2["retexam_macedema"])

df2["retexam_mac_change"] = np.where(df4["ret_os_mac_change"]=='No', 'NO' ,'YES')
df2["retexam_mac_change"] = np.where(df4["ret_os_mac_change"]=='Not Tested', 'NOT TESTED',df2["retexam_mac_change"])
df2["retexam_mac_change"] = np.where(df4["ret_os_mac_change"].isnull(), np.nan,df2["retexam_mac_change"])

df2["retexam_rpe_change"] = np.where(df4["ret_os_rpe_change"]=='No', 'NO' ,'YES')
df2["retexam_rpe_change"] = np.where(df4["ret_os_rpe_change"]=='Not Tested', 'NOT TESTED',df2["retexam_rpe_change"])
df2["retexam_rpe_change"] = np.where(df4["ret_os_rpe_change"].isnull(), np.nan,df2["retexam_rpe_change"])

df2["retexam_pvr"] = np.where(df4["ret_os_pvr"]=='No', 'NO' ,'YES')
df2["retexam_pvr"] = np.where(df4["ret_os_pvr"]=='Not Tested', 'NOT TESTED',df2["retexam_pvr"])
df2["retexam_pvr"] = np.where(df4["ret_os_pvr"].isnull(), np.nan,df2["retexam_pvr"])

df2["retexam_retdystrophy"] = np.where(df4["ret_os_retdystrophy"]=='No', 'NO' ,'YES')
df2["retexam_retdystrophy"] = np.where(df4["ret_os_retdystrophy"]=='Not Tested', 'NOT TESTED',df2["retexam_retdystrophy"])
df2["retexam_retdystrophy"] = np.where(df4["ret_os_retdystrophy"].isnull(), np.nan,df2["retexam_retdystrophy"])

df2["retexam_drusmac_change"] = np.nan

df2["retexam_other"] = df4["ret_os_other"].str.upper()


df2["retexam_comments"] = np.array(df4["ret_os_other_notes"])
df2["site"] = np.array(df4["site"])
df2["source"] = 'REDCap'

df["ret_staff"] = np.nan
df["ret_dilated"] = np.nan

H = list(df.columns.values)
FIN = pd.concat([df, df2[H]], ignore_index=True)
FIN = pd.concat([FIN, df3[H]], ignore_index=True)


FIN = pd.merge(FIN, ind[["OPHID_Text", "researchid"]], how='left', on='OPHID_Text')

FIN = FIN[~FIN['maskedid'].isnull()]

FIN = pd.merge(FIN, ind[["OPHID_Text_1", "maskedid"]], how='left', on='maskedid')
FIN["OPHID_Text"] = np.where(FIN["OPHID_Text"].isnull(), FIN["OPHID_Text_1"], FIN["OPHID_Text"])

FIN['maskedid'] = np.where(FIN['maskedid'].isnull(), FIN['researchid'], FIN['maskedid'])
FIN['BioBID'] = np.where(FIN['researchid'].isnull(), FIN['maskedid'], FIN['researchid'])
FIN['BioBID'] = np.where( (FIN['researchid'].str.contains('OG')) | (FIN['researchid'].str.contains('ED')), FIN['BioBID'], np.nan)
FIN["site"] = FIN['maskedid'].str[:2]

FIN = FIN[name]

FIN.to_csv(redcapeyechart + 'Results/' + extension +'_RetinaExam_RedEC.csv', index=False)


In [25]:
L1 = [i for i in L_name if '20170322_BloodPressure' in i]
df = pd.read_csv(EyeChart + L1[0], encoding = "ISO-8859-1")
#df = pd.read_csv(EyeChart + "/20170322_BloodPressure.csv", encoding = "ISO-8859-1")
df['visit_id'] = np.nan
df["source"] = 'EyeChart'
df2 = pd.read_csv(redcapeyechart + extension + "_BloodPressure.csv", encoding = "ISO-8859-1")

name = 'maskedid	OPHID_Text	BioBID	site	visit_id	bp_date	bp_time	bp_operator	declined	pre_averaged	arm	is_pregnant	systolic1	diastolic1	systolic2	diastolic2	systolic3	diastolic3	bp_comments	source'
name = name.split('\t')

df1 = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
df1["OPHID_Text"] = np.nan
df1["bp_date"] = np.array(df2["anth_date"])
df1['visit_id'] = np.array(df2["visit_id"])
df1["bp_time"] = np.nan
df1["bp_operator"] = np.nan

#When REDCap variable anth_bp_arm is blank
#and anth_bp_bool = "Yes", combined variable arm should show "Not Reported"
# if anth_bp_bool is 1 then Not Tested otherwise if blank then NOT reported 
df1["declined"] = np.where(df2["anth_bp_bool"] == 'Yes', 0, 1)
df1["pre_averaged"] = np.where(df2["anth_bp_bp1avg"] == 'Yes', 1, 0)
df1["anth_bp_bp1avg"] = np.array(df2["anth_bp_bp1avg"])
df1["arm"] = np.where((df2["anth_bp_arm"].isnull()) & (df2["anth_bp_bool"] == 'Yes') , "NOT REPORTED", "NOT REPORTED")
df1["arm"] = np.where(df2["anth_bp_arm"].isin(np.array(["Right"])), "Right", df1["arm"])
df1["arm"] = np.where(df2["anth_bp_arm"].isin(np.array(["Left"])), "Left", df1["arm"])
df1["arm"] = df1["arm"].fillna('NOT REPORTED')


df1["is_pregnant"] = np.nan



df1["diastolic1"] = np.array(df2["anth_bp_diastolic1"])
df1["systolic1"] = np.array(df2["anth_bp_systolic1"])
df1["systolic2"] = np.array(df2["anth_bp_systolic2"])

df1["diastolic2"] = np.array(df2["anth_bp_diastolic2"])
df1["systolic3"] = np.array(df2["anth_bp_systolic3"])
df1["diastolic3"] = np.array(df2["anth_bp_diastolic3"])

df1["bp_comments"] = np.array(df2["anth_bp_note"])

df1["source"] = 'REDCap'
df1["arm"] = df1["arm"].fillna('NOT REPORTED')
df["anth_bp_bp1avg"] = np.nan

H = list(df.columns.values)
FIN = pd.concat([df, df1[H]], ignore_index=True)


FIN = pd.merge(FIN, ind[["OPHID_Text", "researchid"]], how='left', on='OPHID_Text')
    

FIN = FIN[~FIN['maskedid'].isnull()]

FIN = pd.merge(FIN, ind[["OPHID_Text_1", "maskedid"]], how='left', on='maskedid')
FIN["OPHID_Text"] = np.where(FIN["OPHID_Text"].isnull(), FIN["OPHID_Text_1"], FIN["OPHID_Text"])

FIN['maskedid'] = np.where(FIN['maskedid'].isnull(), FIN['researchid'], FIN['maskedid'])
FIN['BioBID'] = np.where(FIN['researchid'].isnull(), FIN['maskedid'], FIN['researchid'])
FIN['BioBID'] = np.where( (FIN['researchid'].str.contains('OG')) | (FIN['researchid'].str.contains('ED')), FIN['BioBID'], np.nan)
FIN["site"] = FIN['maskedid'].str[:2]


FIN = FIN[name]
FIN.to_csv(redcapeyechart + 'Results/' + extension + '_BloodPressure_RedEC.csv', index=False)



In [26]:
L1 = [i for i in L_name if '20170322_FarAcSnellen' in i]
df = pd.read_csv(EyeChart + L1[0], encoding = "ISO-8859-1")

#df = pd.read_csv(EyeChart + "20170322_FarAcSnellen.csv", encoding = "ISO-8859-1")
name = 'maskedid	OPHID_Text	BioBID	site	visit_id	faracu_date	faracu_eye	meas_type	faracu_base	faracu_adjust	faracu_operator	faracu_comments	source'
name = name.split('\t')

df["source"] = 'EyeChart'
df["visit_id"] = np.nan
df2 = pd.read_csv(redcapeyechart + extension + "_VisionTests.csv", encoding = "ISO-8859-1")



df1 = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
df1["OPHID_Text"] = np.nan
df1["vt_staff"] = np.array(df2["vt_staff"])
df1["visit_id"] = np.array(df2["visit_id"])
df1["faracu_eye"] = 'OD'
df1["faracu_date"] = np.array(df2["vt_date"])

df1["meas_type"] = np.array(df2["vt_os_faracutype"])
df1["faracu_base"] = df2["vt_od_faracunumer"].astype(str).apply(lambda x: x.split('.')[0],1) + '|' + df2["vt_od_faracudenom"].astype(str).apply(lambda x: x.split('.')[0],1) 
df1["faracu_adjust"] = np.array(df2["vt_od_faracuadj"])

df1["faracu_operator"] = np.array(df2["vt_staff"])
df1["faracu_comments"] = np.nan

df1["source"] = 'REDCap'

df3 = df1.copy(deep=True)

df1 = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
df1["OPHID_Text"] = np.nan
df1["vt_staff"] = np.array(df2["vt_staff"])
df1["visit_id"] = np.array(df2["visit_id"])
df1["faracu_eye"] = 'OS'
df1["faracu_date"] = np.array(df2["vt_date"])

df1["meas_type"] = np.array(df2["vt_os_faracutype"])
df1["faracu_base"] = df2["vt_os_faracunumer"].astype(str).apply(lambda x: x.split('.')[0],1) + '|' + df2["vt_os_faracudenom"].astype(str).apply(lambda x: x.split('.')[0],1)
df1["faracu_adjust"] = np.array(df2["vt_os_faracuadj"])

df1["faracu_operator"] = np.array(df2["vt_staff"])
df1["faracu_comments"] = np.nan

df1["source"] = 'REDCap'

df1["faracu_base"] = np.where(df1["meas_type"] == 'Not Reported', '', df1["faracu_base"])
df1["faracu_adjust"] = np.where(df1["meas_type"] == 'Not Reported', '', df1["faracu_adjust"])

df["vt_staff"] = np.nan

H = list(df.columns.values)
FIN = pd.concat([df, df1[H]], ignore_index=True)
FIN = pd.concat([FIN, df3[H]], ignore_index=True)

FIN = pd.merge(FIN, ind[["OPHID_Text", "researchid"]], how='left', on='OPHID_Text')

FIN = FIN[~FIN['maskedid'].isnull()]

FIN = pd.merge(FIN, ind[["OPHID_Text_1", "maskedid"]], how='left', on='maskedid')
FIN["OPHID_Text"] = np.where(FIN["OPHID_Text"].isnull(), FIN["OPHID_Text_1"], FIN["OPHID_Text"])

FIN['maskedid'] = np.where(FIN['maskedid'].isnull(), FIN['researchid'], FIN['maskedid'])
FIN['BioBID'] = np.where(FIN['researchid'].isnull(), FIN['maskedid'], FIN['researchid'])
FIN['BioBID'] = np.where( (FIN['researchid'].str.contains('OG')) | (FIN['researchid'].str.contains('ED')), FIN['BioBID'], np.nan)
FIN["site"] = FIN['maskedid'].str[:2]
FIN = FIN[name]
FIN["faracu_base"] = np.where(FIN["faracu_base"]=='nan|nan', "Not Tested", FIN["faracu_base"])
FIN = FIN.fillna('')
FIN.to_csv(redcapeyechart + 'Results/' + extension + '_FarAcSnellen_RedEC.csv', index=False)


In [27]:
L1 = [i for i in L_name if '20170322_FarRefraction' in i]
df = pd.read_csv(EyeChart + L1[0], encoding = "ISO-8859-1")

#df = pd.read_csv(EyeChart + "20170322_FarRefraction.csv", encoding = "ISO-8859-1")

name = ['maskedid',
'OPHID_Text',
'BioBID',
'site',
'visit_id',
'farrefsubj_date',
'farrefsubj_eye',
'meas_type',
'sphere',
'cylinder',
'axis',
'trans_sphere',
'trans_cylinder',
'trans_axis',
'sphere_equiv',
'used_adages_bl09',
'farrefsubj_operator',
'farrefsubj_comments',
'source',
]
df["source"] = 'EyeChart'
df['visit_id'] = np.nan
df2 = pd.read_csv(redcapeyechart + extension + "_VisionTests.csv")


df1 = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
df1["OPHID_Text"] = np.nan
df1["vt_staff"] = np.array(df2["vt_staff"])
df1["farrefsubj_eye"] = 'OD'
df1["farrefsubj_date"] = np.array(df2["vt_date"])
df1["visit_id"] = np.array(df2["visit_id"])
df1["meas_type"] = np.where(df2["vt_od_farreftype"].isnull(), 'Not Tested', df2["vt_od_farreftype"])
df1["sphere"] = np.array(df2["vt_od_sphere"])
df1["cylinder"] = np.array(df2["vt_od_cyl"])

df1["axis"] = np.array(df2["vt_od_axis"])
df1["trans_sphere"] = np.nan

df1["trans_axis"] = np.nan
df1["trans_cylinder"] = np.nan

df1["sphere_equiv"] = df1["sphere"] + (df1["cylinder"]/2.)

df1["used_adages_bl09"] = np.nan
df1["farrefsubj_comments"] = np.nan
df1["farrefsubj_operator"] = np.array(df2["vt_staff"])

df1["source"] = 'REDCap'


df3 = df1.copy(deep=True)

df1 = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
df1["OPHID_Text"] = np.nan
df1["vt_staff"] = np.array(df2["vt_staff"])
df1["farrefsubj_eye"] = 'OS'
df1["farrefsubj_date"] = np.array(df2["vt_date"])
df1["visit_id"] = np.array(df2["visit_id"])
df1["meas_type"] = np.where(df2["vt_os_farreftype"].isnull(), 'Not Tested', df2["vt_os_farreftype"])
df1["sphere"] = np.array(df2["vt_os_sphere"])
df1["cylinder"] = np.array(df2["vt_os_cyl"])

df1["axis"] = np.array(df2["vt_os_axis"])
df1["trans_sphere"] = np.nan

df1["trans_axis"] = np.nan
df1["trans_cylinder"] = np.nan
df1["sphere"] = np.where(df1["sphere"].isin(np.array(["PL", "-3/50", ".0.50", "plano", "sph"])), np.nan, df1["sphere"])
df1["cylinder"] = np.where(df1["cylinder"].isin(np.array(["PL", "-3/50", ".0.50", "plano", "sph", "DS"])), np.nan, df1["cylinder"])
df1["sphere_equiv"] = df1["sphere"].astype(float) + (df1["cylinder"].astype(float)/2.)

df1["used_adages_bl09"] = np.nan
df1["farrefsubj_comments"] = np.nan
df1["farrefsubj_operator"] = np.array(df2["vt_staff"])

df1["source"] = 'REDCap'


df["vt_staff"] = np.nan

H = list(df.columns.values)
FIN = pd.concat([df, df1[H]], ignore_index=True)
FIN = pd.concat([FIN, df3[H]], ignore_index=True)

FIN = pd.merge(FIN, ind[["OPHID_Text", "researchid"]], how='left', on='OPHID_Text')

FIN = FIN[~FIN['maskedid'].isnull()]

FIN = pd.merge(FIN, ind[["OPHID_Text_1", "maskedid"]], how='left', on='maskedid')
FIN["OPHID_Text"] = np.where(FIN["OPHID_Text"].isnull(), FIN["OPHID_Text_1"], FIN["OPHID_Text"])

FIN['maskedid'] = np.where(FIN['maskedid'].isnull(), FIN['researchid'], FIN['maskedid'])
FIN['BioBID'] = np.where(FIN['researchid'].isnull(), FIN['maskedid'], FIN['researchid'])
FIN['BioBID'] = np.where( (FIN['researchid'].str.contains('OG')) | (FIN['researchid'].str.contains('ED')), FIN['BioBID'], np.nan)
FIN["site"] = FIN['maskedid'].str[:2]
FIN = FIN[name]
FIN = FIN.fillna('')
FIN.to_csv(redcapeyechart + 'Results/' + extension +'_FarRefraction_RedEC.csv', index=False)




In [28]:
L1

['20170322_FarRefraction_CLEAN.csv']

In [29]:
L1 = [i for i in L_name if '_Gonio' in i]
df = pd.read_csv(EyeChart + L1[0], encoding = "ISO-8859-1")

#df = pd.read_csv(EyeChart + "20170420_Gonio.csv", encoding = "ISO-8859-1")
name = 'maskedid	OPHID_Text	BioBID	site	visit_id	gonio_date	gonio_eye	gonio_shafer_used	gonio_meas	gonio_meas_nonshafer	dr_gon_oac	gonio_operator	gonio_comments	source'
name = name.split('\t')

df["source"] = 'EyeChart'
df['visit_id'] = np.nan
df20 = pd.read_csv(redcapeyechart + extension + "_Gonioscopy.csv", encoding = "ISO-8859-1")

df2 = df20[(df20["dr_gonio"] == 'OS') | (df20["dr_gonio"] == 'OU')]
df1 = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
df1["OPHID_Text"] = np.nan
df1["dr_gon_oac"] = np.array(df2["dr_gon_oac"])
df1["gonio_eye"] = 'OS'
df1["gonio_date"] = np.array(df2["dr_date"])
df1["visit_id"] = np.array(df2["visit_id"])
df1["gonio_shafer_used"] = np.nan

df1["gonio_meas"] = np.array(df2["dr_gon_os_grade"])
df1["gonio_meas_nonshafer"] = np.nan
df1["gonio_operator"] = np.nan
df1["gonio_comments"] = np.nan

df1["source"] = 'REDCap'


df3 = df1.copy(deep=True)
df2 = df20[(df20["dr_gonio"] == 'OD') | (df20["dr_gonio"] == 'OU')]
df1 = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])

df1["OPHID_Text"] = np.nan
df1["dr_gon_oac"] = np.array(df2["dr_gon_oac"])
df1["gonio_eye"] = 'OD'
df1["gonio_date"] = np.array(df2["dr_date"])
df1["visit_id"] = np.array(df2["visit_id"])
df1["gonio_shafer_used"] = np.nan

df1["gonio_meas"] = np.array(df2["dr_gon_od_grade"])
df1["gonio_meas_nonshafer"] = np.nan
df1["gonio_operator"] = np.nan
df1["gonio_comments"] = np.nan

df1["source"] = 'REDCap'


df["dr_gon_oac"] = np.nan

H = list(df.columns.values)
FIN = pd.concat([df, df1[H]], ignore_index=True)
FIN = pd.concat([FIN, df3[H]], ignore_index=True)

FIN = pd.merge(FIN, ind[["OPHID_Text", "researchid"]], how='left', on='OPHID_Text')

FIN = FIN[~FIN['maskedid'].isnull()]

FIN = pd.merge(FIN, ind[["OPHID_Text_1", "maskedid"]], how='left', on='maskedid')
FIN["OPHID_Text"] = np.where(FIN["OPHID_Text"].isnull(), FIN["OPHID_Text_1"], FIN["OPHID_Text"])

FIN['maskedid'] = np.where(FIN['maskedid'].isnull(), FIN['researchid'], FIN['maskedid'])
FIN['BioBID'] = np.where(FIN['researchid'].isnull(), FIN['maskedid'], FIN['researchid'])
FIN['BioBID'] = np.where( (FIN['researchid'].str.contains('OG')) | (FIN['researchid'].str.contains('ED')), FIN['BioBID'], np.nan)
FIN["site"] = FIN['maskedid'].str[:2]

FIN = FIN[name]
FIN = FIN.fillna('')


FIN.to_csv(redcapeyechart + 'Results/' + extension +'_Gonioscopy_RedEC.csv', index=False)



In [30]:

L1 = [i for i in L_name if '_HeartRate' in i]
df = pd.read_csv(EyeChart + L1[0], encoding = "ISO-8859-1")
#df = pd.read_csv(EyeChart + "20170322_HeartRate.csv", encoding = "ISO-8859-1")
name = 'maskedid	OPHID_Text	BioBID	site	visit_id	hr_date	hr_operator	declined	pre_averaged	is_pregnant	bpm1	bpm2	bpm3	hr_comments	source'
name = name.split('\t')

df["site"] = df['maskedid'].str[:2]
df["source"] = 'EyeChart'
df["visit_id"] = np.nan
df2 = pd.read_csv(redcapeyechart + extension + '_HeartRate.csv')


df1 = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
df1["OPHID_Text"] = np.nan
df1["hr_date"] = np.array(df2["anth_date"])
df1["visit_id"] = np.array(df2["visit_id"])
df1["hr_operator"] = np.nan
df1["declined"] = np.where(df2["anth_hr_bool"] == "Yes", 0, 1)
df1["maskedid"] = np.array(df2["maskedID"])
df1["pre_averaged"] = np.nan
df1["is_pregnant"] = np.nan
df1["bpm1"] = np.array(df2["anth_hr_hr1"])
df1["bpm2"] = np.array(df2["anth_hr_hr2"])
df1["bpm3"] = np.array(df2["anth_hr_hr3"])
df1["hr_comments"] = np.array(df2["anth_hr_note"])
df1["source"] = 'REDCap'
df1["site"] = df1['maskedid'].str[:2]

H = list(df.columns.values)
FIN = pd.concat([df, df1[H]], ignore_index=True)


FIN = pd.merge(FIN, ind[["OPHID_Text", "researchid"]], how='left', on='OPHID_Text')

FIN = FIN[~FIN['maskedid'].isnull()]

FIN = pd.merge(FIN, ind[["OPHID_Text_1", "maskedid"]], how='left', on='maskedid')
FIN["OPHID_Text"] = np.where(FIN["OPHID_Text"].isnull(), FIN["OPHID_Text_1"], FIN["OPHID_Text"])

FIN['maskedid'] = np.where(FIN['maskedid'].isnull(), FIN['researchid'], FIN['maskedid'])
FIN['BioBID'] = np.where(FIN['researchid'].isnull(), FIN['maskedid'], FIN['researchid'])
FIN['BioBID'] = np.where( (FIN['researchid'].str.contains('OG')) | (FIN['researchid'].str.contains('ED')), FIN['BioBID'], np.nan)
FIN["site"] = FIN['maskedid'].str[:2]

FIN = FIN[name]
FIN = FIN.fillna('')
FIN.to_csv(redcapeyechart + 'Results/' + extension + '_HeartRate_RedEC.csv', index=False)

In [31]:
L1 = [i for i in L_name if '_Heights' in i]
df = pd.read_csv(EyeChart + L1[0], encoding = "ISO-8859-1")

#df = pd.read_csv(EyeChart + "20170322_Heights.csv", encoding = "ISO-8859-1")
name = 'maskedid	OPHID_Text	BioBID	site	visit_id	height_date	declined	anth_ht_sreport	height_ft	height_in	height_8ths	hunched_posture	prosthesis	height_comment	source'
name = name.split('\t')

df["source"] = 'EyeChart'
df['visit_id'] = np.nan
df["site"] = df['maskedid'].str[:2]
df2 = pd.read_csv(redcapeyechart + extension +  "_Height.csv")


df1 = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
df1["OPHID_Text"] = np.nan
df1["height_date"] = np.array(df2["anth_date"])
df1["declined"] = np.nan
df1["height_ft"] = np.array(df2["anth_ht_ft"])
df1["height_in"] = np.array(df2["anth_ht_inches"])
df1["height_8ths"] = np.array(df2["anth_ht_eighths"])
df1["hunched_posture"] = np.where(df2["anth_ht_posture"] == 'No', 0, 1)
df1["prosthesis"] = np.where(df2["anth_ht_prosthesis"] == 'No', 0, 1)
df1["height_comment"] = np.array(df2["anth_ht_note"])
df1["visit_id"] = np.array(df2["visit_id"])
df1["height_8ths"] = np.nan
df1["source"] = 'REDCap'
df1["anth_ht_sreport"] = np.array(df2["anth_ht_sreport"])
df1["site"] = df1['maskedid'].str[:2]
df["anth_ht_sreport"] = np.nan

H = list(df.columns.values)
FIN = pd.concat([df, df1[H]], ignore_index=True)


FIN = pd.merge(FIN, ind[["OPHID_Text", "researchid"]], how='left', on='OPHID_Text')

FIN = FIN[~FIN['maskedid'].isnull()]

FIN = pd.merge(FIN, ind[["OPHID_Text_1", "maskedid"]], how='left', on='maskedid')
FIN["OPHID_Text"] = np.where(FIN["OPHID_Text"].isnull(), FIN["OPHID_Text_1"], FIN["OPHID_Text"])

FIN['maskedid'] = np.where(FIN['maskedid'].isnull(), FIN['researchid'], FIN['maskedid'])
FIN['BioBID'] = np.where(FIN['researchid'].isnull(), FIN['maskedid'], FIN['researchid'])
FIN['BioBID'] = np.where( (FIN['researchid'].str.contains('OG')) | (FIN['researchid'].str.contains('ED')), FIN['BioBID'], np.nan)
FIN["site"] = FIN['maskedid'].str[:2]
FIN = FIN[name]
FIN = FIN.fillna('')
FIN.to_csv(redcapeyechart + 'Results/' + extension +'_Height_RedEC.csv', index=False)

In [32]:
L1 = [i for i in L_name if '_Pachymetry' in i]
df = pd.read_csv(EyeChart + L1[0], encoding = "ISO-8859-1")

#df = pd.read_csv( EyeChart + "20170322_Pachymetry.csv", encoding = "ISO-8859-1")
name = 'maskedid	OPHID_Text	BioBID	site	visit_id	pachy_date	et_staff	pachy_eye	pachy_meas_1	pachy_meas_2	pachy_meas_3	pachy_avg	pachy_stddev	pachy_comments	source'
name = name.split('\t')

df["source"] = 'EyeChart'
df['visit_id'] = np.nan
df['et_staff'] = np.nan
df["site"] = df['maskedid'].str[:2]
df2 = pd.read_csv(redcapeyechart + extension + "_Pachymetry.csv")



df1 = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
df1["OPHID_Text"] = np.nan
df1["pachy_eye"] = 'OS'
df1["pachy_date"] = np.array(df2["et_date"])

df1["pachy_meas_1"] = np.array(df2["et_os_pachy1"])
df1["visit_id"] = np.array(df2["visit_id"])
df1["pachy_meas_2"] = np.array(df2["et_os_pachy2"])
df1["pachy_meas_3"] = np.array(df2["et_os_pachy3"])

df1["pachy_avg"] = np.array(df2["et_os_pachy_avg"])
df1["pachy_stddev"] = np.nan
df1["pachy_comments"] = np.array(df2["et_pachynotes"])
df1["site"] = df1['maskedid'].str[:2]
df1["et_staff"] = np.array(df2["et_staff"])
df1["source"] = 'REDCap'

df3 = df1.copy(deep=True)

df1 = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
df1["OPHID_Text"] = np.nan
df1["pachy_eye"] = 'OD'
df1["pachy_date"] = np.array(df2["et_date"])

df1["pachy_meas_1"] = np.array(df2["et_od_mm_pachy1"])
df1["visit_id"] = np.array(df2["visit_id"])
df1["pachy_meas_2"] = np.array(df2["et_od_pachy2"])
df1["pachy_meas_3"] = np.array(df2["et_od_pachy3"])

df1["pachy_avg"] = np.array(df2["et_od_pachy_avg"])
df1["pachy_stddev"] = np.nan
df1["pachy_comments"] = np.array(df2["et_pachynotes"])
df1["site"] = df1['maskedid'].str[:2]
df1["et_staff"] = np.array(df2["et_staff"])

df1["source"] = 'REDCap'

df1["et_staff"] = np.nan
H = list(df.columns.values)
FIN = pd.concat([df, df1[H]], ignore_index=True)
FIN = pd.concat([FIN, df3[H]], ignore_index=True)

FIN = pd.merge(FIN, ind[["OPHID_Text", "researchid"]], how='left', on='OPHID_Text')

FIN = FIN[~FIN['maskedid'].isnull()]

FIN = pd.merge(FIN, ind[["OPHID_Text_1", "maskedid"]], how='left', on='maskedid')
FIN["OPHID_Text"] = np.where(FIN["OPHID_Text"].isnull(), FIN["OPHID_Text_1"], FIN["OPHID_Text"])

FIN['maskedid'] = np.where(FIN['maskedid'].isnull(), FIN['researchid'], FIN['maskedid'])
FIN['BioBID'] = np.where(FIN['researchid'].isnull(), FIN['maskedid'], FIN['researchid'])
FIN['BioBID'] = np.where( (FIN['researchid'].str.contains('OG')) | (FIN['researchid'].str.contains('ED')), FIN['BioBID'], np.nan)
FIN["site"] = FIN['maskedid'].str[:2]

FIN = FIN[name]
FIN = FIN.fillna('')
FIN.to_csv(redcapeyechart + 'Results/' + extension +'_Pachymetry_RedEC.csv', index=False)



In [33]:
L1 = [i for i in L_name if '_Weight' in i]
df = pd.read_csv(EyeChart + L1[0], encoding = "ISO-8859-1")

#df = pd.read_csv(EyeChart + "20170322_Weight.csv", encoding = "ISO-8859-1")
name = 'maskedid	OPHID_Text	BioBID	site	visit_id	weight_date	declined	weight_pt_report	weight	prosthesis	is_pregnant	exceeds_capacity	weight_comment	source'
name = name.split('\t')

df["source"] = 'EyeChart'
df["visit_id"] = np.nan
df2 = pd.read_csv(redcapeyechart + extension + "_Weight.csv")




df1 = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
df1["OPHID_Text"] = np.nan
df1["weight_date"] = np.array(df2["anth_date"])
df1["visit_id"] = np.array(df2["visit_id"])
df1["declined"] = np.where(df2['anth_wt_bool']=='Yes', 0,1)

df1["weight"] = np.array(df2["anth_wt_lbs"])

df1["anth_wt_sreport"] = np.array(df2["anth_wt_sreport"])
df1["anth_wt_prosthesis"] = np.array(df2["anth_wt_prosthesis"])
df1["anth_wt_bool"] = np.array(df2["anth_wt_bool"])

df1["prosthesis"] = np.where(df2["anth_wt_prosthesis"]=='No', 0,1)
df1["is_pregnant"] = np.nan
df1["weight_pt_report"] = np.where(df2["anth_wt_sreport"]=='Measured', 0, 1)

df1["exceeds_capacity"] = np.where(df2["anth_wt_exceeds_capacity"] == 'No', 0, 1)
df1["weight_comment"] = np.array(df2["anth_wt_note"])


df1["source"] = 'REDCap'

df["anth_wt_sreport"] = np.nan
df["anth_wt_prosthesis"] = np.nan
df["anth_wt_bool"] = np.nan


H = list(df.columns.values)
FIN = pd.concat([df, df1[H]], ignore_index=True)

FIN = pd.merge(FIN, ind[["OPHID_Text", "researchid"]], how='left', on='OPHID_Text')

FIN = FIN[~FIN['maskedid'].isnull()]

FIN = pd.merge(FIN, ind[["OPHID_Text_1", "maskedid"]], how='left', on='maskedid')
FIN["OPHID_Text"] = np.where(FIN["OPHID_Text"].isnull(), FIN["OPHID_Text_1"], FIN["OPHID_Text"])

FIN['maskedid'] = np.where(FIN['maskedid'].isnull(), FIN['researchid'], FIN['maskedid'])
FIN['BioBID'] = np.where(FIN['researchid'].isnull(), FIN['maskedid'], FIN['researchid'])
FIN['BioBID'] = np.where( (FIN['researchid'].str.contains('OG')) | (FIN['researchid'].str.contains('ED')), FIN['BioBID'], np.nan)
FIN["site"] = FIN['maskedid'].str[:2]

FIN = FIN[name]
FIN = FIN.fillna('')
FIN.to_csv(redcapeyechart + 'Results/' + extension +'_Weight_RedEC.csv', index=False)



In [34]:
L1 = [i for i in L_name if '_AxialLen' in i]
df = pd.read_csv(EyeChart + L1[0], encoding = "ISO-8859-1")

#df = pd.read_csv(EyeChart + "20170322_AxialLen.csv", encoding = "ISO-8859-1")
df["source"] = 'EyeChart'
name = 'maskedid	OPHID_Text	BioBID	site	visit_id	axlen_date	axlen_eye	axlen_meas	axlen_operator	axlen_comments	source'
name = name.split('\t')
df['visit_id'] = np.nan

df2 = pd.read_csv(redcapeyechart + extension +  "_AxialLen.csv")



df1 = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
df1["OPHID_Text"] = np.nan
df1["axlen_eye"] = 'OS'
df1["axlen_date"] = np.array(df2["et_date"])
df1["axlen_meas"] = np.array(df2["et_os_axial"])
df1["axlen_operator"] = np.array(df2["et_staff"])
df1["axlen_comments"] = np.nan
df1["visit_id"] = np.array(df2["visit_id"])
df1["source"] = 'REDCap'

df3 = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
df3["OPHID_Text"] = np.nan
df3["axlen_eye"] = 'OD'
df3["axlen_date"] = np.array(df2["et_date"])
df3["axlen_meas"] = np.array(df2["et_od_axial"])
df3["axlen_operator"] = np.array(df2["et_staff"])
df3["axlen_comments"] = np.nan
df3["visit_id"] = np.array(df2["visit_id"])
df3["source"] = 'REDCap'


H = list(df.columns.values)
FIN = pd.concat([df, df1[H]], ignore_index=True)
FIN = pd.concat([FIN, df3[H]], ignore_index=True)
FIN = FIN[~FIN["axlen_meas"].isnull()]
FIN = pd.merge(FIN, ind[["OPHID_Text", "researchid"]], how='left', on='OPHID_Text')

FIN = FIN[~FIN['maskedid'].isnull()]

FIN = pd.merge(FIN, ind[["OPHID_Text_1", "maskedid"]], how='left', on='maskedid')
FIN["OPHID_Text"] = np.where(FIN["OPHID_Text"].isnull(), FIN["OPHID_Text_1"], FIN["OPHID_Text"])

FIN['maskedid'] = np.where(FIN['maskedid'].isnull(), FIN['researchid'], FIN['maskedid'])
FIN['BioBID'] = np.where(FIN['researchid'].isnull(), FIN['maskedid'], FIN['researchid'])
FIN['BioBID'] = np.where( (FIN['researchid'].str.contains('OG')) | (FIN['researchid'].str.contains('ED')), FIN['BioBID'], np.nan)
FIN["site"] = FIN['maskedid'].str[:2]
FIN = FIN[name]
FIN = FIN.fillna('')
FIN.to_csv(redcapeyechart + 'Results/' + extension + '_AxialLen_RedEC.csv', index=False)

In [35]:
L1 = [i for i in L_name if '_IOP-DIL' in i]
df = pd.read_csv(EyeChart + L1[0], encoding = "ISO-8859-1")

#df = pd.read_csv(EyeChart + "20170322_IOP-DIL.csv", encoding = "ISO-8859-1")
name = 'maskedid	OPHID_Text	BioBID	site	visit_id	iopdil_eye	iopdil_date	iopdil_operator	iopdil_meas	iopdil_time	iopdil_time_not_recorded	iopdil_treated	iop_high	iop_eye_treated	iopdil_comments	ciop_max	date_iop_max	ciop_min	date_iop_min	source'
name = name.split('\t')
df['visit_id'] = np.nan

df["source"] = 'EyeChart'
df2 = pd.read_csv(redcapeyechart + extension + "_IOP.csv")


df2 = df2[df2["iop_dilate"] == "Yes"]

df1 = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
df1["OPHID_Text"] = np.nan
df1["iopdil_eye"] = 'OS'
df1["iopdil_date"] = np.array(df2["iop_date"])
df1["iopdil_operator"] = np.array(df2["iop_staff"])

df1["iop_staff"] = np.array(df2["iop_staff"])
df1["visit_id"] = np.array(df2["visit_id"])

df1["iopdil_time"] = np.array(df2["iop_gldman_time"])
df1["iopdil_time_not_recorded"] = np.nan

df1["iopdil_treated"] = np.where(df2["iop_treated"].isin(np.array(["OU", "OS"])), "YES", "NO")
df1["iopdil_meas"] = np.array(df2["iop_os_gldmnmeas"])
df1["iop_eye_treated"] = np.array(df2["iop_os_treated"])
df1["iop_high"] = np.array(df2["iop_os_high"])
df1["iopdil_comments"] = np.nan
df1["source"] = 'REDCap'


df3 = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
df3["OPHID_Text"] = np.nan
df3["iopdil_eye"] = 'OD'
df3["iopdil_date"] = np.array(df2["iop_date"])
df3["iopdil_operator"] = np.array(df2["iop_staff"])

df3["iop_staff"] = np.array(df2["iop_staff"])
df3["visit_id"] = np.array(df2["visit_id"])

df3["iopdil_time"] = np.array(df2["iop_gldman_time"])
df3["iopdil_time_not_recorded"] = np.nan

df3["iopdil_treated"] = np.where(df2["iop_treated"].isin(np.array(["OU", "OD"])), "YES", "NO")
df3["iopdil_meas"] = np.array(df2["iop_od_gldmnmeas"])
df3["iop_eye_treated"] = np.array(df2["iop_od_treated"])
df3["iop_high"] = np.array(df2["iop_od_high"])

df3["iopdil_comments"] = np.nan
df3["source"] = 'REDCap'

df["iop_staff"] = np.nan
df["iop_high"] = np.nan
df["iopdil_treated"] = np.nan
df["iop_eye_treated"] = np.nan

H = list(df.columns.values)

Res = pd.concat([df, df1[H]], ignore_index=True)   
Res = pd.concat([Res, df3[H]], ignore_index=True)   
Res = Res.sort_values(["maskedid", "iopdil_eye", "iopdil_date"]).reset_index(drop=True)
Res['iopdil_meas_a'] = Res[["iopdil_meas", "iop_high"]].max(axis=1)
Res["ciop_max"] = Res.groupby(["maskedid", "iopdil_eye"]).iopdil_meas_a.transform(lambda x:max(x))
Res1 = Res.copy(deep=True)
Res1["where"] = np.where(Res1["ciop_max"] == Res1["iopdil_meas_a"], 1, 0)
Res1 = Res1[Res1["where"] == 1].drop_duplicates(["maskedid", "iopdil_eye"], keep ='last')
Res1["date_iop_max"] = np.array(Res1["iopdil_date"])
Res1 = Res1[["maskedid", "iopdil_eye", "date_iop_max"]]
Res = pd.merge(Res, Res1, how="left", on=["maskedid", "iopdil_eye"])

# 6) Include iop_od_high and iop_os_high from REDCap in calculation of ciop_max in joined file.
# include in calculation iop high from redcap for Dial and Undial

Res["ciop_min"] = Res.groupby(["maskedid", "iopdil_eye"]).iopdil_meas.transform(lambda x:min(x))
Res1 = Res.copy(deep=True)
Res1["where"] = np.where(Res1["ciop_min"] == Res1["iopdil_meas"], 1, 0)
Res1 = Res1[Res1["where"] == 1].drop_duplicates(["maskedid", "iopdil_eye"], keep ='last')
Res1["date_iop_min"] = np.array(Res1["iopdil_date"])
Res1 = Res1[["maskedid", "iopdil_eye", "date_iop_min"]]
Res = pd.merge(Res, Res1, how="left", on=["maskedid", "iopdil_eye"])

FIN = Res.copy(deep=True)

FIN = pd.merge(FIN, ind[["OPHID_Text", "researchid"]], how='left', on='OPHID_Text')





FIN = FIN[~FIN['maskedid'].isnull()]

FIN = pd.merge(FIN, ind[["OPHID_Text_1", "maskedid"]], how='left', on='maskedid')
FIN["OPHID_Text"] = np.where(FIN["OPHID_Text"].isnull(), FIN["OPHID_Text_1"], FIN["OPHID_Text"])

FIN['maskedid'] = np.where(FIN['maskedid'].isnull(), FIN['researchid'], FIN['maskedid'])
FIN['BioBID'] = np.where(FIN['researchid'].isnull(), FIN['maskedid'], FIN['researchid'])
FIN['BioBID'] = np.where( (FIN['researchid'].str.contains('OG')) | (FIN['researchid'].str.contains('ED')), FIN['BioBID'], np.nan)
FIN["site"] = FIN['maskedid'].str[:2]
FIN = FIN[name]

FIN.to_csv(redcapeyechart + 'Results/' + extension +'_IOP-DIL_RedEC.csv', index=False)



In [36]:
L1 = [i for i in L_name if '_IOP-Und' in i]
df = pd.read_csv(EyeChart + L1[0], encoding = "ISO-8859-1")

#df = pd.read_csv(EyeChart + "20170322_IOP-Und.csv", encoding = "ISO-8859-1")
name = 'maskedid	OPHID_Text	BioBID	site	visit_id	iopundil_date	iopundil_eye	iopundil_time	iopundil_treated	iopundil_meas	iopundil_comments	iop_staff	iop_eye_treated	iop_high	ciop_max	date_iop_max	ciop_min	date_iop_min	source'
name = name.split('\t')
df['visit_id'] = np.nan

df["source"] = 'EyeChart'
df2 = pd.read_csv(redcapeyechart + extension + "_IOP.csv")



df2 = df2[df2["iop_dilate"] == "No"]

df1 = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
df1["OPHID_Text"] = np.nan
df1["iopundil_eye"] = 'OS'
df1["iop_staff"] = np.array(df2["iop_staff"])
df1["iopundil_date"] = np.array(df2["iop_date"])
df1["iopundil_time"] = np.array(df2["iop_gldman_time"])
df1["iop_eye_treated"] = np.array(df2["iop_os_treated"])
df1["iop_high"] = np.array(df2["iop_os_high"])

df1["iopundil_treated"] = np.where(df2["iop_treated"].isin(np.array(["OU", "OS"])), "YES", "NO")
df1["iopundil_meas"] = np.array(df2["iop_os_gldmnmeas"])
df1["visit_id"] = np.array(df2["visit_id"])
df1["iopundil_comments"] = np.nan
df1["source"] = 'REDCap'


df3 = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
df3["OPHID_Text"] = np.nan
df3["iopundil_eye"] = 'OD'
df3["iop_staff"] = np.array(df2["iop_staff"])
df3["iopundil_date"] = np.array(df2["iop_date"])
df3["iopundil_time"] = np.array(df2["iop_gldman_time"])
df3["iop_eye_treated"] = np.array(df2["iop_od_treated"])
df3["iop_high"] = np.array(df2["iop_od_high"])
df3["iopundil_treated"] = np.where(df2["iop_treated"].isin(np.array(["OU", "OD"])), "YES", "NO")
df3["iopundil_meas"] = np.array(df2["iop_od_gldmnmeas"])
df3["iopundil_comments"] = np.nan
df3["visit_id"] = np.array(df2["visit_id"])
df3["source"] = 'REDCap'

df["iop_staff"] = np.nan
df["iop_eye_treated"] = np.nan
df["iop_high"] = np.nan
H = list(df.columns.values)

Res = pd.concat([df, df1[H]], ignore_index=True)   
Res = pd.concat([Res, df3[H]], ignore_index=True)
Res = Res.sort_values(["maskedid", "iopundil_eye", "iopundil_date"]).reset_index(drop=True)
Res["maskedid"] = Res["maskedid"].astype(str)
Res ["iopundil_eye"] = Res["iopundil_eye"].astype(str)
Res['iopundil_meas_a'] = Res[["iopundil_meas", "iop_high"]].max(axis=1)
Res["ciop_max"] = Res.groupby(["maskedid", "iopundil_eye"]).iopundil_meas_a.transform(lambda x:x.max())
Res1 = Res.copy(deep=True)



Res1["where"] = np.where(Res1["ciop_max"] == Res1["iopundil_meas_a"], 1, 0)
Res3 = Res1.copy(deep=True)
Res1 = Res1[Res1["where"] == 1].drop_duplicates(["maskedid", "iopundil_eye"], keep ='last')
Res1["date_iop_max"] = np.array(Res1["iopundil_date"])
Res1 = Res1[["maskedid", "iopundil_eye", "date_iop_max"]]
Res = pd.merge(Res, Res1, how="inner", on=["maskedid", "iopundil_eye"])


Res["ciop_min"] = Res.groupby(["maskedid", "iopundil_eye"]).iopundil_meas.transform(lambda x: x.min())
Res1 = Res.copy(deep=True)
Res1["where"] = np.where(Res1["ciop_min"] == Res1["iopundil_meas"], 1, 0)
Res1 = Res1[Res1["where"] == 1].drop_duplicates(["maskedid", "iopundil_eye"], keep ='last')
Res1["date_iop_min"] = np.array(Res1["iopundil_date"])
Res1 = Res1[["maskedid", "iopundil_eye", "date_iop_min"]]
Res = pd.merge(Res, Res1, how="inner", on=["maskedid", "iopundil_eye"])





FIN = Res.copy(deep=True)
FIN = FIN[~FIN['maskedid'].isnull()]
FIN = pd.merge(FIN, ind[["OPHID_Text", "researchid"]], how='left', on='OPHID_Text')

FIN = pd.merge(FIN, ind[["OPHID_Text_1", "maskedid"]], how='left', on='maskedid')
FIN["OPHID_Text"] = np.where(FIN["OPHID_Text"].isnull(), FIN["OPHID_Text_1"], FIN["OPHID_Text"])

FIN['maskedid'] = np.where(FIN['maskedid'].isnull(), FIN['researchid'], FIN['maskedid'])
FIN['BioBID'] = np.where(FIN['researchid'].isnull(), FIN['maskedid'], FIN['researchid'])
FIN['BioBID'] = np.where( (FIN['researchid'].str.contains('OG')) | (FIN['researchid'].str.contains('ED')), FIN['BioBID'], np.nan)
FIN['maskedid'] = np.where(FIN['maskedid']=='nan', FIN['BioBID'], FIN['maskedid'])
FIN["site"] = FIN['maskedid'].str[:2]
FIN = FIN[name]
FIN.to_csv(redcapeyechart + 'Results/' + extension +'_IOP_Und_RedEC.csv', index=False)



In [37]:
Res3[Res3['maskedid']=='SD1759']

,OPHID_Text,maskedid,iopundil_date,iopundil_eye,iopundil_time,iopundil_treated,iopundil_meas,iopundil_comments,visit_id,source,iop_staff,iop_eye_treated,iop_high,iopundil_meas_a,ciop_max,where
30752,OPH-100768,SD1759,01/07/1997,OD,NaN,NaN,19.0,NaN,NaN,EyeChart,NaN,NaN,NaN,19.0,30.0,0
30753,OPH-100768,SD1759,01/16/2002,OD,NaN,NaN,17.0,NaN,NaN,EyeChart,NaN,NaN,NaN,17.0,30.0,0
30754,OPH-100768,SD1759,01/16/2015,OD,10:05:00,NaN,15.0,NaN,NaN,EyeChart,NaN,NaN,NaN,15.0,30.0,0
30755,OPH-100768,SD1759,01/30/1995,OD,NaN,NaN,14.0,NaN,NaN,EyeChart,NaN,NaN,NaN,14.0,30.0,0
30756,OPH-100768,SD1759,02/01/2000,OD,NaN,NaN,18.0,NaN,NaN,EyeChart,NaN,NaN,NaN,18.0,30.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30837,OPH-100768,SD1759,11/03/2010,OS,11:15:00,NaN,4.0,NaN,NaN,EyeChart,NaN,NaN,NaN,4.0,29.0,0
30838,OPH-100768,SD1759,11/08/2013,OS,12:03:00,NaN,29.0,NaN,NaN,EyeChart,NaN,NaN,NaN,29.0,29.0,1
30839,OPH-100768,SD1759,11/10/2011,OS,15:48:00,NaN,3.0,NaN,NaN,EyeChart,NaN,NaN,NaN,3.0,29.0,0
30840,OPH-100768,SD1759,11/20/2015,OS,14:50:07,NO,4.0,NaN,NaN,EyeChart,NaN,NaN,NaN,4.0,29.0,0


In [38]:
df2[df2['maskedID']=='SD1759']

,maskedID,site,visit_id,iop_date,iop_staff,iop_dilate,iop_treated,iop_gldman,iop_gldman_time,iop_od_gldmnmeas,...,iop_ora_os_ch,iop_ora_os_ws,iop_pasc,iop_pasc_time,iop_pasc_odm,iop_pasc_odopa,iop_pasc_odq,iop_pasc_osm,iop_pasc_osopa,iop_pasc_osq
1849,SD1759,SD,Baseline,08/03/2018,Onsite Staff,No,OU,OU,12:21,18.0,...,10.0,5.8,Not tested,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1850,SD1759,SD,Month 6,03/01/2019,Onsite Staff,No,OU,OU,11:32,18.0,...,9.5,8.7,Not tested,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1851,SD1759,SD,Month 12,08/30/2019,"Andrew Camp, MD",No,OD,OU,13:58,21.0,...,NaN,NaN,Not tested,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1852,SD1759,SD,Month 18,02/26/2020,Onsite Staff,No,OU,OS,10:58,NaN,...,8.9,9.2,OU,10:02,23.0,NaN,NaN,4.0,NaN,NaN


In [39]:
FIN[FIN['maskedid']=='SD1759']

,maskedid,OPHID_Text,BioBID,site,visit_id,iopundil_date,iopundil_eye,iopundil_time,iopundil_treated,iopundil_meas,iopundil_comments,iop_staff,iop_eye_treated,iop_high,ciop_max,date_iop_max,ciop_min,date_iop_min,source
30725,SD1759,OPH-100768,NaN,SD,NaN,01/07/1997,OD,NaN,NaN,19.0,NaN,NaN,NaN,NaN,30.0,12/21/1994,10.0,11/20/2015,EyeChart
30726,SD1759,OPH-100768,NaN,SD,NaN,01/16/2002,OD,NaN,NaN,17.0,NaN,NaN,NaN,NaN,30.0,12/21/1994,10.0,11/20/2015,EyeChart
30727,SD1759,OPH-100768,NaN,SD,NaN,01/16/2015,OD,10:05:00,NaN,15.0,NaN,NaN,NaN,NaN,30.0,12/21/1994,10.0,11/20/2015,EyeChart
30728,SD1759,OPH-100768,NaN,SD,NaN,01/30/1995,OD,NaN,NaN,14.0,NaN,NaN,NaN,NaN,30.0,12/21/1994,10.0,11/20/2015,EyeChart
30729,SD1759,OPH-100768,NaN,SD,NaN,02/01/2000,OD,NaN,NaN,18.0,NaN,NaN,NaN,NaN,30.0,12/21/1994,10.0,11/20/2015,EyeChart
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30810,SD1759,OPH-100768,NaN,SD,NaN,11/03/2010,OS,11:15:00,NaN,4.0,NaN,NaN,NaN,NaN,29.0,11/08/2013,2.0,08/13/2013,EyeChart
30811,SD1759,OPH-100768,NaN,SD,NaN,11/08/2013,OS,12:03:00,NaN,29.0,NaN,NaN,NaN,NaN,29.0,11/08/2013,2.0,08/13/2013,EyeChart
30812,SD1759,OPH-100768,NaN,SD,NaN,11/10/2011,OS,15:48:00,NaN,3.0,NaN,NaN,NaN,NaN,29.0,11/08/2013,2.0,08/13/2013,EyeChart
30813,SD1759,OPH-100768,NaN,SD,NaN,11/20/2015,OS,14:50:07,NO,4.0,NaN,NaN,NaN,NaN,29.0,11/08/2013,2.0,08/13/2013,EyeChart


In [40]:
L1 = [i for i in L_name if '_OcularProcedures' in i]
df = pd.read_csv(EyeChart + L1[0], encoding = "ISO-8859-1")
#df = pd.read_csv(EyeChart + '20170322_OcularProcedures.csv', encoding = "ISO-8859-1")
name = 'maskedid	OPHID_Text	BioBID	site	visit_id	op_eye	sys_op_id	name	op_date_rep	op_date	op_age_rep	op_details	source'
name = name.split('\t')
df['visit_id'] = np.nan

df1 = pd.read_csv(redcapeyechart + extension +  '_OcularSurgeriesProcedures.csv')
df["site"] = df['maskedid'].str[:2]
df["source"] = 'EyeChart'
df1 = df1[~df1["maskedID"].isnull()]


df2 = df1[(df1["catarctIOL"] == 'Yes') & (df1["catarctIOL_eye"].isin(np.array(['OD', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'osp_date',
 'catarctIOL',
 'catarctIOL_eye',
 'catarctIOL_dateknwn',
 'catarctIOL_datepxOD',
 'catarctIOL_agepxOD',
 'catarctIOL_datepxOS',
 'catarctIOL_agepxOS']]




A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["op_eye"] = 'OD'

A['visit_id'] = np.array(df2["visit_id"])
A['op_age_rep'] = np.array(df2["catarctIOL_agepxOD"])

A["name"] = 'Cataract Ext by Phacoemul w/PCIOL Impl'
A["OPHID_Text"] = np.nan
A["sys_op_id"] = 291
A["op_date"] = np.array(df2["catarctIOL_datepxOD"])
A["op_date_rep"] = np.array(df2["osp_date"])
A["op_details"] = np.nan
A["source"] = 'REDCap'

df2 = df1[(df1["catarctIOL"] == 'Yes') & (df1["catarctIOL_eye"].isin(np.array(['OS', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'osp_date',
 'catarctIOL',
 'catarctIOL_eye',
 'catarctIOL_dateknwn',
 'catarctIOL_datepxOD',
 'catarctIOL_agepxOD',
 'catarctIOL_datepxOS',
 'catarctIOL_agepxOS']]




B = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
B["site"] = df2["site"]
B["op_eye"] = 'OS'

B['visit_id'] = np.array(df2["visit_id"])
B['op_age_rep'] = np.array(df2["catarctIOL_agepxOS"])


B["name"] = 'Cataract Ext by Phacoemul w/PCIOL Impl'
B["OPHID_Text"] = np.nan
B["sys_op_id"] = 291
B["op_date"] = np.array(df2["catarctIOL_datepxOS"])
B["op_date_rep"] = np.array(df2["osp_date"])
B["op_details"] = np.nan
B["source"] = 'REDCap'

A1 = A.copy(deep=True)
B1 = B.copy(deep=True)

df2 = df1[(df1["catarctnoIOL"] == 'Yes') & (df1["catarctnoIOL__eye"].isin(np.array(['OD', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'osp_date',
 'catarctnoIOL',
 'catarctnoIOL__eye',
 'catarctnoIOL__dateknwn',
 'catarctnoIOL__datepxOD',
 'catarctnoIOL__agepxOD',
 'catarctnoIOL__datepxOS',
 'catarctnoIOL__agepxOS',]]




A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["op_eye"] = 'OD'

A['visit_id'] = np.array(df2["visit_id"])
A['op_age_rep'] = np.array(df2["catarctnoIOL__agepxOD"])

A["name"] = 'Cataract Ext without IOL Impl'
A["OPHID_Text"] = np.nan
A["sys_op_id"] = 292
A["op_date"] = np.array(df2["catarctnoIOL__datepxOD"])
A["op_date_rep"] = np.array(df2["osp_date"])
A["op_details"] = np.nan
A["source"] = 'REDCap'

df2 = df1[(df1["catarctnoIOL"] == 'Yes') & (df1["catarctnoIOL__eye"].isin(np.array(['OS', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'osp_date',
 'catarctnoIOL',
 'catarctnoIOL__eye',
 'catarctnoIOL__dateknwn',
 'catarctnoIOL__datepxOD',
 'catarctnoIOL__agepxOD',
 'catarctnoIOL__datepxOS',
 'catarctnoIOL__agepxOS',]]




B = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
B["site"] = df2["site"]
B["op_eye"] = 'OS'

B['visit_id'] = np.array(df2["visit_id"])
B['op_age_rep'] = np.array(df2["catarctnoIOL__agepxOS"])

B["name"] = 'Cataract Ext without IOL Impl'
B["OPHID_Text"] = np.nan
B["sys_op_id"] = 292
B["op_date"] = np.array(df2["catarctnoIOL__datepxOS"])
B["op_date_rep"] = np.array(df2["osp_date"])
B["op_details"] = np.nan
B["source"] = 'REDCap'

A2 = A.copy(deep=True)
B2 = B.copy(deep=True)

df2 = df1[(df1["combined"] == 'Yes') & (df1["combined__eye"].isin(np.array(['OD', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'osp_date',
 'combined',
 'combined__eye',
 'combined__dateknwn',
 'combined__datepxOD',
 'combined__agepxOD',
 'combined__datepxOS',
 'combined__agepxOS',]]




A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["op_eye"] = 'OD'

A['visit_id'] = np.array(df2["visit_id"])
A['op_age_rep'] = np.array(df2["combined__agepxOD"])

A["name"] = 'Combined Cataract Ext by Phacoemul w/PCIOL & TRAB'
A["OPHID_Text"] = np.nan
A["sys_op_id"] = 294
A["op_date_rep"] = np.array(df2["osp_date"])
A["op_date"] = np.array(df2["combined__datepxOD"])
A["op_details"] = np.nan
A["source"] = 'REDCap'

df2 = df1[(df1["combined"] == 'Yes') & (df1["combined__eye"].isin(np.array(['OS', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'osp_date',
 'combined',
 'combined__eye',
 'combined__dateknwn',
 'combined__datepxOD',
 'combined__agepxOD',
 'combined__datepxOS',
 'combined__agepxOS',]]




B = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
B["site"] = df2["site"]
B["op_eye"] = 'OS'

B['visit_id'] = np.array(df2["visit_id"])
B['op_age_rep'] = np.array(df2["combined__agepxOS"])


B["name"] = 'Combined Cataract Ext by Phacoemul w/PCIOL & TRAB'
B["OPHID_Text"] = np.nan
B["sys_op_id"] = 294
B["op_date"] = np.array(df2["combined__datepxOS"])
B["op_date_rep"] = np.array(df2["osp_date"])
B["op_details"] = np.nan
B["source"] = 'REDCap'

A3 = A.copy(deep=True)
B3 = B.copy(deep=True)

df2 = df1[(df1["laserrettear"] == 'Yes') & (df1["laserrettear__eye"].isin(np.array(['OD', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'osp_date',
 'laserrettear',
 'laserrettear__eye',
 'laserrettear__dateknwn',
 'laserrettear__datepxOD',
 'laserrettear__agepxOD',
 'laserrettear__datepxOS',
 'laserrettear__agepxOS',]]




A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["op_eye"] = 'OD'
A['visit_id'] = np.array(df2["visit_id"])
A['op_age_rep'] = np.array(df2["laserrettear__agepxOD"])


A["name"] = 'Laser retinal tear or detachment'
A["OPHID_Text"] = np.nan
A["sys_op_id"] = 155
A["op_date"] = np.array(df2["laserrettear__datepxOD"])
A["op_date_rep"] = np.array(df2["osp_date"])
A["op_details"] = np.nan
A["source"] = 'REDCap'

df2 = df1[(df1["laserrettear"] == 'Yes') & (df1["laserrettear__eye"].isin(np.array(['OS', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'osp_date',
 'laserrettear',
 'laserrettear__eye',
 'laserrettear__dateknwn',
 'laserrettear__datepxOD',
 'laserrettear__agepxOD',
 'laserrettear__datepxOS',
 'laserrettear__agepxOS',]]




B = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
B["site"] = df2["site"]
B["op_eye"] = 'OS'

B['visit_id'] = np.array(df2["visit_id"])
B['op_age_rep'] = np.array(df2["laserrettear__agepxOS"])

B["name"] = 'Laser retinal tear or detachment'
B["OPHID_Text"] = np.nan
B["sys_op_id"] = 155
B["op_date"] = np.array(df2["laserrettear__datepxOS"])
B["op_date_rep"] = np.array(df2["osp_date"])
B["op_details"] = np.nan
B["source"] = 'REDCap'

A4 = A.copy(deep=True)
B4 = B.copy(deep=True)


df2 = df1[(df1["lasertrab__typeOD"] == 'Selective (SLT)') & (df1["lasertrab"] == 'Yes') & (df1["lasertrab__eye"].isin(np.array(['OD', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'osp_date',
 'lasertrab',
 'lasertrab__eye',
 'lasertrab__dateknwn',
 'lasertrab__typeOD',
 'lasertrab__datepxOD',
 'lasertrab__agepxOD',
 'lasertrab__typeOS',
 'lasertrab__datepxOS',
 'lasertrab__agepxOS',]]




A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["op_eye"] = 'OD'

A['visit_id'] = np.array(df2["visit_id"])
A['op_age_rep'] = np.array(df2["lasertrab__agepxOD"])


A["name"] = 'Selective Laser Trabeculoplasty (SLT)'
A["OPHID_Text"] = np.nan
A["sys_op_id"] = 238
A["op_date"] = np.array(df2["lasertrab__datepxOD"])
A["op_date_rep"] = np.array(df2["osp_date"])
A["op_details"] = np.nan
A["source"] = 'REDCap'

df2 = df1[(df1["lasertrab__typeOS"] == 'Selective (SLT)') & (df1["lasertrab"] == 'Yes') & (df1["lasertrab__eye"].isin(np.array(['OS', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'osp_date',
 'lasertrab',
 'lasertrab__eye',
 'lasertrab__dateknwn',
 'lasertrab__typeOD',
 'lasertrab__datepxOD',
 'lasertrab__agepxOD',
 'lasertrab__typeOS',
 'lasertrab__datepxOS',
 'lasertrab__agepxOS',]]




B = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
B["site"] = df2["site"]
B["op_eye"] = 'OS'

B['visit_id'] = np.array(df2["visit_id"])
B['op_age_rep'] = np.array(df2["lasertrab__agepxOS"])


B["name"] = 'Selective Laser Trabeculoplasty (SLT)'
B["OPHID_Text"] = np.nan
B["sys_op_id"] = 238
B["op_date"] = np.array(df2["lasertrab__datepxOS"])
B["op_date_rep"] = np.array(df2["osp_date"])
B["op_details"] = np.nan
B["source"] = 'REDCap'

A5 = A.copy(deep=True)
B5 = B.copy(deep=True)

df2 = df1[(df1["lasertrab__typeOD"] == 'Argon (ALT)') & (df1["lasertrab"] == 'Yes') & (df1["lasertrab__eye"].isin(np.array(['OD', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'osp_date',
 'lasertrab',
 'lasertrab__eye',
 'lasertrab__dateknwn',
 'lasertrab__typeOD',
 'lasertrab__datepxOD',
 'lasertrab__agepxOD',
 'lasertrab__typeOS',
 'lasertrab__datepxOS',
 'lasertrab__agepxOS',]]




A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["op_eye"] = 'OD'

A['visit_id'] = np.array(df2["visit_id"])
A['op_age_rep'] = np.array(df2["lasertrab__agepxOD"])

A["name"] = 'Argon Laser Trabeculoplasty'
A["OPHID_Text"] = np.nan
A["sys_op_id"] = 234
A["op_date"] = np.array(df2["lasertrab__datepxOD"])
A["op_date_rep"] = np.array(df2["osp_date"])
A["op_details"] = np.nan
A["source"] = 'REDCap'

df2 = df1[(df1["lasertrab__typeOS"] == 'Argon (ALT)') & (df1["lasertrab"] == 'Yes') & (df1["lasertrab__eye"].isin(np.array(['OS', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'osp_date',
 'lasertrab',
 'lasertrab__eye',
 'lasertrab__dateknwn',
 'lasertrab__typeOD',
 'lasertrab__datepxOD',
 'lasertrab__agepxOD',
 'lasertrab__typeOS',
 'lasertrab__datepxOS',
 'lasertrab__agepxOS',]]




B = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
B["site"] = df2["site"]
B["op_eye"] = 'OS'

B['visit_id'] = np.array(df2["visit_id"])
B['op_age_rep'] = np.array(df2["lasertrab__agepxOS"])


B["name"] = 'Argon Laser Trabeculoplasty'
B["OPHID_Text"] = np.nan
B["sys_op_id"] = 234
B["op_date"] = np.array(df2["lasertrab__datepxOS"])
B["op_date_rep"] = np.array(df2["osp_date"])
B["op_details"] = np.nan
B["source"] = 'REDCap'

A6 = A.copy(deep=True)
B6 = B.copy(deep=True)


df2 = df1[(df1["lasertrab__typeOD"].isnull()) & (df1["lasertrab"] == 'Yes') & (df1["lasertrab__eye"].isin(np.array(['OD', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'osp_date',
 'lasertrab',
 'lasertrab__eye',
 'lasertrab__dateknwn',
 'lasertrab__typeOD',
 'lasertrab__datepxOD',
 'lasertrab__agepxOD',
 'lasertrab__typeOS',
 'lasertrab__datepxOS',
 'lasertrab__agepxOS',]]




A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["op_eye"] = 'OD'

A['visit_id'] = np.array(df2["visit_id"])
A['op_age_rep'] = np.array(df2["lasertrab__agepxOD"])


A["name"] = 'Laser Trabeculoplasty unspecified'
A["OPHID_Text"] = np.nan
A["sys_op_id"] = 132
A["op_date"] = np.array(df2["lasertrab__datepxOD"])
A["op_date_rep"] = np.array(df2["osp_date"])
A["op_details"] = np.nan
A["source"] = 'REDCap'

df2 = df1[(df1["lasertrab__typeOS"].isnull()) & (df1["lasertrab"] == 'Yes') & (df1["lasertrab__eye"].isin(np.array(['OS', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'osp_date',
 'lasertrab',
 'lasertrab__eye',
 'lasertrab__dateknwn',
 'lasertrab__typeOD',
 'lasertrab__datepxOD',
 'lasertrab__agepxOD',
 'lasertrab__typeOS',
 'lasertrab__datepxOS',
 'lasertrab__agepxOS',]]




B = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
B["site"] = df2["site"]
B["op_eye"] = 'OS'

B['visit_id'] = np.array(df2["visit_id"])
B['op_age_rep'] = np.array(df2["lasertrab__agepxOS"])

B["name"] = 'Laser Trabeculoplasty unspecified'
B["OPHID_Text"] = np.nan
B["sys_op_id"] = 132
B["op_date"] = np.array(df2["lasertrab__datepxOS"])
B["op_date_rep"] = np.array(df2["osp_date"])
B["op_details"] = np.nan
B["source"] = 'REDCap'

A7 = A.copy(deep=True)
B7 = B.copy(deep=True)

df2 = df1[(df1["lasikrefract"] == 'Yes') & (df1["lasikrefract__eye"].isin(np.array(['OD', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'osp_date',
 'lasikrefract',
 'lasikrefract__eye',
 'lasikrefract__dateknwn',
 'lasikrefract__datepxOD',
 'lasikrefract__agepxOD',
 'lasikrefract__datepxOS',
 'lasikrefract__agepxOS',]]




A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["op_eye"] = 'OD'

A['visit_id'] = np.array(df2["visit_id"])
A['op_age_rep'] = np.array(df2["lasikrefract__agepxOD"])

A["name"] = 'LASIK or other refractive surgery'
A["OPHID_Text"] = np.nan
A["sys_op_id"] = 243
A["op_date"] = np.array(df2["lasikrefract__datepxOD"])
A["op_date_rep"] = np.array(df2["osp_date"])
A["op_details"] = np.nan
A["source"] = 'REDCap'

df2 = df1[(df1["lasikrefract"] == 'Yes') & (df1["lasikrefract__eye"].isin(np.array(['OS', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'osp_date',
 'laserrettear',
 'lasikrefract',
 'lasikrefract__eye',
 'lasikrefract__dateknwn',
 'lasikrefract__datepxOD',
 'lasikrefract__agepxOD',
 'lasikrefract__datepxOS',
 'lasikrefract__agepxOS',]]




B = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
B["site"] = df2["site"]
B["op_eye"] = 'OS'

B['visit_id'] = np.array(df2["visit_id"])
B['op_age_rep'] = np.array(df2["lasikrefract__agepxOS"])


B["name"] = 'LASIK or other refractive surgery'
B["OPHID_Text"] = np.nan
B["sys_op_id"] = 243
B["op_date"] = np.array(df2["lasikrefract__datepxOS"])
B["op_date_rep"] = np.array(df2["osp_date"])
B["op_details"] = np.nan
B["source"] = 'REDCap'

A8 = A.copy(deep=True)
B8 = B.copy(deep=True)

df2 = df1[(df1["lpi"] == 'Yes') & (df1["lpi__eye"].isin(np.array(['OD', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'osp_date',
 'lpi',
 'lpi__eye',
 'lpi__dateknwn',
 'lpi__datepxOD',
 'lpi__agepxOD',
 'lpi__datepxOS',
 'lpi__agepxOS',]]




A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["op_eye"] = 'OD'

A['visit_id'] = np.array(df2["visit_id"])
A['op_age_rep'] = np.array(df2["lpi__agepxOD"])


A["name"] = 'LPI for narrow, occludable, or closed angles'
A["OPHID_Text"] = np.nan
A["sys_op_id"] = 191
A["op_date"] = np.array(df2["lpi__datepxOD"])
A["op_date_rep"] = np.array(df2["osp_date"])
A["op_details"] = np.nan
A["source"] = 'REDCap'

df2 = df1[(df1["lpi"] == 'Yes') & (df1["lpi__eye"].isin(np.array(['OS', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'osp_date',
 'lpi',
 'lpi__eye',
 'lpi__dateknwn',
 'lpi__datepxOD',
 'lpi__agepxOD',
 'lpi__datepxOS',
 'lpi__agepxOS',]]




B = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
B["site"] = df2["site"]
B["op_eye"] = 'OS'

B['visit_id'] = np.array(df2["visit_id"])
B['op_age_rep'] = np.array(df2["lpi__agepxOS"])

B["name"] = 'LPI for narrow, occludable, or closed angles'
B["OPHID_Text"] = np.nan
B["sys_op_id"] = 191
B["op_date"] = np.array(df2["lpi__datepxOS"])
B["op_date_rep"] = np.array(df2["osp_date"])
B["op_details"] = np.nan
B["source"] = 'REDCap'

A9 = A.copy(deep=True)
B9 = B.copy(deep=True)

df2 = df1[(df1["tearduct"] == 'Yes') & (df1["tearduct__eye"].isin(np.array(['OD', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'osp_date',
 'tearduct',
 'tearduct__eye',
 'tearduct__dateknwn',
 'tearduct__datepxOD',
 'tearduct__agepxOD',
 'tearduct__datepxOS',
 'tearduct__agepxOS',]]




A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["op_eye"] = 'OD'

A['visit_id'] = np.array(df2["visit_id"])
A['op_age_rep'] = np.array(df2["tearduct__agepxOD"])


A["name"] = 'Lid or Tear Duct Procedure'
A["OPHID_Text"] = np.nan
A["sys_op_id"] = 236
A["op_date"] = np.array(df2["tearduct__datepxOD"])
A["op_date_rep"] = np.array(df2["osp_date"])
A["op_details"] = np.nan
A["source"] = 'REDCap'

df2 = df1[(df1["tearduct"] == 'Yes') & (df1["tearduct__eye"].isin(np.array(['OS', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'osp_date',
 'tearduct',
 'tearduct__eye',
 'tearduct__dateknwn',
 'tearduct__datepxOD',
 'tearduct__agepxOD',
 'tearduct__datepxOS',
 'tearduct__agepxOS',]]




B = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
B["site"] = df2["site"]
B["op_eye"] = 'OS'

B['visit_id'] = np.array(df2["visit_id"])
B['op_age_rep'] = np.array(df2["tearduct__agepxOS"])


B["name"] = 'Lid or Tear Duct Procedure'
B["OPHID_Text"] = np.nan
B["sys_op_id"] = 236
B["op_date"] = np.array(df2["tearduct__datepxOS"])
B["op_date_rep"] = np.array(df2["osp_date"])
B["op_details"] = np.nan
B["source"] = 'REDCap'

A10 = A.copy(deep=True)
B10 = B.copy(deep=True)

df2 = df1[(df1["radkeratot"] == 'Yes') & (df1["radkeratot__eye"].isin(np.array(['OD', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'osp_date',
 'radkeratot',
 'radkeratot__eye',
 'radkeratot__dateknwn',
 'radkeratot__datepxOD',
 'radkeratot__agepxOD',
 'radkeratot__datepxOS',
 'radkeratot__agepxOS',]]




A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["op_eye"] = 'OD'

A['visit_id'] = np.array(df2["visit_id"])
A['op_age_rep'] = np.array(df2["radkeratot__agepxOD"])


A["name"] = 'Radial Keratotomy (RK)'
A["OPHID_Text"] = np.nan
A["sys_op_id"] = 174
A["op_date"] = np.array(df2["radkeratot__datepxOD"])
A["op_date_rep"] = np.array(df2["osp_date"])
A["op_details"] = np.nan
A["source"] = 'REDCap'

df2 = df1[(df1["radkeratot"] == 'Yes') & (df1["radkeratot__eye"].isin(np.array(['OS', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'osp_date',
 'radkeratot',
 'radkeratot__eye',
 'radkeratot__dateknwn',
 'radkeratot__datepxOD',
 'radkeratot__agepxOD',
 'radkeratot__datepxOS',
 'radkeratot__agepxOS',]]




B = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
B["site"] = df2["site"]
B["op_eye"] = 'OS'

B['visit_id'] = np.array(df2["visit_id"])
B['op_age_rep'] = np.array(df2["radkeratot__agepxOS"])

B["name"] = 'Radial Keratotomy (RK)'
B["OPHID_Text"] = np.nan
B["sys_op_id"] = 174
B["op_date"] = np.array(df2["radkeratot__datepxOS"])
B["op_date_rep"] = np.array(df2["osp_date"])
B["op_details"] = np.nan
B["source"] = 'REDCap'

A11 = A.copy(deep=True)
B11 = B.copy(deep=True)


df2 = df1[(df1["trab"] == 'Yes') & (df1["trab__eye"].isin(np.array(['OD', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'osp_date',
 'trab',
 'trab__eye',
 'trab__dateknwn',
 'trab__typeOD',
 'trab__datepxOD',
 'trab__agepxOD',
 'trab__typeOS ',
 'trab__datepxOS',
 'trab__agepxOS',]]




A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["op_eye"] = 'OD'

A['visit_id'] = np.array(df2["visit_id"])
A['op_age_rep'] = np.array(df2["trab__agepxOD"])

A["name"] = 'Trabeculectomy'
A["OPHID_Text"] = np.nan
A["sys_op_id"] = 185
A["op_date"] = np.array(df2["trab__datepxOD"])
A["op_date_rep"] = np.array(df2["osp_date"])
A["op_details"] = np.nan
A["source"] = 'REDCap'

df2 = df1[(df1["trab"] == 'Yes') & (df1["trab__eye"].isin(np.array(['OS', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'osp_date',
 'trab',
 'trab__eye',
 'trab__dateknwn',
 'trab__typeOD',
 'trab__datepxOD',
 'trab__agepxOD',
 'trab__typeOS ',
 'trab__datepxOS',
 'trab__agepxOS',]]




B = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
B["site"] = df2["site"]
B["op_eye"] = 'OS'

B['visit_id'] = np.array(df2["visit_id"])
B['op_age_rep'] = np.array(df2["trab__agepxOS"])


B["name"] = 'Trabeculectomy'
B["OPHID_Text"] = np.nan
B["sys_op_id"] = 185
B["op_date"] = np.array(df2["trab__datepxOS"])
B["op_date_rep"] = np.array(df2["osp_date"])
B["op_details"] = np.nan
B["source"] = 'REDCap'

A12 = A.copy(deep=True)
B12 = B.copy(deep=True)

df2 = df1[(df1["yag"] == 'Yes') & (df1["yag__eye"].isin(np.array(['OD', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'osp_date',
 'yag',
 'yag__eye',
 'yag__dateknwn',
 'yag__datepxOD',
 'yag__agepxOD',
 'yag__datepxOS',
 'yag__agepxOS',]]




A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["op_eye"] = 'OD'

A['visit_id'] = np.array(df2["visit_id"])
A['op_age_rep'] = np.array(df2["yag__agepxOD"])


A["name"] = 'YAG laser capsulotomy'
A["OPHID_Text"] = np.nan
A["sys_op_id"] = 187
A["op_date"] = np.array(df2["yag__datepxOD"])
A["op_date_rep"] = np.array(df2["osp_date"])
A["op_details"] = np.nan
A["source"] = 'REDCap'

df2 = df1[(df1["yag"] == 'Yes') & (df1["yag__eye"].isin(np.array(['OS', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'osp_date',
 'yag',
 'yag__eye',
 'yag__dateknwn',
 'yag__datepxOD',
 'yag__agepxOD',
 'yag__datepxOS',
 'yag__agepxOS',]]




B = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
B["site"] = df2["site"]
B["op_eye"] = 'OS'

B['visit_id'] = np.array(df2["visit_id"])
B['op_age_rep'] = np.array(df2["yag__agepxOS"])


B["name"] = 'YAG laser capsulotomy'
B["OPHID_Text"] = np.nan
B["sys_op_id"] = 187
B["op_date"] = np.array(df2["yag__datepxOS"])
B["op_date_rep"] = np.array(df2["osp_date"])
B["op_details"] = np.nan
B["source"] = 'REDCap'

A13 = A.copy(deep=True)
B13 = B.copy(deep=True)

df2 = df1[(df1["othrocsurg1"] == 'Yes') & (df1["othrocsurg1__eye"].isin(np.array(['OD', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'osp_date',
 'othrocsurg1',
 'othrocsurg1_name',
 'othrocsurg1_name2',
 'othrocsurg1__eye',
 'othrocsurg1__dateknwn',
 'othrocsurg1__datepxOD',
 'othrocsurg1__agepxOD',
 'othrocsurg1__datepxOS',
 'othrocsurg1__agepxOS',]]




A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["op_eye"] = 'OD'

A['visit_id'] = np.array(df2["visit_id"])
A['op_age_rep'] = np.array(df2["othrocsurg1__agepxOD"])


A["name"] = np.array(df2["othrocsurg1_name"])
A["OPHID_Text"] = np.nan
A["sys_op_id"] = np.nan
A["op_date"] = np.array(df2["othrocsurg1__datepxOD"])
A["op_date_rep"] = np.array(df2["osp_date"])
A["op_details"] = np.array(df2["othrocsurg1_name2"])
A["source"] = 'REDCap'

df2 = df1[(df1["othrocsurg1"] == 'Yes') & (df1["othrocsurg1__eye"].isin(np.array(['OS', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'osp_date',
 'othrocsurg1',
 'othrocsurg1_name',
 'othrocsurg1_name2',
 'othrocsurg1__eye',
 'othrocsurg1__dateknwn',
 'othrocsurg1__datepxOD',
 'othrocsurg1__agepxOD',
 'othrocsurg1__datepxOS',
 'othrocsurg1__agepxOS',]]




B = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
B["site"] = df2["site"]
B["op_eye"] = 'OS'

B['visit_id'] = np.array(df2["visit_id"])
B['op_age_rep'] = np.array(df2["othrocsurg1__agepxOS"])


B["name"] = np.array(df2["othrocsurg1_name"])
B["OPHID_Text"] = np.nan
B["sys_op_id"] = np.nan
B["op_date"] = np.array(df2["othrocsurg1__datepxOS"])
B["op_date_rep"] = np.array(df2["osp_date"])
B["op_details"] = np.array(df2["othrocsurg1_name2"])
B["source"] = 'REDCap'

A14 = A.copy(deep=True)
B14 = B.copy(deep=True)

df2 = df1[(df1["othrocsurg2"] == 'Yes') & (df1["othrocsurg2__eye"].isin(np.array(['OD', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'osp_date',
 'othrocsurg2',
 'othrocsurg2_name',
 'othrocsurg2_name2',
 'othrocsurg2__eye',
 'othrocsurg2__dateknwn',
 'othrocsurg2__datepxOD',
 'othrocsurg2__agepxOD',
 'othrocsurg2__datepxOS',
 'othrocsurg2__agepxOS',]]




A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["op_eye"] = 'OD'

A['visit_id'] = np.array(df2["visit_id"])
A['op_age_rep'] = np.array(df2["othrocsurg2__agepxOD"])

A["name"] = np.array(df2["othrocsurg2_name"])


A["OPHID_Text"] = np.nan
A["sys_op_id"] = np.nan
A["op_date"] = np.array(df2["othrocsurg2__datepxOD"])
A["op_date_rep"] = np.array(df2["osp_date"])
A["op_details"] = np.array(df2["othrocsurg2_name2"])
A["source"] = 'REDCap'

df2 = df1[(df1["othrocsurg2"] == 'Yes') & (df1["othrocsurg2__eye"].isin(np.array(['OS', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'osp_date',
 'othrocsurg2',
 'othrocsurg2_name',
 'othrocsurg2_name2',
 'othrocsurg2__eye',
 'othrocsurg2__dateknwn',
 'othrocsurg2__datepxOD',
 'othrocsurg2__agepxOD',
 'othrocsurg2__datepxOS',
 'othrocsurg2__agepxOS',]]




B = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
B["site"] = df2["site"]
B["op_eye"] = 'OS'

B['visit_id'] = np.array(df2["visit_id"])
B['op_age_rep'] = np.array(df2["othrocsurg2__agepxOS"])


B["name"] = np.array(df2["othrocsurg2_name"])
B["OPHID_Text"] = np.nan
B["sys_op_id"] = np.nan
B["op_date"] = np.array(df2["othrocsurg2__datepxOS"])
B["op_date_rep"] = np.array(df2["osp_date"])
B["op_details"] = np.array(df2["othrocsurg2_name2"])
B["source"] = 'REDCap'

A15 = A.copy(deep=True)
B15 = B.copy(deep=True)

df2 = df1[(df1["othrocsurg3"] == 'Yes') & (df1["othrocsurg3__eye"].isin(np.array(['OD', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'osp_date',
 'othrocsurg3',
 'othrocsurg3_name',
 'othrocsurg3_name2',
 'othrocsurg3__eye',
 'othrocsurg3__dateknwn',
 'othrocsurg3__datepxOD',
 'othrocsurg3__agepxOD',
 'othrocsurg3__datepxOS',
 'othrocsurg3__agepxOS',]]




A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["op_eye"] = 'OD'

A['visit_id'] = np.array(df2["visit_id"])
A['op_age_rep'] = np.array(df2["othrocsurg3__agepxOD"])

A["name"] = np.array(df2["othrocsurg3_name"])
A["OPHID_Text"] = np.nan
A["sys_op_id"] = np.nan
A["op_date"] = np.array(df2["othrocsurg3__datepxOD"])
A["op_date_rep"] = np.array(df2["osp_date"])
A["op_details"] = np.array(df2["othrocsurg3_name2"])
A["source"] = 'REDCap'

df2 = df1[(df1["othrocsurg3"] == 'Yes') & (df1["othrocsurg3__eye"].isin(np.array(['OS', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'osp_date',
 'othrocsurg3',
 'othrocsurg3_name',
 'othrocsurg3_name2',
 'othrocsurg3__eye',
 'othrocsurg3__dateknwn',
 'othrocsurg3__datepxOD',
 'othrocsurg3__agepxOD',
 'othrocsurg3__datepxOS',
 'othrocsurg3__agepxOS',]]




B = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
B["site"] = df2["site"]
B["op_eye"] = 'OS'

B['visit_id'] = np.array(df2["visit_id"])
B['op_age_rep'] = np.array(df2["othrocsurg3__agepxOS"])

B["name"] = np.array(df2["othrocsurg3_name"])
B["OPHID_Text"] = np.nan
B["sys_op_id"] = np.nan
B["op_date"] = np.array(df2["othrocsurg3__datepxOS"])
B["op_date_rep"] = np.array(df2["osp_date"])
B["op_details"] = np.array(df2["othrocsurg3_name2"])
B["source"] = 'REDCap'

A16 = A.copy(deep=True)
B16 = B.copy(deep=True)

df2 = df1[(df1["othrocsurg4"] == 'Yes') & (df1["othrocsurg4__eye"].isin(np.array(['OD', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'osp_date',
 'othrocsurg4',
 'othrocsurg4_name',
 'othrocsurg4_name2',
 'othrocsurg4__eye',
 'othrocsurg4__dateknwn',
 'othrocsurg4__datepxOD',
 'othrocsurg4__agepxOD',
 'othrocsurg4__datepxOS',
 'othrocsurg4__agepxOS',]]




A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["op_eye"] = 'OD'

A['visit_id'] = np.array(df2["visit_id"])
A['op_age_rep'] = np.array(df2["othrocsurg4__agepxOD"])


A["name"] = np.array(df2["othrocsurg4_name"])
A["OPHID_Text"] = np.nan
A["sys_op_id"] = np.nan
A["op_date"] = np.array(df2["othrocsurg4__datepxOD"])
A["op_date_rep"] = np.array(df2["osp_date"])
A["op_details"] = np.array(df2["othrocsurg4_name2"])
A["source"] = 'REDCap'

df2 = df1[(df1["othrocsurg4"] == 'Yes') & (df1["othrocsurg4__eye"].isin(np.array(['OS', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'osp_date',
 'othrocsurg4',
 'othrocsurg4_name',
 'othrocsurg4_name2',
 'othrocsurg4__eye',
 'othrocsurg4__dateknwn',
 'othrocsurg4__datepxOD',
 'othrocsurg4__agepxOD',
 'othrocsurg4__datepxOS',
 'othrocsurg4__agepxOS',]]




B = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
B["site"] = df2["site"]
B["op_eye"] = 'OS'

B['visit_id'] = np.array(df2["visit_id"])
B['op_age_rep'] = np.array(df2["othrocsurg4__agepxOS"])


B["name"] = np.array(df2["othrocsurg4_name"])
B["OPHID_Text"] = np.nan
B["sys_op_id"] = np.nan
B["op_date"] = np.array(df2["othrocsurg4__datepxOS"])
B["op_date_rep"] = np.array(df2["osp_date"])
B["op_details"] = np.array(df2["othrocsurg4_name2"])
B["source"] = 'REDCap'

A17 = A.copy(deep=True)
B17 = B.copy(deep=True)

df2 = df1[(df1["othrocsurg5"] == 'Yes') & (df1["othrocsurg5__eye"].isin(np.array(['OD', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'osp_date',
 'othrocsurg5',
 'othrocsurg5_name',
 'othrocsurg5_name2',
 'othrocsurg5__eye',
 'othrocsurg5__dateknwn',
 'othrocsurg5__datepxOD',
 'othrocsurg5__agepxOD',
 'othrocsurg5__datepxOS',
 'othrocsurg5__agepxOS',]]




A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["op_eye"] = 'OD'

A['visit_id'] = np.array(df2["visit_id"])
A['op_age_rep'] = np.array(df2["othrocsurg5__agepxOD"])


A["name"] = np.array(df2["othrocsurg5_name"])
A["OPHID_Text"] = np.nan
A["sys_op_id"] = np.nan
A["op_date"] = np.array(df2["othrocsurg5__datepxOD"])
A["op_date_rep"] = np.array(df2["osp_date"])
A["op_details"] = np.array(df2["othrocsurg5_name2"])
A["source"] = 'REDCap'

df2 = df1[(df1["othrocsurg5"] == 'Yes') & (df1["othrocsurg5__eye"].isin(np.array(['OS', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'osp_date',
 'othrocsurg5',
 'othrocsurg5_name',
 'othrocsurg5_name2',
 'othrocsurg5__eye',
 'othrocsurg5__dateknwn',
 'othrocsurg5__datepxOD',
 'othrocsurg5__agepxOD',
 'othrocsurg5__datepxOS',
 'othrocsurg5__agepxOS',]]




B = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
B["site"] = df2["site"]
B["op_eye"] = 'OS'

B['visit_id'] = np.array(df2["visit_id"])
B['op_age_rep'] = np.array(df2["othrocsurg5__agepxOS"])


B["name"] = np.array(df2["othrocsurg5_name"])
B["OPHID_Text"] = np.nan
B["sys_op_id"] = np.nan
B["op_date"] = np.array(df2["othrocsurg5__datepxOS"])
B["op_date_rep"] = np.array(df2["osp_date"])
B["op_details"] = np.array(df2["othrocsurg5_name2"])
B["source"] = 'REDCap'

A18 = A.copy(deep=True)
B18 = B.copy(deep=True)



df2 = df1[(df1["othrocsurg6"] == 'Yes') & (df1["othrocsurg6__eye"].isin(np.array(['OD', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'osp_date',
 'othrocsurg6',
 'othrocsurg6_name',
 'othrocsurg6_name2',
 'othrocsurg6__eye',
 'othrocsurg6__dateknwn',
 'othrocsurg6__datepxOD',
 'othrocsurg6__agepxOD',
 'othrocsurg6__datepxOS',
 'othrocsurg6__agepxOS',]]




A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["op_eye"] = 'OD'

A['visit_id'] = np.array(df2["visit_id"])
A['op_age_rep'] = np.array(df2["othrocsurg6__agepxOD"])


A["name"] = np.array(df2["othrocsurg6_name"])
A["OPHID_Text"] = np.nan
A["sys_op_id"] = np.nan
A["op_date"] = np.array(df2["othrocsurg6__datepxOD"])
A["op_date_rep"] = np.array(df2["osp_date"])
A["op_details"] = np.array(df2["othrocsurg6_name2"])
A["source"] = 'REDCap'

df2 = df1[(df1["othrocsurg6"] == 'Yes') & (df1["othrocsurg6__eye"].isin(np.array(['OS', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'osp_date',
 'othrocsurg6',
 'othrocsurg6_name',
 'othrocsurg6_name2',
 'othrocsurg6__eye',
 'othrocsurg6__dateknwn',
 'othrocsurg6__datepxOD',
 'othrocsurg6__agepxOD',
 'othrocsurg6__datepxOS',
 'othrocsurg6__agepxOS',]]




B = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
B["site"] = df2["site"]
B["op_eye"] = 'OS'

B['visit_id'] = np.array(df2["visit_id"])
B['op_age_rep'] = np.array(df2["othrocsurg6__agepxOS"])


B["name"] = np.array(df2["othrocsurg6_name"])
B["OPHID_Text"] = np.nan
B["sys_op_id"] = np.nan
B["op_date"] = np.array(df2["othrocsurg6__datepxOS"])
B["op_date_rep"] = np.array(df2["osp_date"])
B["op_details"] = np.array(df2["othrocsurg6_name2"])
B["source"] = 'REDCap'

A19 = A.copy(deep=True)
B19 = B.copy(deep=True)

df2 = df1[(df1["othrocsurg_more"] == 'Yes') & (df1["othrocsurg6__eye"].isin(np.array(['OS', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'osp_date',
 'othrocsurg_more',
 'othrocsurg_morename',]]




B = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
B["site"] = df2["site"]
B["op_eye"] = np.nan

B['visit_id'] = np.array(df2["visit_id"])
B['op_age_rep'] = np.nan


B["name"] = np.array(df2["othrocsurg_morename"])
B["OPHID_Text"] = np.nan
B["sys_op_id"] = np.nan
B["op_date"] = np.nan
B["op_date_rep"] = np.array(df2["osp_date"])
B["op_details"] = np.nan
B["source"] = 'REDCap'

df2 = df1[(df1["othrocsurg_more"] == 'Yes') & (df1["othrocsurg6__eye"].isin(np.array(['OS', 'OD'])))][['maskedID',
 'site',
 'visit_id',
 'osp_date',
 'othrocsurg_more',
 'othrocsurg_morename',]]




A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["op_eye"] = np.nan

A['visit_id'] = np.array(df2["visit_id"])
A['op_age_rep'] = np.nan


A["name"] = np.array(df2["othrocsurg_morename"])
A["OPHID_Text"] = np.nan
A["sys_op_id"] = np.nan
A["op_date"] = np.nan
A["op_date_rep"] = np.array(df2["osp_date"])
A["op_details"] = np.nan
A["source"] = 'REDCap'

A20 = A.copy(deep=True)

df['visit_id'] = np.nan
df['op_age_rep'] = np.nan

H = list(df.columns.values)
FIN = pd.concat([df, A1[H]], ignore_index=True)
FIN = pd.concat([FIN, A2[H]], ignore_index=True)
FIN = pd.concat([FIN, A3[H]], ignore_index=True)
FIN = pd.concat([FIN, A4[H]], ignore_index=True)
FIN = pd.concat([FIN, A5[H]], ignore_index=True)
FIN = pd.concat([FIN, A6[H]], ignore_index=True)
FIN = pd.concat([FIN, A7[H]], ignore_index=True)
FIN = pd.concat([FIN, A8[H]], ignore_index=True)
FIN = pd.concat([FIN, A9[H]], ignore_index=True)
FIN = pd.concat([FIN, A10[H]], ignore_index=True)
FIN = pd.concat([FIN, A11[H]], ignore_index=True)
FIN = pd.concat([FIN, A12[H]], ignore_index=True)
FIN = pd.concat([FIN, A13[H]], ignore_index=True)
FIN = pd.concat([FIN, A14[H]], ignore_index=True)
FIN = pd.concat([FIN, A15[H]], ignore_index=True)
FIN = pd.concat([FIN, A16[H]], ignore_index=True)
FIN = pd.concat([FIN, A17[H]], ignore_index=True)
FIN = pd.concat([FIN, A18[H]], ignore_index=True)
FIN = pd.concat([FIN, A19[H]], ignore_index=True)
FIN = pd.concat([FIN, A20[H]], ignore_index=True)
FIN = pd.concat([FIN, B1[H]], ignore_index=True)
FIN = pd.concat([FIN, B2[H]], ignore_index=True)
FIN = pd.concat([FIN, B3[H]], ignore_index=True)
FIN = pd.concat([FIN, B4[H]], ignore_index=True)
FIN = pd.concat([FIN, B5[H]], ignore_index=True)
FIN = pd.concat([FIN, B6[H]], ignore_index=True)
FIN = pd.concat([FIN, B7[H]], ignore_index=True)
FIN = pd.concat([FIN, B8[H]], ignore_index=True)
FIN = pd.concat([FIN, B9[H]], ignore_index=True)
FIN = pd.concat([FIN, B10[H]], ignore_index=True)
FIN = pd.concat([FIN, B11[H]], ignore_index=True)
FIN = pd.concat([FIN, B12[H]], ignore_index=True)
FIN = pd.concat([FIN, B13[H]], ignore_index=True)
FIN = pd.concat([FIN, B14[H]], ignore_index=True)
FIN = pd.concat([FIN, B15[H]], ignore_index=True)
FIN = pd.concat([FIN, B16[H]], ignore_index=True)
FIN = pd.concat([FIN, B17[H]], ignore_index=True)
FIN = pd.concat([FIN, B18[H]], ignore_index=True)
FIN = pd.concat([FIN, B19[H]], ignore_index=True)
#FIN = pd.concat([FIN, B20[H]], ignore_index=True)

FIN = pd.merge(FIN, ind[["OPHID_Text", "researchid"]], how='left', on='OPHID_Text')





FIN = FIN[~FIN['maskedid'].isnull()]

FIN = pd.merge(FIN, ind[["OPHID_Text_1", "maskedid"]], how='left', on='maskedid')
FIN["OPHID_Text"] = np.where(FIN["OPHID_Text"].isnull(), FIN["OPHID_Text_1"], FIN["OPHID_Text"])

FIN['maskedid'] = np.where(FIN['maskedid'].isnull(), FIN['researchid'], FIN['maskedid'])
FIN['BioBID'] = np.where(FIN['researchid'].isnull(), FIN['maskedid'], FIN['researchid'])
FIN['BioBID'] = np.where( (FIN['researchid'].str.contains('OG')) | (FIN['researchid'].str.contains('ED')), FIN['BioBID'], np.nan)
FIN["site"] = FIN['maskedid'].str[:2]
FIN = FIN[name]
FIN = FIN.fillna('')
FIN["op_date"] = pd.to_datetime(FIN["op_date"],  errors='coerce').dt.strftime('%m/%d/%Y')
FIN["op_date"] = np.where(FIN["op_date"]=='NaT', '', FIN["op_date"])
FIN.to_csv(redcapeyechart + 'Results/' + extension + '_OcularSurgProc_RedEC.csv', encoding='utf-8', index=False)


In [41]:
L1 = [i for i in L_name if '_SmoAlc' in i]
df = pd.read_csv(EyeChart + L1[0], encoding = "ISO-8859-1")

#df = pd.read_csv(EyeChart + '20170322_SmoAlc.csv', encoding = "ISO-8859-1")
df1 = pd.read_csv(redcapeyechart + extension + '_AlcoholTobacco.csv')
df["site"] = df['maskedid'].str[:2]
df["source"] = 'EyeChart'
name = 'maskedid	OPHID_Text	BioBID	site	visit_id	smoke_alc_date	smoke_cur	smoke_ever	at_tob_types___1	at_tob_types___2	at_tob_types___3	at_tob_types___4	at_tob_types___5	at_tob_types___6	at_tob_types___7	smoke_agequit	smoke_atleast100	smoke_last30days	smoke_agestarted	smoke_avgperday	smoke_alc_operator	smoke_non_cig	smoke_non_cig_info	alc_cur	alc_wineperweek	alc_wine24hrs	alc_beerperweek	alc_beer24hrs	alc_hardperweek	alc_hard24hrs	alc_other	alc_otherperweek	alc_other24hrs	source'
name = name.split('\t')
df2 = df1.copy(deep=True)

A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["visit_id"] = np.array(df2["visit_id"])
A["OPHID_Text"] = np.nan
A["smoke_alc_date"] = np.array(df2["at_date"])
A["smoke_cur"] = np.array(df2["at_tob_current_use"])
A["smoke_cur"] = A["smoke_cur"].str.upper()

A["smoke_ever"] = np.array(df2["at_tob_past_use"])
A["smoke_ever"] = A["smoke_ever"].str.upper()

A["smoke_agequit"] = np.array(df2["at_tob_age_quit"])

A["smoke_atleast100"] = np.array(df2["at_tob_100cigs"])
A["smoke_atleast100"] = A["smoke_atleast100"].str.upper()


A["smoke_last30days"] = np.array(df2["at_tob_cigs_30days"])
A["smoke_last30days"] = A["smoke_last30days"].str.upper()

A["smoke_agestarted"] = np.array(df2["at_tob_use_age"])

A["smoke_avgperday"] = np.array(df2["at_tob_daily_cigs"])

A["smoke_alc_operator"] = np.nan
A["smoke_non_cig"] = np.nan
A["smoke_non_cig_info"] = np.nan

A["at_tob_types___1"] = np.array(df2["at_tob_types___1"])
A["at_tob_types___2"] = np.array(df2["at_tob_types___2"])
A["at_tob_types___3"] = np.array(df2["at_tob_types___3"])
A["at_tob_types___4"] = np.array(df2["at_tob_types___4"])
A["at_tob_types___5"] = np.array(df2["at_tob_types___5"])
A["at_tob_types___6"] = np.array(df2["at_tob_types___6"])
A["at_tob_types___7"] = np.array(df2["at_tob_types___7"])


    

A["alc_cur"] = np.array(df2["at_alc_current"])
A["alc_cur"] = A["alc_cur"].str.upper()

A["alc_wineperweek"] = np.array(df2["at_alc_wine_wk"])

A["alc_wine24hrs"] = np.array(df2["at_alc_wine_24hr"])

A["alc_beerperweek"] = np.array(df2["at_alc_beer_wk"])
A["alc_beer24hrs"] = np.array(df2["at_alc_beer_24hr"])

A["alc_hardperweek"] = np.array(df2["at_alc_liquor_wk"])
A["alc_hard24hrs"] = np.array(df2["at_alc_liquor_24hr"])

A["alc_other"] = np.nan
A["alc_otherperweek"] = np.nan
A["alc_other24hrs"] = np.nan
A["source"] = 'REDCap'
df['visit_id'] = np.nan
df["at_tob_types___1"] = np.nan
df["at_tob_types___2"] = np.nan
df["at_tob_types___3"] = np.nan
df["at_tob_types___4"] = np.nan
df["at_tob_types___5"] = np.nan
df["at_tob_types___6"] = np.nan
df["at_tob_types___7"] = np.nan

H = list(df.columns.values)
FIN = pd.concat([df, A[H]], ignore_index=True)
FIN = pd.merge(FIN, ind[["OPHID_Text", "researchid"]], how='left', on='OPHID_Text')





FIN = FIN[~FIN['maskedid'].isnull()]

FIN = pd.merge(FIN, ind[["OPHID_Text_1", "maskedid"]], how='left', on='maskedid')
FIN["OPHID_Text"] = np.where(FIN["OPHID_Text"].isnull(), FIN["OPHID_Text_1"], FIN["OPHID_Text"])

FIN['maskedid'] = np.where(FIN['maskedid'].isnull(), FIN['researchid'], FIN['maskedid'])
FIN['BioBID'] = np.where(FIN['researchid'].isnull(), FIN['maskedid'], FIN['researchid'])
FIN['BioBID'] = np.where( (FIN['researchid'].str.contains('OG')) | (FIN['researchid'].str.contains('ED')), FIN['BioBID'], np.nan)
FIN["site"] = FIN['maskedid'].str[:2]
FIN = FIN[name]

FIN = FIN.fillna('')


FIN.to_csv(redcapeyechart + 'Results/' + extension + '_AlcoholTobacco_RedEC.csv', index=False)

In [42]:
L1 = [i for i in L_name if '_IOP-ICT' in i]
df = pd.read_csv(EyeChart + L1[0], encoding = "ISO-8859-1")

#df = pd.read_csv(EyeChart + '20170322_IOP-ICT.csv', encoding = "ISO-8859-1")
df1 = pd.read_csv(redcapeyechart + extension + '_IOP.csv')
df["site"] = df['maskedid'].str[:2]
df["source"] = 'EyeChart'
name = 'maskedid	OPHID_Text	BioBID	site	visit_id	iopict_eye	iopict_date	iopict_time	iopict_operator	iopict_dilation	iopict_conf	iopict_meas	iop_treated	iopict_comments	source'
name = name.split('\t')
df2 = df1[ (df1["iop_ict"].isin(np.array(['OD', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'iop_date',
 'iop_staff',
 'iop_dilate',
 'iop_treated',
 'iop_ict',
 'iop_ict_time',
 'iop_ict_od_meas',
 'iop_ict_od_conf',
 'iop_ict_os_meas',
 'iop_ist_os_conf','iop_od_treated',]]




A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = np.array(df2["site"])
A["iopict_eye"] = 'OD'

A["visit_id"] = np.array(df2["visit_id"])
A["iop_treated"] = np.array(df2["iop_treated"])


A["iopict_date"] = np.array(df2["iop_date"])
A["OPHID_Text"] = np.nan
A["iopict_time"] = np.array(df2["iop_ict_time"])
A["iopict_operator"] = np.array(df2["iop_staff"])
A["iopict_dilation"] = np.where(df2["iop_dilate"] == 'Yes', 'DILATED', 'UNDILATED')
A["iopict_conf"] = np.array(df2["iop_ict_od_conf"])


A["iopict_meas"] = np.array(df2["iop_ict_od_meas"])
A["iopict_comments"] = np.nan

A["source"] = 'REDCap'

B = A.copy(deep=True)

df2 = df1[ (df1["iop_ict"].isin(np.array(['OU', 'OS'])))][['maskedID',
 'site',
 'visit_id',
 'iop_date',
 'iop_staff',
 'iop_dilate',
 'iop_treated',
 'iop_ict',
 'iop_ict_time',
 'iop_ict_od_meas',
 'iop_ict_od_conf',
 'iop_ict_os_meas',
 'iop_ist_os_conf','iop_od_treated',]]




A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = np.array(df2["site"])
A["iopict_eye"] = 'OS'
A["visit_id"] = np.array(df2["visit_id"])
A["iop_treated"] = np.array(df2["iop_treated"])

A["iopict_date"] = np.array(df2["iop_date"])
A["OPHID_Text"] = np.nan
A["iopict_time"] = np.array(df2["iop_ict_time"])
A["iopict_operator"] = np.array(df2["iop_staff"])
A["iopict_dilation"] = np.where(df2["iop_dilate"] == 'Yes', 'DILATED', 'UNDILATED')
A["iopict_conf"] = np.array(df2["iop_ist_os_conf"])


A["iopict_meas"] = np.array(df2["iop_ict_os_meas"])
A["iopict_comments"] = np.nan

A["source"] = 'REDCap'

df["visit_id"] = np.nan
df["iop_treated"] = np.nan

H = list(df.columns.values)
FIN = pd.concat([df, A[H]], ignore_index=True)
FIN = pd.concat([FIN, B[H]], ignore_index=True)



FIN = pd.merge(FIN, ind[["OPHID_Text", "researchid"]], how='left', on='OPHID_Text')





FIN = FIN[~FIN['maskedid'].isnull()]

FIN = pd.merge(FIN, ind[["OPHID_Text_1", "maskedid"]], how='left', on='maskedid')
FIN["OPHID_Text"] = np.where(FIN["OPHID_Text"].isnull(), FIN["OPHID_Text_1"], FIN["OPHID_Text"])

FIN['maskedid'] = np.where(FIN['maskedid'].isnull(), FIN['researchid'], FIN['maskedid'])
FIN['BioBID'] = np.where(FIN['researchid'].isnull(), FIN['maskedid'], FIN['researchid'])
FIN['BioBID'] = np.where( (FIN['researchid'].str.contains('OG')) | (FIN['researchid'].str.contains('ED')), FIN['BioBID'], np.nan)
FIN["site"] = FIN['maskedid'].str[:2]
FIN = FIN[name]

FIN = FIN.fillna('')

FIN.to_csv(redcapeyechart + 'Results/' + extension +'_IOP_ICT_RedEC.csv', index=False)


In [43]:
def check_duplicates(df, col):
    return df[~df[col].duplicated()]

In [85]:
L1 = [i for i in L_name if '_OcularMedications_wGlaucomaMeds' in i]
df = pd.read_csv(EyeChart + L1[0], encoding = "ISO-8859-1")

#df = pd.read_csv(EyeChart + '20170322_OcularMedications_wGlaucomaMeds.csv', encoding = "ISO-8859-1")
name = 'maskedid	OPHID_Text	BioBID	site	visit_id	om_rep	om_eye	sys_om_id	name	glaucoma_med	om_date_rep	om_date	om_age_rep	om_details	source'
name = name.split('\t')

df1 = pd.read_csv(redcapeyechart + extension +'_OcularMedications.csv')

df1['ocmed1_eye'] = np.where(df1["ocmed1_bool"] == 'No', 'OU', df1['ocmed1_eye'])
df1['ocmed2_eye'] = np.where(df1["ocmed2_bool"] == 'No', 'OU', df1['ocmed2_eye'])
df1['ocmed3_eye'] = np.where(df1["ocmed3_bool"] == 'No', 'OU', df1['ocmed3_eye'])
df1['ocmed4_eye'] = np.where(df1["ocmed4_bool"] == 'No', 'OU', df1['ocmed4_eye'])


df10 = pd.read_csv(redcapeyechart + 'glau_med.csv')#, error_bad_lines=False)
df10 = check_duplicates(df10, "name")
Glau = df10[df10['cGlaucoma.Med'] == 'Yes'][["name"]].reset_index(drop=True)
Glau["glaucoma_med"] = 'Yes'

df["site"] = df['maskedid'].str[:2]
df["source"] = 'EyeChart'



df2 = df1[ ((df1["ocmed1_bool"] == 'Yes')| (df1["ocmed1_bool"] == 'No')) & (df1["ocmed1_eye"].isin(np.array(['OD', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'ocmed_date',
 'ocmed1',
 'ocmed1_other',
 'ocmed1_eye', 'ocmed1_bool',
 'ocmed1_startdate',
 'ocmed1_startdateOD',
 'ocmed1_startageOD',
 'ocmed1_startdateOS',
 'ocmed1_startageOS']]




A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]

A["om_eye"] = 'OD'
A["name"] = np.where(df2["ocmed1"].isnull(), 'No Current Ocular Medications', df2["ocmed1"])
A["OPHID_Text"] = np.nan
A["sys_om_id"] = np.nan
A['om_rep'] = np.array(df2["ocmed1_bool"])
A['visit_id'] = np.array(df2["visit_id"])
A["om_age_rep"] = np.array(df2["ocmed1_startageOD"])

A["om_date"] = np.array(df2["ocmed1_startdateOD"])
A["om_date"] = A["om_date"].fillna("UNKNOWN")
A["om_date_rep"] = np.array(df2["ocmed_date"])
A["om_details"] = np.array(df2["ocmed1_other"])
A["source"] = 'REDCap'

A["name_temp"] = A["name"]
A.loc[A["name_temp"] == "OTHER (Specify)", "name_temp"] = np.array(A["om_details"][A["om_details"].notnull()])

A = pd.merge(A,Glau, how='left', left_on = "name_temp", right_on='name')
A = A.rename(columns={"name_x":"name"})
A["glaucoma_med"] = A["glaucoma_med"].fillna('No')


#A["name"] = np.array(df2["ocmed1"])


B = A.copy(deep=True)
df2 = df1[ ((df1["ocmed1_bool"] == 'Yes')| (df1["ocmed1_bool"] == 'No')) & (df1["ocmed1_eye"].isin(np.array(['OS', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'ocmed_date',
 'ocmed1',
 'ocmed1_other',
 'ocmed1_eye', 'ocmed1_bool',
 'ocmed1_startdate',
 'ocmed1_startdateOD',
 'ocmed1_startageOD',
 'ocmed1_startdateOS',
 'ocmed1_startageOS']]




A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["name"] = np.where(df2["ocmed1"].isnull(), 'No Current Ocular Medications', df2["ocmed1"])
A['visit_id'] = np.array(df2["visit_id"])
A['om_rep'] = np.array(df2["ocmed1_bool"])
A["om_age_rep"] = np.array(df2["ocmed1_startageOS"])

A["om_eye"] = 'OS'
A["name"] = np.array(df2["ocmed1"])
A["OPHID_Text"] = np.nan
A["sys_om_id"] = np.nan

A["om_date"] = np.array(df2["ocmed1_startdateOS"])
A["om_date"] = A["om_date"].fillna("UNKNOWN")
A["om_date_rep"] = np.array(df2["ocmed_date"])

A["om_details"] = np.array(df2["ocmed1_other"])
A["source"] = 'REDCap'

A["name_temp"] = A["name"]
A.loc[A["name_temp"] == "OTHER (Specify)", "name_temp"] = np.array(A["om_details"][A["om_details"].notnull()])

A = pd.merge(A,Glau, how='left', left_on = "name_temp", right_on='name')
A = A.rename(columns={"name_x":"name"})
A["glaucoma_med"] = A["glaucoma_med"].fillna('No')

A1 = A.copy(deep=True)
B1 = B.copy(deep=True)





df2 = df1[ (df1["ocmed2_bool"] == 'Yes') & (df1["ocmed2_eye"].isin(np.array(['OD', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'ocmed_date',
 'ocmed2',

 'ocmed2_other',
 'ocmed2_eye', 'ocmed2_bool',
 'ocmed2_startdate',
 'ocmed2_startdateOD',
 'ocmed2_startageOD',
 'ocmed2_startdateOS',
 'ocmed2_startageOS',]]




A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["om_eye"] = 'OD'
A["name"] = np.where(df2["ocmed2"].isnull(), 'No Current Ocular Medications', df2["ocmed2"])
A["OPHID_Text"] = np.nan
A["sys_om_id"] = np.nan

A['visit_id'] = np.array(df2["visit_id"])
A['om_rep'] = np.array(df2["ocmed2_bool"])
A["om_age_rep"] = np.array(df2["ocmed2_startageOD"])

A["om_date"] = np.array(df2["ocmed2_startdateOD"])
A["om_date"] = A["om_date"].fillna("UNKNOWN")
A["om_date_rep"] = np.array(df2["ocmed_date"])
A["om_details"] = np.array(df2["ocmed2_other"])
A["source"] = 'REDCap'

A["name_temp"] = A["name"]
A.loc[A["name_temp"] == "OTHER (Specify)", "name_temp"] = np.array(A["om_details"][A["om_details"].notnull()])

A = pd.merge(A,Glau, how='left', left_on = "name_temp", right_on='name')
A = A.rename(columns={"name_x":"name"})
A["glaucoma_med"] = A["glaucoma_med"].fillna('No')


B = A.copy(deep=True)
df2 = df1[ (df1["ocmed2_bool"] == 'Yes') & (df1["ocmed2_eye"].isin(np.array(['OS', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'ocmed_date',
 'ocmed2',

 'ocmed2_other',
 'ocmed2_eye', 'ocmed2_bool',
 'ocmed2_startdate',
 'ocmed2_startdateOD',
 'ocmed2_startageOD',
 'ocmed2_startdateOS',
 'ocmed2_startageOS',]]




A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["om_eye"] = 'OS'
A["name"] = np.where(df2["ocmed2"].isnull(), 'No Current Ocular Medications', df2["ocmed2"])
A["OPHID_Text"] = np.nan
A["sys_om_id"] = np.nan
A['om_rep'] = np.array(df2["ocmed2_bool"])
A['visit_id'] = np.array(df2["visit_id"])
A["om_age_rep"] = np.array(df2["ocmed2_startageOS"])

A["om_date"] = np.array(df2["ocmed2_startdateOS"])
A["om_date"] = A["om_date"].fillna("UNKNOWN")
A["om_date_rep"] = np.array(df2["ocmed_date"])
A["om_details"] = np.array(df2["ocmed2_other"])
A["source"] = 'REDCap'

A["name_temp"] = A["name"]
A.loc[A["name_temp"] == "OTHER (Specify)", "name_temp"] = np.array(A["om_details"][A["om_details"].notnull()])

A = pd.merge(A,Glau, how='left', left_on = "name_temp", right_on='name')
A = A.rename(columns={"name_x":"name"})
A["glaucoma_med"] = A["glaucoma_med"].fillna('No')

A2 = A.copy(deep=True)
B2 = B.copy(deep=True)


df2 = df1[ (df1["ocmed3_bool"] == 'Yes') & (df1["ocmed3_eye"].isin(np.array(['OD', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'ocmed_date',
 'ocmed3',

 'ocmed3_other',
 'ocmed3_eye', 'ocmed3_bool',
 'ocmed3_startdate',
 'ocmed3_startdateOD',
 'ocmed3_startageOD',
 'ocmed3_startdateOS',
 'ocmed3_startageOS',]]




A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["om_eye"] = 'OD'
A["name"] = np.where(df2["ocmed3"].isnull(), 'No Current Ocular Medications', df2["ocmed3"])
A["OPHID_Text"] = np.nan
A["sys_om_id"] = np.nan
A['om_rep'] = np.array(df2["ocmed3_bool"])
A['visit_id'] = np.array(df2["visit_id"])
A["om_age_rep"] = np.array(df2["ocmed3_startageOD"])


A["om_date"] = np.array(df2["ocmed3_startdateOD"])
A["om_date"] = A["om_date"].fillna("UNKNOWN")
A["om_date_rep"] = np.array(df2["ocmed_date"])
A["om_details"] = np.array(df2["ocmed3_other"])
A["source"] = 'REDCap'

A["name_temp"] = A["name"]
A.loc[A["name_temp"] == "OTHER (Specify)", "name_temp"] = np.array(A["om_details"][A["om_details"].notnull()])

A = pd.merge(A,Glau, how='left', left_on = "name_temp", right_on='name')
A = A.rename(columns={"name_x":"name"})
A["glaucoma_med"] = A["glaucoma_med"].fillna('No')


B = A.copy(deep=True)
df2 = df1[ (df1["ocmed3_bool"] == 'Yes') & (df1["ocmed3_eye"].isin(np.array(['OS', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'ocmed_date',
 'ocmed3',
 'ocmed3_other',
 'ocmed3_eye', 'ocmed3_bool',
 'ocmed3_startdate',
 'ocmed3_startdateOD',
 'ocmed3_startageOD',
 'ocmed3_startdateOS',
 'ocmed3_startageOS',]]




A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["om_eye"] = 'OS'
A["name"] = np.where(df2["ocmed3"].isnull(), 'No Current Ocular Medications', df2["ocmed3"])
A["OPHID_Text"] = np.nan
A["sys_om_id"] = np.nan

A['visit_id'] = np.array(df2["visit_id"])
A['om_rep'] = np.array(df2["ocmed3_bool"])
A["om_age_rep"] = np.array(df2["ocmed3_startageOS"])


A["om_date"] = np.array(df2["ocmed3_startdateOS"])
A["om_date"] = A["om_date"].fillna("UNKNOWN")
A["om_date_rep"] = np.array(df2["ocmed_date"])
A["om_details"] = np.array(df2["ocmed3_other"])
A["source"] = 'REDCap'

A["name_temp"] = A["name"]
A.loc[A["name_temp"] == "OTHER (Specify)", "name_temp"] = np.array(A["om_details"][A["om_details"].notnull()])

A = pd.merge(A,Glau, how='left', left_on = "name_temp", right_on='name')
A = A.rename(columns={"name_x":"name"})
A["glaucoma_med"] = A["glaucoma_med"].fillna('No')

A3 = A.copy(deep=True)
B3 = B.copy(deep=True)



df2 = df1[ (df1["ocmed4_bool"] == 'Yes') & (df1["ocmed4_eye"].isin(np.array(['OD', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'ocmed_date',
 'ocmed4',
 'ocmed4_other',
 'ocmed4_eye', 'ocmed4_bool',
 'ocmed4_startdate',
 'ocmed4_startdateOD',
 'ocmed4_startageOD',
 'ocmed4_startdateOS',
 'ocmed4_startageOS',]]




A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["om_eye"] = 'OD'
A["name"] = np.where(df2["ocmed4"].isnull(), 'No Current Ocular Medications', df2["ocmed4"])
A["OPHID_Text"] = np.nan
A["sys_om_id"] = np.nan
A['om_rep'] = np.array(df2["ocmed4_bool"])
A['visit_id'] = np.array(df2["visit_id"])
A["om_age_rep"] = np.array(df2["ocmed4_startageOD"])


A["om_date"] = np.array(df2["ocmed4_startdateOD"])
A["om_date"] = A["om_date"].fillna("UNKNOWN")
A["om_date_rep"] = np.array(df2["ocmed_date"])
A["om_details"] = np.array(df2["ocmed4_other"])
A["source"] = 'REDCap'


A["name_temp"] = A["name"]
A.loc[A["name_temp"] == "OTHER (Specify)", "name_temp"] = np.array(A["om_details"][A["om_details"].notnull()])

A = pd.merge(A,Glau, how='left', left_on = "name_temp", right_on='name')
A = A.rename(columns={"name_x":"name"})
A["glaucoma_med"] = A["glaucoma_med"].fillna('No')


B = A.copy(deep=True)
df2 = df1[ (df1["ocmed4_bool"] == 'Yes') & (df1["ocmed4_eye"].isin(np.array(['OS', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'ocmed_date',
 'ocmed4',
 'ocmed4_other',
 'ocmed4_eye', 'ocmed4_bool',
 'ocmed4_startdate',
 'ocmed4_startdateOD',
 'ocmed4_startageOD',
 'ocmed4_startdateOS',
 'ocmed4_startageOS',]]




A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["om_eye"] = 'OS'
A["name"] = np.array(df2["ocmed4"])
A["OPHID_Text"] = np.nan
A["sys_om_id"] = np.nan
A['om_rep'] = np.array(df2["ocmed4_bool"])
A['visit_id'] = np.array(df2["visit_id"])
A["om_age_rep"] = np.array(df2["ocmed4_startageOS"])


A["om_date"] = np.array(df2["ocmed4_startdateOS"])
A["om_date"] = A["om_date"].fillna("UNKNOWN")
A["om_date_rep"] = np.array(df2["ocmed_date"])
A["om_details"] = np.array(df2["ocmed4_other"])
A["source"] = 'REDCap'

A["name_temp"] = A["name"]
A.loc[A["name_temp"] == "OTHER (Specify)", "name_temp"] = np.array(A["om_details"][A["om_details"].notnull()])

A = pd.merge(A,Glau, how='left', left_on = "name_temp", right_on='name')
A = A.rename(columns={"name_x":"name"})
A["glaucoma_med"] = A["glaucoma_med"].fillna('No')


A4 = A.copy(deep=True)
B4 = B.copy(deep=True)

df['om_rep'] = np.nan
df['visit_id'] = np.nan
df["om_age_rep"] = np.nan

H = list(df.columns.values)
FIN = pd.concat([df, A1[H]], ignore_index=True)
FIN = pd.concat([FIN, B1[H]], ignore_index=True)
FIN = pd.concat([FIN, A2[H]], ignore_index=True)
FIN = pd.concat([FIN, B2[H]], ignore_index=True)
FIN = pd.concat([FIN, A3[H]], ignore_index=True)
FIN = pd.concat([FIN, B3[H]], ignore_index=True)
FIN = pd.concat([FIN, A4[H]], ignore_index=True)
FIN = pd.concat([FIN, B4[H]], ignore_index=True)

FIN = pd.merge(FIN, ind[["OPHID_Text", "researchid"]], how='left', on='OPHID_Text')


FIN = FIN[~FIN['maskedid'].isnull()]

FIN = pd.merge(FIN, ind[["OPHID_Text_1", "maskedid"]], how='left', on='maskedid')
FIN["OPHID_Text"] = np.where(FIN["OPHID_Text"].isnull(), FIN["OPHID_Text_1"], FIN["OPHID_Text"])

FIN['maskedid'] = np.where(FIN['maskedid'].isnull(), FIN['researchid'], FIN['maskedid'])
FIN['BioBID'] = np.where(FIN['researchid'].isnull(), FIN['maskedid'], FIN['researchid'])
FIN['BioBID'] = np.where( (FIN['researchid'].str.contains('OG')) | (FIN['researchid'].str.contains('ED')), FIN['BioBID'], np.nan)
FIN["site"] = FIN['maskedid'].str[:2]
FIN = FIN.drop('glaucoma_med', 1)

idx = FIN.loc[FIN["name"] == "OTHER (Specify)", "name"].index

FIN["name_temp"] = FIN["name"]
FIN.loc[FIN["name_temp"] == "OTHER (Specify)", "name_temp"] = np.array(FIN["om_details"].iloc[idx].str.strip())

FIN = pd.merge(FIN, Glau, how='left', left_on = "name_temp", right_on='name')
FIN = FIN.rename(columns={"name_x":"name"})

FIN["glaucoma_med"] = FIN["glaucoma_med"].fillna('No')
FIN = FIN[name]

FIN = FIN.fillna('')
FIN["om_date"] = np.where(FIN["om_date"] == 'UNKOWN', '', FIN["om_date"])
FIN["om_date"] = pd.to_datetime(FIN["om_date"],  errors='coerce').dt.strftime('%m/%d/%Y')
FIN["om_date"] = np.where(FIN["om_date"]=='NaT', 'UNKOWN', FIN["om_date"])

FIN['glaucoma_med'] = np.where(FIN['glaucoma_med']=='?', 'No', FIN['glaucoma_med'])
FIN.to_csv(redcapeyechart + 'Results/' + extension +'_OcularMedications_RedEC.csv', index=False)


/var/folders/7m/n2zn61593k18rvyz907d2skc0000gn/T/ipykernel_30504/211649229.py:409: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  FIN = FIN.drop('glaucoma_med', 1)


In [45]:
df2 = df1[ ((df1["ocmed1_bool"] == 'Yes')| (df1["ocmed1_bool"] == 'No')) & (df1["ocmed1_eye"].isin(np.array(['OD', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'ocmed_date',
 'ocmed1',
 'ocmed1_other',
 'ocmed1_eye', 'ocmed1_bool',
 'ocmed1_startdate',
 'ocmed1_startdateOD',
 'ocmed1_startageOD',
 'ocmed1_startdateOS',
 'ocmed1_startageOS']]




A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]

A["om_eye"] = 'OD'
A["name"] = np.where(df2["ocmed1"].isnull(), 'No Current Ocular Medications', df2["ocmed1"])
A["OPHID_Text"] = np.nan
A["sys_om_id"] = np.nan
A['om_rep'] = np.array(df2["ocmed1_bool"])
A['visit_id'] = np.array(df2["visit_id"])
A["om_age_rep"] = np.array(df2["ocmed1_startageOD"])
A = pd.merge(A,Glau, how='left', on= 'name')
A["glaucoma_med"] = A["glaucoma_med"].fillna('No')

#A["name"] = np.array(df2["ocmed1"])

A["om_date"] = np.array(df2["ocmed1_startdateOD"])



A["om_date"] = A["om_date"].fillna("UNKNOWN")
A["om_date_rep"] = np.array(df2["ocmed_date"])


In [46]:
##KEEP

L1 = [i for i in L_name if '_OcularConditions' in i]
df = pd.read_csv(EyeChart + L1[0], encoding = "ISO-8859-1")
#df = pd.read_csv(EyeChart + '20170322_OcularConditions.csv', encoding = "ISO-8859-1")

name = 'maskedid	OPHID_Text	BioBID	site	visit_id	oc_eye	sys_oc_id	name	oc_date_rep	oc_date_dx	oc_agedx	oc_tx	oc_tx1stdate	oc_tx1stage	oc_details	source'
name = name.split('\t')
df["visit_id"] = np.nan
df['oc_agedx'] = np.nan

df1 = pd.read_csv(redcapeyechart + extension + '_OcularConditions.csv')
df["site"] = df['maskedid'].str[:2]
df["source"] = 'EyeChart'





df2 = df1[ (df1["myopia"] == 'Yes') & (df1["myopiaeye"].isin(np.array(['OD', 'OU'])))]
[['maskedID',
 'site',
 'visit_id',
 'oc_date',
 'myopia',
 'myopiaeye',
 'myopia_datedx',
 'myopia_datedxOD',
 'myopia_agedxOD',
 'myopia_agedxOS',
 'myopia_datedxOS',
 'myopia_tx',
 'myopia_tx1stdate',
 'myopia_tx1stdateOD',
 'myopia_tx1stageOD',
 'myopia_tx1stdateOS',
 'myopia_tx1stageOS',]]

VAR = 'myopia'
df2[VAR + "_tx1stdate"] = df2[VAR + "_tx1stdate"].astype('str')
df2[VAR + "_datedx"] = df2[VAR + "_datedx"].astype('str')
A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["oc_eye"] = 'OD'
A["name"] = 'Myopia'
A["OPHID_Text"] = np.nan
A["sys_oc_id"] = 238
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='Yes', df2[VAR + "_datedxOD"], np.nan)
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='No', 'UNKNOWN', A["oc_date_dx"])
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='nan', 'Not Reported', A["oc_date_dx"])
A["oc_date_rep"] = np.array(df2["oc_date"])
A["oc_details"] = np.nan

A["visit_id"] = np.array(df2["visit_id"])
A["oc_agedx"] = np.array(df2[VAR + "_agedxOD"])
A["oc_tx"] = np.array(df2[VAR + "_tx"])
A["oc_tx"]  = A["oc_tx"] .fillna('Not Reported')

A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='Yes', df2[VAR + "_tx1stdateOD"], np.nan)
A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='No', 'UNKNOWN', A["oc_tx1stdate"])
A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='nan', 'Not Reported', A["oc_tx1stdate"])

A["oc_tx1stage"] = np.array(df2[VAR + "_tx1stageOD"])


A["source"] = 'REDCap'

df2 = df1[ (df1["myopia"] == 'Yes') & (df1["myopiaeye"].isin(np.array(['OS', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'oc_date',
 'myopia',
 'myopiaeye',
 'myopia_datedx',
 'myopia_datedxOD',
 'myopia_agedxOD',
  'myopia_agedxOS',
 'myopia_datedxOS',
 'myopia_tx',
 'myopia_tx1stdate',
 'myopia_tx1stdateOD',
 'myopia_tx1stageOD',
 'myopia_tx1stdateOS',
 'myopia_tx1stageOS',]]

df2[VAR + "_tx1stdate"] = df2[VAR + "_tx1stdate"].astype('str')
df2[VAR + "_datedx"] = df2[VAR + "_datedx"].astype('str')


B = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
B["site"] = df2["site"]
B["oc_eye"] = 'OS'
B["name"] = 'Myopia'
B["OPHID_Text"] = np.nan
B["sys_oc_id"] = 238
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='Yes', df2[VAR + "_datedxOS"], np.nan)
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='No', 'UNKNOWN', B["oc_date_dx"])
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='nan', 'Not Reported', B["oc_date_dx"])
B["oc_date_rep"] = np.array(df2["oc_date"])
B["oc_details"] = np.nan

B["visit_id"] = np.array(df2["visit_id"])
B["oc_agedx"] = np.array(df2[VAR + "_agedxOS"])
B["oc_tx"] = np.array(df2[VAR + "_tx"])
B["oc_tx"]  = B["oc_tx"] .fillna('Not Reported')

B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='Yes', df2[VAR + "_tx1stdateOS"], np.nan)
B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='No', 'UNKNOWN', B["oc_tx1stdate"])
B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='nan', 'Not Reported', B["oc_tx1stdate"])

B["oc_tx1stage"] = np.array(df2[VAR + "_tx1stageOS"])

B["source"] = 'REDCap'

A1 = A.copy(deep=True)
B1 = B.copy(deep=True)

df2 = df1[ (df1["presbyopia"] == 'Yes') & (df1["presbyopiaeye"].isin(np.array(['OD', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'oc_date',
 'presbyopia',
 'presbyopiaeye',
 'presbyopia_datedx',
 'presbyopia_datedxOD',
 'presbyopia_agedxOD',
 'presbyopia_datedxOS',
 'presbyopia_tx',
 'presbyopia_tx1stdate',
 'presbyopia_tx1stdateOD',
 'presbyopia_tx1stageOD',
 'presbyopia_tx1stdateOS',
 'presbyopia_tx1stageOS',]]


VAR = 'presbyopia'
df2[VAR + "_tx1stdate"] = df2[VAR + "_tx1stdate"].astype('str')
df2[VAR + "_datedx"] = df2[VAR + "_datedx"].astype('str')
A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["oc_eye"] = 'OD'
A["name"] = 'Presbyopia'
A["OPHID_Text"] = np.nan
A["sys_oc_id"] = 238
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='Yes', df2[VAR + "_datedxOD"], np.nan)
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='No', 'UNKNOWN', A["oc_date_dx"])
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='nan', 'Not Reported', A["oc_date_dx"])
A["oc_date_rep"] = np.array(df2["oc_date"])
A["oc_details"] = np.nan

A["visit_id"] = np.array(df2["visit_id"])
A["oc_agedx"] = np.array(df2[VAR + "_agedxOD"])
A["oc_tx"] = np.array(df2[VAR + "_tx"])
A["oc_tx"]  = A["oc_tx"] .fillna('Not Reported')

A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='Yes', df2[VAR + "_tx1stdateOD"], np.nan)
A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='No', 'UNKNOWN', A["oc_tx1stdate"])
A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='nan', 'Not Reported', A["oc_tx1stdate"])

A["oc_tx1stage"] = np.array(df2[VAR + "_tx1stageOD"])

A["source"] = 'REDCap'

df2 = df1[ (df1["presbyopia"] == 'Yes') & (df1["presbyopiaeye"].isin(np.array(['OS', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'oc_date',
 'presbyopia',
 'presbyopiaeye',
 'presbyopia_datedx',
 'presbyopia_datedxOD',
 'presbyopia_agedxOD',
 'presbyopia_agedxOS',
 'presbyopia_datedxOS',
 'presbyopia_tx',
 'presbyopia_tx1stdate',
 'presbyopia_tx1stdateOD',
 'presbyopia_tx1stageOD',
 'presbyopia_tx1stdateOS',
 'presbyopia_tx1stageOS',]]



df2[VAR + "_tx1stdate"] = df2[VAR + "_tx1stdate"].astype('str')
df2[VAR + "_datedx"] = df2[VAR + "_datedx"].astype('str')
B = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
B["site"] = df2["site"]
B["oc_eye"] = 'OS'
B["name"] = 'Presbyopia'
B["OPHID_Text"] = np.nan
B["sys_oc_id"] = 238
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='Yes', df2[VAR + "_datedxOS"], np.nan)
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='No', 'UNKNOWN', B["oc_date_dx"])
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='nan', 'Not Reported', B["oc_date_dx"])
B["oc_date_rep"] = np.array(df2["oc_date"])
B["oc_details"] = np.nan

B["visit_id"] = np.array(df2["visit_id"])
B["oc_agedx"] = np.array(df2[VAR + "_agedxOS"])
B["oc_tx"] = np.array(df2[VAR + "_tx"])
B["oc_tx"]  = B["oc_tx"] .fillna('Not Reported')

B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='Yes', df2[VAR + "_tx1stdateOS"], np.nan)
B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='No', 'UNKNOWN', B["oc_tx1stdate"])
B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='nan', 'Not Reported', B["oc_tx1stdate"])

B["oc_tx1stage"] = np.array(df2[VAR + "_tx1stageOS"])

B["source"] = 'REDCap'

A2 = A.copy(deep=True)
B2 = B.copy(deep=True)

df2 = df1[ (df1["corndyst"] == 'Yes') & (df1["corndysteye"].isin(np.array(['OD', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'oc_date',
 'corndyst',
 'corndysteye',
 'corndyst_datedx',
 'corndyst_datedxOD',
 'corndyst_agedxOD',
 'corndyst_datedxOS',
 'corndyst_tx',
 'corndyst_tx1stdate',
 'corndyst_tx1stdateOD',
 'corndyst_tx1stageOD',
 'corndyst_tx1stdateOS',
 'corndyst_tx1stageOS',]]


VAR = 'corndyst'
df2[VAR + "_tx1stdate"] = df2[VAR + "_tx1stdate"].astype('str')
df2[VAR + "_datedx"] = df2[VAR + "_datedx"].astype('str')
A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["oc_eye"] = 'OD'
A["name"] = 'Corneal Dystrophy'
A["OPHID_Text"] = np.nan
A["sys_oc_id"] = 100
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='Yes', df2[VAR + "_datedxOD"], np.nan)
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='No', 'UNKNOWN', A["oc_date_dx"])
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='nan', 'Not Reported', A["oc_date_dx"])
A["oc_date_rep"] = np.array(df2["oc_date"])
A["oc_details"] = np.nan

A["visit_id"] = np.array(df2["visit_id"])
A["oc_agedx"] = np.array(df2[VAR + "_agedxOD"])
A["oc_tx"] = np.array(df2[VAR + "_tx"])
A["oc_tx"]  = A["oc_tx"] .fillna('Not Reported')

A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='Yes', df2[VAR + "_tx1stdateOD"], np.nan)
A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='No', 'UNKNOWN', A["oc_tx1stdate"])
A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='nan', 'Not Reported', A["oc_tx1stdate"])

A["oc_tx1stage"] = np.array(df2[VAR + "_tx1stageOD"])


A["source"] = 'REDCap'

df2 = df1[ (df1["corndyst"] == 'Yes') & (df1["corndysteye"].isin(np.array(['OS', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'oc_date',
 'corndyst',
 'corndysteye',
 'corndyst_datedx',
 'corndyst_datedxOD',
 'corndyst_agedxOD',
  'corndyst_agedxOS',
 'corndyst_datedxOS',

 'corndyst_tx',
 'corndyst_tx1stdate',
 'corndyst_tx1stdateOD',
 'corndyst_tx1stageOD',
 'corndyst_tx1stdateOS',
 'corndyst_tx1stageOS',]]


df2[VAR + "_tx1stdate"] = df2[VAR + "_tx1stdate"].astype('str')
df2[VAR + "_datedx"] = df2[VAR + "_datedx"].astype('str')

B = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
B["site"] = df2["site"]
B["oc_eye"] = 'OS'
B["name"] = 'Corneal Dystrophy'
B["OPHID_Text"] = np.nan
B["sys_oc_id"] = 100
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='Yes', df2[VAR + "_datedxOS"], np.nan)
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='No', 'UNKNOWN', B["oc_date_dx"])
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='nan', 'Not Reported', B["oc_date_dx"])
B["oc_date_rep"] = np.array(df2["oc_date"])
B["oc_details"] = np.nan

B["visit_id"] = np.array(df2["visit_id"])
B["oc_agedx"] = np.array(df2[VAR + "_agedxOS"])
B["oc_tx"] = np.array(df2[VAR + "_tx"])
B["oc_tx"]  = B["oc_tx"] .fillna('Not Reported')

B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='Yes', df2[VAR + "_tx1stdateOS"], np.nan)
B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='No', 'UNKNOWN', B["oc_tx1stdate"])
B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='nan', 'Not Reported', B["oc_tx1stdate"])

B["oc_tx1stage"] = np.array(df2[VAR + "_tx1stageOS"])


B["source"] = 'REDCap'

A3 = A.copy(deep=True)
B3 = B.copy(deep=True)


df2 = df1[ (df1["dryeye"] == 'Yes') & (df1["dryeyeeye"].isin(np.array(['OD', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'oc_date',
 'dryeye',
 'dryeyeeye',
 'dryeye_datedx',
 'dryeye_datedxOD',
 'dryeye_agedxOD',

 'dryeye_datedxOS',

 'dryeye_tx',
 'dryeye_tx1stdate',
 'dryeye_tx1stdateOD',
 'dryeye_tx1stageOD',
 'dryeye_tx1stdateOS',
 'dryeye_tx1stageOS',]]

VAR = 'dryeye'
df2[VAR + "_tx1stdate"] = df2[VAR + "_tx1stdate"].astype('str')
df2[VAR + "_datedx"] = df2[VAR + "_datedx"].astype('str')

A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["oc_eye"] = 'OD'
A["name"] = 'Dry Eyes'
A["OPHID_Text"] = np.nan
A["sys_oc_id"] = 218
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='Yes', df2[VAR + "_datedxOD"], np.nan)
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='No', 'UNKNOWN', A["oc_date_dx"])
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='nan', 'Not Reported', A["oc_date_dx"])
A["oc_date_rep"] = np.array(df2["oc_date"])
A["oc_details"] = np.nan

A["visit_id"] = np.array(df2["visit_id"])
A["oc_agedx"] = np.array(df2[VAR + "_agedxOD"])
A["oc_tx"] = np.array(df2[VAR + "_tx"])
A["oc_tx"]  = A["oc_tx"] .fillna('Not Reported')

A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='Yes', df2[VAR + "_tx1stdateOD"], np.nan)
A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='No', 'UNKNOWN', A["oc_tx1stdate"])
A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='nan', 'Not Reported', A["oc_tx1stdate"])

A["oc_tx1stage"] = np.array(df2[VAR + "_tx1stageOD"])

A["source"] = 'REDCap'

df2 = df1[ (df1["dryeye"] == 'Yes') & (df1["dryeyeeye"].isin(np.array(['OS', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'oc_date',
 'dryeye',
 'dryeyeeye',
 'dryeye_datedx',
 'dryeye_datedxOD',
 'dryeye_agedxOD',
 'dryeye_agedxOS',
 'dryeye_datedxOS',

 'dryeye_tx',
 'dryeye_tx1stdate',
 'dryeye_tx1stdateOD',
 'dryeye_tx1stageOD',
 'dryeye_tx1stdateOS',
 'dryeye_tx1stageOS',]]

df2[VAR + "_tx1stdate"] = df2[VAR + "_tx1stdate"].astype('str')
df2[VAR + "_datedx"] = df2[VAR + "_datedx"].astype('str')


B = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
B["site"] = df2["site"]
B["oc_eye"] = 'OS'
B["name"] = 'Dry Eyes'
B["OPHID_Text"] = np.nan
B["sys_oc_id"] = 218
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='Yes', df2[VAR + "_datedxOS"], np.nan)
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='No', 'UNKNOWN', B["oc_date_dx"])
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='nan', 'Not Reported', B["oc_date_dx"])
B["oc_date_rep"] = np.array(df2["oc_date"])
B["oc_details"] = np.nan

B["visit_id"] = np.array(df2["visit_id"])
B["oc_agedx"] = np.array(df2[VAR + "_agedxOS"])
B["oc_tx"] = np.array(df2[VAR + "_tx"])
B["oc_tx"]  = B["oc_tx"] .fillna('Not Reported')

B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='Yes', df2[VAR + "_tx1stdateOS"], np.nan)
B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='No', 'UNKNOWN', B["oc_tx1stdate"])
B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='nan', 'Not Reported', B["oc_tx1stdate"])

B["oc_tx1stage"] = np.array(df2[VAR + "_tx1stageOS"])


B["source"] = 'REDCap'

A4 = A.copy(deep=True)
B4 = B.copy(deep=True)

df2 = df1[ (df1["cataract"] == 'Yes') & (df1["cataracteye"].isin(np.array(['OD', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'oc_date',
 'cataract',
 'cataracteye',
 'cataract_datedx',
 'cataract_datedxOD',
 'cataract_agedxOD',

 'cataract_datedxOS',

 'cataract_tx',
 'cataract_tx1stdate',
 'cataract_tx1stdateOD',
 'cataract_tx1stageOD',
 'cataract_tx1stdateOS',
 'cataract_tx1stageOS',]]



VAR = 'cataract'
df2[VAR + "_tx1stdate"] = df2[VAR + "_tx1stdate"].astype('str')
df2[VAR + "_datedx"] = df2[VAR + "_datedx"].astype('str')
A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["oc_eye"] = 'OD'
A["name"] = 'Cataract'
A["OPHID_Text"] = np.nan
A["sys_oc_id"] = 15
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='Yes', df2[VAR + "_datedxOD"], np.nan)
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='No', 'UNKNOWN', A["oc_date_dx"])
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='nan', 'Not Reported', A["oc_date_dx"])
A["oc_date_rep"] = np.array(df2["oc_date"])
A["oc_details"] = np.nan

A["visit_id"] = np.array(df2["visit_id"])
A["oc_agedx"] = np.array(df2[VAR + "_agedxOD"])
A["oc_tx"] = np.array(df2[VAR + "_tx"])
A["oc_tx"]  = A["oc_tx"] .fillna('Not Reported')

A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='Yes', df2[VAR + "_tx1stdateOD"], np.nan)
A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='No', 'UNKNOWN', A["oc_tx1stdate"])
A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='nan', 'Not Reported', A["oc_tx1stdate"])

A["oc_tx1stage"] = np.array(df2[VAR + "_tx1stageOD"])

A["source"] = 'REDCap'

df2 = df1[ (df1["cataract"] == 'Yes') & (df1["cataracteye"].isin(np.array(['OS', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'oc_date',
 'cataract',
 'cataracteye',
 'cataract_datedx',
 'cataract_datedxOD',
 'cataract_agedxOD',
 'cataract_agedxOS',
 'cataract_datedxOS',

 'cataract_tx',
 'cataract_tx1stdate',
 'cataract_tx1stdateOD',
 'cataract_tx1stageOD',
 'cataract_tx1stdateOS',
 'cataract_tx1stageOS',]]

df2[VAR + "_tx1stdate"] = df2[VAR + "_tx1stdate"].astype('str')
df2[VAR + "_datedx"] = df2[VAR + "_datedx"].astype('str')


B = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
B["site"] = df2["site"]
B["oc_eye"] = 'OS'
B["name"] = 'Cataract'
B["OPHID_Text"] = np.nan
B["sys_oc_id"] = 15
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='Yes', df2[VAR + "_datedxOS"], np.nan)
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='No', 'UNKNOWN', B["oc_date_dx"])
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='nan', 'Not Reported', B["oc_date_dx"])
B["oc_date_rep"] = np.array(df2["oc_date"])
B["oc_details"] = np.nan

B["visit_id"] = np.array(df2["visit_id"])
B["oc_agedx"] = np.array(df2[VAR + "_agedxOS"])
B["oc_tx"] = np.array(df2[VAR + "_tx"])
B["oc_tx"]  = B["oc_tx"] .fillna('Not Reported')

B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='Yes', df2[VAR + "_tx1stdateOS"], np.nan)
B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='No', 'UNKNOWN', B["oc_tx1stdate"])
B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='nan', 'Not Reported', B["oc_tx1stdate"])

B["oc_tx1stage"] = np.array(df2[VAR + "_tx1stageOS"])

B["source"] = 'REDCap'

A5 = A.copy(deep=True)
B5 = B.copy(deep=True)


df2 = df1[(df1["glauc_type"] == 'Primary Open Angle (POAG)') & (df1["glauc"] == 'Yes') & (df1["glauceye"].isin(np.array(['OD', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'oc_date',
 'glauc',
 'glauc_type',
 'glauc_typeother',
 'glauceye',
 'glauc_datedx',
 'glauc_datedxOD',
 'glauc_agedxOD',
 'glauc_datedxOS',

 'glauc_tx',
 'glauc_tx1stdate',
 'glauc_tx1stdateOD',
 'glauc_tx1stageOD',
 'glauc_tx1stdateOS',
 'glauc_tx1stageOS',]]


VAR = 'glauc'
df2[VAR + "_tx1stdate"] = df2[VAR + "_tx1stdate"].astype('str')
df2[VAR + "_datedx"] = df2[VAR + "_datedx"].astype('str')
A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["oc_eye"] = 'OD'
A["name"] = 'Glaucoma, POAG'
A["OPHID_Text"] = np.nan
A["sys_oc_id"] = 135
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='Yes', df2[VAR + "_datedxOD"], np.nan)
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='No', 'UNKNOWN', A["oc_date_dx"])
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='nan', 'Not Reported', A["oc_date_dx"])
A["oc_date_rep"] = np.array(df2["oc_date"])
A["oc_details"] = np.nan

A["visit_id"] = np.array(df2["visit_id"])
A["oc_agedx"] = np.array(df2[VAR + "_agedxOD"])
A["oc_tx"] = np.array(df2[VAR + "_tx"])
A["oc_tx"]  = A["oc_tx"] .fillna('Not Reported')

A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='Yes', df2[VAR + "_tx1stdateOD"], np.nan)
A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='No', 'UNKNOWN', A["oc_tx1stdate"])
A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='nan', 'Not Reported', A["oc_tx1stdate"])

A["oc_tx1stage"] = np.array(df2[VAR + "_tx1stageOD"])


A["source"] = 'REDCap'

df2 = df1[ (df1["glauc_type"] == 'Primary Open Angle (POAG)') & (df1["glauc"] == 'Yes') & (df1["glauceye"].isin(np.array(['OS', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'oc_date',
 'glauc',
 'glauc_type',
 'glauc_typeother',
 'glauceye',
 'glauc_datedx',
 'glauc_datedxOD',
 'glauc_agedxOD',
 'glauc_agedxOS',
 'glauc_datedxOS',

 'glauc_tx',
 'glauc_tx1stdate',
 'glauc_tx1stdateOD',
 'glauc_tx1stageOD',
 'glauc_tx1stdateOS',
 'glauc_tx1stageOS',]]


df2[VAR + "_tx1stdate"] = df2[VAR + "_tx1stdate"].astype('str')
df2[VAR + "_datedx"] = df2[VAR + "_datedx"].astype('str')

B = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
B["site"] = df2["site"]
B["oc_eye"] = 'OS'
B["name"] = 'Glaucoma, POAG'
B["OPHID_Text"] = np.nan
B["sys_oc_id"] = 135
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='Yes', df2[VAR + "_datedxOS"], np.nan)
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='No', 'UNKNOWN', B["oc_date_dx"])
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='nan', 'Not Reported', B["oc_date_dx"])
B["oc_date_rep"] = np.array(df2["oc_date"])
B["oc_details"] = np.nan

B["visit_id"] = np.array(df2["visit_id"])
B["oc_agedx"] = np.array(df2[VAR + "_agedxOS"])
B["oc_tx"] = np.array(df2[VAR + "_tx"])
B["oc_tx"]  = B["oc_tx"] .fillna('Not Reported')

B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='Yes', df2[VAR + "_tx1stdateOS"], np.nan)
B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='No', 'UNKNOWN', B["oc_tx1stdate"])
B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='nan', 'Not Reported', B["oc_tx1stdate"])

B["oc_tx1stage"] = np.array(df2[VAR + "_tx1stageOS"])


B["source"] = 'REDCap'

A6 = A.copy(deep=True)
B6 = B.copy(deep=True)


df2 = df1[~(df1["glauc_type"] == 'Primary Open Angle (POAG)') & (df1["glauc"] == 'Yes') & (df1["glauceye"].isin(np.array(['OD', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'oc_date',
 'glauc',
 'glauc_type',
 'glauc_typeother',
 'glauceye',
 'glauc_datedx',
 'glauc_datedxOD',
 'glauc_agedxOD',

 'glauc_datedxOS',

 'glauc_tx',
 'glauc_tx1stdate',
 'glauc_tx1stdateOD',
 'glauc_tx1stageOD',
 'glauc_tx1stdateOS',
 'glauc_tx1stageOS',]]

VAR = 'glauc'

df2[VAR + "_tx1stdate"] = df2[VAR + "_tx1stdate"].astype('str')
df2[VAR + "_datedx"] = df2[VAR + "_datedx"].astype('str')
A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["oc_eye"] = 'OD'
A["name"] = np.array(df2["glauc_typeother"]) #'Glaucoma (other than POAG)'
A["OPHID_Text"] = np.nan
A["sys_oc_id"] = np.nan
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='Yes', df2[VAR + "_datedxOD"], np.nan)
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='No', 'UNKNOWN', A["oc_date_dx"])
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='nan', 'Not Reported', A["oc_date_dx"])
A["oc_date_rep"] = np.array(df2["oc_date"])
A["oc_details"] = np.array(df2["glauc_type"])

A["visit_id"] = np.array(df2["visit_id"])
A["oc_agedx"] = np.array(df2[VAR + "_agedxOD"])
A["oc_tx"] = np.array(df2[VAR + "_tx"])
A["oc_tx"]  = A["oc_tx"] .fillna('Not Reported')

A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='Yes', df2[VAR + "_tx1stdateOD"], np.nan)
A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='No', 'UNKNOWN', A["oc_tx1stdate"])
A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='nan', 'Not Reported', A["oc_tx1stdate"])

A["oc_tx1stage"] = np.array(df2[VAR + "_tx1stageOD"])


A["source"] = 'REDCap'

df2 = df1[ ~(df1["glauc_type"] == 'Primary Open Angle (POAG)') & (df1["glauc"] == 'Yes') & (df1["glauceye"].isin(np.array(['OS', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'oc_date',
 'glauc',
 'glauc_type',
 'glauc_typeother',
 'glauceye',
 'glauc_datedx',
 'glauc_datedxOD',
 'glauc_agedxOD',
  'glauc_agedxOS',
 'glauc_datedxOS',

 'glauc_tx',
 'glauc_tx1stdate',
 'glauc_tx1stdateOD',
 'glauc_tx1stageOD',
 'glauc_tx1stdateOS',
 'glauc_tx1stageOS',]]

df2[VAR + "_tx1stdate"] = df2[VAR + "_tx1stdate"].astype('str')
df2[VAR + "_datedx"] = df2[VAR + "_datedx"].astype('str')


B = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
B["site"] = df2["site"]
B["oc_eye"] = 'OS'
B["name"] = np.array(df2["glauc_typeother"]) #'Glaucoma (other than POAG)'
B["OPHID_Text"] = np.nan
B["sys_oc_id"] = np.nan
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='Yes', df2[VAR + "_datedxOS"], np.nan)
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='No', 'UNKNOWN', B["oc_date_dx"])
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='nan', 'Not Reported', B["oc_date_dx"])
B["oc_date_rep"] = np.array(df2["oc_date"])
B["oc_details"] = np.array(df2["glauc_type"])

B["visit_id"] = np.array(df2["visit_id"])
B["oc_agedx"] = np.array(df2[VAR + "_agedxOS"])
B["oc_tx"] = np.array(df2[VAR + "_tx"])
B["oc_tx"]  = B["oc_tx"] .fillna('Not Reported')

B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='Yes', df2[VAR + "_tx1stdateOS"], np.nan)
B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='No', 'UNKNOWN', B["oc_tx1stdate"])
B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='nan', 'Not Reported', B["oc_tx1stdate"])

B["oc_tx1stage"] = np.array(df2[VAR + "_tx1stageOS"])

B["source"] = 'REDCap'

A7 = A.copy(deep=True)
B7 = B.copy(deep=True)


df2 = df1[ (df1["macdgen"] == 'Yes') & (df1["macdgeneye"].isin(np.array(['OD', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'oc_date',
 'macdgen',
 'macdgen_type',
 'macdgeneye',
 'macdgen_datedx',
 'macdgen_datedxOD',
 'macdgen_agedxOD',

 'macdgen_datedxOS',

 'macdgen_tx',
 'macdgen_tx1stdate',
 'macdgen_tx1stdateOD',
 'macdgen_tx1stageOD',
 'macdgen_tx1stdateOS',
 'macdgen_tx1stageOS',]]

VAR = 'macdgen'

df2[VAR + "_tx1stdate"] = df2[VAR + "_tx1stdate"].astype('str')
df2[VAR + "_datedx"] = df2[VAR + "_datedx"].astype('str')
A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["oc_eye"] = 'OD'
A["name"] = 'Macular Degeneration'
A["OPHID_Text"] = np.nan
A["sys_oc_id"] = np.nan
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='Yes', df2[VAR + "_datedxOD"], np.nan)
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='No', 'UNKNOWN', A["oc_date_dx"])
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='nan', 'Not Reported', A["oc_date_dx"])
A["oc_date_rep"] = np.array(df2["oc_date"])
A["oc_details"] = np.nan

A["visit_id"] = np.array(df2["visit_id"])
A["oc_agedx"] = np.array(df2[VAR + "_agedxOD"])
A["oc_tx"] = np.array(df2[VAR + "_tx"])
A["oc_tx"]  = A["oc_tx"] .fillna('Not Reported')

A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='Yes', df2[VAR + "_tx1stdateOD"], np.nan)
A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='No', 'UNKNOWN', A["oc_tx1stdate"])
A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='nan', 'Not Reported', A["oc_tx1stdate"])

A["oc_tx1stage"] = np.array(df2[VAR + "_tx1stageOD"])

A["source"] = 'REDCap'

df2 = df1[ (df1["macdgen"] == 'Yes') & (df1["macdgeneye"].isin(np.array(['OS', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'oc_date',
 'macdgen',
 'macdgen_type',
 'macdgeneye',
 'macdgen_datedx',
 'macdgen_datedxOD',
 'macdgen_agedxOD',
'macdgen_agedxOS',
 'macdgen_datedxOS',

 'macdgen_tx',
 'macdgen_tx1stdate',
 'macdgen_tx1stdateOD',
 'macdgen_tx1stageOD',
 'macdgen_tx1stdateOS',
 'macdgen_tx1stageOS',]]

df2[VAR + "_tx1stdate"] = df2[VAR + "_tx1stdate"].astype('str')
df2[VAR + "_datedx"] = df2[VAR + "_datedx"].astype('str')


B = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
B["site"] = df2["site"]
B["oc_eye"] = 'OS'
B["name"] = 'Macular Degeneration'
B["OPHID_Text"] = np.nan
B["sys_oc_id"] = np.nan
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='Yes', df2[VAR + "_datedxOS"], np.nan)
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='No', 'UNKNOWN', B["oc_date_dx"])
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='nan', 'Not Reported', B["oc_date_dx"])
B["oc_date_rep"] = np.array(df2["oc_date"])
B["oc_details"] = np.nan

B["visit_id"] = np.array(df2["visit_id"])
B["oc_agedx"] = np.array(df2[VAR + "_agedxOS"])
B["oc_tx"] = np.array(df2[VAR + "_tx"])
B["oc_tx"]  = B["oc_tx"] .fillna('Not Reported')

B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='Yes', df2[VAR + "_tx1stdateOS"], np.nan)
B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='No', 'UNKNOWN', B["oc_tx1stdate"])
B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='nan', 'Not Reported', B["oc_tx1stdate"])

B["oc_tx1stage"] = np.array(df2[VAR + "_tx1stageOS"])

B["source"] = 'REDCap'

A8 = A.copy(deep=True)
B8 = B.copy(deep=True)


df2 = df1[ (df1["dr"] == 'Yes') & (df1["dreye"].isin(np.array(['OD', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'oc_date',
 'dr',
 'dreye',
 'dr_datedx',
 'dr_datedxOD',
 'dr_agedxOD',

 'dr_datedxOS',

 'dr_tx',
 'dr_tx1stdate',
 'dr_tx1stdateOD',
 'dr_tx1stageOD',
 'dr_tx1stdateOS',
 'dr_tx1stageOS',]]

VAR = 'dr'
df2[VAR + "_tx1stdate"] = df2[VAR + "_tx1stdate"].astype('str')
df2[VAR + "_datedx"] = df2[VAR + "_datedx"].astype('str')

A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["oc_eye"] = 'OD'

A["name"] = 'Diabetic Retinopathy'
A["OPHID_Text"] = np.nan
A["sys_oc_id"] = 206
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='Yes', df2[VAR + "_datedxOD"], np.nan)
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='No', 'UNKNOWN', A["oc_date_dx"])
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='nan', 'Not Reported', A["oc_date_dx"])
A["oc_date_rep"] = np.array(df2["oc_date"])
A["oc_details"] = np.nan

A["visit_id"] = np.array(df2["visit_id"])
A["oc_agedx"] = np.array(df2[VAR + "_agedxOD"])
A["oc_tx"] = np.array(df2[VAR + "_tx"])
A["oc_tx"]  = A["oc_tx"] .fillna('Not Reported')

A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='Yes', df2[VAR + "_tx1stdateOD"], np.nan)
A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='No', 'UNKNOWN', A["oc_tx1stdate"])
A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='nan', 'Not Reported', A["oc_tx1stdate"])

A["oc_tx1stage"] = np.array(df2[VAR + "_tx1stageOD"])

A["source"] = 'REDCap'

df2 = df1[ (df1["dr"] == 'Yes') & (df1["dreye"].isin(np.array(['OS', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'oc_date',
 'dr',
 'dreye',
 'dr_datedx',
 'dr_datedxOD',
 'dr_agedxOD',
  'dr_agedxOS',
 'dr_datedxOS',

 'dr_tx',
 'dr_tx1stdate',
 'dr_tx1stdateOD',
 'dr_tx1stageOD',
 'dr_tx1stdateOS',
 'dr_tx1stageOS',]]
df2[VAR + "_tx1stdate"] = df2[VAR + "_tx1stdate"].astype('str')
df2[VAR + "_datedx"] = df2[VAR + "_datedx"].astype('str')



B = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
B["site"] = df2["site"]
B["oc_eye"] = 'OS'
B["name"] = 'Diabetic Retinopathy'
B["OPHID_Text"] = np.nan
B["sys_oc_id"] = 106
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='Yes', df2[VAR + "_datedxOS"], np.nan)
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='No', 'UNKNOWN', B["oc_date_dx"])
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='nan', 'Not Reported', B["oc_date_dx"])
B["oc_date_rep"] = np.array(df2["oc_date"])
B["oc_details"] = np.nan

B["visit_id"] = np.array(df2["visit_id"])
B["oc_agedx"] = np.array(df2[VAR + "_agedxOS"])
B["oc_tx"] = np.array(df2[VAR + "_tx"])
B["oc_tx"]  = B["oc_tx"] .fillna('Not Reported')

B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='Yes', df2[VAR + "_tx1stdateOS"], np.nan)
B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='No', 'UNKNOWN', B["oc_tx1stdate"])
B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='nan', 'Not Reported', B["oc_tx1stdate"])

B["oc_tx1stage"] = np.array(df2[VAR + "_tx1stageOS"])

B["source"] = 'REDCap'

A9 = A.copy(deep=True)
B9 = B.copy(deep=True)

df2 = df1[ (df1["diabmaced"] == 'Yes') & (df1["diabmacedeye"].isin(np.array(['OD', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'oc_date',
 'diabmaced',
 'diabmacedeye',
 'diabmaced_datedx',
 'diabmaced_datedxOD',
 'diabmaced_agedxOD',

 'diabmaced_datedxOS',

 'diabmaced_tx',
 'diabmaced_tx1stdate',
 'diabmaced_tx1stdateOD',
 'diabmaced_tx1stageOD',
 'diabmaced_tx1stdateOS',
 'diabmaced_tx1stageOS',]]

VAR = 'diabmaced'

df2[VAR + "_tx1stdate"] = df2[VAR + "_tx1stdate"].astype('str')
df2[VAR + "_datedx"] = df2[VAR + "_datedx"].astype('str')

A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["oc_eye"] = 'OD'
A["name"] = 'Diabetic Macular Edema'
A["OPHID_Text"] = np.nan
A["sys_oc_id"] = 22
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='Yes', df2[VAR + "_datedxOD"], np.nan)
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='No', 'UNKNOWN', A["oc_date_dx"])
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='nan', 'Not Reported', A["oc_date_dx"])
A["oc_date_rep"] = np.array(df2["oc_date"])
A["oc_details"] = np.nan

A["visit_id"] = np.array(df2["visit_id"])
A["oc_agedx"] = np.array(df2[VAR + "_agedxOD"])
A["oc_tx"] = np.array(df2[VAR + "_tx"])
A["oc_tx"]  = A["oc_tx"] .fillna('Not Reported')

A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='Yes', df2[VAR + "_tx1stdateOD"], np.nan)
A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='No', 'UNKNOWN', A["oc_tx1stdate"])
A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='nan', 'Not Reported', A["oc_tx1stdate"])

A["oc_tx1stage"] = np.array(df2[VAR + "_tx1stageOD"])

A["source"] = 'REDCap'

df2 = df1[ (df1["diabmaced"] == 'Yes') & (df1["diabmacedeye"].isin(np.array(['OS', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'oc_date',
 'diabmaced',
 'diabmacedeye',
 'diabmaced_datedx',
 'diabmaced_datedxOD',
 'diabmaced_agedxOD',
  'diabmaced_agedxOS',
 'diabmaced_datedxOS',

 'diabmaced_tx',
 'diabmaced_tx1stdate',
 'diabmaced_tx1stdateOD',
 'diabmaced_tx1stageOD',
 'diabmaced_tx1stdateOS',
 'diabmaced_tx1stageOS',]]

df2[VAR + "_tx1stdate"] = df2[VAR + "_tx1stdate"].astype('str')
df2[VAR + "_datedx"] = df2[VAR + "_datedx"].astype('str')


B = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
B["site"] = df2["site"]
B["oc_eye"] = 'OS'
B["name"] = 'Diabetic Macular Edema'
B["OPHID_Text"] = np.nan
B["sys_oc_id"] = 22
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='Yes', df2[VAR + "_datedxOS"], np.nan)
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='No', 'UNKNOWN', B["oc_date_dx"])
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='nan', 'Not Reported', B["oc_date_dx"])
B["oc_date_rep"] = np.array(df2["oc_date"])
B["oc_details"] = np.nan

B["visit_id"] = np.array(df2["visit_id"])
B["oc_agedx"] = np.array(df2[VAR + "_agedxOS"])
B["oc_tx"] = np.array(df2[VAR + "_tx"])
B["oc_tx"]  = B["oc_tx"] .fillna('Not Reported')

B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='Yes', df2[VAR + "_tx1stdateOS"], np.nan)
B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='No', 'UNKNOWN', B["oc_tx1stdate"])
B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='nan', 'Not Reported', B["oc_tx1stdate"])

B["oc_tx1stage"] = np.array(df2[VAR + "_tx1stageOS"])

B["source"] = 'REDCap'

A10 = A.copy(deep=True)
B10 = B.copy(deep=True)


df2 = df1[ (df1["vesocclu"] == 'Yes') & (df1["vesocclueye"].isin(np.array(['OD', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'oc_date',
 'vesocclu',
 'vesocclueye',
 'vesocclu_datedx',
 'vesocclu_datedxOD',
 'vesocclu_agedxOD',

 'vesocclu_datedxOS',

 'vesocclu_tx',
 'vesocclu_tx1stdate',
 'vesocclu_tx1stdateOD',
 'vesocclu_tx1stageOD',
 'vesocclu_tx1stdateOS',
 'vesocclu_tx1stageOS',]]


VAR = 'vesocclu'
df2[VAR + "_tx1stdate"] = df2[VAR + "_tx1stdate"].astype('str')
df2[VAR + "_datedx"] = df2[VAR + "_datedx"].astype('str')
A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["oc_eye"] = 'OD'
A["name"] = 'vesocclu'
A["OPHID_Text"] = np.nan
A["sys_oc_id"] = np.nan
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='Yes', df2[VAR + "_datedxOD"], np.nan)
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='No', 'UNKNOWN', A["oc_date_dx"])
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='nan', 'Not Reported', A["oc_date_dx"])
A["oc_date_rep"] = np.array(df2["oc_date"])
A["oc_details"] = np.nan

A["visit_id"] = np.array(df2["visit_id"])
A["oc_agedx"] = np.array(df2[VAR + "_agedxOD"])
A["oc_tx"] = np.array(df2[VAR + "_tx"])
A["oc_tx"]  = A["oc_tx"] .fillna('Not Reported')

A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='Yes', df2[VAR + "_tx1stdateOD"], np.nan)
A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='No', 'UNKNOWN', A["oc_tx1stdate"])
A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='nan', 'Not Reported', A["oc_tx1stdate"])

A["oc_tx1stage"] = np.array(df2[VAR + "_tx1stageOD"])

A["source"] = 'REDCap'

df2 = df1[ (df1["vesocclu"] == 'Yes') & (df1["vesocclueye"].isin(np.array(['OS', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'oc_date',
 'vesocclu',
 'vesocclueye',
 'vesocclu_datedx',
 'vesocclu_datedxOD',
 'vesocclu_agedxOD',
 'vesocclu_agedxOS',
 'vesocclu_datedxOS',

 'vesocclu_tx',
 'vesocclu_tx1stdate',
 'vesocclu_tx1stdateOD',
 'vesocclu_tx1stageOD',
 'vesocclu_tx1stdateOS',
 'vesocclu_tx1stageOS',]]


df2[VAR + "_tx1stdate"] = df2[VAR + "_tx1stdate"].astype('str')
df2[VAR + "_datedx"] = df2[VAR + "_datedx"].astype('str')

B = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
B["site"] = df2["site"]
B["oc_eye"] = 'OS'
B["name"] = 'vesocclu'
B["OPHID_Text"] = np.nan
B["sys_oc_id"] = np.nan
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='Yes', df2[VAR + "_datedxOS"], np.nan)
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='No', 'UNKNOWN', B["oc_date_dx"])
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='nan', 'Not Reported', B["oc_date_dx"])
B["oc_date_rep"] = np.array(df2["oc_date"])
B["oc_details"] = np.nan

B["visit_id"] = np.array(df2["visit_id"])
B["oc_agedx"] = np.array(df2[VAR + "_agedxOS"])
B["oc_tx"] = np.array(df2[VAR + "_tx"])
B["oc_tx"]  = B["oc_tx"] .fillna('Not Reported')

B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='Yes', df2[VAR + "_tx1stdateOS"], np.nan)
B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='No', 'UNKNOWN', B["oc_tx1stdate"])
B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='nan', 'Not Reported', B["oc_tx1stdate"])

B["oc_tx1stage"] = np.array(df2[VAR + "_tx1stageOS"])

B["source"] = 'REDCap'

A11 = A.copy(deep=True)
B11 = B.copy(deep=True)


df2 = df1[ (df1["retdet"] == 'Yes') & (df1["retdeteye"].isin(np.array(['OD', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'oc_date',
 'retdet',
 'retdeteye',
 'retdet_datedx',
 'retdet_datedxOD',
 'retdet_agedxOD',

 'retdet_datedxOS',

 'retdet_tx',
 'retdet_tx1stdate',
 'retdet_tx1stdateOD',
 'retdet_tx1stageOD',
 'retdet_tx1stdateOS',
 'retdet_tx1stageOS',]]

VAR = 'retdet'

df2[VAR + "_tx1stdate"] = df2[VAR + "_tx1stdate"].astype('str')
df2[VAR + "_datedx"] = df2[VAR + "_datedx"].astype('str')
A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["oc_eye"] = 'OD'
A["name"] = 'Retinal Detachment'
A["OPHID_Text"] = np.nan
A["sys_oc_id"] = 79
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='Yes', df2[VAR + "_datedxOD"], np.nan)
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='No', 'UNKNOWN', A["oc_date_dx"])
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='nan', 'Not Reported', A["oc_date_dx"])
A["oc_date_rep"] = np.array(df2["oc_date"])
A["oc_details"] = np.nan

A["visit_id"] = np.array(df2["visit_id"])
A["oc_agedx"] = np.array(df2[VAR + "_agedxOD"])
A["oc_tx"] = np.array(df2[VAR + "_tx"])
A["oc_tx"]  = A["oc_tx"] .fillna('Not Reported')

A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='Yes', df2[VAR + "_tx1stdateOD"], np.nan)
A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='No', 'UNKNOWN', A["oc_tx1stdate"])
A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='nan', 'Not Reported', A["oc_tx1stdate"])

A["oc_tx1stage"] = np.array(df2[VAR + "_tx1stageOD"])


A["source"] = 'REDCap'

df2 = df1[ (df1["retdet"] == 'Yes') & (df1["retdeteye"].isin(np.array(['OS', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'oc_date',
 'retdet',
 'retdeteye',
 'retdet_datedx',
 'retdet_datedxOD',
 'retdet_agedxOD',
  'retdet_agedxOS',
 'retdet_datedxOS',

 'retdet_tx',
 'retdet_tx1stdate',
 'retdet_tx1stdateOD',
 'retdet_tx1stageOD',
 'retdet_tx1stdateOS',
 'retdet_tx1stageOS',]]



df2[VAR + "_tx1stdate"] = df2[VAR + "_tx1stdate"].astype('str')
df2[VAR + "_datedx"] = df2[VAR + "_datedx"].astype('str')
B = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
B["site"] = df2["site"]
B["oc_eye"] = 'OS'
B["name"] = 'Retinal Detachment'
B["OPHID_Text"] = np.nan
B["sys_oc_id"] = 79
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='Yes', df2[VAR + "_datedxOS"], np.nan)
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='No', 'UNKNOWN', B["oc_date_dx"])
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='nan', 'Not Reported', B["oc_date_dx"])
B["oc_date_rep"] = np.array(df2["oc_date"])
B["oc_details"] = np.nan

B["visit_id"] = np.array(df2["visit_id"])
B["oc_agedx"] = np.array(df2[VAR + "_agedxOS"])
B["oc_tx"] = np.array(df2[VAR + "_tx"])
B["oc_tx"]  = B["oc_tx"] .fillna('Not Reported')

B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='Yes', df2[VAR + "_tx1stdateOS"], np.nan)
B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='No', 'UNKNOWN', B["oc_tx1stdate"])
B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='nan', 'Not Reported', B["oc_tx1stdate"])

B["oc_tx1stage"] = np.array(df2[VAR + "_tx1stageOS"])

B["source"] = 'REDCap'

A12 = A.copy(deep=True)
B12 = B.copy(deep=True)

###
df2 = df1[ (df1["retdys"] == 'Yes') & (df1["retdyseye"].isin(np.array(['OD', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'oc_date',
 'retdys',
 'retdyseye',
 'retdys_datedx',
 'retdys_datedxOD',
 'retdys_agedxOD',

 'retdys_datedxOS',

 'retdys_tx',
 'retdys_tx1stdate',
 'retdys_tx1stdateOD',
 'retdys_tx1stageOD',
 'retdys_tx1stdateOS',
 'retdys_tx1stageOS',]]

VAR = 'retdys'
df2[VAR + "_tx1stdate"] = df2[VAR + "_tx1stdate"].astype('str')
df2[VAR + "_datedx"] = df2[VAR + "_datedx"].astype('str')


A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["oc_eye"] = 'OD'
A["name"] = 'Retinal Dyst'
A["OPHID_Text"] = np.nan
A["sys_oc_id"] = 79
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='Yes', df2[VAR + "_datedxOD"], np.nan)
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='No', 'UNKNOWN', A["oc_date_dx"])
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='nan', 'Not Reported', A["oc_date_dx"])
A["oc_date_rep"] = np.array(df2["oc_date"])
A["oc_details"] = np.nan

A["visit_id"] = np.array(df2["visit_id"])
A["oc_agedx"] = np.array(df2[VAR + "_agedxOD"])
A["oc_tx"] = np.array(df2[VAR + "_tx"])
A["oc_tx"]  = A["oc_tx"] .fillna('Not Reported')

A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='Yes', df2[VAR + "_tx1stdateOD"], np.nan)
A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='No', 'UNKNOWN', A["oc_tx1stdate"])
A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='nan', 'Not Reported', A["oc_tx1stdate"])

A["oc_tx1stage"] = np.array(df2[VAR + "_tx1stageOD"])


A["source"] = 'REDCap'

df2 = df1[ (df1["retdys"] == 'Yes') & (df1["retdyseye"].isin(np.array(['OS', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'oc_date',
 'retdys',
 'retdyseye',
 'retdys_datedx',
 'retdys_datedxOD',
 'retdys_agedxOD',
  'retdys_agedxOS',
 'retdys_datedxOS',

 'retdys_tx',
 'retdys_tx1stdate',
 'retdys_tx1stdateOD',
 'retdys_tx1stageOD',
 'retdys_tx1stdateOS',
 'retdys_tx1stageOS',]]


df2[VAR + "_tx1stdate"] = df2[VAR + "_tx1stdate"].astype('str')
df2[VAR + "_datedx"] = df2[VAR + "_datedx"].astype('str')

B = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
B["site"] = df2["site"]
B["oc_eye"] = 'OS'
B["name"] = 'Retinal Dyst'
B["OPHID_Text"] = np.nan
B["sys_oc_id"] = 79
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='Yes', df2[VAR + "_datedxOS"], np.nan)
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='No', 'UNKNOWN', B["oc_date_dx"])
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='nan', 'Not Reported', B["oc_date_dx"])
B["oc_date_rep"] = np.array(df2["oc_date"])
B["oc_details"] = np.nan

B["visit_id"] = np.array(df2["visit_id"])
B["oc_agedx"] = np.array(df2[VAR + "_agedxOS"])
B["oc_tx"] = np.array(df2[VAR + "_tx"])
B["oc_tx"]  = B["oc_tx"] .fillna('Not Reported')

B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='Yes', df2[VAR + "_tx1stdateOS"], np.nan)
B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='No', 'UNKNOWN', B["oc_tx1stdate"])
B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='nan', 'Not Reported', B["oc_tx1stdate"])

B["oc_tx1stage"] = np.array(df2[VAR + "_tx1stageOS"])

B["source"] = 'REDCap'

A16 = A.copy(deep=True)
B16 = B.copy(deep=True)
###
df2 = df1[ (df1["eyeinjury"] == 'Yes') & (df1["eyeinjuryeye"].isin(np.array(['OD', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'oc_date',
 'eyeinjury',
 'eyeinjuryeye',
 'eyeinjury_datedx',
 'eyeinjury_datedxOD',
 'eyeinjury_agedxOD',

 'eyeinjury_datedxOS',

 'eyeinjury_tx',
 'eyeinjury_tx1stdate',
 'eyeinjury_tx1stdateOD',
 'eyeinjury_tx1stageOD',
 'eyeinjury_tx1stdateOS',
 'eyeinjury_tx1stageOS',]]

VAR = 'eyeinjury'
df2[VAR + "_tx1stdate"] = df2[VAR + "_tx1stdate"].astype('str')
df2[VAR + "_datedx"] = df2[VAR + "_datedx"].astype('str')

A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["oc_eye"] = 'OD'
A["name"] = 'Eye Injury'
A["OPHID_Text"] = np.nan
A["sys_oc_id"] = np.nan
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='Yes', df2[VAR + "_datedxOD"], np.nan)
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='No', 'UNKNOWN', A["oc_date_dx"])
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='nan', 'Not Reported', A["oc_date_dx"])
A["oc_date_rep"] = np.array(df2["oc_date"])
A["oc_details"] = np.nan

A["visit_id"] = np.array(df2["visit_id"])
A["oc_agedx"] = np.array(df2[VAR + "_agedxOD"])
A["oc_tx"] = np.array(df2[VAR + "_tx"])
A["oc_tx"]  = A["oc_tx"] .fillna('Not Reported')

A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='Yes', df2[VAR + "_tx1stdateOD"], np.nan)
A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='No', 'UNKNOWN', A["oc_tx1stdate"])
A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='nan', 'Not Reported', A["oc_tx1stdate"])

A["oc_tx1stage"] = np.array(df2[VAR + "_tx1stageOD"])

A["source"] = 'REDCap'

df2 = df1[ (df1["eyeinjury"] == 'Yes') & (df1["eyeinjuryeye"].isin(np.array(['OS', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'oc_date',
 'eyeinjury',
 'eyeinjuryeye',
 'eyeinjury_datedx',
 'eyeinjury_datedxOD',
 'eyeinjury_agedxOD',
  'eyeinjury_agedxOS',
 'eyeinjury_datedxOS',

 'eyeinjury_tx',
 'eyeinjury_tx1stdate',
 'eyeinjury_tx1stdateOD',
 'eyeinjury_tx1stageOD',
 'eyeinjury_tx1stdateOS',
 'eyeinjury_tx1stageOS',]]

df2[VAR + "_tx1stdate"] = df2[VAR + "_tx1stdate"].astype('str')
df2[VAR + "_datedx"] = df2[VAR + "_datedx"].astype('str')


B = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
B["site"] = df2["site"]
B["oc_eye"] = 'OS'
B["name"] = 'Eye Injury'
B["OPHID_Text"] = np.nan
B["sys_oc_id"] = np.nan
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='Yes', df2[VAR + "_datedxOS"], np.nan)
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='No', 'UNKNOWN', B["oc_date_dx"])
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='nan', 'Not Reported', B["oc_date_dx"])
B["oc_date_rep"] = np.array(df2["oc_date"])
B["oc_details"] = np.nan

B["visit_id"] = np.array(df2["visit_id"])
B["oc_agedx"] = np.array(df2[VAR + "_agedxOS"])
B["oc_tx"] = np.array(df2[VAR + "_tx"])
B["oc_tx"]  = B["oc_tx"] .fillna('Not Reported')

B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='Yes', df2[VAR + "_tx1stdateOS"], np.nan)
B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='No', 'UNKNOWN', B["oc_tx1stdate"])
B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='nan', 'Not Reported', B["oc_tx1stdate"])

B["oc_tx1stage"] = np.array(df2[VAR + "_tx1stageOS"])


B["source"] = 'REDCap'

A13 = A.copy(deep=True)
B13 = B.copy(deep=True)


df2 = df1[ (df1["othercond1_rpt"] == 'Yes') & (df1["othercond1eye"].isin(np.array(['OD', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'oc_date',
 'othercond1_rpt',
 'othercond1_name',
 'othercond1_drops',
 'othercond1',
 'othercond1eye',
 'othercond1_datedx',
 'othercond1_datedxOD',
 'othercond1_agedxOD',

 'othercond1_datedxOS',

 'othercond1_tx',
 'othercond1_tx1stdate',
 'othercond1_tx1stdateOD',
 'othercond1_tx1stageOD',
 'othercond1_tx1stdateOS',
 'othercond1_tx1stageOS',]]

VAR = 'othercond1'
df2[VAR + "_tx1stdate"] = df2[VAR + "_tx1stdate"].astype('str')
df2[VAR + "_datedx"] = df2[VAR + "_datedx"].astype('str')

A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["oc_eye"] = 'OD'
A["name"] = np.array(df2["othercond1_name"])
A["OPHID_Text"] = np.nan
A["sys_oc_id"] = np.nan
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='Yes', df2[VAR + "_datedxOD"], np.nan)
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='No', 'UNKNOWN', A["oc_date_dx"])
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='nan', 'Not Reported', A["oc_date_dx"])
A["oc_date_rep"] = np.array(df2["oc_date"])
A["oc_details"] = np.array(df2["othercond1"])

A["visit_id"] = np.array(df2["visit_id"])
A["oc_agedx"] = np.array(df2[VAR + "_agedxOD"])
A["oc_tx"] = np.array(df2[VAR + "_tx"])
A["oc_tx"]  = A["oc_tx"] .fillna('Not Reported')

A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='Yes', df2[VAR + "_tx1stdateOD"], np.nan)
A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='No', 'UNKNOWN', A["oc_tx1stdate"])
A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='nan', 'Not Reported', A["oc_tx1stdate"])

A["oc_tx1stage"] = np.array(df2[VAR + "_tx1stageOD"])


A["source"] = 'REDCap'

df2 = df1[ (df1["othercond1_rpt"] == 'Yes') & (df1["othercond1eye"].isin(np.array(['OS', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'oc_date',
 'othercond1_rpt',
 'othercond1_name',
 'othercond1_drops',
 'othercond1',
 'othercond1eye',
 'othercond1_datedx',
 'othercond1_datedxOD',
 'othercond1_agedxOD',
 'othercond1_agedxOS',
 'othercond1_datedxOS',

 'othercond1_tx',
 'othercond1_tx1stdate',
 'othercond1_tx1stdateOD',
 'othercond1_tx1stageOD',
 'othercond1_tx1stdateOS',
 'othercond1_tx1stageOS',]]

df2[VAR + "_tx1stdate"] = df2[VAR + "_tx1stdate"].astype('str')
df2[VAR + "_datedx"] = df2[VAR + "_datedx"].astype('str')


B = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
B["site"] = df2["site"]
B["oc_eye"] = 'OS'
B["name"] = np.array(df2["othercond1_name"])
B["OPHID_Text"] = np.nan
B["sys_oc_id"] = np.nan
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='Yes', df2[VAR + "_datedxOS"], np.nan)
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='No', 'UNKNOWN', B["oc_date_dx"])
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='nan', 'Not Reported', B["oc_date_dx"])
B["oc_date_rep"] = np.array(df2["oc_date"])
B["oc_details"] = np.array(df2["othercond1"])

B["visit_id"] = np.array(df2["visit_id"])
B["oc_agedx"] = np.array(df2[VAR + "_agedxOS"])
B["oc_tx"] = np.array(df2[VAR + "_tx"])
B["oc_tx"]  = B["oc_tx"] .fillna('Not Reported')

B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='Yes', df2[VAR + "_tx1stdateOS"], np.nan)
B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='No', 'UNKNOWN', B["oc_tx1stdate"])
B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='nan', 'Not Reported', B["oc_tx1stdate"])

B["oc_tx1stage"] = np.array(df2[VAR + "_tx1stageOS"])

B["source"] = 'REDCap'

A14 = A.copy(deep=True)
B14 = B.copy(deep=True)

df2 = df1[ (df1["othercond2_rpt"] == 'Yes') & (df1["othercond2eye"].isin(np.array(['OD', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'oc_date',
 'othercond2_rpt',
 'othercond2_name',
 'othercond2_drops',
 'othercond2',
 'othercond2eye',
 'othercond2_datedx',
 'othercond2_datedxOD',
 'othercond2_agedxOD',

 'othercond2_datedxOS',

 'othercond2_tx',
 'othercond2_tx1stdate',
 'othercond2_tx1stdateOD',
 'othercond2_tx1stageOD',
 'othercond2_tx1stdateOS',
 'othercond2_tx1stageOS',]]

VAR = 'othercond2'

df2[VAR + "_tx1stdate"] = df2[VAR + "_tx1stdate"].astype('str')
df2[VAR + "_datedx"] = df2[VAR + "_datedx"].astype('str')
A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["oc_eye"] = 'OD'
A["name"] = np.array(df2["othercond2_name"])
A["OPHID_Text"] = np.nan
A["sys_oc_id"] = np.nan
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='Yes', df2[VAR + "_datedxOD"], np.nan)
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='No', 'UNKNOWN', A["oc_date_dx"])
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='nan', 'Not Reported', A["oc_date_dx"])
A["oc_date_rep"] = np.array(df2["oc_date"])
A["oc_details"] = np.array(df2["othercond2"])

A["visit_id"] = np.array(df2["visit_id"])
A["oc_agedx"] = np.array(df2[VAR + "_agedxOD"])
A["oc_tx"] = np.array(df2[VAR + "_tx"])
A["oc_tx"]  = A["oc_tx"] .fillna('Not Reported')

A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='Yes', df2[VAR + "_tx1stdateOD"], np.nan)
A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='No', 'UNKNOWN', A["oc_tx1stdate"])
A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='nan', 'Not Reported', A["oc_tx1stdate"])

A["oc_tx1stage"] = np.array(df2[VAR + "_tx1stageOD"])

A["source"] = 'REDCap'

df2 = df1[ (df1["othercond2_rpt"] == 'Yes') & (df1["othercond2eye"].isin(np.array(['OS', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'oc_date',
 'othercond2_rpt',
 'othercond2_name',
 'othercond2_drops',
 'othercond2',
 'othercond2eye',
 'othercond2_datedx',
 'othercond2_datedxOD',
 'othercond2_agedxOD',
  'othercond2_agedxOS',
 'othercond2_datedxOS',

 'othercond2_tx',
 'othercond2_tx1stdate',
 'othercond2_tx1stdateOD',
 'othercond2_tx1stageOD',
 'othercond2_tx1stdateOS',
 'othercond2_tx1stageOS',]]


df2[VAR + "_tx1stdate"] = df2[VAR + "_tx1stdate"].astype('str')
df2[VAR + "_datedx"] = df2[VAR + "_datedx"].astype('str')

B = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
B["site"] = df2["site"]
B["oc_eye"] = 'OS'
B["name"] = np.array(df2["othercond2_name"])
B["OPHID_Text"] = np.nan
B["sys_oc_id"] = np.nan
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='Yes', df2[VAR + "_datedxOS"], np.nan)
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='No', 'UNKNOWN', B["oc_date_dx"])
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='nan', 'Not Reported', B["oc_date_dx"])
B["oc_date_rep"] = np.array(df2["oc_date"])
B["oc_details"] = np.array(df2["othercond2"])

B["visit_id"] = np.array(df2["visit_id"])
B["oc_agedx"] = np.array(df2[VAR + "_agedxOS"])
B["oc_tx"] = np.array(df2[VAR + "_tx"])
B["oc_tx"]  = B["oc_tx"] .fillna('Not Reported')

B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='Yes', df2[VAR + "_tx1stdateOS"], np.nan)
B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='No', 'UNKNOWN', B["oc_tx1stdate"])
B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='nan', 'Not Reported', B["oc_tx1stdate"])

B["oc_tx1stage"] = np.array(df2[VAR + "_tx1stageOS"])

B["source"] = 'REDCap'

A15 = A.copy(deep=True)
B15 = B.copy(deep=True)



df2 = df1[ (df1["othercond3_rpt"] == 'Yes') & (df1["othercond3eye"].isin(np.array(['OD', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'oc_date',
 'othercond3_rpt',
 'othercond3_name',
 'othercond3_drops',
 'othercond3',
 'othercond3eye',
 'othercond3_datedx',
 'othercond3_datedxOD',
 'othercond3_agedxOD',

 'othercond3_datedxOS',

 'othercond3_tx',
 'othercond3_tx1stdate',
 'othercond3_tx1stdateOD',
 'othercond3_tx1stageOD',
 'othercond3_tx1stdateOS',
 'othercond3_tx1stageOS',]]

VAR = 'othercond3'
df2[VAR + "_tx1stdate"] = df2[VAR + "_tx1stdate"].astype('str')
df2[VAR + "_datedx"] = df2[VAR + "_datedx"].astype('str')

A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["oc_eye"] = 'OD'
A["name"] = np.array(df2["othercond3_name"])
A["OPHID_Text"] = np.nan
A["sys_oc_id"] = np.nan
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='Yes', df2[VAR + "_datedxOD"], np.nan)
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='No', 'UNKNOWN', A["oc_date_dx"])
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='nan', 'Not Reported', A["oc_date_dx"])
A["oc_date_rep"] = np.array(df2["oc_date"])
A["oc_details"] = np.array(df2["othercond3"])

A["visit_id"] = np.array(df2["visit_id"])
A["oc_agedx"] = np.array(df2[VAR + "_agedxOD"])
A["oc_tx"] = np.array(df2[VAR + "_tx"])
A["oc_tx"]  = A["oc_tx"] .fillna('Not Reported')

A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='Yes', df2[VAR + "_tx1stdateOD"], np.nan)
A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='No', 'UNKNOWN', A["oc_tx1stdate"])
A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='nan', 'Not Reported', A["oc_tx1stdate"])

A["oc_tx1stage"] = np.array(df2[VAR + "_tx1stageOD"])

A["source"] = 'REDCap'

df2 = df1[ (df1["othercond3_rpt"] == 'Yes') & (df1["othercond3eye"].isin(np.array(['OS', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'oc_date',
 'othercond3_rpt',
 'othercond3_name',
 'othercond3_drops',
 'othercond3',
 'othercond3eye',
 'othercond3_datedx',
 'othercond3_datedxOD',
 'othercond3_agedxOD',
  'othercond3_agedxOS',
 'othercond3_datedxOS',

 'othercond3_tx',
 'othercond3_tx1stdate',
 'othercond3_tx1stdateOD',
 'othercond3_tx1stageOD',
 'othercond3_tx1stdateOS',
 'othercond3_tx1stageOS',]]

df2[VAR + "_tx1stdate"] = df2[VAR + "_tx1stdate"].astype('str')
df2[VAR + "_datedx"] = df2[VAR + "_datedx"].astype('str')


B = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
B["site"] = df2["site"]
B["oc_eye"] = 'OS'
B["name"] = np.array(df2["othercond3_name"])
B["OPHID_Text"] = np.nan
B["sys_oc_id"] = np.nan
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='Yes', df2[VAR + "_datedxOS"], np.nan)
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='No', 'UNKNOWN', B["oc_date_dx"])
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='nan', 'Not Reported', B["oc_date_dx"])
B["oc_date_rep"] = np.array(df2["oc_date"])
B["oc_details"] = np.array(df2["othercond3"])

B["visit_id"] = np.array(df2["visit_id"])
B["oc_agedx"] = np.array(df2[VAR + "_agedxOS"])
B["oc_tx"] = np.array(df2[VAR + "_tx"])
B["oc_tx"]  = B["oc_tx"] .fillna('Not Reported')

B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='Yes', df2[VAR + "_tx1stdateOS"], np.nan)
B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='No', 'UNKNOWN', B["oc_tx1stdate"])
B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='nan', 'Not Reported', B["oc_tx1stdate"])

B["oc_tx1stage"] = np.array(df2[VAR + "_tx1stageOS"])

B["source"] = 'REDCap'

A17 = A.copy(deep=True)
B17 = B.copy(deep=True)



df2 = df1[ (df1["othercond4_rpt"] == 'Yes') & (df1["othercond4eye"].isin(np.array(['OD', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'oc_date',
 'othercond4_rpt',
 'othercond4_name',
 'othercond4_drops',
 'othercond4',
 'othercond4eye',
 'othercond4_datedx',
 'othercond4_datedxOD',
 'othercond4_agedxOD',

 'othercond4_datedxOS',

 'othercond4_tx',
 'othercond4_tx1stdate',
 'othercond4_tx1stdateOD',
 'othercond4_tx1stageOD',
 'othercond4_tx1stdateOS',
 'othercond4_tx1stageOS',]]

VAR = 'othercond4'
df2[VAR + "_tx1stdate"] = df2[VAR + "_tx1stdate"].astype('str')
df2[VAR + "_datedx"] = df2[VAR + "_datedx"].astype('str')

A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["oc_eye"] = 'OD'
A["name"] = np.array(df2["othercond4_name"])
A["OPHID_Text"] = np.nan
A["sys_oc_id"] = np.nan
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='Yes', df2[VAR + "_datedxOD"], np.nan)
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='No', 'UNKNOWN', A["oc_date_dx"])
A["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='nan', 'Not Reported', A["oc_date_dx"])
A["oc_date_rep"] = np.array(df2["oc_date"])
A["oc_details"] = np.array(df2["othercond4"])

A["visit_id"] = np.array(df2["visit_id"])
A["oc_agedx"] = np.array(df2[VAR + "_agedxOD"])
A["oc_tx"] = np.array(df2[VAR + "_tx"])
A["oc_tx"]  = A["oc_tx"] .fillna('Not Reported')

A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='Yes', df2[VAR + "_tx1stdateOD"], np.nan)
A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='No', 'UNKNOWN', A["oc_tx1stdate"])
A["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='nan', 'Not Reported', A["oc_tx1stdate"])

A["oc_tx1stage"] = np.array(df2[VAR + "_tx1stageOD"])

A["source"] = 'REDCap'

df2 = df1[ (df1["othercond4_rpt"] == 'Yes') & (df1["othercond4eye"].isin(np.array(['OS', 'OU'])))][['maskedID',
 'site',
 'visit_id',
 'oc_date',
 'othercond4_rpt',
 'othercond4_name',
 'othercond4_drops',
 'othercond4',
 'othercond4eye',
 'othercond4_datedx',
 'othercond4_datedxOD',
 'othercond4_agedxOD',
 'othercond4_agedxOS',
 'othercond4_datedxOS',

 'othercond4_tx',
 'othercond4_tx1stdate',
 'othercond4_tx1stdateOD',
 'othercond4_tx1stageOD',
 'othercond4_tx1stdateOS',
 'othercond4_tx1stageOS',]]


VAR = 'othercond4'
df2[VAR + "_tx1stdate"] = df2[VAR + "_tx1stdate"].astype('str')
df2[VAR + "_datedx"] = df2[VAR + "_datedx"].astype('str')
B = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
B["site"] = df2["site"]
B["oc_eye"] = 'OS'
B["name"] = np.array(df2["othercond4_name"])
B["OPHID_Text"] = np.nan
B["sys_oc_id"] = np.nan
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='Yes', df2[VAR + "_datedxOS"], np.nan)
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='No', 'UNKNOWN', B["oc_date_dx"])
B["oc_date_dx"] = np.where(df2[VAR + "_datedx"]=='nan', 'Not Reported', B["oc_date_dx"])
B["oc_date_rep"] = np.array(df2["oc_date"])
B["oc_details"] = np.array(df2["othercond4"])

B["visit_id"] = np.array(df2["visit_id"])
B["oc_agedx"] = np.array(df2[VAR + "_agedxOS"])
B["oc_tx"] = np.array(df2[VAR + "_tx"])
B["oc_tx"]  = B["oc_tx"] .fillna('Not Reported')

B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='Yes', df2[VAR + "_tx1stdateOS"], np.nan)
B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='No', 'UNKNOWN', B["oc_tx1stdate"])
B["oc_tx1stdate"] = np.where(df2[VAR + "_tx1stdate"]=='nan', 'Not Reported', B["oc_tx1stdate"])

B["oc_tx1stage"] = np.array(df2[VAR + "_tx1stageOS"])

B["source"] = 'REDCap'

A18 = A.copy(deep=True)
B18 = B.copy(deep=True)


df2 = df1[df1['cond_add']=='Yes'][['maskedID',
 'site',
 'visit_id',
 'oc_date','cond_add',
 'cond_more',]]

A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["oc_eye"] = np.nan
A["name"] = np.array(df2["cond_more"])
A["OPHID_Text"] = np.nan
A["sys_oc_id"] = np.nan
A["oc_date_dx"] = np.nan
A["oc_details"] = np.nan

A["visit_id"] = np.array(df2["visit_id"])
A["oc_agedx"] = np.nan
A["oc_tx"] = np.nan

A["oc_tx1stdate"] = np.nan
A["oc_tx1stage"] = np.nan
A["oc_date_rep"] = np.array(df2["oc_date"])
A["source"] = 'REDCap'

A20 = A.copy(deep=True)

df["oc_tx"] = np.nan
df["oc_tx1stdate"] = np.nan
df["oc_tx1stage"] = np.nan

H = list(df.columns.values)
FIN = pd.concat([df, A1[H]], ignore_index=True)
FIN = pd.concat([FIN, A2[H]], ignore_index=True)
FIN = pd.concat([FIN, A3[H]], ignore_index=True)
FIN = pd.concat([FIN, A4[H]], ignore_index=True)
FIN = pd.concat([FIN, A5[H]], ignore_index=True)
FIN = pd.concat([FIN, A6[H]], ignore_index=True)
FIN = pd.concat([FIN, A7[H]], ignore_index=True)
FIN = pd.concat([FIN, A8[H]], ignore_index=True)
FIN = pd.concat([FIN, A9[H]], ignore_index=True)
FIN = pd.concat([FIN, A10[H]], ignore_index=True)
FIN = pd.concat([FIN, A11[H]], ignore_index=True)
FIN = pd.concat([FIN, A12[H]], ignore_index=True)
FIN = pd.concat([FIN, A13[H]], ignore_index=True)
FIN = pd.concat([FIN, A14[H]], ignore_index=True)
FIN = pd.concat([FIN, A15[H]], ignore_index=True)
FIN = pd.concat([FIN, A16[H]], ignore_index=True)
FIN = pd.concat([FIN, A17[H]], ignore_index=True)
FIN = pd.concat([FIN, A18[H]], ignore_index=True)

FIN = pd.concat([FIN, B1[H]], ignore_index=True)
FIN = pd.concat([FIN, B2[H]], ignore_index=True)
FIN = pd.concat([FIN, B3[H]], ignore_index=True)
FIN = pd.concat([FIN, B4[H]], ignore_index=True)
FIN = pd.concat([FIN, B5[H]], ignore_index=True)
FIN = pd.concat([FIN, B6[H]], ignore_index=True)
FIN = pd.concat([FIN, B7[H]], ignore_index=True)
FIN = pd.concat([FIN, B8[H]], ignore_index=True)
FIN = pd.concat([FIN, B9[H]], ignore_index=True)
FIN = pd.concat([FIN, B10[H]], ignore_index=True)
FIN = pd.concat([FIN, B11[H]], ignore_index=True)
FIN = pd.concat([FIN, B12[H]], ignore_index=True)
FIN = pd.concat([FIN, B13[H]], ignore_index=True)
FIN = pd.concat([FIN, B14[H]], ignore_index=True)
FIN = pd.concat([FIN, B15[H]], ignore_index=True)
FIN = pd.concat([FIN, B16[H]], ignore_index=True)
FIN = pd.concat([FIN, B17[H]], ignore_index=True)
FIN = pd.concat([FIN, B18[H]], ignore_index=True)
FIN = pd.concat([FIN, A20[H]], ignore_index=True)


#FIN['glauc_typeother'] = np.where(FIN['oc_details'] == 'Other, Combined, Inflammatory', 'Yes', np.nan)
FIN = pd.merge(FIN, ind[["OPHID_Text", "researchid"]], how='left', on='OPHID_Text')





FIN = FIN[~FIN['maskedid'].isnull()]

FIN = pd.merge(FIN, ind[["OPHID_Text_1", "maskedid"]], how='left', on='maskedid')
FIN["OPHID_Text"] = np.where(FIN["OPHID_Text"].isnull(), FIN["OPHID_Text_1"], FIN["OPHID_Text"])

FIN['maskedid'] = np.where(FIN['maskedid'].isnull(), FIN['researchid'], FIN['maskedid'])
FIN['BioBID'] = np.where(FIN['researchid'].isnull(), FIN['maskedid'], FIN['researchid'])
FIN['BioBID'] = np.where( (FIN['researchid'].str.contains('OG')) | (FIN['researchid'].str.contains('ED')), FIN['BioBID'], np.nan)
FIN["site"] = FIN['maskedid'].str[:2]
FIN = FIN[name]

FIN = FIN.fillna('')

FIN.to_csv(redcapeyechart + 'Results/' + extension + '_OcularConditions_RedEC.csv', index=False)


/var/folders/7m/n2zn61593k18rvyz907d2skc0000gn/T/ipykernel_30504/2035954581.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[VAR + "_tx1stdate"] = df2[VAR + "_tx1stdate"].astype('str')
/var/folders/7m/n2zn61593k18rvyz907d2skc0000gn/T/ipykernel_30504/2035954581.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[VAR + "_datedx"] = df2[VAR + "_datedx"].astype('str')


In [47]:
L1 = [i for i in L_name if '_ONHExam' in i]
df = pd.read_csv(EyeChart + L1[0], encoding = "ISO-8859-1")
#df = pd.read_csv(EyeChart + '20170322_ONHExam.csv', encoding = "ISO-8859-1")

name = 'maskedid	OPHID_Text	BioBID	site	visit_id	onhexam_date	onh_staff	onh_expectedgp	onh_expecgpundet_note	onh_dilat	onhexam_eye	onhexam_status	onhexam_abn	onhexam_large_cdr	onhexam_rim_thinning	onhexam_excavation	onhexam_disc_hem	onhexam_asymmetry	onhexam_rnfl_defect	onhexam_tilted_disc	onhexam_ppa	onhexam_drawing_oag	onh_clear_prog	onhexam_other	onhexam_comments	created_at	updated_at	source'
name = name.split('\t')
df["visit_id"] = np.nan
df['onh_staff'] = np.nan
df['onh_expecgpundet_note'] = np.nan
df["onh_expectedgp"] = np.nan
df['onh_clear_prog'] = np.nan
df['onh_dilat'] = np.nan
df1 = pd.read_csv(redcapeyechart + extension + '_ONHExam.csv')
df["site"] = df['maskedid'].str[:2]
df["source"] = 'EyeChart'



df2 = df1[df1["onh_od_bool"] == "Yes"][['maskedID',
 'site',
 'visit_id',
 'onh_date',
 'onh_staff',
 'onh_expectedgp',
 'onh_expecgpundet_note',
 'onh_dilat',
 'onh_od_bool',
 'onh_od_status',
 'onh_od_large_cdr',
 'onh_od_rnfl_defect',
 'onh_od_rim_thinning',
 'onh_od_tilted_disk',
 'onh_od_disk_hem',
 'onh_od_drawing_oag',
 'onh_od_ppa',
 'onh_od_asymmetry',
 'onh_od_clear_prog',
 'onh_od_other',
 'onh_od_other_notes',]]





A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["onhexam_eye"] = 'OD'
A["onh_staff"] = np.array(df2['onh_staff'])
A["onh_expecgpundet_note"] = np.array(df2['onh_expecgpundet_note'])
A["onh_expectedgp"] = np.array(df2['onh_expectedgp'])
A['onh_clear_prog'] = np.array(df2['onh_od_clear_prog'])
A['onh_dilat'] = np.array(df2['onh_dilat'])
A["OPHID_Text"] = np.nan
A["onhexam_date"] = np.array(df2["onh_date"])
#A["onhexam_status"] = np.where(df2["onh_expectedgp"] == "Glaucoma", "GON", df2["onh_expectedgp"])
#A["onhexam_status"] = np.where(df2["onh_expectedgp"] == "Normal", "NO GON", A["onhexam_status"])
A["onhexam_status"] = np.array(df2["onh_od_status"])
A["onhexam_status"] = A["onhexam_status"].str.upper()
A["visit_id"] = np.array(df2["visit_id"])
A["onhexam_abn"] = np.array(df2["onh_od_status"])
A["onhexam_abn"] = A["onhexam_abn"].str.upper()

A["onhexam_large_cdr"] = np.array(df2["onh_od_large_cdr"])
A["onhexam_large_cdr"] = A["onhexam_large_cdr"].str.upper()

A["onhexam_rim_thinning"] = np.array(df2["onh_od_rim_thinning"])
A["onhexam_rim_thinning"] = A["onhexam_rim_thinning"].str.upper()

A["onhexam_excavation"] = np.nan

A["onhexam_disc_hem"] = np.array(df2["onh_od_disk_hem"])
A["onhexam_disc_hem"] = A["onhexam_disc_hem"].str.upper()

A["onhexam_asymmetry"] = np.array(df2["onh_od_asymmetry"])
A["onhexam_asymmetry"] = A["onhexam_asymmetry"].str.upper()

A["onhexam_rnfl_defect"] = np.array(df2["onh_od_rnfl_defect"])
A["onhexam_rnfl_defect"] = A["onhexam_rnfl_defect"].str.upper()

A["onhexam_tilted_disc"] = np.array(df2["onh_od_tilted_disk"])
A["onhexam_tilted_disc"] = A["onhexam_tilted_disc"].str.upper()

A["onhexam_ppa"] = np.array(df2["onh_od_ppa"])
A["onhexam_ppa"] = A["onhexam_ppa"].str.upper()

A["onhexam_drawing_oag"] = np.array(df2["onh_od_drawing_oag"])
A["onhexam_drawing_oag"] = A["onhexam_drawing_oag"].str.upper()

    
A["onhexam_other"] = np.array(df2["onh_od_other"])
A["onhexam_comments"] = np.array(df2["onh_od_other_notes"])
A["created_at"] = np.array(df2["onh_date"])
A["updated_at"] = np.nan
A["source"] = 'REDCap'

B = A.copy(deep=True)


df2 = df1[df1["onh_os_bool"] == "Yes"][['maskedID',
 'site',
 'visit_id',
 'onh_date',
 'onh_staff',
 'onh_expectedgp',
 'onh_expecgpundet_note',
 'onh_dilat',
 'onh_os_bool',
 'onh_os_status',
 'onh_os_large_cdr',
 'onh_os_rnfl_defect',
 'onh_os_rim_thinning',
 'onh_os_tilted_disk',
 'onh_os_disk_hem',
 'onh_os_drawing_oag',
 'onh_os_ppa',
 'onh_os_asymmetry',
 'onh_os_clear_prog',
 'onh_os_other',
 'onh_os_other_notes',]]





A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]
A["onhexam_eye"] = 'OS'
A["onh_staff"] = np.array(df2['onh_staff'])
A["onh_expecgpundet_note"] = np.array(df2['onh_expecgpundet_note'])
A["onh_expectedgp"] = np.array(df2['onh_expectedgp'])
A['onh_clear_prog'] = np.array(df2['onh_os_clear_prog'])
A['onh_dilat'] = np.array(df2['onh_dilat'])
A["OPHID_Text"] = np.nan
A["onhexam_date"] = np.array(df2["onh_date"])
#A["onhexam_status"] = np.where(df2["onh_expectedgp"] == "Glaucoma", "GON", df2["onh_expectedgp"])
#A["onhexam_status"] = np.where(df2["onh_expectedgp"] == "Normal", "NO GON", A["onhexam_status"])
A["onhexam_status"] = np.array(df2["onh_os_status"])
A["onhexam_status"] = A["onhexam_status"].str.upper()
A["visit_id"] = np.array(df2["visit_id"])
A["onhexam_abn"] = np.array(df2["onh_os_status"])
A["onhexam_abn"] = A["onhexam_abn"].str.upper()

A["onhexam_large_cdr"] = np.array(df2["onh_os_large_cdr"])
A["onhexam_large_cdr"] = A["onhexam_large_cdr"].str.upper()

A["onhexam_rim_thinning"] = np.array(df2["onh_os_rim_thinning"])
A["onhexam_rim_thinning"] = A["onhexam_rim_thinning"].str.upper()

A["onhexam_excavation"] = np.nan

A["onhexam_disc_hem"] = np.array(df2["onh_os_disk_hem"])
A["onhexam_disc_hem"] = A["onhexam_disc_hem"].str.upper()

A["onhexam_asymmetry"] = np.array(df2["onh_os_asymmetry"])
A["onhexam_asymmetry"] = A["onhexam_asymmetry"].str.upper()

A["onhexam_rnfl_defect"] = np.array(df2["onh_os_rnfl_defect"])
A["onhexam_rnfl_defect"] = A["onhexam_rnfl_defect"].str.upper()

A["onhexam_tilted_disc"] = np.array(df2["onh_os_tilted_disk"])
A["onhexam_tilted_disc"] = A["onhexam_tilted_disc"].str.upper()

A["onhexam_ppa"] = np.array(df2["onh_os_ppa"])
A["onhexam_ppa"] = A["onhexam_ppa"].str.upper()

A["onhexam_drawing_oag"] = np.array(df2["onh_os_drawing_oag"])
A["onhexam_drawing_oag"] = A["onhexam_drawing_oag"].str.upper()

    
A["onhexam_other"] = np.array(df2["onh_os_other"])
A["onhexam_comments"] = np.array(df2["onh_os_other_notes"])
A["created_at"] = np.array(df2["onh_date"])
A["updated_at"] = np.nan
A["source"] = 'REDCap'

H = list(df.columns.values)
FIN = pd.concat([df, A[H]], ignore_index=True)
FIN = pd.concat([FIN, B[H]], ignore_index=True)

FIN = pd.merge(FIN, ind[["OPHID_Text", "researchid"]], how='left', on='OPHID_Text')





FIN = FIN[~FIN['maskedid'].isnull()]

FIN = pd.merge(FIN, ind[["OPHID_Text_1", "maskedid"]], how='left', on='maskedid')
FIN["OPHID_Text"] = np.where(FIN["OPHID_Text"].isnull(), FIN["OPHID_Text_1"], FIN["OPHID_Text"])

FIN['maskedid'] = np.where(FIN['maskedid'].isnull(), FIN['researchid'], FIN['maskedid'])
FIN['BioBID'] = np.where(FIN['researchid'].isnull(), FIN['maskedid'], FIN['researchid'])
FIN['BioBID'] = np.where( (FIN['researchid'].str.contains('OG')) | (FIN['researchid'].str.contains('ED')), FIN['BioBID'], np.nan)
FIN["site"] = FIN['maskedid'].str[:2]

FIN = FIN[name]

FIN.to_csv(redcapeyechart + 'Results/' + extension + '_ONHExam_RedEC.csv', index=False)


In [48]:
L1 = [i for i in L_name if '_PhysAct' in i]
df = pd.read_csv(EyeChart + L1[0], encoding = "ISO-8859-1")

#df = pd.read_csv(EyeChart + '20170322_PhysAct.csv')

df1 = pd.read_csv(redcapeyechart + extension + '_PhysicalActivity.csv', encoding = "ISO-8859-1")
name = 'maskedid	OPHID_Text	BioBID	site	visit_id	physact_date	physact_operator	physact_employed	physact_workactlvl	physact_lstwk_physexer	physact_lstwk_cycling	physact_lstwk_walking	physact_lstwk_house	physact_lstwk_garden	physact_walkpace	physact_comments	source'
name = name.split('\t')

df["site"] = df['maskedid'].str[:2]
df["source"] = 'EyeChart'



df2 = df1.copy(deep=True)

A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
A["site"] = df2["site"]

A["OPHID_Text"] = np.nan
A["physact_date"] = np.array(df2["pa_date"])
A["visit_id"] = np.array(df2["visit_id"])
A["physact_operator"] = np.nan

A["physact_employed"] = np.array(df2["pa_phys_emp"])
A["physact_employed"] = A["physact_employed"].str.upper()

A["physact_workactlvl"] = np.where(df2["pa_phys_emp_type"]== 'I spend most of my time at work sitting (e.g. in an office)', 1, np.nan )
A["physact_workactlvl"] = np.where(df2["pa_phys_emp_type"]== 'I spend most of my time at work standing or walking. However, my work does not require much intense physical effort (e.g. shop assistant, hairdresser, security guard, etc.)', 2, A["physact_workactlvl"] )
A["physact_workactlvl"] = np.where(df2["pa_phys_emp_type"]== 'My work involves definite physical effort including handling of heavy objects and use of tools (e.g. plumber, electrician, carpenter, cleaner, hospital nurse, gardener, postal delivery workers, etc.)', 3, A["physact_workactlvl"] )

A['physact_workactlvl'] = A['physact_workactlvl'].fillna('NOT REPORTED')

A["physact_lstwk_physexer"] = np.where(df2["pa_phys_exercise"] == '3 hours or more', 4, np.nan)
A["physact_lstwk_physexer"] = np.where(df2["pa_phys_exercise"] == '1 hour but less than 3 hours', 2, A["physact_lstwk_physexer"])
A["physact_lstwk_physexer"] = np.where(df2["pa_phys_exercise"] == 'Some but less than 1 hour', 1,A["physact_lstwk_physexer"])
A["physact_lstwk_physexer"] = np.where(df2["pa_phys_exercise"] == 'None', 1,A["physact_lstwk_physexer"])
A['physact_lstwk_physexer'] = A['physact_lstwk_physexer'].fillna('NOT REPORTED')

A["physact_lstwk_cycling"] = np.where(df2["pa_phys_cycling"] == '3 hours or more', 4, df2["pa_phys_cycling"])
A["physact_lstwk_cycling"] = np.where(df2["pa_phys_cycling"] == '1 hour but less than 3 hours', 3, A["physact_lstwk_cycling"])
A["physact_lstwk_cycling"] = np.where(df2["pa_phys_cycling"] == 'Some but less than 1 hour', 2,A["physact_lstwk_cycling"])
A["physact_lstwk_cycling"] = np.where(df2["pa_phys_cycling"] == 'None', 1,A["physact_lstwk_cycling"])
A['physact_lstwk_cycling'] = A['physact_lstwk_cycling'].fillna('NOT REPORTED')

A["physact_lstwk_walking"] = np.where(df2["pa_phys_walk"] == '3 hours or more', 4, df2["pa_phys_walk"])
A["physact_lstwk_walking"] = np.where(df2["pa_phys_walk"] == '1 hour but less than 3 hours', 3, A["physact_lstwk_walking"])
A["physact_lstwk_walking"] = np.where(df2["pa_phys_walk"] == 'Some but less than 1 hour', 2,A["physact_lstwk_walking"])
A["physact_lstwk_walking"] = np.where(df2["pa_phys_walk"] == 'None', 1,A["physact_lstwk_walking"])
A['physact_lstwk_walking'] = A['physact_lstwk_walking'].fillna('NOT REPORTED')

A["physact_lstwk_house"] = np.where(df2["pa_phys_domestic"] == '3 hours or more', 4, df2["pa_phys_domestic"])
A["physact_lstwk_house"] = np.where(df2["pa_phys_domestic"] == '1 hour but less than 3 hours', 3, A["physact_lstwk_house"])
A["physact_lstwk_house"] = np.where(df2["pa_phys_domestic"] == 'Some but less than 1 hour', 2,A["physact_lstwk_house"])
A["physact_lstwk_house"] = np.where(df2["pa_phys_domestic"] == 'None', 1,A["physact_lstwk_house"])
A['physact_lstwk_house'] = A['physact_lstwk_house'].fillna('NOT REPORTED')


A["physact_lstwk_garden"] = np.where(df2["pa_phys_garden"] == '3 hours or more', 4, df2["pa_phys_garden"])
A["physact_lstwk_garden"] = np.where(df2["pa_phys_garden"] == '1 hour but less than 3 hours', 3, A["physact_lstwk_garden"])
A["physact_lstwk_garden"] = np.where(df2["pa_phys_garden"] == 'Some but less than 1 hour', 2,A["physact_lstwk_garden"])
A["physact_lstwk_garden"] = np.where(df2["pa_phys_garden"] == 'None', 1, A["physact_lstwk_garden"])

A["physact_walkpace"] = np.array(df2["pa_phys_pace"])


A["physact_walkpace"] = np.where(df2["pa_phys_pace"] == 'Brisk or striding (more than 5 mph)', 5, df2["pa_phys_pace"])
A["physact_walkpace"] = np.where(df2["pa_phys_pace"] == 'Fairly briskly (4-5 mph)', 4, A["physact_walkpace"])
A["physact_walkpace"] = np.where(df2["pa_phys_pace"] == 'Average or normal (2-3 mph)', 3,A["physact_walkpace"])
A["physact_walkpace"] = np.where(df2["pa_phys_pace"] == 'Casual strolling (up to 2 mph)', 2, A["physact_walkpace"])
A["physact_walkpace"] = np.where(df2["pa_phys_pace"] == 'Not walking at all', 1, A["physact_walkpace"])

A["physact_comments"] = np.nan
A["source"] = 'REDCap'
A['visit_id'] = np.array(df2["visit_id"])

df['visit_id'] = np.nan
H = list(df.columns.values)

FIN = pd.concat([df, A[H]], ignore_index=True)


FIN = pd.merge(FIN, ind[["OPHID_Text", "researchid"]], how='left', on='OPHID_Text')





FIN = FIN[~FIN['maskedid'].isnull()]

FIN = pd.merge(FIN, ind[["OPHID_Text_1", "maskedid"]], how='left', on='maskedid')
FIN["OPHID_Text"] = np.where(FIN["OPHID_Text"].isnull(), FIN["OPHID_Text_1"], FIN["OPHID_Text"])

FIN['maskedid'] = np.where(FIN['maskedid'].isnull(), FIN['researchid'], FIN['maskedid'])
FIN['BioBID'] = np.where(FIN['researchid'].isnull(), FIN['maskedid'], FIN['researchid'])
FIN['BioBID'] = np.where( (FIN['researchid'].str.contains('OG')) | (FIN['researchid'].str.contains('ED')), FIN['BioBID'], np.nan)
FIN["site"] = FIN['maskedid'].str[:2]

FIN = FIN[name]
FIN.to_csv(redcapeyechart + 'Results/' + extension + '_PhysicalActivity_RedEC.csv', index=False)

In [49]:
L1 = [i for i in L_name if '_AnteriorSeg' in i]
df = pd.read_csv(EyeChart + L1[0], encoding = "ISO-8859-1")
#df = pd.read_csv(EyeChart + '20170322_AnteriorSeg.csv')

df2 = pd.read_csv(redcapeyechart + extension + '_AnteriorSeg.csv', encoding = "ISO-8859-1")




df["site"] = df['maskedid'].str[:2]
df["source"] = 'EyeChart'


df1 = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
df1["OPHID_Text"] = np.nan
df1["antseg_eye"] = 'OD'
df1["antseg_date"] = np.array(df2["dr_date"])
df1["antseg_cornopac_abn"] = np.where(df2["dr_od_opacity"].isin(np.array(["No"])), "NO", np.nan)
df1["antseg_cornopac_abn"] = np.where(df2["dr_od_opacity"].isin(np.array(["Yes"])), "YES", df1["antseg_cornopac_abn"])
df1["antseg_cornopac_abn"] = np.where(df2["dr_od_opacity"].isin(np.array(["Not Tested"])), "NOT TESTED", df1["antseg_cornopac_abn"])
df1["antseg_cornopac_abn"] = np.where(df2["dr_od_antseg"]=='No', "NOT TESTED", df1["antseg_cornopac_abn"])
df1["antseg_cornopac_abn"] = np.where(df2["dr_antseg"]=='No', "NOT TESTED", df1["antseg_cornopac_abn"])


df1["antseg_cornguttata_abn"] = np.where(df2["dr_od_guttata"].isin(np.array(["No"])), "NO", np.nan)
df1["antseg_cornguttata_abn"] = np.where(df2["dr_od_guttata"].isin(np.array(["Yes"])), "YES", df1["antseg_cornguttata_abn"])
df1["antseg_cornguttata_abn"] = np.where(df2["dr_od_guttata"].isin(np.array(["Not Tested"])), "NOT TESTED", df1["antseg_cornguttata_abn"])
df1["antseg_cornguttata_abn"] = np.where(df2["dr_od_antseg"]=='No', "NOT TESTED", df1["antseg_cornguttata_abn"])
df1["antseg_cornguttata_abn"] = np.where(df2["dr_antseg"]=='No', "NOT TESTED", df1["antseg_cornguttata_abn"])


df1["antseg_cornedema_abn"] = np.where(df2["dr_od_edema"].isin(np.array(["No"])), "NO", np.nan)
df1["antseg_cornedema_abn"] = np.where(df2["dr_od_edema"].isin(np.array(["Yes"])), "YES", df1["antseg_cornedema_abn"])
df1["antseg_cornedema_abn"] = np.where(df2["dr_od_edema"].isin(np.array(["Not Tested"])), "NOT TESTED", df1["antseg_cornedema_abn"])
df1["antseg_cornedema_abn"] = np.where(df2["dr_od_antseg"]=='No', "NOT TESTED", df1["antseg_cornedema_abn"])
df1["antseg_cornedema_abn"] = np.where(df2["dr_antseg"]=='No', "NOT TESTED", df1["antseg_cornedema_abn"])



df1["antseg_cornbleb"] = np.where(df2["dr_od_bleb"].isin(np.array(["No Bleb"])), "NO BLEB", np.nan)
df1["antseg_cornbleb"] = np.where(df2["dr_od_bleb"].isin(np.array(["Intact"])), "INTACT", df1["antseg_cornbleb"])
df1["antseg_cornbleb"] = np.where(df2["dr_od_bleb"].isin(np.array(["Failed"])), "FAILED", df1["antseg_cornbleb"])
df1["antseg_cornbleb"] = np.where(df2["dr_od_bleb"].isin(np.array(["Not Tested"])), "NOT TESTED", df1["antseg_cornbleb"])
df1["antseg_cornbleb"] = np.where(df2["dr_od_antseg"]=='No', "NOT TESTED", df1["antseg_cornbleb"])
df1["antseg_cornbleb"] = np.where(df2["dr_antseg"]=='No', "NOT TESTED", df1["antseg_cornbleb"])


df1["antseg_cornkps_abn"] = np.nan


df1["antseg_lensopac_abn"] = np.nan

df1["antseg_lensexfol_abn"] = np.where(df2["dr_od_lensexfol"].isin(np.array(["No"])), "NO", np.nan)
df1["antseg_lensexfol_abn"] = np.where(df2["dr_od_lensexfol"].isin(np.array(["Yes"])), "YES", df1["antseg_lensexfol_abn"])
df1["antseg_lensexfol_abn"] = np.where(df2["dr_od_lensexfol"].isin(np.array(["Not Tested"])), "NOT TESTED", df1["antseg_lensexfol_abn"])
df1["antseg_lensexfol_abn"] = np.where(df2["dr_od_antseg"]=='No', "NOT TESTED", df1["antseg_lensexfol_abn"])
df1["antseg_lensexfol_abn"] = np.where(df2["dr_antseg"]=='No', "NOT TESTED", df1["antseg_lensexfol_abn"])


df1["antseg_irisexfol_abn"] = np.where(df2["dr_od_irisexfol"].isin(np.array(["No"])), "NO", np.nan)
df1["antseg_irisexfol_abn"] = np.where(df2["dr_od_irisexfol"].isin(np.array(["Yes"])), "YES", df1["antseg_irisexfol_abn"])
df1["antseg_irisexfol_abn"] = np.where(df2["dr_od_irisexfol"].isin(np.array(["Not Tested"])), "NOT TESTED", df1["antseg_irisexfol_abn"])
df1["antseg_irisexfol_abn"] = np.where(df2["dr_od_antseg"]=='No', "NOT TESTED", df1["antseg_irisexfol_abn"])
df1["antseg_irisexfol_abn"] = np.where(df2["dr_antseg"]=='No', "NOT TESTED", df1["antseg_irisexfol_abn"])



df1["antseg_iriscorect_abn"] = np.nan

df1["antseg_iristransill_abn"] = np.where(df2["dr_od_transdef"].isin(np.array(["No"])), "NO", np.nan)
df1["antseg_iristransill_abn"] = np.where(df2["dr_od_transdef"].isin(np.array(["Yes"])), "YES", df1["antseg_iristransill_abn"])
df1["antseg_iristransill_abn"] = np.where(df2["dr_od_transdef"].isin(np.array(["Not Tested"])), "NOT TESTED", df1["antseg_iristransill_abn"])
df1["antseg_iristransill_abn"] = np.where(df2["dr_od_antseg"]=='No', "NOT TESTED", df1["antseg_iristransill_abn"])
df1["antseg_iristransill_abn"] = np.where(df2["dr_antseg"]=='No', "NOT TESTED", df1["antseg_iristransill_abn"])



df1["antseg_irissynechia_abn"] = np.where(df2["dr_od_synechia"].isin(np.array(["No"])), "NO", np.nan)
df1["antseg_irissynechia_abn"] = np.where(df2["dr_od_synechia"].isin(np.array(["Yes"])), "YES", df1["antseg_irissynechia_abn"])
df1["antseg_irissynechia_abn"] = np.where(df2["dr_od_synechia"].isin(np.array(["Not Tested"])), "NOT TESTED", df1["antseg_irissynechia_abn"])
df1["antseg_irissynechia_abn"] = np.where(df2["dr_od_antseg"]=='No', "NOT TESTED", df1["antseg_irissynechia_abn"])
df1["antseg_irissynechia_abn"] = np.where(df2["dr_antseg"]=='No', "NOT TESTED", df1["antseg_irissynechia_abn"])




df1["antseg_other"] = np.nan
df1["antseg_status"] = np.nan

df1["antseg_lenscataract"] = np.where(df2["dr_od_cataract"].isin(np.array(["Phakic"])), "PHAKIC", np.nan)
df1["antseg_lenscataract"] = np.where(df2["dr_od_cataract"].isin(np.array(["Pseudo"])), "PSEUDOPHAKIC", df1["antseg_lenscataract"])
df1["antseg_lenscataract"] = np.where(df2["dr_od_cataract"].isin(np.array(["APHAKIC"])), "Aphakic", df1["antseg_lenscataract"])
df1["antseg_lenscataract"] = np.where(df2["dr_od_cataract"].isin(np.array(["Not Tested"])), "NOT TESTED", df1["antseg_lenscataract"])
df1["antseg_lenscataract"] = np.where(df2["dr_od_cataract"].isin(np.array(["No"])), "NO", df1["antseg_lenscataract"])
df1["antseg_lenscataract"] = np.where(df2["dr_od_cataract"].isin(np.array(["Type Unknown"])), "TYPE UNK", df1["antseg_lenscataract"])
df1["antseg_lenscataract"] = np.where(df2["dr_od_antseg"]=='No', "NOT TESTED", df1["antseg_lenscataract"])
df1["antseg_lenscataract"] = np.where(df2["dr_antseg"]=='No', "NOT TESTED", df1["antseg_lenscataract"])



df1["antseg_comments"] = np.array(df2["dr_od_antseg_other"])

df1["source"] = 'REDCap'
df1["visit_id"] = np.array(df2["visit_id"])


df3 = df1.copy(deep=True)

df1 = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])
df1["OPHID_Text"] = np.nan
df1["antseg_eye"] = 'OS'
df1["antseg_date"] = np.array(df2["dr_date"])
df1["antseg_cornopac_abn"] = np.where(df2["dr_os_opacity"].isin(np.array(["No"])), "NO", np.nan)
df1["antseg_cornopac_abn"] = np.where(df2["dr_os_opacity"].isin(np.array(["Yes"])), "YES", df1["antseg_cornopac_abn"])
df1["antseg_cornopac_abn"] = np.where(df2["dr_os_opacity"].isin(np.array(["Not Tested"])), "NOT TESTED", df1["antseg_cornopac_abn"])
df1["antseg_cornopac_abn"] = np.where(df2["dr_os_antseg"]=='No', "NOT TESTED", df1["antseg_cornopac_abn"])
df1["antseg_cornopac_abn"] = np.where(df2["dr_antseg"]=='No', "NOT TESTED", df1["antseg_cornopac_abn"])


df1["antseg_cornguttata_abn"] = np.where(df2["dr_os_guttata"].isin(np.array(["No"])), "NO", np.nan)
df1["antseg_cornguttata_abn"] = np.where(df2["dr_os_guttata"].isin(np.array(["Yes"])), "YES", df1["antseg_cornguttata_abn"])
df1["antseg_cornguttata_abn"] = np.where(df2["dr_os_guttata"].isin(np.array(["Not Tested"])), "NOT TESTED", df1["antseg_cornguttata_abn"])
df1["antseg_cornguttata_abn"] = np.where(df2["dr_os_antseg"]=='No', "NOT TESTED", df1["antseg_cornguttata_abn"])
df1["antseg_cornguttata_abn"] = np.where(df2["dr_antseg"]=='No', "NOT TESTED", df1["antseg_cornguttata_abn"])


df1["antseg_cornedema_abn"] = np.where(df2["dr_os_edema"].isin(np.array(["No"])), "NO", np.nan)
df1["antseg_cornedema_abn"] = np.where(df2["dr_os_edema"].isin(np.array(["Yes"])), "YES", df1["antseg_cornedema_abn"])
df1["antseg_cornedema_abn"] = np.where(df2["dr_os_edema"].isin(np.array(["Not Tested"])), "NOT TESTED", df1["antseg_cornedema_abn"])
df1["antseg_cornedema_abn"] = np.where(df2["dr_os_antseg"]=='No', "NOT TESTED", df1["antseg_cornedema_abn"])
df1["antseg_cornedema_abn"] = np.where(df2["dr_antseg"]=='No', "NOT TESTED", df1["antseg_cornedema_abn"])


df1["antseg_cornbleb"] = np.where(df2["dr_os_bleb"].isin(np.array(["No Bleb"])), "NO BLEB", np.nan)
df1["antseg_cornbleb"] = np.where(df2["dr_os_bleb"].isin(np.array(["Intact"])), "INTACT", df1["antseg_cornbleb"])
df1["antseg_cornbleb"] = np.where(df2["dr_os_bleb"].isin(np.array(["Failed"])), "FAILED", df1["antseg_cornbleb"])
df1["antseg_cornbleb"] = np.where(df2["dr_os_bleb"].isin(np.array(["Not Tested"])), "NOT TESTED", df1["antseg_cornbleb"])
df1["antseg_cornbleb"] = np.where(df2["dr_os_antseg"]=='No', "NOT TESTED", df1["antseg_cornbleb"])
df1["antseg_cornbleb"] = np.where(df2["dr_antseg"]=='No', "NOT TESTED", df1["antseg_cornbleb"])



df1["antseg_cornkps_abn"] = np.nan


df1["antseg_lensopac_abn"] = np.nan

df1["antseg_lensexfol_abn"] = np.where(df2["dr_os_lensexfol"].isin(np.array(["No"])), "NO", np.nan)
df1["antseg_lensexfol_abn"] = np.where(df2["dr_os_lensexfol"].isin(np.array(["Yes"])), "YES", df1["antseg_lensexfol_abn"])
df1["antseg_lensexfol_abn"] = np.where(df2["dr_os_lensexfol"].isin(np.array(["Not Tested"])), "NOT TESTED", df1["antseg_lensexfol_abn"])
df1["antseg_lensexfol_abn"] = np.where(df2["dr_os_antseg"]=='No', "NOT TESTED", df1["antseg_lensexfol_abn"])
df1["antseg_lensexfol_abn"] = np.where(df2["dr_antseg"]=='No', "NOT TESTED", df1["antseg_lensexfol_abn"])


df1["antseg_irisexfol_abn"] = np.where(df2["dr_os_irisexfol"].isin(np.array(["No"])), "NO", np.nan)
df1["antseg_irisexfol_abn"] = np.where(df2["dr_os_irisexfol"].isin(np.array(["Yes"])), "YES", df1["antseg_irisexfol_abn"])
df1["antseg_irisexfol_abn"] = np.where(df2["dr_os_irisexfol"].isin(np.array(["Not Tested"])), "NOT TESTED", df1["antseg_irisexfol_abn"])
df1["antseg_irisexfol_abn"] = np.where(df2["dr_os_antseg"]=='No', "NOT TESTED", df1["antseg_irisexfol_abn"])
df1["antseg_irisexfol_abn"] = np.where(df2["dr_antseg"]=='No', "NOT TESTED", df1["antseg_irisexfol_abn"])



df1["antseg_iriscorect_abn"] = np.nan

df1["antseg_iristransill_abn"] = np.where(df2["dr_os_transdef"].isin(np.array(["No"])), "NO", np.nan)
df1["antseg_iristransill_abn"] = np.where(df2["dr_os_transdef"].isin(np.array(["Yes"])), "YES", df1["antseg_iristransill_abn"])
df1["antseg_iristransill_abn"] = np.where(df2["dr_os_transdef"].isin(np.array(["Not Tested"])), "NOT TESTED", df1["antseg_iristransill_abn"])
df1["antseg_iristransill_abn"] = np.where(df2["dr_os_antseg"]=='No', "NOT TESTED", df1["antseg_iristransill_abn"])
df1["antseg_iristransill_abn"] = np.where(df2["dr_antseg"]=='No', "NOT TESTED", df1["antseg_iristransill_abn"])


df1["antseg_irissynechia_abn"] = np.where(df2["dr_os_synechia"].isin(np.array(["No"])), "NO", np.nan)
df1["antseg_irissynechia_abn"] = np.where(df2["dr_os_synechia"].isin(np.array(["Yes"])), "YES", df1["antseg_irissynechia_abn"])
df1["antseg_irissynechia_abn"] = np.where(df2["dr_os_synechia"].isin(np.array(["Not Tested"])), "NOT TESTED", df1["antseg_irissynechia_abn"])
df1["antseg_irissynechia_abn"] = np.where(df2["dr_os_antseg"]=='No', "NOT TESTED", df1["antseg_irissynechia_abn"])
df1["antseg_irissynechia_abn"] = np.where(df2["dr_antseg"]=='No', "NOT TESTED", df1["antseg_irissynechia_abn"])


df1["antseg_other"] = np.nan
df1["antseg_status"] = np.nan

df1["antseg_lenscataract"] = np.where(df2["dr_oa_cataract"].isin(np.array(["Phakic"])), "PHAKIC", np.nan)
df1["antseg_lenscataract"] = np.where(df2["dr_oa_cataract"].isin(np.array(["Pseudo"])), "PSEUDOPHAKIC", df1["antseg_lenscataract"])
df1["antseg_lenscataract"] = np.where(df2["dr_oa_cataract"].isin(np.array(["APHAKIC"])), "Aphakic", df1["antseg_lenscataract"])
df1["antseg_lenscataract"] = np.where(df2["dr_oa_cataract"].isin(np.array(["Not Tested"])), "NOT TESTED", df1["antseg_lenscataract"])
df1["antseg_lenscataract"] = np.where(df2["dr_oa_cataract"].isin(np.array(["No"])), "NO", df1["antseg_lenscataract"])
df1["antseg_lenscataract"] = np.where(df2["dr_oa_cataract"].isin(np.array(["Type Unknown"])), "TYPE UNK", df1["antseg_lenscataract"])
df1["antseg_lenscataract"] = np.where(df2["dr_os_antseg"]=='No', "NOT TESTED", df1["antseg_lenscataract"])
df1["antseg_lenscataract"] = np.where(df2["dr_antseg"]=='No', "NOT TESTED", df1["antseg_lenscataract"])


df1["antseg_comments"] = np.array(df2["dr_os_antseg_other"])

df1["source"] = 'REDCap'
df1["visit_id"] = np.array(df2["visit_id"])
df1["site"] = df1['maskedid'].str[:2]
df3["site"] = df3['maskedid'].str[:2]

H = list(df.columns.values)
H = H + ['visit_id']
df['visit_id'] = np.nan
FIN = pd.concat([df, df1[H]], ignore_index=True)   
FIN = pd.concat([FIN, df3[H]], ignore_index=True)   




FIN = pd.merge(FIN, ind[["OPHID_Text", "researchid"]], how='left', on='OPHID_Text')





FIN = FIN[~FIN['maskedid'].isnull()]

FIN = pd.merge(FIN, ind[["OPHID_Text_1", "maskedid"]], how='left', on='maskedid')
FIN["OPHID_Text"] = np.where(FIN["OPHID_Text"].isnull(), FIN["OPHID_Text_1"], FIN["OPHID_Text"])

FIN['maskedid'] = np.where(FIN['maskedid'].isnull(), FIN['researchid'], FIN['maskedid'])
FIN['BioBID'] = np.where(FIN['researchid'].isnull(), FIN['maskedid'], FIN['researchid'])
FIN['BioBID'] = np.where( (FIN['researchid'].str.contains('OG')) | (FIN['researchid'].str.contains('ED')), FIN['BioBID'], np.nan)
FIN["site"] = FIN['maskedid'].str[:2]


FIN = FIN.drop('researchid', 1)
name = ['maskedid',
'OPHID_Text',
'BioBID',
'site',
'visit_id',
'antseg_date',
'antseg_eye',
'antseg_cornopac_abn',
'antseg_cornguttata_abn',
'antseg_cornedema_abn',
'antseg_cornbleb',
'antseg_cornkps_abn',
'antseg_lensopac_abn',
'antseg_lensexfol_abn',
'antseg_irisexfol_abn',
'antseg_iriscorect_abn',
'antseg_iristransill_abn',
'antseg_irissynechia_abn',
'antseg_other',
'antseg_status',
'antseg_lenscataract',
'antseg_comments',
'source']
FIN = FIN[name]
FIN.to_csv(redcapeyechart + 'Results/' + extension + '_AnteriorSeg_RedEC.csv', index=False)



/var/folders/7m/n2zn61593k18rvyz907d2skc0000gn/T/ipykernel_30504/3296120754.py:225: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  FIN = FIN.drop('researchid', 1)


In [50]:
FIN[FIN['maskedid']=='ED10271']

,maskedid,OPHID_Text,BioBID,site,visit_id,antseg_date,antseg_eye,antseg_cornopac_abn,antseg_cornguttata_abn,antseg_cornedema_abn,...,antseg_lensexfol_abn,antseg_irisexfol_abn,antseg_iriscorect_abn,antseg_iristransill_abn,antseg_irissynechia_abn,antseg_other,antseg_status,antseg_lenscataract,antseg_comments,source
30866,ED10271,NaN,NaN,ED,Baseline,02/10/2014,OS,NOT TESTED,NOT TESTED,NOT TESTED,...,NOT TESTED,NOT TESTED,NaN,NOT TESTED,NOT TESTED,NaN,NaN,NOT TESTED,NaN,REDCap
33649,ED10271,NaN,NaN,ED,Baseline,02/10/2014,OD,NOT TESTED,NOT TESTED,NOT TESTED,...,NOT TESTED,NOT TESTED,NaN,NOT TESTED,NOT TESTED,NaN,NaN,NOT TESTED,NaN,REDCap


In [51]:

L1 = [i for i in L_name if '_Hospitalization' in i]
df = pd.read_csv(EyeChart + L1[0], encoding = "ISO-8859-1")
#df = pd.read_csv(EyeChart + '20170322_Hospitalization.csv', encoding = "ISO-8859-1")
df["site"] = np.array(df['maskedid'].str[:2])
df["source"] = 'EyeChart'

name = 'maskedid	OPHID_Text	BioBID	site	visit_id	hosp_date_rep	hosp_start_date	hosp_end_date	hosp_reason	REDCapExport	source'
name = name.split('\t')


df1 = pd.read_csv(redcapeyechart + extension + '_OcularConditions.csv')

df2 = df1[df1['hosp_bool']=='Yes'][['maskedID', 'site', 'visit_id', 'oc_date', 'hosp_bool',
       'hosp_start', 'hosp_end', 'hosp_reason']]


A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])

A["OPHID_Text"] = np.nan
A["hosp_date_rep"] = np.array(df2["oc_date"])
A["visit_id"] = np.array(df2["visit_id"])
A["hosp_start_date"] = np.array(df2["hosp_start"])
A["hosp_end_date"] = np.array(df2["hosp_end"])
A["hosp_reason"] = np.array(df2["hosp_reason"])
A["REDCapExport"] = 'OcularConditions'
A["site"] = np.array(df2["maskedID"].str[:2])
A["source"] = 'REDCap'


A1 = A.copy(deep=True)
df1 = pd.read_csv(redcapeyechart + extension + '_NonOcularSurgeriesProcedures.csv', encoding = "ISO-8859-1")
df1 = df1[~df1["maskedID"].isnull()]


df2 = df1[df1["nosp_hosp_bool"] == 'Yes'][['maskedID',
 'site',
 'visit_id',
 'nosp_date',
 u'nosp_hosp_bool',
 u'nosp_hosp_start',
 u'nosp_hosp_end',
 u'nosp_hosp_reason']]

A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])

A["OPHID_Text"] = np.nan
A["hosp_date_rep"] = np.array(df2["nosp_date"])
A["visit_id"] = np.array(df2["visit_id"])
A["hosp_start_date"] = np.array(df2["nosp_hosp_start"])
A["hosp_end_date"] = np.array(df2["nosp_hosp_end"])
A["hosp_reason"] = np.array(df2["nosp_hosp_reason"])
A["REDCapExport"] = 'NonOcularSurgeriesProcedures'
A["source"] = 'REDCap'
A["site"] = np.array(df2["maskedID"].str[:2])
A2 = A.copy(deep=True)




df1 = pd.read_csv(redcapeyechart + extension + '_NonOcularConditions.csv')
df1["noc_date"] = pd.to_datetime(df1["noc_date"],  errors='coerce').dt.strftime('%m/%d/%Y')

df2 = df1[df1["noc_hosp_bool"] == 'Yes'][['maskedID',
 'site',
 'visit_id',
 'noc_date',
 'noc_hosp_bool',
 'noc_hosp_start',
 'noc_hosp_end',
 'noc_hosp_reason']]


A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])

A["OPHID_Text"] = np.nan
A["hosp_date_rep"] = np.array(df2["noc_date"])
A["visit_id"] = np.array(df2["visit_id"])
A["hosp_start_date"] = np.array(df2["noc_hosp_start"])
A["hosp_end_date"] = np.array(df2["noc_hosp_end"])
A["hosp_reason"] = np.array(df2["noc_hosp_reason"])
A["REDCapExport"] = 'NonOcularConditions'
A["source"] = 'REDCap'
A["site"] = np.array(df2["maskedID"].str[:2])
A3 = A.copy(deep=True)





df1 = pd.read_csv(redcapeyechart + extension +  '_OcularSurgeriesProcedures.csv')

df1 = df1[~df1["maskedID"].isnull()]


df2 = df1[(df1["hospocular_cond"] == 'Yes')][['maskedID',
 'site',
 'visit_id',
 'osp_date',
 'hospocular_cond',
 'hospocular_condstrtdate',
 'hospocular_condenddate',
 'hosocular_reason']]

A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])


A["OPHID_Text"] = np.nan
A["hosp_date_rep"] = np.array(df2["osp_date"])
A["visit_id"] = np.array(df2["visit_id"])
A["hosp_start_date"] = np.array(df2["hospocular_condstrtdate"])
A["hosp_end_date"] = np.array(df2["hospocular_condenddate"])
A["hosp_reason"] = np.array(df2["hosocular_reason"])
A["REDCapExport"] = 'OcularSurgeriesProcedures'
A["source"] = 'REDCap'
A["site"] = np.array(df2["maskedID"].str[:2])
A4 = A.copy(deep=True)

df['REDCapExport'] = np.nan
df['visit_id'] = np.nan
H = list(df.columns.values)




FIN = pd.concat([df, A1[H]], ignore_index=True)
FIN = pd.concat([FIN, A2[H]], ignore_index=True)
FIN = pd.concat([FIN, A3[H]], ignore_index=True)
FIN = pd.concat([FIN, A4[H]], ignore_index=True)


FIN = pd.merge(FIN, ind[["OPHID_Text", "researchid"]], how='left', on='OPHID_Text')





FIN = FIN[~FIN['maskedid'].isnull()]

FIN = pd.merge(FIN, ind[["OPHID_Text_1", "maskedid"]], how='left', on='maskedid')
FIN["OPHID_Text"] = np.where(FIN["OPHID_Text"].isnull(), FIN["OPHID_Text_1"], FIN["OPHID_Text"])

FIN['maskedid'] = np.where(FIN['maskedid'].isnull(), FIN['researchid'], FIN['maskedid'])
FIN['BioBID'] = np.where(FIN['researchid'].isnull(), FIN['maskedid'], FIN['researchid'])
FIN['BioBID'] = np.where( (FIN['researchid'].str.contains('OG')) | (FIN['researchid'].str.contains('ED')), FIN['BioBID'], np.nan)
FIN["site"] = FIN['maskedid'].str[:2]
FIN = FIN[name]
FIN = FIN.fillna('')
FIN.to_csv(redcapeyechart + 'Results/' + extension + '_Hospitalization_RedEC.csv', encoding='utf-8', index=False)



In [52]:
L1 = [i for i in L_name if '_Keratometry' in i]
df = pd.read_csv(EyeChart + L1[0], encoding = "ISO-8859-1")
#df = pd.read_csv(EyeChart + '20170322_Hospitalization.csv', encoding = "ISO-8859-1")
df["site"] = np.array(df['maskedid'].str[:2])
df["source"] = 'EyeChart'

name = 'maskedid	OPHID_Text	BioBID	site	visit_id	k_date	k_eye	k_operator	et_kerat_unit	k_meas_1	k_meas_2	k_avg	k_stddev	k_comments'
name = name.split('\t')
name = name + ['source']

df1 = pd.read_csv(redcapeyechart + extension + '_Keratometry.csv')

df2 = df1[df1['et_keratom'].isin(np.array(['OD', 'OU']))]

A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])


A["OPHID_Text"] = np.nan

A["visit_id"] = np.array(df2["visit_id"])
A["k_date"] = np.array(df2["et_date"])
A["k_eye"] = "OD"
A["k_operator"] = np.array(df2["et_staff"])

A["et_kerat_unit"] = np.array(df2["et_kerat_unit"])

A["k_meas_1"] = np.where(df2["et_kerat_unit"]=='Millimeters',df2["et_od_keratom1mm"], np.nan)
A["k_meas_1"] = np.where(df2["et_kerat_unit"]=='Diopters',df2["et_od_kertaom1diop"], A["k_meas_1"])

A["k_meas_2"] = np.where(df2["et_kerat_unit"]=='Millimeters',df2["et_od_keratom2mm"], np.nan)
A["k_meas_2"] = np.where(df2["et_kerat_unit"]=='Diopters',df2["et_od_kertaom2diop"], A["k_meas_2"])


A["k_avg"] = np.nan
A["k_stddev"] = np.nan
A["k_comments"] = np.array(df2["et_keratomnotes"])

A["source"] = 'REDCap'
A["site"] = np.array(df2["maskedID"].str[:2])

B = A.copy(deep=True)

df2 = df1[df1['et_keratom'].isin(np.array(['OS', 'OU']))]

A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])


A["OPHID_Text"] = np.nan

A["visit_id"] = np.array(df2["visit_id"])
A["k_date"] = np.array(df2["et_date"])
A["k_eye"] = "OS"
A["k_operator"] = np.array(df2["et_staff"])

A["et_kerat_unit"] = np.array(df2["et_kerat_unit"])

A["k_meas_1"] = np.where(df2["et_kerat_unit"]=='Millimeters',df2["et_os_keratom1mm"], np.nan)
A["k_meas_1"] = np.where(df2["et_kerat_unit"]=='Diopters',df2["et_os_kertaom1diop"], A["k_meas_1"])

A["k_meas_2"] = np.where(df2["et_kerat_unit"]=='Millimeters',df2["et_os_keratom2mm"], np.nan)
A["k_meas_2"] = np.where(df2["et_kerat_unit"]=='Diopters',df2["et_os_kertaom2diop"], A["k_meas_2"])


A["k_avg"] = np.nan
A["k_stddev"] = np.nan
A["k_comments"] = np.array(df2["et_keratomnotes"])

A["source"] = 'REDCap'
A["site"] = np.array(df2["maskedID"].str[:2])


df['visit_id'] = np.nan
df['et_kerat_unit'] = np.nan
H = list(df.columns.values)

FIN = pd.concat([df, A[H]], ignore_index=True)
FIN = pd.concat([FIN, B[H]], ignore_index=True)


FIN = pd.merge(FIN, ind[["OPHID_Text", "researchid"]], how='left', on='OPHID_Text')





FIN = FIN[~FIN['maskedid'].isnull()]

FIN = pd.merge(FIN, ind[["OPHID_Text_1", "maskedid"]], how='left', on='maskedid')
FIN["OPHID_Text"] = np.where(FIN["OPHID_Text"].isnull(), FIN["OPHID_Text_1"], FIN["OPHID_Text"])

FIN['maskedid'] = np.where(FIN['maskedid'].isnull(), FIN['researchid'], FIN['maskedid'])
FIN['BioBID'] = np.where(FIN['researchid'].isnull(), FIN['maskedid'], FIN['researchid'])
FIN['BioBID'] = np.where( (FIN['researchid'].str.contains('OG')) | (FIN['researchid'].str.contains('ED')), FIN['BioBID'], np.nan)
FIN["site"] = FIN['maskedid'].str[:2]
FIN = FIN[name]
FIN = FIN.fillna('')
FIN.to_csv(redcapeyechart + 'Results/' + extension + '_Keratometry_RedEC.csv', encoding='utf-8', index=False)



In [53]:
FIN[FIN['maskedid'] == 'AL0491']

,maskedid,OPHID_Text,BioBID,site,visit_id,k_date,k_eye,k_operator,et_kerat_unit,k_meas_1,k_meas_2,k_avg,k_stddev,k_comments,source
4352,AL0491,OPH-102843,,AL,,09/26/2003,OD,,,7.8,8.0,7.9,0.1,,EyeChart
4353,AL0491,OPH-102843,,AL,,09/26/2003,OS,,,7.76,7.85,7.81,0.05,,EyeChart
8621,AL0491,,,AL,Baseline,2018-11-09,OS,Onsite Staff,Diopters,43.3,43.74,,,,REDCap
8872,AL0491,,,AL,Baseline,2018-11-09,OD,Onsite Staff,Diopters,42.62,43.13,,,,REDCap


In [54]:
name_1 = 'dob	gender	dod	ethnicity_form_date	ethnicity	caribbean_other	hispanic_other	race_black	race_caucasian	race_american_indian_or_alaskan_native	race_asian_chinese	race_asian_filipino	race_asian_indian	race_asian_japanese	race_asian_korean	race_asian_vietnamese	race_asian_other	asian_other	race_asian_group_unknown	race_asian_group_declined	race_middle_eastern	race_native_hawaiian_or_pacific_islander	race_other	race_other_spec	race_unknown	race_decline	race_primary	race_primary_nih	jewish		jewish_other	birthplace_country	birthplace_us_state	birthplace_specific_country	birthplace_specific_country_other	year_entered_usa_bool	year_entered_usa	mom_ethnicity	mom_caribbean_other	mom_hispanic_other	mom_race_black	mom_race_caucasian	mom_race_american_indian_or_alaskan_native	mom_race_asian_chinese	mom_race_asian_filipino	mom_race_asian_indian	mom_race_asian_japanese	mom_race_asian_korean	mom_race_asian_vietnamese	mom_race_asian_other	mom_asian_other	mom_race_asian_group_unknown	mom_race_asian_group_declined	mom_race_middle_eastern	mom_race_native_hawaiian_or_pacific_islander	mom_race_other	mom_race_other_spec	mom_race_unknown	mom_race_decline	mom_race_primary	mom_race_primary_nih	mom_jewish		mom_jewish_other	mom_birthplace_country	mom_birthplace_us_state	mom_birthplace_specific_country	mom_birthplace_specific_country_other	mom_entered_usa	mom_year_entered_usa_bool	mom_year_entered_usa	dad_ethnicity	dad_caribbean_other	dad_hispanic_other	dad_race_black	dad_race_caucasian	dad_race_american_indian_or_alaskan_native	dad_race_asian_chinese	dad_race_asian_filipino	dad_race_asian_indian	dad_race_asian_japanese	dad_race_asian_korean	dad_race_asian_vietnamese	dad_race_asian_other	dad_asian_other	dad_race_asian_group_unknown	dad_race_asian_group_declined	dad_race_middle_eastern	dad_race_native_hawaiian_or_pacific_islander	dad_race_other	dad_race_other_spec	dad_race_unknown	dad_race_decline	dad_race_primary	dad_race_primary_nih	dad_jewish		dad_jewish_other	dad_birthplace_country	dad_birthplace_us_state	dad_birthplace_specific_country	dad_birthplace_specific_country_other	dad_entered_usa	dad_year_entered_usa_bool	dad_year_entered_usa	mm_born	mm_state	mm_othercountry	mm_otherspecify	mm_entry_year	mm_year_bool	mm_yearusa	mf_born	mf_state	mf_othercountry	mf_otherspecify	mf_entry_year	mf_year_bool	mf_yearusa	fm_born	fm_state	fm_othercountry	fm_otherspecify	fm_entry_year	fm_year_bool	fm_yearusa	ff_born	ff_state	ff_othercountry	ff_otherspecify	ff_entry_year	ff_year_bool	ff_yearusa'
name_1 = name_1.split('\t')

In [55]:
name_2 = 'dob	gender	date_of_death	XXX	ethnicity	XXX	ethnicity_hispanic_other_info	race_black_or_african_american	race_caucasian	race_american_indian_or_alaska_native	race_asian_chinese	race_asian_filipino	race_asian_indian	race_asian_japanese	race_asian_korean	race_asian_vietnamese	race_asian_other	race_asian_other_info	race_asian_group_unknown	race_asian_group_declined	race_middle_eastern	race_native_hawaiian_or_other_pacific_islander	XXX	XXX	XXX	race_declined	race_primary	race_nih	jewishanc_notjewish		jewishanc_other_info	birthplace_country	birthplace_usstate	XXX	XXX	XXX	arrived_in_us_year	ethnicity_mother	XXX	ethnicity_mother_hispanic_other_info	race_mother_black_or_african_american	race_mother_caucasian	race_mother_american_indian_or_alaska_native	race_mother_asian_chinese	race_mother_asian_filipino	race_mother_asian_indian	race_mother_asian_japanese	race_mother_asian_korean	race_mother_asian_vietnamese	race_mother_asian_other	race_mother_asian_other_info	race_mother_asian_group_unknown	race_mother_asian_group_declined	race_mother_middle_eastern	race_mother_native_hawaiian_or_other_pacific_islander	XXX	XXX	XXX	race_mother_declined	race_mother_primary	XXX	jewishanc_mother_notjewish		jewishanc_mother_other_info	birthplace_mother_country	birthplace_mother_usstate	XXX	XXX	XXX	XXX	arrived_in_us_year_mother	ethnicity_father	XXX	ethnicity_father_hispanic_other_info	race_father_black_or_african_american	race_father_caucasian	race_father_american_indian_or_alaska_native	race_father_asian_chinese	race_father_asian_filipino	race_father_asian_indian	race_father_asian_japanese	race_father_asian_korean	race_father_asian_vietnamese	race_father_asian_other	race_father_asian_other	race_father_asian_group_unknown	race_father_asian_group_declined	race_father_middle_eastern	race_father_native_hawaiian_or_other_pacific_islander	XXX	XXX	XXX	race_father_declined	race_father_primary	XXX	jewishanc_father_notjewish		jewishanc_father_other_info	birthplace_father_country	birthplace_father_usstate	XXX	XXX	XXX	XXX	arrived_in_us_year_father	birthplace_matgm_country	birthplace_matgm_usstate	XXX	XXX	XXX	XXX	arrived_in_us_year_matgm	birthplace_matgf_country	birthplace_matgf_usstate	XXX	XXX	XXX	XXX	arrived_in_us_year_matgf	birthplace_patgm_country	birthplace_patgm_usstate	XXX	XXX	XXX	XXX	arrived_in_us_year_patgm	birthplace_patgf_country	birthplace_patgf_usstate	XXX	XXX	XXX	XXX	arrived_in_us_year_patgf'
name_2 = name_2.split('\t')

In [56]:
L1 = [i for i in L_name if '_DemographicswClass' in i]
df = pd.read_csv(EyeChart + L1[0], encoding = "ISO-8859-1")
#df = pd.read_csv(EyeChart + '20170322_Hospitalization.csv', encoding = "ISO-8859-1")
df["site"] = np.array(df['maskedid'].str[:2])
df["source"] = 'EyeChart'


name = 'maskedid	OPHID_Text	BioBID	site	dob	gender	dod	ethnicity_form_date	ethnicity	caribbean_other	hispanic_other	race_black	race_caucasian	race_american_indian_or_alaskan_native	race_asian_chinese	race_asian_filipino	race_asian_indian	race_asian_japanese	race_asian_korean	race_asian_vietnamese	race_asian_other	asian_other	race_asian_group_unknown	race_asian_group_declined	race_middle_eastern	race_native_hawaiian_or_pacific_islander	race_other	race_other_spec	race_unknown	race_decline	race_primary	race_primary_nih	jewish		jewish_other	birthplace_country	birthplace_us_state	birthplace_specific_country	birthplace_specific_country_other	year_entered_usa_bool	year_entered_usa	mom_ethnicity	mom_caribbean_other	mom_hispanic_other	mom_race_black	mom_race_caucasian	mom_race_american_indian_or_alaskan_native	mom_race_asian_chinese	mom_race_asian_filipino	mom_race_asian_indian	mom_race_asian_japanese	mom_race_asian_korean	mom_race_asian_vietnamese	mom_race_asian_other	mom_asian_other	mom_race_asian_group_unknown	mom_race_asian_group_declined	mom_race_middle_eastern	mom_race_native_hawaiian_or_pacific_islander	mom_race_other	mom_race_other_spec	mom_race_unknown	mom_race_decline	mom_race_primary	mom_race_primary_nih	mom_jewish		mom_jewish_other	mom_birthplace_country	mom_birthplace_us_state	mom_birthplace_specific_country	mom_birthplace_specific_country_other	mom_entered_usa	mom_year_entered_usa_bool	mom_year_entered_usa	dad_ethnicity	dad_caribbean_other	dad_hispanic_other	dad_race_black	dad_race_caucasian	dad_race_american_indian_or_alaskan_native	dad_race_asian_chinese	dad_race_asian_filipino	dad_race_asian_indian	dad_race_asian_japanese	dad_race_asian_korean	dad_race_asian_vietnamese	dad_race_asian_other	dad_asian_other	dad_race_asian_group_unknown	dad_race_asian_group_declined	dad_race_middle_eastern	dad_race_native_hawaiian_or_pacific_islander	dad_race_other	dad_race_other_spec	dad_race_unknown	dad_race_decline	dad_race_primary	dad_race_primary_nih	dad_jewish		dad_jewish_other	dad_birthplace_country	dad_birthplace_us_state	dad_birthplace_specific_country	dad_birthplace_specific_country_other	dad_entered_usa	dad_year_entered_usa_bool	dad_year_entered_usa	mm_born	mm_state	mm_othercountry	mm_otherspecify	mm_entry_year	mm_year_bool	mm_yearusa	mf_born	mf_state	mf_othercountry	mf_otherspecify	mf_entry_year	mf_year_bool	mf_yearusa	fm_born	fm_state	fm_othercountry	fm_otherspecify	fm_entry_year	fm_year_bool	fm_yearusa	ff_born	ff_state	ff_othercountry	ff_otherspecify	ff_entry_year	ff_year_bool	ff_yearusa'

name = [i for i in name.split('\t') if i !='']
name = name + ['source']
df2 = pd.read_csv(redcapeyechart + extension + '_DemoData.csv')
df2["source"] = 'REDCap'
df2["site"] = np.array(df2["maskedID"].str[:2])
df2 = df2.rename(columns={'maskedID': 'maskedid'})
A = pd.DataFrame(np.array(df["maskedid"]), columns=["maskedid"])


A["OPHID_Text"] = np.nan
df2["OPHID_Text"] = np.nan

A["site"] = np.array(df["site"])
A['source'] = 'EyeChart'
for i in range(len(name_1)):
    if name_1[i]=='asian_other':
        A[name_1[i]] = np.where(df['race_asian_group_not_reported']==1, 'NOT REPORTED', np.nan)
        A[name_1[i]] = np.where(df['race_asian_group_not_reported']==0, np.nan, A[name_1[i]])
        A[name_1[i]] = np.where(df['race_asian_group_not_reported'].isnull(), np.nan, df[name_2[i]])
    elif name_1[i]=='race_primary':
        A[name_1[i]] = np.where(df['race_not_reported']==1, 'NOT REPORTED', np.nan)
        A[name_1[i]] = np.where(df['race_not_reported']==0, np.nan, A[name_1[i]])
        A[name_1[i]] = np.where(df['race_not_reported'].isnull(), np.nan, df[name_2[i]])
    elif name_1[i]=='jewish':
        A[name_1[i]] = np.where(df['jewishanc_notjewish']==1, 'Not Jewish', np.nan)
        A[name_1[i]] = np.where(df['jewishanc_ashkenazi']==1, 'Ashkenazi', A[name_1[i]])
        A[name_1[i]] = np.where(df['jewishanc_sephardi']==1, 'Sephardi', A[name_1[i]])
        A[name_1[i]] = np.where(df['jewishanc_other']==1, 'Other', A[name_1[i]])
        A[name_1[i]] = np.where(df['jewishanc_unknown']==1, 'Unknown', A[name_1[i]])
        A[name_1[i]] = np.where(df['jewishanc_notreported']==1, 'Not Reported', A[name_1[i]])
        A[name_1[i]] = np.where(df['jewishanc_declined']==1, 'Decline', A[name_1[i]])
        A[name_1[i]] = np.where(df['race_not_reported']==0, np.nan, A[name_1[i]])
    
    elif name_1[i]=='mom_asian_other':
        A[name_1[i]] = np.where(df['race_mother_asian_group_not_reported']==1, 'NOT REPORTED', np.nan)
        A[name_1[i]] = np.where(df['race_mother_asian_group_not_reported']==0, np.nan, A[name_1[i]])
        A[name_1[i]] = np.where(df['race_mother_asian_group_not_reported'].isnull(), np.nan, df[name_2[i]])
     
    elif name_1[i]=='mom_race_primary':
        A[name_1[i]] = np.where(df['race_mother_not_reported']==1, 'NOT REPORTED', np.nan)
        A[name_1[i]] = np.where(df['race_mother_not_reported']==0, np.nan, A[name_1[i]])
        A[name_1[i]] = np.where(df['race_mother_not_reported'].isnull(), np.nan, df[name_2[i]])
    elif name_1[i]=='mom_jewish':
        A[name_1[i]] = np.where(df['jewishanc_mother_notjewish']==1, 'Not Jewish', np.nan)
        A[name_1[i]] = np.where(df['jewishanc_mother_ashkenazi']==1, 'Ashkenazi', A[name_1[i]])
        A[name_1[i]] = np.where(df['jewishanc_mother_sephardi']==1, 'Sephardi', A[name_1[i]])
        A[name_1[i]] = np.where(df['jewishanc_mother_other']==1, 'Other', A[name_1[i]])
        A[name_1[i]] = np.where(df['jewishanc_mother_unknown']==1, 'Unknown', A[name_1[i]])
        A[name_1[i]] = np.where(df['jewishanc_mother_notreported']==1, 'Not Reported', A[name_1[i]])
        A[name_1[i]] = np.where(df['jewishanc_mother_declined']==1, 'Decline', A[name_1[i]])
        A[name_1[i]] = np.where(df['race_mother_not_reported']==0, np.nan, A[name_1[i]])
    elif name_1[i]=='dad_asian_other':
        A[name_1[i]] = np.where(df['race_father_asian_group_not_reported']==1, 'NOT REPORTED', np.nan)
        A[name_1[i]] = np.where(df['race_father_asian_group_not_reported']==0, np.nan, A[name_1[i]])
        A[name_1[i]] = np.where(df['race_father_asian_group_not_reported'].isnull(), np.nan, df[name_2[i]])
     
    elif name_1[i]=='dad_race_primary':
        A[name_1[i]] = np.where(df['race_father_not_reported']==1, 'NOT REPORTED', np.nan)
        A[name_1[i]] = np.where(df['race_father_not_reported']==0, np.nan, A[name_1[i]])
        A[name_1[i]] = np.where(df['race_father_not_reported'].isnull(), np.nan, df[name_2[i]])
    elif name_1[i]=='dad_jewish':
        A[name_1[i]] = np.where(df['jewishanc_father_notjewish']==1, 'Not Jewish', np.nan)
        A[name_1[i]] = np.where(df['jewishanc_father_ashkenazi']==1, 'Ashkenazi', A[name_1[i]])
        A[name_1[i]] = np.where(df['jewishanc_father_sephardi']==1, 'Sephardi', A[name_1[i]])
        A[name_1[i]] = np.where(df['jewishanc_father_other']==1, 'Other', A[name_1[i]])
        A[name_1[i]] = np.where(df['jewishanc_father_unknown']==1, 'Unknown', A[name_1[i]])
        A[name_1[i]] = np.where(df['jewishanc_father_notreported']==1, 'Not Reported', A[name_1[i]])
        A[name_1[i]] = np.where(df['jewishanc_father_declined']==1, 'Decline', A[name_1[i]])
        A[name_1[i]] = np.where(df['race_father_not_reported']==0, np.nan, A[name_1[i]])    
    elif name_1[i] !='':
        if name_2[i] !='XXX':
            A[name_1[i]] = np.array(df[name_2[i]])
        else:
            A[name_1[i]] = np.nan
                
H = list(df2.columns.values)

FIN = pd.concat([df2, A[H]], ignore_index=True)



FIN = pd.merge(FIN, ind[["OPHID_Text", "researchid"]], how='left', on='OPHID_Text')





FIN = FIN[~FIN['maskedid'].isnull()]

FIN = pd.merge(FIN, ind[["OPHID_Text_1", "maskedid"]], how='left', on='maskedid')
FIN["OPHID_Text"] = np.where(FIN["OPHID_Text"].isnull(), FIN["OPHID_Text_1"], FIN["OPHID_Text"])

FIN['maskedid'] = np.where(FIN['maskedid'].isnull(), FIN['researchid'], FIN['maskedid'])
FIN['BioBID'] = np.where(FIN['researchid'].isnull(), FIN['maskedid'], FIN['researchid'])
FIN['BioBID'] = np.where( (FIN['researchid'].str.contains('OG')) | (FIN['researchid'].str.contains('ED')), FIN['BioBID'], np.nan)
FIN["site"] = FIN['maskedid'].str[:2]
FIN = FIN[name]
FIN = FIN.fillna('')
FIN = FIN.drop_duplicates('maskedid')
FIN.to_csv(redcapeyechart + 'Results/' + extension + '_DemoData_RedEC.csv', encoding='utf-8', index=False)

/Users/nicolebrye/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (4,12,27,35,40,57,64,69,86,93,98,104,105,108,109,112,113,117,120,124,128,130,131) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/nicolebrye/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (7,26,34,39,72,121) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/var/folders/7m/n2zn61593k18rvyz907d2skc0000gn/T/ipykernel_30504/2634119251.py:81: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  A[name_1[i]] = np.array(df[name_2[i]])
/var

ValueError: You are trying to merge on float64 and object columns. If you wish to proceed you should use pd.concat

In [ ]:
FIN

In [ ]:
name_3 = 'maskedid	OPHID_Text	site	dob	gender	date_of_death	marital_status	has_insurance	education_highest	income_level	num_people_in_household	other_languages_spoken'
name_3 = name_3.split('\t')

L1 = [i for i in L_name if '_DemographicswClass' in i]
df = pd.read_csv(EyeChart + L1[0], encoding = "ISO-8859-1")
df = df[name_3]
#df = pd.read_csv(EyeChart + '20170322_Hospitalization.csv', encoding = "ISO-8859-1")
df["site"] = np.array(df['maskedid'].str[:2])
df["source"] = 'EyeChart'
df["visit_id"] = np.nan

name = 'maskedid	OPHID_Text	BioBID	site	visit_id	dob	gender	date_of_death	soec_date	marital_status	has_insurance	education_highest	income_level	num_people_in_household	other_languages_spoken'
name = name.split('\t')
name = name + ['source']
df2 = pd.read_csv(redcapeyechart + extension + '_SocioEconomic.csv')
df2["source"] = 'REDCap'
df2["site"] = np.array(df2["maskedID"].str[:2])
df2 = df2[df2['soec_date'].notnull()]


A = pd.DataFrame(np.array(df2["maskedID"]), columns=["maskedid"])


A["OPHID_Text"] = np.nan

A["visit_id"] = np.array(df2["visit_id"])
A["dob"] = np.array(df2["dob"])
A["gender"] = np.array(df2["gender"])
A["date_of_death"] = np.array(df2["dod"])

A["soec_date"] = np.array(df2["soec_date"])
A["marital_status"] = np.array(df2["marital"])
A["has_insurance"] = np.array(df2["insurance"])

A["education_highest"] = np.array(df2["education"])
A["income_level"] = np.array(df2["income"])
A["num_people_in_household"] = np.array(df2["house"])
A["other_languages_spoken"] = np.array(df2["other_than_english"])
A["other_languages_spoken"] = np.where((df2['other_than_english']=="Yes") & (df2['lang_spanish']==1), 'spanish', A["other_languages_spoken"])
A["other_languages_spoken"] = np.where((df2['other_than_english']=="Yes") & (df2['lang_french']==1), 'french', A["other_languages_spoken"])

A["other_languages_spoken"] = np.where((df2['other_than_english']=="Yes") & (df2['lang_american_sign_language']==1), 'american_sign_language', A["other_languages_spoken"])


A["other_languages_spoken"] = np.where((df2['other_than_english']=="Yes") & (df2['lang_german']==1), 'german', A["other_languages_spoken"])

A["other_languages_spoken"] = np.where((df2['other_than_english']=="Yes") & (df2['lang_italian']==1), 'italian', A["other_languages_spoken"])

A["other_languages_spoken"] = np.where((df2['other_than_english']=="Yes") & (df2['lang_japanese']==1), 'japanese', A["other_languages_spoken"])

A["other_languages_spoken"] = np.where((df2['other_than_english']=="Yes") & (df2['lang_chinese']==1), 'chinese', A["other_languages_spoken"])

A["other_languages_spoken"] = np.where((df2['other_than_english']=="Yes") & (df2['lang_korean']==1), 'korean', A["other_languages_spoken"])

A["other_languages_spoken"] = np.where((df2['other_than_english']=="Yes") & (df2['lang_american_sign_language']==1), 'american_sign_language', A["other_languages_spoken"])


A["other_languages_spoken"] = np.where((df2['other_than_english']=="Yes") & (df2['lang_vietnamese']==1), 'vietnamese', A["other_languages_spoken"])

A["other_languages_spoken"] = np.where((df2['other_than_english']=="Yes") & (df2['lang_arabic']==1), 'arabic', A["other_languages_spoken"])

A["other_languages_spoken"] = np.where((df2['other_than_english']=="Yes") & (df2['lang_farsi']==1), 'farsi', A["other_languages_spoken"])

A["other_languages_spoken"] = np.where((df2['other_than_english']=="Yes") & (df2['lang_latin']==1), 'latin', A["other_languages_spoken"])

A["other_languages_spoken"] = np.where((df2['other_than_english']=="Yes") & (df2['lang_russian']==1), 'russian', A["other_languages_spoken"])

A["other_languages_spoken"] = np.where((df2['other_than_english']=="Yes") & (df2['lang_greek']==1), 'greek', A["other_languages_spoken"])

A["other_languages_spoken"] = np.where((df2['other_than_english']=="Yes") & (df2['lang_portuguese']==1), 'portuguese', A["other_languages_spoken"])

A["other_languages_spoken"] = np.where((df2['other_than_english']=="Yes") & (df2['lang_hebrew']==1), 'hebrew', A["other_languages_spoken"])

A["other_languages_spoken"] = np.where((df2['other_than_english']=="Yes") & (df2['lang_other']==1), df2['lang_other_specify'], A["other_languages_spoken"])
df2['multiple_languages'] = df2[['lang_spanish', 'lang_french',
       'lang_american_sign_language', 'lang_german', 'lang_italian',
       'lang_japanese', 'lang_chinese', 'lang_korean', 'lang_vietnamese',
       'lang_arabic', 'lang_farsi', 'lang_latin', 'lang_russian',
       'lang_greek', 'lang_portuguese', 'lang_hebrew', 'lang_other']].fillna(0).apply(sum,axis=1)

A["other_languages_spoken"] = np.where((df2['other_than_english']=="Yes") & (df2['multiple_languages']>1), 'Multiple languages', A["other_languages_spoken"])


A["source"] = 'REDCap'
A["site"] = np.array(df2["maskedID"].str[:2])


df['visit_id'] = np.nan
df['soec_date'] = np.nan


H = list(df.columns.values)

FIN = pd.concat([df, A[H]], ignore_index=True)



FIN = pd.merge(FIN, ind[["OPHID_Text", "researchid"]], how='left', on='OPHID_Text')





FIN = FIN[~FIN['maskedid'].isnull()]

FIN = pd.merge(FIN, ind[["OPHID_Text_1", "maskedid"]], how='left', on='maskedid')
FIN["OPHID_Text"] = np.where(FIN["OPHID_Text"].isnull(), FIN["OPHID_Text_1"], FIN["OPHID_Text"])

FIN['maskedid'] = np.where(FIN['maskedid'].isnull(), FIN['researchid'], FIN['maskedid'])
FIN['BioBID'] = np.where(FIN['researchid'].isnull(), FIN['maskedid'], FIN['researchid'])
FIN['BioBID'] = np.where( (FIN['researchid'].str.contains('OG')) | (FIN['researchid'].str.contains('ED')), FIN['BioBID'], np.nan)
FIN["site"] = FIN['maskedid'].str[:2]
FIN = FIN[name]
FIN = FIN.fillna('')
FIN.to_csv(redcapeyechart + 'Results/' + extension + '_SocioEconomic_RedEC.csv', encoding='utf-8', index=False)

In [ ]:
FIN